Copyright 2025 Google DeepMind Technologies Limited

All software is licensed under the Apache License, Version 2.0 (Apache 2.0);
you may not use this file except in compliance with the Apache 2.0 license.
You may obtain a copy of the Apache 2.0 license at:
https://www.apache.org/licenses/LICENSE-2.0

All other materials are licensed under the Creative Commons Attribution 4.0
International License (CC-BY). You may obtain a copy of the CC-BY license at:
https://creativecommons.org/licenses/by/4.0/legalcode

Unless required by applicable law or agreed to in writing, all software and
materials distributed here under the Apache 2.0 or CC-BY licenses are
distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND,
either express or implied. See the licenses for the specific language governing
permissions and limitations under those licenses.

This is not an official Google product.

# Tensor decompositions (algorithms for matrix multiplication)

This section contains several low-rank tensor decompositions, discovered by AlphaEvolve, and the code to verify their correctness. A tensor with parameters $\langle n,m,p\rangle$ corresponds to multiplying an $n \times m$ and an $m \times p$ matrix, and the rank of the decomposition corresponds to the number of scalar multiplications needed to perform the matrix multiplication.

In [ ]:
#@title Verification function
import numpy as np

def verify_tensor_decomposition(decomposition: tuple[np.ndarray, np.ndarray, np.ndarray], n: int, m: int, p: int, rank: int):
  """Verifies the correctness of the tensor decomposition.

  Args:
    decomposition: a tuple of 3 factor matrices with the same number of columns.
      (The number of columns specifies the rank of the decomposition.) To
      construct a tensor, we take the outer product of the i-th column of the
      three factor matrices, for 1 <= i <= rank, and add up all these outer
      products.
    n: the first parameter of the matrix multiplication tensor.
    m: the second parameter of the matrix multiplication tensor.
    p: the third parameter of the matrix multiplication tensor.
    rank: the expected rank of the decomposition.

  Raises:
    AssertionError: If the decomposition does not have the correct rank, or if
    the decomposition does not construct the 3D tensor which corresponds to
    multiplying an n x m matrix by an m x p matrix.
  """
  # Check that each factor matrix has the correct shape.
  factor_matrix_1, factor_matrix_2, factor_matrix_3 = decomposition
  assert factor_matrix_1.shape == (n * m, rank), f'Expected shape of factor matrix 1 is {(n * m, rank)}. Actual shape is {factor_matrix_1.shape}.'
  assert factor_matrix_2.shape == (m * p, rank), f'Expected shape of factor matrix 1 is {(m * p, rank)}. Actual shape is {factor_matrix_2.shape}.'
  assert factor_matrix_3.shape == (p * n, rank), f'Expected shape of factor matrix 1 is {(p * n, rank)}. Actual shape is {factor_matrix_3.shape}.'

  # Form the matrix multiplication tensor <n, m, p>.
  matmul_tensor = np.zeros((n * m, m * p, p * n), dtype=np.int32)
  for i in range(n):
    for j in range(m):
      for k in range(p):
        matmul_tensor[i * m + j][j * p + k][k * n + i] = 1

  # Check that the tensor is correctly constructed.
  constructed_tensor = np.einsum('il,jl,kl -> ijk', *decomposition)
  assert np.array_equal(constructed_tensor, matmul_tensor), f'Tensor constructed by decomposition does not match the matrix multiplication tensor <{(n,m,p)}>: {constructed_tensor}.'
  print(f'Verified a decomposition of rank {rank} for matrix multiplication tensor <{n},{m},{p}>.')

  # Print the set of values used in the decomposition.
  np.set_printoptions(linewidth=100)
  print('This decomposition uses these factor entries:\n', np.array2string(np.unique(np.vstack((factor_matrix_1, factor_matrix_2, factor_matrix_3))), separator=', '))

## Rank-32 decomposition of <2,4,5> over 0.5*Z

In [ ]:
#@title Data

decomposition_245 = (np.array([[-0. ,  0.5, -0. ,  0.5,  0. ,  0. ,  0.5,  0. , -0.5, -0.5, -0. ,
        -0.5, -0. , -0. ,  0.5,  0. ,  0.5, -1. , -0.5, -0. , -0.5, -0. ,
         0.5, -0. , -0.5,  0.5, -1. ,  0. ,  0.5,  0. ,  0.5, -0. ],
       [ 0.5, -0.5,  0. ,  0.5, -0. ,  0.5, -0.5, -0. ,  0. , -0.5,  0. ,
         0. , -0.5,  0.5,  0. ,  0.5,  0. , -0. , -0. , -0.5, -0. ,  0. ,
         0. ,  0.5, -0.5,  0. , -0. ,  0. ,  0. ,  0. , -0. ,  0.5],
       [ 0. , -0. , -0.5, -0. ,  0.5, -0.5, -0. ,  0. , -0. ,  0. ,  0. ,
         0.5, -0.5, -0. ,  0.5,  0. ,  0.5, -0.5,  0.5,  0. ,  0. , -0.5,
        -0.5,  0. , -0. ,  0.5,  0.5, -0. ,  0. , -0. ,  0. ,  0. ],
       [-0.5,  0. ,  0.5, -1. ,  0.5,  0. ,  0. , -0. , -0.5,  0. , -0. ,
         0. , -0. , -0.5, -0. ,  0.5, -0. , -0.5, -0. , -0.5, -0.5,  0.5,
        -0. ,  0.5, -1. ,  0. ,  0.5, -0. , -0.5, -0. , -0.5, -0.5],
       [ 0. ,  0.5,  0. ,  0.5,  0. ,  0. , -0.5, -0. , -0.5, -0. , -0. ,
         0.5, -0. , -0. ,  0. , -0. , -0. ,  0. ,  0. , -0. , -0. , -0. ,
        -0. , -0. , -0.5, -0.5,  0. ,  0.5,  0.5, -0. ,  0. , -0. ],
       [-0.5, -0.5, -0. , -0.5, -0. , -0.5,  0.5, -0.5, -0. ,  0. ,  0.5,
         0. ,  0.5, -0.5, -0. , -0. ,  0.5, -0. , -0.5,  0.5, -0. , -0. ,
        -0. , -0.5,  0.5, -0. ,  0. ,  0.5,  0. ,  0. ,  0. , -0. ],
       [-0. ,  0. ,  0.5,  0. , -0. ,  0.5, -0. ,  0.5,  0. ,  0. ,  0.5,
        -0.5,  0.5, -0.5, -0. ,  0. , -0.5,  0.5, -0.5,  0. ,  0. , -0.5,
        -0. ,  0.5, -0. , -0.5, -0.5, -0. ,  0. , -0.5, -0. , -0. ],
       [ 0.5, -0. , -0.5, -0. , -0. , -0. ,  0. ,  0. , -0.5,  0. ,  0. ,
        -0. , -0. ,  0. , -0. , -0. , -0. , -0.5, -0. ,  0.5, -0. ,  0.5,
        -0. ,  0. , -0. , -0. ,  0.5,  0. , -0.5, -0.5,  0. , -0. ]],
      dtype=np.float32), np.array([[ 0.5, -0. , -0. ,  0. , -0. , -0.5,  0. ,  0. ,  1. , -0. , -0. ,
        -0. ,  0.5, -0.5,  0.5,  0. , -0. , -0.5, -0. ,  0.5, -0.5,  0. ,
        -0.5, -0.5,  0. , -0. ,  0.5,  0. , -1. ,  0. ,  0.5,  0. ],
       [ 0.5,  0. ,  0. ,  1. ,  0. ,  0.5,  2. ,  0. , -0. , -0. ,  0. ,
         0. , -0.5,  0.5, -0.5,  0. ,  0. ,  0.5, -0. ,  0.5,  0.5,  0. ,
         0.5,  0.5, -1. ,  0. , -0.5, -0. ,  0. , -0. , -0.5,  0. ],
       [ 0.5,  0. , -0. , -0. ,  0. , -0.5, -0. , -1. , -0. ,  0. ,  1. ,
         1. ,  0.5, -0.5, -0.5,  0. , -1. ,  0.5,  1. ,  0.5, -0.5, -0. ,
         0.5, -0.5, -0. ,  1. , -0.5, -2. , -0. , -0. ,  0.5, -0. ],
       [-0.5, -0. , -0. ,  0. ,  0. ,  0.5,  0. ,  0. , -0. ,  0. , -0. ,
        -1. , -0.5,  0.5, -0.5,  0. , -0. , -0.5,  0. , -0.5,  0.5,  0. ,
         0.5,  0.5, -0. , -1. ,  0.5, -0. ,  0. , -0. , -0.5,  0. ],
       [ 0. ,  0. ,  0. ,  1. , -0. , -1. ,  0. , -0. , -0. , -0. , -0. ,
        -1. , -1. , -0. , -0. , -1. , -1. , -0. , -1. , -0. , -1. , -0. ,
        -0. , -0. ,  1. ,  1. , -0. , -0. , -0. , -0. , -1. , -1. ],
       [ 0.5, -2. , -0. ,  1. ,  0. , -0.5,  0. ,  0. ,  1. ,  0. , -0. ,
         0. ,  0.5, -0.5,  0.5,  1. , -0. , -0.5, -0. ,  0.5,  0.5, -0. ,
        -0.5, -0.5, -1. , -0. ,  0.5, -0. , -1. , -0. , -0.5, -1. ],
       [ 0.5, -0. ,  0. ,  0. ,  0. ,  0.5,  0. , -0. , -0. , -0. ,  0. ,
        -0. , -0.5,  0.5, -0.5, -1. ,  0. ,  0.5, -0. ,  0.5, -0.5,  0. ,
         0.5,  0.5, -0. ,  0. , -0.5, -0. , -0. ,  0. ,  0.5,  1. ],
       [-0.5, -0. , -0. , -0. , -0. , -0.5,  0. ,  1. ,  0. ,  0. , -1. ,
         0. ,  0.5,  0.5,  0.5, -0. ,  0. , -0.5, -0. , -0.5,  0.5, -0. ,
        -0.5,  0.5, -0. ,  0. ,  0.5, -0. , -0. , -0. , -0.5,  0. ],
       [ 0.5,  0. , -0. , -0. , -0. ,  0.5,  0. ,  0. ,  0. ,  2. ,  0. ,
         0. , -0.5, -0.5,  0.5, -0. ,  1. ,  0.5, -1. ,  0.5, -0.5, -0. ,
        -0.5, -0.5, -0. ,  0. , -0.5, -0. , -0. ,  0. ,  0.5, -0. ],
       [ 0. , -0. , -0. , -1. ,  0. , -1. ,  0. ,  0. , -0. ,  0. , -0. ,
        -1. , -1. ,  0. , -0. ,  1. , -1. , -0. , -1. , -0. ,  1. , -0. ,
        -0. , -0. , -1. ,  1. , -0. , -0. , -0. , -0. ,  1. ,  1. ],
       [ 0. ,  0. , -0. , -0.5,  0. ,  0.5,  0. , -0. ,  1. ,  0. ,  0. ,
         0.5,  0.5,  0. ,  1. ,  0.5,  0.5, -1. ,  0.5, -0. ,  0.5,  2. ,
         1. , -0. , -0.5, -0.5, -1. ,  0. ,  1. , -0. ,  0.5,  0.5],
       [ 0. ,  0. , -0. ,  0.5,  2. , -0.5,  0. ,  0. , -0. ,  0. , -0. ,
         0.5, -0.5,  1. ,  0. ,  0.5,  0.5, -0. ,  0.5,  0. , -0.5, -0. ,
         0. , -1. ,  0.5, -0.5, -0. , -0. , -0. ,  0. , -0.5,  0.5],
       [-0. , -0. ,  0. , -0.5,  0. ,  0.5,  0. , -1. , -0. , -0. , -1. ,
        -0.5,  0.5, -0. ,  0. ,  0.5, -0.5,  0. , -0.5, -0. ,  0.5,  0. ,
        -0. ,  0. , -0.5,  0.5, -0. , -0. , -0. ,  0. ,  0.5,  0.5],
       [-0. ,  0. ,  0. ,  0.5, -0. , -0.5, -0. , -0. ,  0. ,  0. , -0. ,
         0.5, -0.5,  0. , -1. , -0.5, -0.5,  0. , -0.5, -0. , -0.5,  0. ,
        -1. , -0. ,  0.5, -0.5,  0. ,  0. ,  0. , -0. , -0.5, -0.5],
       [-1. ,  0. , -0. , -0. , -0. ,  1. , -0. , -0. ,  0. , -0. ,  0. ,
         0. , -1. ,  1. ,  1. ,  0. , -0. , -1. , -0. , -1. ,  1. ,  0. ,
        -1. ,  1. ,  0. , -0. ,  1. , -0. , -0. ,  0. , -1. , -0. ],
       [-0. , -0. ,  0. , -0.5, -0. ,  0.5, -0. ,  0. ,  1. ,  0. ,  0. ,
         0.5,  0.5,  0. , -0. ,  0.5,  0.5, -0. ,  0.5,  0. , -0.5, -0. ,
        -0. , -0. , -0.5, -0.5, -0. ,  0. ,  1. ,  0. , -0.5,  0.5],
       [-1. , -0. , -0. , -0.5,  0. , -0.5, -0. ,  0. ,  0. ,  0. , -0. ,
        -0.5, -0.5,  0. ,  0. , -0.5, -0.5, -0. , -0.5,  1. ,  0.5,  0. ,
         0. , -0. , -0.5,  0.5,  0. , -0. ,  0. ,  0. ,  0.5, -0.5],
       [ 1. , -0. , -0. ,  0.5,  0. ,  0.5,  0. ,  1. , -0. , -0. ,  1. ,
         0.5,  0.5, -1. ,  0. , -0.5,  0.5,  0. ,  0.5, -1. , -0.5,  0. ,
        -0. ,  1. ,  0.5, -0.5, -0. ,  0. ,  0. ,  2. , -0.5, -0.5],
       [ 0. ,  0. ,  2. ,  0.5,  0. , -0.5, -0. ,  0. , -0. , -0. , -0. ,
         0.5, -0.5,  0. ,  0. , -0.5, -0.5, -1. , -0.5, -0. ,  0.5, -0. ,
        -0. ,  0. ,  0.5, -0.5, -1. , -0. , -0. ,  0. ,  0.5, -0.5],
       [-1. , -0. , -0. ,  0. ,  0. ,  1. , -0. ,  0. , -0. ,  0. ,  0. ,
        -0. , -1. ,  1. , -1. , -0. ,  0. ,  1. ,  0. , -1. , -1. ,  0. ,
         1. ,  1. , -0. ,  0. , -1. , -0. ,  0. , -0. ,  1. , -0. ]],
      dtype=np.float32), np.array([[ 0. ,  0.5,  0.5,  0.5, -0. , -0. ,  0.5,  0. , -0. , -0. , -0. ,
        -0. , -0. ,  0. ,  0. ,  0. , -0. ,  0.5,  0. , -0. ,  1. , -0.5,
        -0. , -0. ,  0.5,  0. , -0.5, -0. , -0. ,  0. ,  1. ,  0. ],
       [-0. ,  0.5, -0.5, -0.5, -0. , -0. , -0.5,  0. , -1. , -0. , -0. ,
        -0. ,  0. ,  0. ,  0. ,  0. , -0. , -0.5,  0. , -0. , -1. , -0.5,
         0. , -0. , -0.5, -0. ,  0.5,  0. , -1. ,  0. , -1. ,  0. ],
       [ 0. ,  0.5, -0. ,  0.5,  1. , -0. ,  0.5, -0. ,  0. ,  0. , -0. ,
         0. ,  0. , -0. ,  0. , -1. , -0. ,  0. ,  0. , -0. ,  0. ,  0. ,
         0. ,  0. ,  0.5,  0. ,  0. , -0. ,  0. ,  0. , -0. ,  1. ],
       [-1. ,  0.5,  0. ,  0.5, -0. ,  0. , -0.5,  0. ,  0. , -0. ,  0. ,
        -0. , -0. , -1. ,  0. , -1. , -0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0. , -1. ,  0.5, -0. ,  0. , -0. , -0. , -1. ,  0. ,  1. ],
       [-0. , -0. ,  0. , -0. ,  1. , -1. ,  0. , -1. , -0. , -1. , -1. ,
        -0. , -1. ,  1. ,  0. ,  0. , -1. , -0. , -1. , -0. ,  0. ,  0. ,
         0. ,  1. , -0. , -0. ,  0. , -0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -0. , -0. ,  0. ,  0. , -1. , -0. ,  0. , -1. ,
         0. ,  0. , -0. , -0. ,  0. ,  0. ,  0. ,  0. , -0. , -0. ,  0. ,
         0. , -0. ,  0. , -0. ,  0. , -1. ,  0. , -1. ,  0. ,  0. ],
       [-0. , -0. ,  0.5, -0. ,  0. ,  0. ,  0. ,  0. , -0. , -1. , -0. ,
         0. , -0. ,  0. , -1. ,  0. , -0. ,  0.5, -0. , -0. , -0. , -0.5,
         1. , -0. , -0. ,  0. , -0.5,  0. , -0. , -0. ,  0. ,  0. ],
       [ 0. , -0. , -0.5,  0. ,  0. ,  0. , -0. , -0. , -0. ,  0. ,  0. ,
        -1. ,  0. , -0. , -1. ,  0. ,  1. ,  0.5,  1. ,  0. , -0. , -0.5,
         1. ,  0. ,  0. ,  1. , -0.5, -1. , -0. ,  0. ,  0. , -0. ],
       [ 0. ,  0. , -0. , -0. , -0. , -0.5, -0. , -0.5, -0. ,  0. ,  0.5,
        -0. ,  0.5, -0.5,  0.5,  0.5, -0.5,  0. ,  0.5, -0. ,  0.5,  0. ,
         0.5,  0.5, -0. ,  0. ,  0. , -0. ,  0. , -0. , -0.5,  0.5],
       [-0.5, -0. ,  0. ,  0.5,  0. ,  0. ,  0. , -0.5, -0.5, -0. ,  0.5,
        -0.5, -0. , -0.5,  0.5,  0.5, -0.5, -0.5,  0.5, -0.5, -0.5, -0. ,
         0.5,  0.5, -0.5, -0.5, -0.5,  0. ,  0.5,  0. ,  0.5,  0.5]],
      dtype=np.float32))

In [ ]:
#@title Verification
n = 2
m = 4
p = 5
rank = 32

verify_tensor_decomposition(decomposition_245, n, m, p, rank)

## Rank-45 decomposition of <2,4,7> over Z

In [ ]:
#@title Data

decomposition_247 = (np.array([[ 0., -1., -1.,  0.,  1.,  1.,  1., -1., -1.,  0., -0., -0.,  0.,
         0., -0.,  1., -0., -1.,  0.,  0., -0., -0., -0., -1., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  1., -1., -0., -0., -0., -0.,  1., -0.,
        -0.,  0., -1., -1., -0.,  0.],
       [ 0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  1., -0.,  0., -1.,
        -0., -0., -0., -0., -0.,  1., -0.,  1., -0.,  1.,  1., -1., -1.,
         0., -1., -0.,  0., -0., -1., -0., -0.,  1., -1., -0., -0.,  0.,
         0., -0.,  1.,  0., -0.,  1.],
       [ 0.,  0., -0.,  0.,  1.,  1., -0., -1.,  0., -1.,  0., -0., -0.,
         1., -0.,  1.,  1.,  0., -0.,  0.,  0., -1., -0., -1.,  0.,  0.,
        -0.,  0., -0.,  0., -1.,  0., -1., -0.,  0.,  0.,  1.,  1., -0.,
        -0.,  0., -1., -1., -1., -0.],
       [-0.,  1., -0.,  0., -1., -0., -1.,  0.,  1.,  0.,  0.,  0.,  0.,
        -0., -1., -1.,  0.,  1., -0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,
        -0., -0., -0.,  0.,  1., -0., -0.,  0., -0., -0., -0., -1.,  0.,
         0.,  0., -0.,  0.,  0., -0.],
       [-1., -1., -1., -0.,  1.,  1.,  0., -1., -0., -0., -0., -0.,  0.,
        -0., -0.,  1.,  1., -1., -0.,  0.,  0., -0., -0., -0., -0., -0.,
         0.,  0.,  1., -1., -0., -0.,  0., -0.,  1., -0., -0., -0.,  0.,
         1.,  0., -1., -0., -0.,  0.],
       [ 1.,  0., -0.,  0., -1., -0., -0.,  0., -1.,  1.,  0.,  1.,  0.,
        -0., -0., -0., -1., -0., -0., -0.,  1., -0.,  1., -0.,  0., -0.,
        -1., -1., -1., -0.,  0., -0.,  0., -0.,  0., -1., -0., -0., -0.,
        -1., -1.,  1., -1.,  0.,  1.],
       [-1.,  0.,  0., -1.,  1., -0.,  0., -1.,  1., -1., -0., -0., -0.,
         0.,  0.,  1.,  1.,  0., -0., -0., -1., -1., -0.,  0.,  0., -0.,
         0., -0.,  1., -0.,  0.,  0.,  0.,  1., -0.,  0., -0., -0., -0.,
         0.,  1., -1.,  0., -0., -0.],
       [ 0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,  1., -1.,  0.,
         1., -0., -1., -0.,  1.,  1.,  1., -0., -0.,  1.,  0.,  0.,  0.,
        -1., -0., -1.,  0.,  0.,  0., -0., -1., -0., -1., -0.,  0., -1.,
         0.,  0.,  0., -1., -0., -0.]], dtype=np.float32), np.array([[-0.,  0.,  0.,  1.,  1.,  1., -0., -1.,  0., -0., -0., -1., -0.,
        -1.,  0.,  0.,  1.,  0.,  1., -0., -0.,  1.,  0., -0.,  0., -1.,
         0., -0.,  0., -0.,  0., -1., -0., -1.,  1., -0.,  0.,  1., -0.,
         1., -0., -0., -0., -1.,  0.],
       [-0.,  1.,  1.,  1., -0.,  1., -0., -0., -1., -0.,  0.,  0.,  0.,
         0., -1.,  0.,  0., -0.,  0.,  0., -1., -0.,  0.,  1.,  1., -0.,
         0., -1., -1., -0., -0., -0.,  0.,  0.,  1.,  1., -1.,  0., -1.,
         1.,  0., -0., -1.,  0., -1.],
       [ 0.,  0.,  1., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,
        -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
        -0.,  0., -0., -0.,  0.,  1.,  0., -0., -1., -0., -0.,  0., -0.,
        -0.,  0., -0., -0.,  0., -0.],
       [-0.,  1.,  1., -0., -1., -0.,  0.,  0., -0.,  0.,  0.,  1., -0.,
         1.,  0., -1., -1., -1., -1., -0.,  0., -1.,  0., -0.,  0.,  1.,
         0.,  0., -0., -1., -1.,  1.,  0.,  0., -1.,  0., -0.,  0., -1.,
        -1., -0., -0.,  0.,  1., -0.],
       [-0., -1., -0., -0.,  0.,  0.,  1.,  0.,  0.,  0., -0.,  0., -0.,
         0.,  1., -0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,
         0.,  0.,  0.,  1., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0., -0., -0.,  0.],
       [ 0., -0.,  0., -0.,  0.,  1.,  0.,  0.,  0., -0., -0., -0., -0.,
         0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -0., -0., -0., -1., -0., -0.,  1., -0., -0., -0., -1., -0., -0.,
         0.,  0., -0.,  0., -0.,  0.],
       [ 0., -0., -1., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
         0., -0., -0., -1., -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -0.],
       [ 0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -1., -1.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0., -0., -0.,  0., -0., -0., -0.,
         0.,  1., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,  1., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0.,  0.,  1.],
       [-0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  1.,
         0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,
         0., -1., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -0.],
       [-0.,  1.,  1.,  0., -1., -0., -0.,  0.,  0.,  0., -0.,  1., -0.,
         1., -0., -1.,  0., -1., -1.,  0., -0., -0.,  0.,  0., -0.,  1.,
         0., -0.,  0., -1., -1., -0.,  0., -0., -1., -0.,  0., -0., -1.,
        -1., -0., -1.,  0.,  1., -0.],
       [ 0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,
         0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -1., -0.,
        -1., -0.,  0., -0.,  0.,  0., -0., -0.,  0., -1.,  0., -0.,  1.,
         0., -0., -0., -0., -0., -0.],
       [ 0.,  0.,  0.,  1., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0., -0., -0., -0., -0.,  0., -0., -1., -0.,  0., -0.,  1.,  0.,
         0.,  0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,
         0.,  1., -0., -0.,  0., -0.],
       [-0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,
        -0.,  1., -0., -1., -0.,  0.,  0.,  0., -1., -0., -0.,  0., -0.,
        -1.,  0.,  0.,  0.,  0., -0.],
       [ 0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -1., -0., -0.,
         1.,  0., -0., -0.,  0.,  0., -0., -0., -1.,  0., -0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  1., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0.,  0.],
       [-0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  1., -0., -0., -0.,
        -0., -0., -0.,  0., -0., -0.,  0.,  1.,  0., -0., -0., -0.,  0.,
        -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  1., -0.,  0.,
        -0.,  0.,  0., -0.,  0.,  1.],
       [-1., -0., -1., -1., -0., -1., -0.,  1., -0.,  0.,  0., -0.,  0.,
         0.,  0.,  0., -1.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,
        -1., -0.,  0.,  0.,  1., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  1., -0., -0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,  0., -0.,
        -0., -0.,  0., -0.,  1.,  0.],
       [ 0., -0., -1., -1.,  0., -1., -1.,  1.,  1., -1.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -1., -1., -0.,
        -1.,  1., -0., -1., -0.,  0.,  0., -0., -1., -1.,  0.,  0.,  1.,
        -1.,  0.,  0., -0., -0.,  0.],
       [-0.,  0., -0.,  1.,  0.,  0., -0., -0., -0.,  0., -0.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,  0., -0.,
        -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  1.,  0.,  0.,
        -0., -0., -0.,  0., -0.,  0.],
       [ 0.,  0.,  1.,  1.,  0.,  1., -0., -1., -0.,  0.,  0., -0.,  0.,
        -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,
        -0.,  0., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0.,  0.],
       [ 0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -1., -0., -0.,
        -0., -0.,  0.,  0.,  0.,  0., -1., -0.,  0., -0., -0., -0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0.,  0.],
       [-0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
        -0., -1.,  0., -0., -0., -0., -0., -0.,  0., -1., -0.,  0.,  0.,
         0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  1., -0., -0.,  0.,
         0., -0., -0.,  0., -0., -1.],
       [ 0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0., -0.,  0.,  1.,  1., -0., -0., -0., -0., -0., -1.,
         0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,
        -0., -0.,  0., -0.,  0.,  0.],
       [ 0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
         1.,  0.,  0., -0.,  0., -0.,  1.,  0., -0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -1.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0.,  1.,  0.],
       [-0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  1.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -1.,
        -0.,  0.,  0.,  0.,  0., -0.],
       [-0., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,
        -0., -0., -0.,  0.,  1.,  0.,  0.,  1., -0., -1., -1., -1.,  0.,
         0.,  1.,  0., -1.,  0., -0.,  1., -0., -1., -0.,  0., -0.,  1.,
        -1., -1.,  0.,  1., -0., -0.],
       [ 0., -1., -1., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0.,  1., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  1.,
        -0.,  0., -0.,  0., -0., -0.]], dtype=np.float32), np.array([[ 0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  1., -0.,  0.,
         1., -0., -0.,  0.,  0.,  1., -1.,  0.,  0., -0., -0., -0., -1.,
        -0.,  0.,  0., -0., -1., -0.,  0., -0., -0.,  0., -0.,  1.,  0.,
        -0., -0., -0.,  0.,  0.,  0.],
       [-0.,  0., -0., -0.,  1.,  0., -0.,  0., -0.,  0., -1., -1.,  0.,
         0.,  0., -1.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0., -0.,  0., -0.,  1.,  0., -0.,  0., -1.,  0.,
        -0., -0.,  0., -0.,  0., -0.],
       [ 0., -0., -0., -0., -0., -0.,  0.,  0.,  1., -1.,  0., -0., -0.,
        -0., -0., -0.,  0.,  0., -0., -0.,  1.,  0., -1.,  1., -1.,  0.,
        -1.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  1.,  0., -0.,  0.,
         0., -1.,  0.,  1., -0.,  0.],
       [ 0., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.,
         1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0., -1., -0., -0., -0.,
        -0.,  1.,  0., -0., -0.,  1.],
       [-1.,  0., -0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -0., -0., -1.,
         0., -0.,  0., -1., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -1.,
        -0., -0.,  0.,  0.,  0.,  1., -0.,  0.,  0.,  0., -0.,  0.,  0.,
         0., -0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -1.,  1.,
        -0., -0., -0.,  0., -0.,  1.,  0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  1., -0.,  0., -0., -0.,  0.,  0., -1., -0.,  0., -0.,  0.,
         1., -0., -0.,  0., -0., -0.],
       [ 1., -0., -0., -0., -1.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,
         0.,  0., -0.,  1.,  0.,  0., -0., -0., -0., -0., -0., -0., -0.,
        -0.,  0.,  0., -0., -1., -0., -0., -0.,  0., -0.,  0.,  1.,  0.,
        -0.,  0., -1.,  0., -1., -0.],
       [-1.,  0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
         1., -0., -1., -1., -0., -0., -0.,  0., -1.,  0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0.,  0., -0., -0.,  1., -0.,  0., -0., -1.,  0.,
        -0., -0.,  0.,  0.,  1., -0.],
       [-0.,  0.,  0., -0.,  0.,  0.,  1.,  0., -0.,  0., -0., -0.,  0.,
         0., -1.,  0., -0., -0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,
        -1.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  1., -0., -0.,  0.,
        -0., -0.,  0.,  1.,  0., -0.],
       [ 0.,  1., -0.,  0., -0.,  0., -1., -0.,  1., -0.,  0.,  0., -0.,
        -0., -0., -0.,  0.,  1.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
         1.,  0., -1.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  1.,
        -0.,  0., -0.,  0.,  0., -0.],
       [ 0., -0., -0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0., -0., -0., -1.,  0., -0., -1., -0., -0.,
         0.,  0., -0., -0., -0.,  0., -1.,  0.,  0., -0.,  1.,  0.,  0.,
         0.,  1., -0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  1., -0., -1.,  0., -0.,  0.,  0.,  0.,
         0., -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,
        -0.,  0.,  1., -0.,  0.,  0.,  1., -0.,  0., -0.,  0.,  0., -0.,
         0., -1., -0., -1., -0.,  0.],
       [-1., -1.,  1.,  0.,  1.,  1.,  1.,  0.,  0., -0.,  0., -0., -0.,
         0.,  0.,  0., -1., -0., -0.,  0.,  0.,  0., -0.,  1., -0., -0.,
        -0., -0.,  0., -1.,  0.,  1.,  1.,  0., -1., -0.,  0., -1., -0.,
         0.,  0.,  1., -0.,  0.,  0.],
       [ 1.,  1., -0., -0., -1., -1., -1.,  1.,  1., -0., -0.,  0.,  0.,
        -0., -0.,  1.,  0.,  1., -0.,  0., -0., -0., -0., -0.,  0., -0.,
         0., -0., -1.,  1.,  0.,  0., -1., -0., -0., -0.,  0.,  1., -0.,
         1., -0., -0.,  1., -0., -0.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 2
m = 4
p = 7
rank = 45

verify_tensor_decomposition(decomposition_247, n, m, p, rank)

## Rank-51 decomposition of <2,4,8> over Z


In [ ]:
#@title Data

decomposition_248 = (np.array([[-1., -0., -0.,  1.,  0.,  0., -0.,  1.,  0., -0.,  0.,  0., -0.,
        -1.,  0.,  0.,  1.,  0.,  0.,  1.,  1., -0., -0., -1., -0., -0.,
         0.,  0.,  0.,  1., -0.,  0.,  1., -0.,  0., -0.,  0., -0., -0.,
        -1.,  1., -0., -0., -0., -0.,  0.,  0.,  1.,  0.,  1., -0.],
       [-0.,  0.,  0., -0.,  0., -0.,  0.,  0., -1., -0., -0.,  0.,  0.,
         1.,  0., -0.,  0., -0.,  0., -1.,  0.,  1., -1.,  1.,  0., -0.,
        -0.,  0.,  0.,  0., -1.,  1.,  1., -0.,  0., -0., -0.,  1.,  1.,
         1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -1.],
       [ 0.,  0., -0.,  0., -0.,  0., -0., -1., -1.,  0., -0.,  1., -1.,
         0.,  0., -0., -1.,  0., -0., -0., -1., -0., -1.,  0.,  0., -0.,
        -0., -0., -0., -1.,  1.,  0., -0., -1.,  1., -1., -0., -0., -1.,
         0., -1., -0., -1., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.],
       [-0., -0., -1.,  0.,  0., -0.,  0., -0.,  1., -1.,  0.,  0.,  1.,
        -0., -0.,  0.,  0.,  0.,  1.,  0.,  0., -0., -0.,  0.,  1., -0.,
        -0.,  0.,  1.,  0.,  0., -1.,  0., -0.,  0., -0.,  1.,  0., -1.,
        -0., -0., -1., -0., -1.,  0.,  0.,  1., -1., -0., -1., -0.],
       [-1.,  0., -0.,  1., -0., -1.,  0., -0., -0., -0., -1., -0., -0.,
         0., -1.,  0.,  1., -1., -0., -0., -0., -0., -0.,  0.,  1.,  1.,
         0., -0.,  1., -0., -0.,  0., -0., -0.,  0., -1.,  0.,  0., -0.,
        -1., -0.,  0., -0., -0., -0., -1., -0., -0.,  0., -0.,  1.],
       [-0., -1., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -0., -0.,  1.,  0.,  0., -0., -1., -1.,  0., -1.,  1., -0.,  1.,
         1., -1.,  0.,  0., -1.,  1., -0.,  0.,  0., -0.,  1., -0., -0.,
         1.,  0.,  0., -1., -0.,  1., -0., -0.,  0., -1.,  0., -0.],
       [-0., -1.,  1.,  0., -0.,  0.,  1.,  0.,  0.,  0.,  1., -0., -1.,
         0., -0.,  0., -1., -0., -0.,  1., -1., -1., -1.,  0.,  0., -0.,
         1.,  1.,  0.,  0.,  1.,  0.,  0., -1.,  1.,  0.,  1.,  0.,  0.,
         0., -1.,  0., -0., -0., -0., -0.,  0., -0.,  1.,  0.,  0.],
       [-0.,  1.,  0.,  0., -1.,  1., -1., -0., -0., -1.,  0.,  0.,  1.,
         0.,  1., -1.,  0., -0., -0.,  0., -0.,  1., -0., -1., -0.,  0.,
         0.,  0., -0.,  0.,  0., -1., -0.,  1.,  0., -0., -0., -0.,  0.,
        -0.,  1.,  0.,  1., -1.,  0., -0., -0.,  0.,  1.,  0.,  0.]],
      dtype=np.float32), np.array([[ 0.,  0.,  0.,  1.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0., -0.,  0., -1., -0.,  0., -0., -0., -0., -0.,  0., -0.,
         0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0.],
       [-1., -0.,  0.,  0., -0., -0., -0., -1.,  0., -0., -0., -1.,  0.,
        -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,
         0., -0., -0.,  0., -0., -0.,  0., -0., -0.,  1.,  0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.],
       [-0., -0., -1.,  0., -0., -0., -0., -0.,  0., -0., -0., -0., -1.,
        -0., -1.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,  1.,  0.,  0.,
         0.,  1.,  0., -0., -0., -1.,  0.,  1.,  1.,  0.,  0., -0.,  0.,
         1., -0., -0.,  0., -0., -0.,  0.,  1.,  0., -1.,  0.,  0.],
       [-0., -0.,  0.,  0.,  1.,  1.,  0., -0.,  0., -0., -0., -0.,  0.,
        -0.,  0., -0., -1., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  1.,  1.,  0.,  0., -0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.],
       [-0., -0., -0., -1., -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0.,  0., -0.,  0., -0., -0., -1., -0., -0.,  0.,  0.,  1.,  0.,
         0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -1.],
       [ 0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0., -0., -1., -0., -0.,  0.,  0.,  0., -0., -1.,  0.,
         0., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,
        -0., -0.,  1., -0., -0.,  0., -0.,  0.,  0.,  0., -1.,  0.],
       [-1.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,
        -0., -0.,  0.,  0.,  0.,  1., -0., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  1., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,
         0., -0., -0., -0., -0.,  0., -0., -0.,  1., -0.,  0.,  0.],
       [-1.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,
         0., -0., -0., -0., -0., -0.,  1.,  0., -0., -0.,  0.,  0.],
       [ 0., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0.,  1., -0., -0.,  0.,  0., -0.,  0., -0., -1.,
        -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,
         0.,  0., -0., -0.,  0.,  1., -0., -0., -0.,  0., -0.,  1.],
       [ 0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -1.,  1.,
         0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0.,
        -0.,  0., -0., -0., -0.,  1.,  0., -1., -1.,  0., -0.,  0.,  1.,
         0., -0., -0.,  1.,  0.,  0., -0., -1., -0., -0., -0., -0.],
       [ 0., -0., -1.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -1.,
        -0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
         0.,  1.,  0., -0., -0., -1., -0.,  1.,  1., -0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -0.,  0.,  0.,  1., -0., -1.,  0., -0.],
       [ 0.,  1., -0., -0., -0., -0., -1.,  0.,  0.,  0.,  0., -0.,  0.,
        -0., -0., -0., -0.,  0.,  0., -0.,  0., -1.,  0.,  0.,  0., -0.,
         0.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0., -0., -0.,  0.,  0., -1.,  0., -0., -0., -0.,  0., -0.],
       [ 0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
        -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -1.,  0.,
        -0., -0.,  0.,  0.,  0., -1.,  0., -0., -0., -0.,  0.,  0.],
       [ 0., -1.,  0.,  0., -1., -1.,  1.,  0., -0., -0.,  0.,  0., -0.,
         0., -0., -0., -0.,  1.,  0.,  0.,  1.,  1.,  1., -0.,  1.,  0.,
        -0., -0.,  0., -0.,  0., -0., -1., -1., -1., -0., -0.,  0., -0.,
         0., -1., -1., -0., -0.,  1., -0.,  0.,  0.,  0.,  1., -0.],
       [-0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0.,  0., -1.,  0.,  1.,
         0.,  1., -1., -0., -0.,  1.,  1., -0.,  0., -0., -1., -0., -0.,
        -0., -1.,  1.,  0.,  0.,  1.,  0., -1., -1., -1., -0., -0.,  1.,
         0.,  0.,  0.,  1., -0.,  0., -0., -1.,  1., -0., -0.,  0.],
       [-0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0.,  0., -1.,  0.,  1.,
        -1., -0.,  0., -0., -0.,  0.,  1., -0., -0., -0.,  0., -0., -0.,
        -0., -1., -0.,  0.,  0.,  1.,  0., -1., -1., -1., -0.,  0.,  1.,
         0.,  0.,  0.,  1.,  0.,  0.,  1., -1., -0., -0., -0.,  0.],
       [ 0., -1.,  0., -0., -1.,  0., -0., -0.,  1., -0.,  0., -0., -0.,
        -0.,  0.,  0., -0., -1.,  0.,  0.,  1.,  1., -0., -0.,  0.,  1.,
        -0., -0.,  0., -1., -0., -0., -1., -1., -1., -0.,  1.,  0.,  0.,
         0.,  0., -1., -0.,  0., -0., -0., -0.,  0.,  0.,  0., -1.],
       [ 0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -1., -0.,
        -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0.,  0., -0., -0.,
        -0.,  0., -0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.],
       [ 0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,
         0.,  0., -1., -0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,
         0., -0., -0., -0.,  0.,  0.,  0., -0.,  1.,  0.,  0., -0., -0.,
        -0.,  0., -0.,  1., -0., -0.,  0., -0., -0., -1., -0.,  0.],
       [ 0., -0.,  0., -0.,  1., -0.,  1.,  0.,  0., -0., -0., -0.,  0.,
        -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0., -0.,  1.,  1., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.],
       [-0.,  1., -0.,  0.,  1., -0.,  0., -0., -1.,  0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0., -0., -0.,  0., -1.,  0.,  0.,  0.,  0.,
        -1.,  0., -0., -0.,  0.,  0., -0.,  1.,  1.,  0., -1.,  1.,  0.,
        -0., -0.,  1.,  0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.],
       [-0., -0.,  0.,  0., -0.,  1., -1., -0., -0.,  0., -0.,  0.,  0.,
         0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0., -0., -1., -0.,
         0.,  0.,  0., -1., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,
         0.,  1.,  1., -0.,  0.,  0., -0.,  0.,  0.,  0., -1.,  0.],
       [-0.,  0.,  1.,  0.,  0., -0.,  0.,  1., -0.,  0., -1.,  0.,  1.,
         0.,  1., -1., -0.,  0., -0.,  1., -0., -0., -0., -1., -0., -0.,
        -0., -1.,  1.,  0., -1.,  1.,  0., -1., -1., -1., -0., -0., -0.,
         0.,  0.,  0.,  1., -0.,  0.,  0., -1.,  1., -0.,  0.,  0.],
       [ 0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0., -1.,  0., -0.,
         0., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -1., -1., -0., -0., -0.,
        -0., -0., -0., -0.,  0., -0.,  1.,  0., -0., -0.,  0.,  0.],
       [-0.,  0.,  0., -0.,  0.,  1.,  0.,  0., -0., -0., -0., -0., -0.,
        -0.,  0., -0.,  0., -1., -0.,  0., -0., -0.,  0., -0., -1., -0.,
        -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,
         0., -0.,  0.,  0., -1.,  0., -0., -0.,  0.,  0.,  0.,  0.],
       [ 0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  1.,
        -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0., -1., -1., -0.,  0., -0., -0.,
        -0.,  0., -0., -0.,  0.,  0., -0., -1.,  0., -0., -0.,  0.],
       [ 0., -0., -1.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0., -1.,
        -0.,  0.,  0., -0.,  0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  0.,
         0.,  1.,  0., -0., -0.,  0., -0.,  1.,  1., -0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0., -0.,  0.,  1.,  0., -1.,  0., -0.],
       [-0., -0., -0., -0.,  1., -0.,  0.,  0.,  0.,  0., -0.,  0., -1.,
         0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,
         0.,  0., -0., -0., -0.,  0., -0.,  1.,  1., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.],
       [-0.,  1., -0.,  0.,  1., -0.,  0.,  0.,  0., -0., -0.,  0., -1.,
        -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
        -1.,  0., -0., -0., -0.,  0.,  0.,  1.,  1., -0., -1., -0., -0.,
        -0., -0.,  1.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.],
       [-0., -0.,  0.,  0.,  0.,  1.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0.,  0., -0.,  0., -1., -0.,  0.,  0.,  0.,  0., -0., -1.,  0.,
         0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.],
       [ 0., -0., -0.,  0., -0., -0.,  0., -0., -0.,  1.,  0., -0., -0.,
        -0.,  0.,  0., -0.,  0.,  1., -0.,  0.,  0., -0.,  0.,  0.,  0.,
         0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.],
       [ 0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  1.,  0.,  0., -0.,
         0., -1., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0., -0.,
         0.,  0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0., -0.,  1.,  0.,  0., -0., -0., -0.]],
      dtype=np.float32), np.array([[-0., -0.,  0.,  1., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -1.,  0.,  0., -1., -0.,  1., -0., -1., -0.,
         1.,  0.,  0.,  0., -0., -0., -1.,  0., -0.,  0., -1.,  0.,  0.,
         0.,  0.,  1., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -1.],
       [ 0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0.,  0., -0., -0.,  1., -0.,  0., -0., -0.,  0., -0.,  1., -1.,
        -1., -0.,  0., -0., -0., -0., -0., -0., -0., -0.,  1., -0., -0.,
         0., -0., -1., -0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  0.],
       [ 0.,  0.,  1., -0., -0., -0.,  0., -1.,  0., -0.,  0., -1.,  0.,
         0.,  0., -0.,  0.,  0., -0.,  1., -0.,  0.,  0.,  0.,  0., -0.,
         0.,  1., -0., -0.,  1.,  0.,  0.,  0., -0.,  0., -0.,  0.,  1.,
        -0., -0., -0.,  0.,  0., -0.,  0., -1.,  0., -0., -0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0., -0., -0.,  0., -0.,  1.,  1.,  0.,
        -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        -0., -1.,  0.,  0.,  0.,  0., -0., -1., -1., -1., -0., -0.,  0.,
         0.,  0.,  0., -1., -0.,  0., -0., -0., -0., -1., -0.,  0.],
       [-0., -0.,  1.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -1., -0.,  1., -0., -0., -0.,  1.,  0.,  0., -0., -1., -0.,  0.,
        -0.,  1.,  0., -0.,  1., -1.,  0.,  0.,  0., -0.,  0.,  0.,  1.,
        -0., -0.,  0., -1., -0.,  0., -0.,  0.,  0.,  0., -0., -0.],
       [ 0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,
         1.,  0., -1.,  0.,  0.,  0., -1.,  0., -0.,  0.,  1.,  0.,  0.,
         0., -1., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
        -1.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -1.,  0.,  0.],
       [ 0., -0.,  0.,  0., -0.,  0., -1., -0., -1.,  0., -0., -0., -1.,
        -0., -0.,  0., -0., -0., -0.,  0.,  1., -1., -1., -0., -0.,  0.,
        -1.,  0., -0.,  1.,  0., -0., -0., -1.,  0.,  0.,  1., -0.,  0.,
        -0.,  1.,  0., -0., -0., -0.,  0., -1.,  0., -0.,  0., -0.],
       [ 0., -1.,  0., -0., -1., -0.,  1., -0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -1.,  0., -0., -0., -1., -0., -0.,  0.,  0., -0.,
         1., -0.,  0., -1., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -0., -1., -0., -0.,  0., -0., -0., -0., -0., -0., -0., -0.],
       [ 0.,  0.,  0., -0., -0., -0., -0., -0.,  1.,  0.,  0., -0., -0.,
        -0., -0.,  0.,  0.,  0.,  0.,  0., -1., -0.,  1., -0.,  0., -0.,
         1.,  0., -0.,  0.,  0., -0., -1., -0.,  0.,  0., -1., -1.,  0.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0.],
       [ 0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
         0., -0., -0., -0., -0., -0., -0., -0.,  1.,  0.,  0., -0., -1.,
        -1., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  1., -0.,
        -0., -0.,  0., -0., -0., -1., -0., -0.,  0., -0., -0., -1.],
       [ 0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,
        -0.,  0., -0., -0., -0.,  0., -0.,  1.,  0., -1.,  0., -0.,  0.,
        -1., -0., -0.,  1., -0.,  0., -0.,  0., -0.,  0.,  1.,  0., -0.,
         0., -0., -1.,  0., -0.,  0.,  0.,  0.,  0., -0., -1., -0.],
       [ 0.,  0.,  0., -0.,  0.,  1., -0., -0., -0., -0., -0., -0., -0.,
        -0.,  0.,  0., -1.,  0., -0.,  0., -1., -0., -0., -0., -1., -0.,
         1.,  0.,  0., -1., -0.,  0.,  0., -0.,  0.,  0., -1., -0., -0.,
         0., -1.,  1., -0., -1.,  0., -0.,  0., -0.,  0., -0.,  0.],
       [ 0., -0., -1., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,
         1.,  0.,  0.,  0., -0.,  1., -1., -0., -0.,  0., -0., -0.,  0.,
         0., -1., -0., -0., -1., -0., -0., -0., -0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0., -0.,  1., -0.,  0.,  0.],
       [ 0., -0.,  1., -0., -0., -0., -0.,  0.,  0., -1., -0.,  0., -0.,
        -1.,  1., -0.,  0., -0., -1.,  1.,  0.,  0., -0., -1.,  0., -0.,
        -0.,  1.,  1.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,
         1.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0.],
       [ 1.,  0.,  0.,  0., -0., -0.,  0.,  1.,  0., -0., -0., -0.,  0.,
        -1., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,
         0.,  0., -1., -0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0., -0.,  1., -0., -1., -0., -0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0., -0., -1.,  0.,  0.,
         1., -1.,  0.,  0.,  0., -0., -1.,  0., -0.,  0.,  1.,  0., -0.,
        -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,
        -1.,  0., -0., -0.,  0.,  0., -1., -0., -0., -0.,  0.,  0.]],
      dtype=np.float32))

In [ ]:
#@title Verification
n = 2
m = 4
p = 8
rank = 51

verify_tensor_decomposition(decomposition_248, n, m, p, rank)

##Rank-47 decomposition of <2,5,6> over Z

In [ ]:
#@title Data

decomposition_256 = (np.array([[ 0.,  0.,  1., -0.,  0.,  0., -1., -1.,  0., -0., -1.,  0., -0.,
         0.,  1., -0., -0., -0.,  0.,  0., -1.,  1., -0.,  0.,  1.,  0.,
         0., -0., -0., -1., -0., -1.,  0.,  0.,  0., -0.,  0.,  1., -0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -0.],
       [ 0., -0., -0.,  1., -0., -0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  1., -0., -1.,  0., -0., -1.,  0.,  0.,  0., -0., -1., -1.,
         0., -0.,  0.,  0.,  0.,  0.,  1.,  0., -0., -1.,  0.,  1., -0.,
         0.,  0., -0., -0., -0.,  0., -0.,  0.],
       [-0., -0., -0., -1., -0.,  0., -0., -0.,  0., -1.,  0.,  1.,  0.,
         0.,  0., -1.,  1.,  0., -0., -0., -1.,  0., -1., -1.,  1.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -1., -0., -0.,  1.,  0., -0.,  0.,
         0.,  0., -1., -0., -1.,  1.,  0.,  0.],
       [ 0., -0., -1.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  1.,  0.,
        -0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  0., -1., -1.,
        -0., -1., -1., -0., -0., -0., -0., -0.,  1.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  1., -1.,  1., -0.,  0.],
       [-1., -1., -0., -0., -1.,  0., -1., -1.,  0., -0., -1.,  0.,  0.,
         0.,  1.,  1.,  0.,  1., -0., -0.,  0.,  1., -0., -0., -0.,  0.,
        -0.,  0., -0., -1., -0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
        -0.,  1.,  1., -0.,  0., -0., -0.,  0.],
       [ 0., -0., -1., -1., -1.,  0., -0.,  1.,  1., -1., -0., -0., -0.,
        -0.,  0., -0., -0., -0.,  1., -0., -0.,  0., -0., -0.,  0., -0.,
         1.,  0.,  0., -0., -1., -0., -0.,  1.,  0., -0.,  0., -1., -1.,
         0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.],
       [-0., -0., -0.,  0., -1., -1., -0., -0.,  1., -1., -0.,  0.,  0.,
         1.,  0., -0.,  1.,  0., -0.,  1.,  0.,  1., -0., -0., -0., -0.,
         1., -1., -0.,  0.,  0.,  0., -1.,  0., -0., -0.,  0., -1.,  0.,
         0., -0.,  0.,  0., -0.,  0., -1., -0.],
       [ 0., -1.,  0.,  0.,  0.,  0., -1.,  0.,  0., -0.,  0., -1.,  1.,
         0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  0., -1.,  0.,  1., -0., -0.,  0.,  1.,  0.,  0.,
         1., -0.,  1.,  1.,  0., -0.,  1.,  0.],
       [-0.,  0.,  1.,  1.,  0., -1.,  0.,  0.,  0.,  0., -0., -1.,  0.,
        -0.,  0., -0.,  0., -0., -1.,  0., -0., -0., -0., -0., -0., -0.,
        -0., -0.,  1., -0., -0., -0.,  0., -1.,  0., -0.,  1.,  0., -0.,
        -0., -0., -0.,  0., -0., -1., -1., -0.],
       [ 1.,  1.,  0.,  0.,  0., -0.,  1.,  1.,  1.,  0.,  1., -0.,  0.,
         0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,
        -0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -1.,  0.,  0.,
        -1., -1., -1., -1.,  1., -0., -0.,  1.]], dtype=np.float32), np.array([[ 0., -1., -0., -0.,  0.,  0.,  1., -1.,  0., -0.,  0.,  0., -1.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0., -1., -0., -0., -0., -0.,  0.,  0.,  0., -0.,
        -0.,  1., -0., -0.,  0., -0., -0., -0.],
       [-0.,  0.,  1., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0., -1., -0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  1.,  0., -0.,  0., -0., -0., -0., -0.,  0., -0., -1.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  1.,  0.,  0., -0.,
        -0., -0., -0.,  1.,  0.,  0., -1., -1.,  0., -0., -1., -0., -0.,
        -0., -0.,  0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  1., -0.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0.],
       [ 0.,  0.,  1., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
         0.,  0.,  0., -0., -0., -1., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0.,  1., -0., -0.,  1.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -0., -0., -0.,  0., -0., -0., -0.],
       [-1., -0., -0.,  0., -1.,  0.,  0., -1., -0., -0., -1.,  0., -0.,
         1.,  0., -0., -0., -1., -0.,  0., -0., -1.,  0.,  0.,  0.,  0.,
         1.,  0., -0.,  1., -0.,  1.,  0.,  0.,  0.,  0.,  0., -0., -0.,
         0.,  0., -0.,  0., -0.,  0., -0., -0.],
       [-1., -0.,  0.,  0.,  0.,  0.,  0., -1.,  0., -0., -1., -0., -0.,
        -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0., -0., -0.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0.],
       [-1.,  1., -0., -0.,  0., -0., -1., -0., -0., -1., -1.,  0.,  1.,
         0., -1.,  0., -0.,  0.,  0.,  1.,  1., -1.,  0.,  0., -0., -0.,
         1.,  0., -0.,  1.,  1.,  1.,  0.,  0., -0.,  1.,  0.,  0., -0.,
         0., -1., -0.,  0., -0.,  0., -0.,  0.],
       [-0.,  0., -0., -1., -0., -0.,  0.,  0.,  0., -0.,  0.,  1., -0.,
         0., -0.,  0., -0., -0.,  1.,  0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0., -1.,  0., -1., -0., -1., -1., -0., -0.,  0.,  0.,  1.,
         0., -0.,  0., -0.,  0.,  1.,  1.,  0.],
       [-0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,
         0., -0., -0., -1.,  0.,  0.,  1., -0., -0., -0.,  1., -0.,  0.,
         0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -1., -0.,  0., -0.,
         0., -0., -0.,  0.,  0.,  0., -0., -0.],
       [ 0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,
         0.,  0., -0.,  0., -0., -0.,  1., -0.,  0., -0.,  0.,  0.,  1.,
        -0., -1.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0., -0., -0., -0.,
        -0.,  0.,  0.,  0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0., -0.,
        -1.,  0.,  0.,  0., -0., -0.,  1., -0., -0., -0., -0., -0., -0.,
         0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,
         0.,  0., -0., -0., -0.,  0., -0., -0.],
       [ 0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,
         0., -1., -0., -0.,  0.,  0.,  1., -0., -1., -0., -0.,  0., -0.,
        -0.,  0., -0.,  1., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0., -0.],
       [-0.,  1., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  1.,
        -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,
         0., -1., -1.,  0., -0.,  0.,  0.,  0.],
       [ 0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  1., -0.,
        -0., -0., -0., -0., -0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  0.,
         0., -0., -1.,  0., -1., -0.,  0.,  0., -0.,  0.,  0., -0.,  1.,
         0.,  0., -0.,  0., -0.,  1., -0.,  0.],
       [ 0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
        -0., -0.,  0., -1.,  0., -0.,  1., -0.,  0., -0.,  1., -0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  1., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -0.,  0., -0.,  0.],
       [ 0.,  1., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  1.,
        -0.,  0., -0., -0.,  0., -0., -0., -0.,  0., -1., -0., -0., -0.,
        -0.,  0.,  0., -0., -0., -0., -0., -0.,  1., -0.,  0.,  0., -0.,
        -1., -1., -1., -1.,  1., -0.,  0., -1.],
       [-0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  1.,
         0.,  0., -0., -1., -0., -0.,  1.,  0., -0.,  0., -0., -0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  1., -0.,  0., -0.,  0., -0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.],
       [ 1.,  0.,  0., -0.,  0., -0.,  1.,  0., -0.,  0.,  1.,  0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0., -1.,  0., -0.,  0., -0., -0.,
        -0., -0.,  0., -0.,  0., -1., -0., -0., -0., -0., -0.,  0.,  0.,
        -0., -0., -1., -0., -0., -0.,  0.,  0.],
       [ 0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
         0.,  0.,  1.,  0., -0., -0., -0.,  0.,  0.,  0., -1.,  0.,  0.,
        -1., -0., -0., -1., -0.,  0., -0.,  0.],
       [-0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0., -1., -0.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0.,  1., -0., -0.,  0., -0.,  1.,  0., -0., -0., -0., -1.,
        -0., -0., -0., -0., -0.,  0.,  0.,  0.],
       [ 0.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,
        -0., -0.,  0.,  1.,  0., -0., -1., -1.,  0.,  0., -1., -1.,  0.,
        -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  1., -0.,  1.,  0.,
        -0., -0., -0., -0.,  0.,  0., -0.,  0.],
       [ 0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0.,  1., -0., -0., -0.,  0., -0., -1., -0.,  0., -0.,  0.,
        -0., -0., -0., -0.,  0.,  0., -0., -0.],
       [ 0., -0.,  0.,  0.,  0., -1.,  0.,  0., -0., -0.,  0.,  0.,  0.,
         1., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -1.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -0., -0.],
       [-1.,  1.,  0., -0.,  0.,  0., -1., -0., -0.,  0., -1.,  0.,  1.,
        -0.,  0., -0.,  0.,  0., -0., -0.,  1.,  0., -1., -0., -0., -0.,
        -0.,  0.,  1., -0.,  1.,  1., -0., -1.,  0., -0., -1.,  0., -0.,
        -1., -1., -0., -1., -0., -1.,  0.,  0.],
       [-0.,  1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  1.,
        -0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0., -0.,
        -1., -1.,  0.,  0., -0.,  0.,  0.,  0.],
       [ 0.,  1., -0., -0.,  0., -0.,  0., -0., -0.,  0., -1., -0.,  1.,
         0.,  0.,  1., -0.,  0., -0., -0., -0.,  0., -1., -0.,  0., -0.,
        -0., -0.,  0.,  1.,  0.,  1., -0.,  0.,  0., -0.,  0., -0., -0.,
        -1., -1., -0.,  0.,  1., -0.,  0.,  0.],
       [ 1., -1.,  0.,  0.,  1., -0., -0.,  0., -1.,  0.,  1., -0., -1.,
        -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -1., -0.,  0., -1.,  0., -1., -0., -0., -0.,  0., -0.,  0., -0.,
         1., -0.,  0., -0.,  0., -0., -0., -0.],
       [ 0.,  1., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  1.,
         0.,  0.,  1., -0., -0.,  0., -0., -0.,  0., -1.,  0.,  0.,  0.,
        -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -1., -1.,  0.,  0.,  1.,  0., -0., -1.],
       [ 1.,  0., -0., -0.,  1.,  0., -0.,  0., -1.,  0.,  1.,  0.,  0.,
         0., -0.,  0.,  0.,  1.,  0., -0.,  0., -0., -0.,  0., -0., -0.,
        -1.,  0.,  0., -1.,  0., -1., -0., -0., -0., -0., -0., -0.,  0.,
        -0., -0.,  0., -0., -0., -0., -0.,  0.],
       [ 1.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  1., -0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,
         0., -0.,  0., -1.,  0., -1., -0., -0.,  0., -0., -0., -0., -0.,
        -0., -0.,  0., -0.,  0., -0., -0.,  0.]], dtype=np.float32), np.array([[-0., -1., -0., -0., -1.,  0., -1.,  0., -0., -1.,  0., -0., -0.,
        -0., -0., -1., -0., -1.,  0., -0., -1., -0., -1.,  0., -0., -0.,
         1.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,  1.,  0.,
         1., -0.,  1., -1.,  0.,  0.,  0.,  0.],
       [-0., -1., -0.,  0., -1., -0.,  0., -0.,  0.,  0.,  0.,  1.,  0.,
        -0.,  0.,  0.,  0., -1., -0.,  0.,  0., -0., -1.,  0.,  0., -0.,
         1.,  0.,  0.,  0.,  1.,  0., -0., -0., -0., -0., -1., -0., -0.,
         1.,  1., -0.,  0., -1., -1., -0., -0.],
       [-1., -0.,  1., -1., -0.,  0., -0., -0., -0., -0.,  1.,  0., -0.,
         0.,  0.,  0., -0.,  0., -1., -0., -0.,  0.,  1., -0., -1., -0.,
         0., -0., -0., -0., -0.,  1., -0., -1., -0., -0., -0.,  0., -0.,
         0., -0., -0.,  0.,  1.,  1.,  0.,  1.],
       [ 0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -1., -0.,
        -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  1., -0.,  0.,  0.,
         0.,  0.,  0., -0.,  0., -0., -0., -1., -0., -0.,  0., -0.,  1.,
         0., -0., -0., -0.,  1.,  1., -1.,  1.],
       [ 0.,  0.,  0.,  0., -1., -0.,  0., -0.,  0., -1., -0.,  0.,  0.,
        -0.,  0.,  0.,  0., -1.,  0., -0.,  0., -0.,  0., -1.,  1.,  0.,
         1.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,  0.,  1.,  0.,
        -0., -0., -0.,  0.,  0.,  0., -0.,  0.],
       [-0., -1.,  0., -1., -1.,  0., -0.,  0., -0., -1., -0., -0., -1.,
        -0., -0., -0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
         1., -0.,  0., -0., -0., -0.,  1., -0., -0.,  1.,  0., -0.,  0.,
        -0.,  1., -0.,  0.,  0.,  0.,  1., -0.],
       [ 1., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0., -1., -0., -0.,
         0., -0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -1.,
         0., -0., -0.,  0.,  0., -1., -0., -0., -1.,  0., -0.,  0.,  0.,
         0.,  0.,  0.,  0., -1.,  0.,  0., -1.],
       [-0., -0., -0.,  0., -0., -1., -0., -0.,  0., -0.,  0., -0.,  0.,
        -0., -0., -0.,  0., -0., -1., -0.,  0., -0.,  0.,  0.,  0.,  0.,
        -0.,  1.,  1.,  0., -0.,  0.,  0., -0., -1.,  0.,  1., -0., -1.,
         0.,  0., -0., -1.,  0.,  0., -0., -1.],
       [-0., -0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        -1.,  1., -0., -1.,  1.,  0., -1.,  0., -1.,  0.,  1.,  0.,  1.,
         0.,  1.,  0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  0.],
       [ 0.,  1., -0., -0.,  1.,  1., -0., -0., -1., -0.,  0., -0.,  1.,
        -1., -0.,  0., -0.,  1., -0.,  0., -0., -0.,  0., -0., -0., -0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.,
         0., -1., -0., -0.,  0.,  0.,  0.,  0.],
       [-1., -0., -0.,  0.,  1.,  0., -0.,  0., -0.,  1.,  1.,  0.,  0.,
        -0., -1., -0., -0., -0.,  0., -0.,  1.,  0.,  1.,  0.,  0., -0.,
        -1.,  0., -0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0., -1.,  0.,
         0., -0.,  0., -0.,  1., -0.,  0.,  1.],
       [ 0., -0.,  0., -0., -0., -0., -1., -1.,  1., -0.,  1., -1.,  0.,
        -0.,  0.,  0., -0., -0., -0.,  0., -0., -1.,  1., -0., -0., -0.,
        -1.,  0.,  0.,  1., -1.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
         0.,  0., -0., -0.,  1.,  1.,  0.,  1.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 2
m = 5
p = 6
rank = 47

verify_tensor_decomposition(decomposition_256, n, m, p, rank)

##Rank-23 decomposition of <3,3,3> over Z

In [ ]:
#@title Data
decomposition_333 = (np.array([[ 1.,  0., -0., -0., -1.,  0.,  0., -0., -0.,  1., -0.,  0.,  0.,
        -0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.],
       [-1., -0.,  0.,  0., -0.,  0.,  0.,  0., -1.,  0., -0.,  1., -0.,
         0.,  0.,  0., -0., -0., -1., -0.,  0.,  0.,  1.],
       [ 0., -0., -0., -0.,  0.,  0., -1., -0.,  0., -0.,  1.,  1.,  0.,
         0., -0., -0.,  1.,  0.,  0.,  1.,  0., -0., -0.],
       [ 0., -0.,  1.,  1.,  0., -0., -0., -0.,  0., -0.,  1., -0., -1.,
         0.,  0., -0.,  0.,  0., -0., -0., -1., -0., -0.],
       [-0.,  1.,  0., -1.,  0., -1., -0., -1.,  0., -0., -0.,  1., -0.,
        -0., -0., -0.,  0.,  1., -0.,  1., -0., -0.,  0.],
       [ 0.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  1.,  1., -0.,
        -0.,  0.,  0., -0., -0., -0.,  1.,  0.,  1., -0.],
       [ 1., -0., -1.,  0.,  0.,  0., -0.,  1.,  0., -0.,  0., -0.,  0.,
        -0.,  1., -0.,  0.,  0., -0.,  0., -0.,  0., -1.],
       [-1.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -1., -1., -0.,  0.,  0., -1.,  0., -0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -0.,  0., -0., -0., -1.,
        -1.,  0.,  1., -0., -0., -1., -0.,  0., -1., -0.]], dtype=np.float32), np.array([[ 0., -0., -1., -1.,  1., -0., -0., -1., -0.,  0., -0., -0.,  0.,
         0.,  0.,  0., -0.,  1., -0., -0.,  1., -0.,  0.],
       [-0.,  0., -1.,  0., -0., -0., -0.,  0., -0.,  1., -0., -0.,  0.,
         0.,  0., -0., -0., -0.,  0., -0.,  1., -0., -0.],
       [ 1.,  0., -1.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  1.,
         0.,  0.,  1., -0., -0., -1.,  0., -0., -0.,  1.],
       [ 0., -0.,  0.,  0., -0.,  1.,  0., -0., -0.,  0.,  1.,  1., -1.,
        -1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0., -0.],
       [ 0.,  0.,  0., -0.,  0.,  1., -0.,  1.,  1., -0.,  1.,  1., -1.,
        -1., -1., -0., -0., -0.,  0.,  1.,  1., -0.,  1.],
       [-0.,  0.,  0., -0., -0.,  1.,  0.,  1., -0.,  0., -0., -0.,  0.,
        -1., -1.,  1., -0., -0., -1., -0., -0., -0.,  1.],
       [ 0., -1., -0., -0.,  0., -1.,  0., -0., -0.,  0., -1.,  0.,  1.,
        -0.,  0., -0.,  0., -1., -0., -1., -1.,  0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0.,  1.,
        -0., -0.,  0., -0.,  0.,  0.,  0., -1., -1.,  0.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0., -0., -1., -1.,  0., -0.,  0.,  0., -1.,  0.]], dtype=np.float32), np.array([[-0.,  1.,  0., -0., -1.,  0.,  0., -0.,  1.,  0., -0.,  1., -0.,
         0.,  0., -0., -0.,  0., -0., -1., -0., -0., -0.],
       [-0., -1., -0., -1., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,
        -0.,  0., -0., -0.,  1.,  0.,  0., -0., -0., -0.],
       [-0.,  0.,  0.,  0., -0.,  1.,  0., -1.,  0., -0.,  0.,  0., -0.,
         1., -1., -0., -0.,  1., -0., -0., -0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -0.,  1.,  0., -1.,  1., -0.,  0., -0.,
         0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.],
       [ 0., -0., -0.,  1., -0., -0., -1., -0., -0.,  0., -1.,  0., -0.,
        -0., -0., -0., -0., -1.,  0.,  1., -1., -0., -0.],
       [-0.,  0.,  1., -0.,  0., -1.,  0.,  1.,  1.,  0.,  0.,  0., -1.,
         0.,  0., -0., -0., -1., -1., -0., -1.,  0.,  1.],
       [ 1.,  0.,  0., -0.,  0., -0.,  0.,  0.,  1.,  0.,  0.,  0., -0.,
         0., -1., -0., -1., -0.,  0., -0., -0., -0.,  1.],
       [ 0., -0., -0.,  0.,  0., -1.,  1., -0.,  0.,  0.,  1.,  0., -1.,
        -0., -0., -1., -0., -0., -0., -1., -0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -0., -0.,  0., -1.,  0., -0., -0., -0.,
         0.,  1., -1., -0., -0.,  1.,  0., -0.,  0., -1.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 3
m = 3
p = 3
rank = 23

verify_tensor_decomposition(decomposition_333, n, m, p, rank)

## Rank-54 decomposition of <3,4,6> over 0.5*Z


In [ ]:
#@title Data

decomposition_346 = (np.array([[ 0.5,  0. ,  0. ,  0. , -1. ,  1. ,  0. ,  1. , -1. ,  0. ,  0. ,
         0. ,  0.5,  1. , -0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. ,  0. ,  1. , -1. ,  1. ,  0.5, -0.5,  0. ,  0. ,  0.5,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         1. ,  0. ,  0.5, -1. ,  1. , -1. ,  0. ,  1. ,  0. , -1. ],
       [ 0.5,  0. ,  0. ,  0. ,  1. ,  1. ,  0. ,  1. ,  1. ,  0. ,  0. ,
         0. ,  0.5, -1. ,  0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. ,  0. ,  1. ,  1. , -1. ,  0.5, -0.5,  0. ,  0. , -0.5,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0.5,  0. ,  0. , -1. ,  0. ,  1. ,  0. ,  0. ],
       [-0.5,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -0.5,  0. , -1. ,  0. , -1. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  0. ,  0. , -0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0.5,
        -1. ,  0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ,  0. , -1. ,  1. ,  0. , -1. ,  1. ,  0. ,  0. ,
         0. ,  0.5, -1. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,  0. ,
         0. ,  0.5,  0. ,  0. ,  1. ,  0. , -0.5, -0.5, -1. ,  0. , -0.5,
         0. ,  0. ,  0. ,  0.5,  0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0.5,
         0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ],
       [-0.5,  0. , -1. , -0.5,  0. ,  0. , -1. ,  0. ,  0. , -1. , -1. ,
        -1. ,  0. ,  0. ,  0.5, -1. , -0.5,  0. ,  0. , -0.5,  0.5,  0. ,
         0. , -0.5,  1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -0.5,  0. ,  1. ,  0. , -0.5,  0.5,  0. , -1. , -0.5,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0.5,  0. , -1. ,  0. ],
       [-0.5,  0. ,  0. , -0.5,  0. ,  0. ,  1. ,  0. ,  0. ,  1. , -1. ,
         0. ,  0. ,  0. , -0.5,  1. ,  0.5,  0. ,  0. , -0.5, -0.5,  0. ,
         0. ,  0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0.5,  0. ,  1. ,  0. , -0.5, -0.5,  0. ,  1. ,  0.5,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0.5,  0. ,  0. ,  0. ],
       [ 0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  1. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  1. ,  0. ,
         0. ,  0. , -1. ,  0.5,  0. ,  0. ,  1. ,  0. ,  0. ,  0. , -0.5,
         0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -0.5,  0. ,  0. ,  1. ,  1. ,  0. ,  1. , -1. ,
         0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. , -0.5,  0.5,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,
         0.5,  0. ,  0. , -0.5,  0.5,  0.5,  1. , -1. ,  0.5,  0. , -0.5,
         0. ,  0. ,  0.5,  0. ,  0. , -1. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  1. ,  0.5, -1. ,  1. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0.5,  0.5,  0. ,  0. ,  0. ,  0. , -0.5, -1. ,  0.5,  0. ,  1. ,
         0. ,  0. , -0.5,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0.5,  0. , -1. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  1. ,  0. ,
         1. ,  0. ,  0. , -1. ,  0.5,  0. ,  0. ,  0. ,  0. , -0.5],
       [ 0. ,  0. ,  0. ,  0.5,  1. , -1. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0.5,  0.5,  0. ,  0. ,  0. ,  0. , -0.5, -1. ,  0.5,  0. , -1. ,
         0. ,  0. , -0.5,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -0.5,  0. , -1. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5, -1. ,  0. ,
        -1. ,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0.5],
       [ 0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. , -1. ,
        -0.5,  0. , -1. ,  0. ,  0. ,  0. ,  0.5,  0. , -0.5,  0. , -1. ,
         0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0.5],
       [ 0. ,  1. ,  0. ,  0.5, -1. ,  1. ,  0. ,  0. ,  1. , -1. ,  1. ,
         0. ,  0.5, -1. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,
        -0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
        -0.5,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. , -0.5,  1. ,  0. ,
         0. ,  1. ,  0. ,  0. ,  0.5,  0. ,  0. ,  1. ,  0. ,  0. ]],
      dtype=np.float32), np.array([[ 0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5, -1. ,
         0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ],
       [ 0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
        -1. ,  1. , -0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ],
       [ 0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. , -1. ,  0. ],
       [-1. ,  0. , -1. ,  0. ,  0. , -1. ,  0.5,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. , -1. ,  0. ,  0. ,  1. ,  0. , -1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0.5, -1. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,  1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ],
       [ 0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,
         0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [-1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. , -0.5,  0. ,
         0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  1. ,  1. , -1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,
        -1. ,  1. , -0.5,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  1. ],
       [ 0. , -0.5,  1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ,  0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  1. , -1. ,  0. , -0.5,  0. ,  0. ,  0. ,
        -0.5,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ],
       [ 0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  1. ,  0. ,
         0. , -1. , -1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. , -1. ,  0. ],
       [-1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. , -1. , -1. ,  0.5,  1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  1. ,
         1. ,  0. ,  0. ,  0. ,  1. ,  0. , -1. ,  0. ,  0. , -0.5,  0. ,
         1. ,  0. ,  0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. , -1. ,  0. ,  0. ,  1. ,  1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.5,  0. ,  1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
         0. ,  0. , -1. ,  0. ,  1. , -1. ,  0. , -0.5,  0. ,  0. ,  0. ,
        -0.5,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         1. ,  0. , -0.5,  1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. , -1. , -1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  1. , -1. , -1. ,  2. ,  0. ,  0.5,  0. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. , -1. ,  0. ,  0. , -1. ,  0.5,  0. ,  0. , -1. ,  0. ,
         0. ,  0. ,  0. ,  0. , -1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  1. ,  0. ,
        -1. ,  0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0.5, -1. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ,  1. ,  0. ,  0. ,  1. ,  1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. , -1. ,  0. ,  1. ,  0. ,  0. , -0.5,  0. ,
         0. ,  0. , -0.5,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,
         1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  1. ],
       [ 0. ,  1. , -1. ,  0. , -1. , -1. ,  0. ,  0. ,  0. , -1. ,  0. ,
         0. , -1. ,  0. ,  0. , -1. ,  2. ,  0. , -1. ,  0. ,  0. ,  0. ,
         0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
        -1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  1. ,  0.5,  0. ,
         0. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. , -1. ,  0. ,  0. ],
       [-1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,  0. ,
         0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  1. ,  0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.5,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  0. , -1. ,  0. , -1. ,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. , -1. ,  1. ,  0. ,  0.5,  0. ,  0. ,  0. ,
         0.5,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. , -0.5,  1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. , -1. ,  1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  1. ,  1. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. , -1. ,  0. ,  0. , -1. , -0.5,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0.5, -1. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ,  1. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  0.5,  0. ,
         0. ,  0. , -0.5,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  1. ],
       [ 0. ,  0. , -1. ,  0. ,  1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,
         0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. , -1. ,  0.5,  0. ,
         0. ,  0. ,  0. , -1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ]],
      dtype=np.float32), np.array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,
         0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0.5, -1. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,  1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0.5,  0. , -1. ,  1. ,  0. , -1. ,  0. ,  0. ,  1. ],
       [ 0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,
         1. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. , -1. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  0. ,  1. , -1. ,  0. ,  1. ,  0.5,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,
         0. ,  1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  0. ,  0. , -1. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
        -1. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  1. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -1. ,  0. , -1. ,  0. ,  0.5,  0. ,  1. ,  0. ,  0. , -1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. , -1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. , -0.5, -1. ,  0. ,  0. , -1. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. , -0.5,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,
         1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,
         0. ,  0. ,  0. ,  0. ,  0.5,  0. , -1. ,  0. ,  1. ,  0. ,  0. ,
         0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  1. , -0.5,
         0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,
        -1. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. , -1. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,  0. ,  0. , -0.5,
        -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. , -1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  1. ],
       [ 0. ,  0. ,  0. ,  0. , -0.5, -0.5,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,
         0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [-1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0.5,  0. ,  0. ,  0. ,
         0. ,  0. , -1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ,
         0. ,  0. , -1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0.5,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ]],
      dtype=np.float32))

In [ ]:
#@title Verification
n = 3
m = 4
p = 6
rank = 54

verify_tensor_decomposition(decomposition_346, n, m, p, rank)

##Rank-63 decomposition of <3,4,7> over 0.5*C

In [ ]:
#@title Data

decomposition_347 = (np.array([[ 0. -0.j , -1. +0.j ,  0. +0.j ,  0. +1.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j ,  1. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,
         1. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j , -0. +1.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -1. -0.j ,  0. +0.j ,  0. -0.j ,  0. -1.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  1. +0.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
        -0. +1.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ],
       [ 0. +0.j , -0.5-0.j , -0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0.5+0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0.5+0.j , -0. +0.j ,  0. +0.j ,  0. +0.5j, -0. +0.5j,
        -0.5-0.j , -0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. -0.5j,
         0. +0.j , -0. +0.5j,  0. +0.j , -0. +0.j ,  0. -0.j ,  0.5+0.j ,
        -0.5+0.j ,  0.5+0.j , -0. +0.j , -0. +0.j ,  0. +0.5j, -0.5+0.j ,
         0. -0.5j, -0.5+0.j ,  0. +0.j , -1. +0.j , -0. +0.5j,  0. -0.5j,
        -0.5-0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j ,
         0.5+0.j , -0.5-0.j ,  0. +0.j , -0. +0.j ,  0. -0.5j, -0. +0.5j,
         0. -0.5j, -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,
         0. +0.j ,  0.5+0.j ,  0. +0.j ],
       [ 0. -0.j , -0. +0.j ,  0. -0.j , -1. -0.j ,  0. -0.j , -0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. -0.j ,  0. -1.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  1. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -1.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  1. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. -1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j ,
         1. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
        -0. +1.j , -0. +0.j , -1. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -1. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -1.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [-0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  1. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +1.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -1.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. -1.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +1.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  1. +0.j ,
         0. -0.j ,  0. -1.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. -1.j , -0. +1.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  1. +0.j ,
         0. -0.j , -0. +1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +1.j ,  0. +0.j ],
       [-1. -0.j ,  0. +1.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -1.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -1.j , -0. +0.j , -1. -0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. -1.j ,  0. +0.j ,  0. -0.j ,  0. -1.j ,  0. -0.j ,
        -1. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. -1.j ,
        -0. +0.j , -0. +1.j ,  0. +0.j , -0.5-0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -1.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.5j,  0. -0.5j, -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0. +0.j ,  0. -0.5j, -0. +0.j ,  0.5+0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.5j,  0. +0.5j,  0. +0.j ,  0. +0.j , -0. +0.5j,  0. +0.j ,
         0.5+0.j ,  0.5+0.j , -0. +0.j ,  0.5+0.j , -0. +0.j ,  0. -0.5j,
         0. +0.j , -0. +0.5j,  0.5+0.j , -0. +0.j , -0. +0.j ,  0. -0.5j,
        -0.5+0.j ,  0. -0.5j, -0.5+0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,
         0. -0.5j,  0. +0.j , -0. +0.5j,  0. -0.5j, -0.5+0.j ,  0. -0.j ,
        -0.5-0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j, -0. +0.j , -0. +0.j ,
         0. -0.j ,  0. -0.5j,  1. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j , -1. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -1. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -1.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +1.j ,  0. -0.5j,  0. +0.j ,  1. +0.j ,
         0. -1.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j ,  0. +0.j ,
         1. +0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +1.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j , -0.5+0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ],
       [ 0. -1.j ,  0. +0.j , -1. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +1.j , -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  1. +0.j , -0. +1.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  1. +0.j , -0. +0.j ,  0. +0.j ,
         0. -1.j ,  1. +0.j , -0.5-0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  1. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  1. +0.j ,
         1. +0.j ,  1. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -1.j ,  0. -1.j ,  0. +0.j ,
         0. -0.j ,  1. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j , -0. +1.j ,  0. +0.j , -0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +1.j ,  0. -0.j ,  1. +0.j ,
         0. -0.j , -1. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ,  1. +0.j ,
         1. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j , -0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. +0.5j,  0. -0.j ,
         0.5+0.j ,  0.5+0.j ,  0. +0.5j,  0.5+0.j ,  0.5+0.j ,  0.5+0.j ,
         0. -0.5j, -0. +0.j , -0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.5j,
        -0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,
         0. +0.5j,  0. -0.j , -0. +1.j , -0. +0.5j,  0. +0.j ,  0. +0.j ,
        -0.5-0.j , -0. +0.j ,  0. +0.5j,  0. -0.j , -0. +0.5j,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.j ,  0. -0.5j,
         0. +0.j ,  0. -0.j , -0. +0.5j,  0. +0.5j,  0. +0.5j, -0.5-0.j ,
         0. +0.j , -0.5-0.j , -0.5+0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j ,  0.5+0.j ,  0.5+0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +1.j ,  0. -1.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  1. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j , -0. +0.j , -0. +0.j ,
        -1. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. -1.j ,  0. +0.j , -1. -0.j , -0. +0.j , -1. +0.j , -0. +0.j ,
         0. -0.j ,  0. -0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j , -1. -0.j ,  0. +0.j , -1. -0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +1.j ,  0. -0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. -1.j ,  0. -0.j ,  1. +0.j , -0. +1.j ,
         0. +0.j ,  0. +0.j , -1. -0.j ,  0. +1.j ,  0. -0.j ,  0. +0.j ,
         1. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  1. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  1. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -1.j , -0. +1.j , -0. +0.j ,  1. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -1.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ]]), np.array([[ 0. +0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j , -0. +1.j ,
         0. +0.j ,  0. +0.j , -0. +0.5j,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. +1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j , -1. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j , -1. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.5j,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -1. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j , -0.5-0.j ,  0. -0.j ,  0. +0.5j,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [-0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.5j,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j , -0. +0.5j,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.5j,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0.5+0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0.5+0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. -0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j , -0.5+0.j ,
         0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j ,
         0. -0.j ,  0. -1.j , -0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
         0. -1.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. -0.5j, -0. +0.j ,  0.5+0.j ,  0. -0.j , -1. -0.j , -0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. -0.5j,  0. -1.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,
         0. -1.j , -0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  1. +0.j ,  0. +0.j , -0. +0.j ,  0.5+0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
         0. -0.j , -0. +0.j , -0. +1.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  1. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. -0.5j,
         0. -0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.5j,
         0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j , -1. +0.j , -0. +0.j ,
         0. +0.j , -0.5-0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. -1.j , -0. +0.j , -1. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. -1.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0.5+0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.5j,
         0. +0.5j, -0. +0.j ,  0. +0.j ,  0. -0.5j, -0. +0.j , -0. +0.j ,
         0. -1.j ,  0. +0.j ,  0. -0.5j,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0.5-0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +1.j ,  0. -1.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         1. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.5j, -0. +0.j ,
        -0. +0.j ,  0.5+0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0.5+0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j , -0.5+0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ],
       [ 0. +1.j ,  0. -1.j ,  0. +0.j , -0.5-0.j ,  0. +0.j ,  0.5+0.j ,
         0. +0.j ,  1. +0.j ,  0. -0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.5j,  0.5+0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0.5-0.j , -0. +1.j , -0.5+0.j ,  0. +0.5j, -1. -0.j ,  0. +0.5j,
        -0. +0.5j, -0. +0.5j,  0. +0.j ,  0.5+0.j , -0.5-0.j ,  0. -0.5j,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. -1.j ,  0.5+0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.j , -0.5-0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +1.j ,
         0. +0.5j,  0. -0.5j,  0. +0.5j, -0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +0.5j, -0. +0.j ,  0. +1.j , -0.5-0.j , -0. +1.j ,  0. -0.5j,
        -0. +0.5j, -0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. -1.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. -1.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j , -1. +0.j ,  0. -0.j , -0. +1.j ,
         0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  1. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. -1.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  1. +0.j , -0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -1. -0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -1. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -1. -0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -1. -0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -1.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.j ,  0. +1.j ,  0. -0.j ,  0. -1.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ],
       [-0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -1. +0.j , -0. +0.j ,
         0. -0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  1. +0.j , -0. +0.j , -1. -0.j ,  0. +0.j ,  0. +1.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. -1.j ,
         0. -0.j ,  0. +1.j ,  0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  1. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j , -0. +0.j , -1. -0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -1.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j , -0. +1.j ,
         0. -1.j , -0. +0.j ,  0. +0.j ,  0. +1.j ,  0. -0.j , -0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +1.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -0. +1.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j , -0. +1.j ,  0. +1.j ,  0. +0.j ,
         0. +0.j , -1. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -1. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -1. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j , -1. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +1.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  1. +0.j ],
       [ 0. +0.j ,  0. +0.j , -0.5+0.j ,  0. -0.j ,  0. -0.5j,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +1.j ,  0. +0.j ,  1. +0.j ,
         1. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  1. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j , -0.5+0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j ,
         0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. -1.j , -0. +0.5j,  0. +0.j ,  0.5+0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ],
       [ 0. -0.j ,  0. +0.5j,  0. +0.j ,  1. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j , -0.5+0.j ,  0. +0.j ,
         1. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. -1.j ,  0.5+0.j ,  0. -0.j ,  0. -0.j , -1. -0.j ,  0. -0.j ,
         0. -1.j ,  0. +0.j ,  0. -0.j , -0. +0.5j, -0. +1.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ],
       [-0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +1.j ,  0. -0.5j,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.5j,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0.5-0.j , -0. +0.j ,  0. -0.5j, -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j , -0.5-0.j ,  0. -0.j ,  0.5+0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j , -0. +0.j ,  1. +0.j ,  0. -1.j , -0. +0.j , -0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
        -0. +1.j ,  0. +0.5j, -0. +0.j ,  0. -0.5j,  0. -0.j , -0.5+0.j ,
        -0. +0.j , -1. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,
         0. -1.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0.5+0.j ,
         0. +0.j , -0.5-0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.5j,  0. +0.j ],
       [ 0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.5j,
         0. -0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
        -0.5+0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0.5+0.j ,
        -0.5+0.j ,  0. +0.j , -0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. +0.j ,
         1. +0.j ,  0. +0.j ,  0.5+0.j , -0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.5j,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ],
       [-0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +1.j ,  0.5+0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  1. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,  1. +0.j ,
         0. +0.j , -0. +0.5j, -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j , -0. +0.j , -0.5-0.j ,  1. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.5j,
         0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.5j,  0. -1.j ,
        -0. +1.j , -0. +0.j ,  0. +0.j ],
       [-1. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.5j, -0. +0.j ,  0. +0.5j,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. -1.j ,
         0. +0.j , -0.5+0.j ,  0. +0.5j,  0. +0.j , -0. +0.j ,  0. -1.j ,
        -0. +0.5j,  0. +0.j ,  0. -0.5j, -0.5-0.j ,  0. +0.j , -0.5-0.j ,
         0.5+0.j , -0.5-0.j ,  0. -0.j ,  0. -0.5j,  0. -0.5j,  0.5+0.j ,
         1. +0.j ,  0. +0.j ,  0. +1.j , -1. +0.j ,  0. -0.5j,  0. +0.j ,
         1. +0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j ,  0. -0.j ,  0. -0.j ,
        -0.5-0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,  0. +1.j ,  0. +0.j ,
        -0.5-0.j ,  0. +0.j , -1. -0.j ,  0. -0.5j,  0. -0.j , -0.5+0.j ,
         0.5+0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0.5+0.j , -0. +0.j ,  0. +0.5j,  0. +0.j ,
         0. +0.j , -0. +0.j , -0.5+0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -1. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.5j,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. +0.j , -0. +0.j , -1. -0.j , -0. +0.j , -0.5-0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -1.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0.5+0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.5j,  0. +1.j ,  1. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -1.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,
         0. +0.j ,  0. -1.j ,  0. +0.j ],
       [-0. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -1.j , -0. +0.5j,
         0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.5j,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  1. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.5j,  0. +0.j , -0. +1.j ,  0. +0.j ,
        -0.5+0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +1.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5-0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.5j,  0. -0.j , -0. +0.5j,  0. +0.j ,  0.5+0.j ,
        -0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j ,
        -0. +0.j , -0.5-0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.5j,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  1. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0. +0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j , -1. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
         0.5+0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0.5-0.j ,
         0.5+0.j ,  0. -0.j ,  0. +0.j , -0.5-0.j ,  0. +0.j , -0. +1.j ,
        -1. -0.j ,  0. +0.j , -0.5+0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.5j, -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  1. +0.j ,  0. +0.j ],
       [ 0. +0.j , -0. +0.j , -1. +0.j ,  0. +0.j ,  0. -1.j ,  1. +0.j ,
        -0. +1.j ,  0.5+0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. -0.j ,  0. -0.j ,  0.5+0.j ,  0.5+0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.5j,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.j , -0.5-0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j ,  0. -0.j , -0. +0.5j,
         0. +0.j ,  0. -1.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ],
       [ 1. +0.j , -0. +0.j ,  0. +1.j ,  0. -0.5j, -1. -0.j ,  0. -0.5j,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.5j,  0. +0.j ,  0. -0.5j, -0.5-0.j , -0. +0.j ,  0.5+0.j ,
        -0.5-0.j , -0.5-0.j ,  0. +0.j , -0. +0.5j,  0. -0.5j, -0.5+0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  1. +0.j , -0. +0.5j,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j , -0. +1.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0. +0.j ,
        -0.5+0.j , -0.5-0.j , -0.5-0.j ,  1. +0.j , -0. +0.j , -0. +0.j ,
         0.5+0.j ,  0. +1.j , -1. -0.j ,  0. +0.5j, -0. +0.j ,  0.5+0.j ,
        -0.5+0.j ,  1. +0.j ,  0. +0.j ]]), np.array([[ 0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.j ,  1. +0.j ,  0. -0.j ,  0. +0.j ,
         1. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +1.j ,
         0. -0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,
         0. -0.j , -1. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -1. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j , -1. +0.j ,
         0. -1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  1. +0.j ,  0. +0.j ],
       [-0.5+0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
        -0. +0.j , -0. +0.j , -0. +1.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +1.j ,  0. -1.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j , -0. +1.j , -1. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ],
       [-0. +0.j , -0. +0.j , -0. +1.j ,  0. +0.j ,  0. +0.j ,  0. -1.j ,
         0. +0.j , -0. +0.j , -1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -1. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  1. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j , -1. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. -1.j ,  0. -0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j , -1. +0.j , -0. +0.j , -1. +0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -1. -0.j , -0. +1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -1. +0.j ,
         0. -0.j ,  0. +1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +1.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0.5+0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
        -1. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  1. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  1. +0.j ,  0. -0.j , -1. +0.j ,  0. -0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,
        -1. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. -0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j , -0. +0.j , -1. -0.j ,  1. +0.j , -0. +0.j ,
         0. +0.j , -1. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +1.j ,
        -0. +1.j , -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  1. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -1.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -1.j , -0. +0.j , -0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ],
       [ 0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -1. -0.j ,  0. -0.j ,  0. -0.j , -0. +1.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
         0. -1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -1.j ,  0. -1.j ,
         1. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.j ,  1. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
         0. -1.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ],
       [ 0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +1.j ,  0. -0.j ,
         0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  1. +0.j ,  0. -1.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j , -1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -1.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  1. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -1. +0.j , -1. -0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j , -0. +0.j ,  0. +0.j ,  0. -1.j ,  0. -1.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -1.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +1.j , -0. +0.j ,  0. +0.j ],
       [-0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. -1.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j , -0. +1.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j , -1. -0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -1.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  1. +0.j ,  0. +0.j ],
       [-0.5-0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +1.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j , -0. +1.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  1. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +1.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j , -1. +0.j , -0. +1.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +1.j ,  0. -0.j , -0. +1.j ,  0. +0.j ,
         0. -0.j , -0. +0.j , -1. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j , -1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -1.j ,  0. +0.j ,  0. -1.j ,  0. -0.j , -0. +0.j ,
         0. +0.j , -1. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -1. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j , -0. +0.j , -0. +1.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         1. +0.j ,  0. +0.j , -1. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +1.j ,
        -1. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -1.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +1.j , -0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ],
       [-0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,
        -0. +0.j , -1. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +1.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j , -0. +1.j ,  0. +0.j ,  1. +0.j ,
         0. -0.j , -0. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,
        -1. -0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -1. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j , -1. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -1.j , -0. +0.j ,  1. +0.j ,
         0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j ,
         0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +1.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +1.j ,  0. +0.j , -1. -0.j ,  0. +0.j , -1. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +1.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +1.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. -0.j ,  1. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
        -1. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  1. +0.j , -0. +0.j ,  0. +0.j , -1. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  1. +0.j ,
        -0. +0.j ,  0. -1.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j , -1. +0.j , -1. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ],
       [-0.5+0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -1.j ,  1. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -1.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j , -0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +1.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j , -0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. -1.j ,
         1. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  1. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  1. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. -1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +1.j ,  0. -0.j , -0. +0.j ,
        -1. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. -1.j ,  0. +0.j ,  0. -1.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -1.j , -1. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
        -1. -0.j ,  0. -0.j ,  1. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,
         0. -0.j , -1. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -1.j ,
        -0. +0.j , -1. -0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0. -1.j ,
         1. +0.j ,  0. +1.j ,  0. +0.j , -0. +1.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. -1.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,
        -1. -0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -1.j ,  0. +0.j ],
       [ 0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.j , -0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. -0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0. +0.j ,  0. +0.j ,  1. +0.j ,  0. -0.j , -0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  1. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  1. +0.j ,  0. -0.j ,  0. -0.j , -1. -0.j ,
         0. +0.j ,  0. +0.j , -0. +1.j ,  0. -0.j ,  0. +0.j , -1. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  0. -1.j ,  0. +0.j , -0. +0.j ,
        -0. +0.j ,  0. -1.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
        -1. -0.j , -0. +0.j ,  0. +1.j ,  0. -1.j ,  0. -1.j ,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0. -0.j ,  0. -0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +1.j ,  0. +0.j , -0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.j , -1. +0.j ,  1. +0.j ,  0. +0.j ,
         0. +0.j ,  0. -0.j ,  0. -1.j ,  0. -0.j ,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j ,  0. -0.j ,  1. +0.j ,  0. +0.j , -0. +0.j ,
         0. +1.j ,  0. +0.j ,  0. +0.j ]]))

In [ ]:
#@title Verification
n = 3
m = 4
p = 7
rank = 63

verify_tensor_decomposition(decomposition_347, n, m, p, rank)

##Rank-74 decomposition of <3,4,8> over Z


In [ ]:
#@title Data

decomposition_348 = (np.array([[ 1., -0.,  0., -0.,  0.,  1.,  0., -1.,  0., -1., -0., -0.,  1.,
         1.,  0., -1.,  0., -1., -0., -0.,  0.,  1.,  0.,  1.,  0.,  1.,
        -1., -1., -1.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,
         1.,  1., -1.,  1.,  1.,  0., -0., -0.,  0., -0., -0., -0.,  0.,
        -0., -1., -0., -1.,  1., -1.,  0.,  0., -1., -1., -0.,  0., -0.,
         1., -1., -1.,  1., -0.,  0., -0.,  0., -1.],
       [-0., -0., -0.,  1., -0.,  1., -0.,  0., -1.,  0.,  0., -0.,  1.,
         1., -0., -1., -0.,  0., -0.,  0., -0., -0., -0.,  1., -0., -0.,
         0.,  0.,  0., -1., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0., -0.,  0.,  1., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,
         0., -1., -0.,  0., -0., -1.,  0., -0.,  0.,  0., -0.,  0., -0.,
         1.,  0.,  0., -0.,  0., -0., -0., -0., -1.],
       [-0.,  0.,  0., -0., -0.,  1.,  0., -1.,  0.,  0., -0.,  1., -0.,
        -0., -0.,  0.,  0., -1.,  0., -0.,  0., -0., -0.,  1.,  0., -0.,
         0.,  0., -1.,  0.,  0., -1.,  0., -0.,  0.,  0.,  0., -0.,  1.,
         1., -0., -1.,  1.,  1., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0.,  0.,  0., -1.,  1., -1., -0.,  0.,  0.,  0.,  0., -0.,  1.,
        -0.,  0., -1.,  1., -0.,  0.,  1.,  0., -1.],
       [-1.,  0.,  0.,  1., -0.,  0., -0.,  1., -1., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  1., -1.,
        -0.,  1.,  1.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., -1., -0.,  0., -1.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -0., -0.,  0.,  1.,  0., -0.,  0., -0., -0.,  1., -0., -0.,  0.,
         0., -0., -0., -1., -0.,  0.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0., -0.,  0., -0.,  0.,
         0., -0., -0.,  1., -0., -1., -0.,  0.,  0., -0., -1.,  0.,  0.,
        -0., -0., -0.,  0., -1.,  1., -0.,  0.,  1., -0.,  0.,  0.,  0.,
        -1.,  0.,  1., -1.,  0., -0., -0.,  0.,  1., -1.,  0., -0.,  0.,
        -1., -0., -1.,  1.,  0., -0.,  0.,  0., -0., -0., -1.,  0.,  0.,
         0., -0.,  1., -1., -0., -0., -0., -0.,  1.],
       [-0., -0., -1.,  0., -0.,  0., -1., -0., -0.,  0., -0., -0.,  0.,
         0.,  0., -0., -0., -0., -1.,  1., -1., -0.,  0.,  0., -0., -0.,
         0.,  0., -0., -0., -1., -0.,  1., -0.,  1., -0., -0., -1.,  0.,
         0., -0., -0.,  0., -0.,  0., -0., -0., -0., -1., -1.,  1., -1.,
         0., -0., -1., -0., -0., -0., -0., -1.,  0.,  0., -1.,  0.,  0.,
         0.,  0., -0.,  0.,  1.,  0., -0.,  0.,  1.],
       [-0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  1.,  1.,  0., -0.,  0., -0.,  0., -1., -0., -0.,
         0.,  0.,  1.,  0.,  0.,  1., -0., -0.,  1., -0., -0.,  0., -1.,
        -1., -0.,  1., -1.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,
         0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -1., -1.,
        -0.,  0.,  1., -1., -0., -0.,  0., -0.,  1.],
       [-0., -0.,  0.,  0., -1.,  0., -1., -1., -0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0., -0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0., -0., -0.,  0., -0.,
        -0., -0., -1.,  1., -0., -0., -0., -0., -1.,  0., -1., -0.,  0.,
        -0., -0., -0., -1.,  0.,  0., -0., -1., -0.,  0.,  0., -0., -0.,
         0., -0.,  0., -0., -0., -0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -1.,  0., -0.,  1.,
         0., -1., -1.,  0.,  0.,  1.,  0., -0.,  1., -1., -0., -0.,  1.,
        -0., -1.,  0., -0., -0.,  0., -1.,  0., -1., -1.,  0.,  1.,  0.,
        -0., -0.,  0., -0., -0., -1.,  0.,  1.,  0.,  1., -0.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -1., -0.,  0., -0.,
         1., -1.,  0., -0.,  0., -1., -0., -0.,  0.],
       [ 0.,  0., -0., -0.,  0., -0.,  1., -0.,  0.,  0., -1.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  1., -1., -0., -0.,  0., -0.,  0., -0.,
        -0.,  0., -0.,  0.,  1., -0., -1., -0., -1., -1.,  0.,  1.,  0.,
         0., -0., -0., -0.,  0.,  0., -1., -0., -0.,  1.,  1., -1.,  1.,
         0.,  0.,  1., -0.,  0.,  0., -1.,  1., -0.,  0., -0.,  0.,  0.,
         1.,  0., -0.,  0., -1.,  0., -0.,  0.,  0.],
       [-1., -1.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,
        -1.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,
         1.,  0.,  0., -0., -0.,  0., -1.,  0., -1., -1., -1.,  1., -0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  1.,  0.,  1., -0.,  0., -0.,
         0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,
        -0.,  0.,  0., -0.,  0., -1.,  1., -0.,  0.],
       [-0., -0.,  0., -0., -0., -0.,  1., -0.,  0.,  1.,  0., -0.,  0.,
        -0., -0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  1., -0., -0., -1.,
         0.,  1., -0., -0.,  1.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0.,  0., -0., -1., -1., -0.,  0.,  1., -1., -0.,
        -0., -0., -0., -0., -0.,  0., -1.,  1.,  0., -0., -0., -0., -0.,
         0., -0.,  0.,  0.,  0., -0.,  0., -1.,  0.]], dtype=np.float32), np.array([[ 0., -0., -0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,
         0., -0., -0., -0., -1., -0., -0.,  0., -0., -0., -0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  1.,
        -1.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,
         0.,  0., -0., -0., -0., -0.,  0., -0., -1., -0., -0., -0., -0.,
        -0.,  1., -0., -0., -0.,  0.,  0.,  0.,  0.],
       [-0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,
        -0., -0., -0., -0.,  0.,  1., -0., -0., -1., -0.,  0., -0., -0.,
        -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -1.,  0., -0.,  0.,  0.,  0.,  0.,  1.,
        -0., -0., -1.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,
         0., -0., -0., -0., -0., -0., -0.,  0., -0.],
       [ 0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -1.,  0., -0.,  0.,
        -0., -0.,  0.,  0., -1.,  1.,  1.,  0.,  0.,  1.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -1.,  0., -0.,  1., -0., -0., -0.,  0.,  1.,
        -1.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  1.,
         0., -0.,  0.,  0.,  0., -0.,  0., -1., -1., -0.,  0., -0.,  0.,
         0., -0.,  0., -0., -0., -0., -0.,  0.,  0.],
       [-0.,  1.,  0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,
        -0., -0.,  0., -0., -0., -1.,  0.,  0., -0.,  0., -0.,  0.,  0.,
         1., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  1., -0.,
         0., -0., -0., -0.,  0., -0., -0., -0., -0.,  1., -0., -0., -0.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
        -0.,  1., -0.,  0.,  1., -1.,  0.,  0.,  0.],
       [-0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0., -0.,  0.,  0., -1.,  1., -0., -0.,  0., -0.,  0.,  0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  1.,
        -1., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  1.,
        -1.,  0., -1., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -0., -0., -0.,  0.],
       [ 0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0.,  1.,  0., -0.,  0.,  1.,  0., -0., -0., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -1.,  0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  1.,
         0.,  0., -1., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -1.,
        -0.,  0.,  1.,  0., -0., -0., -0., -0.,  0.],
       [-0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,
         0.,  0., -0., -0., -0.,  0.,  0.,  0., -1.,  0., -1., -0.,  0.,
         0., -0.,  0.,  1., -0., -0., -0.,  0., -0., -0., -0.,  0., -1.,
        -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
        -0.,  1., -0., -0., -0., -1., -0., -0., -0., -0., -0.,  0.,  0.,
         0.,  0., -1., -0.,  0., -0., -0., -0., -0.],
       [ 0.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0., -0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -1.,  0.,  0.,  1., -0.,
         0.,  0., -1., -0., -0., -0., -0.,  0., -0., -0., -0., -0., -1.,
        -0., -1.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  1., -0.,  0., -0.,
        -0.,  0., -1.,  1.,  0., -0., -0., -0., -0.],
       [ 0., -0., -0., -0., -0., -0., -0.,  0., -0., -1., -0., -0., -0.,
         0., -0., -1.,  0., -0., -0.,  0.,  1.,  0., -0.,  0., -0.,  0.,
         0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,
         1., -0.,  1., -1.,  0., -0.,  1.,  0., -0., -0.,  0., -0.,  0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -1., -0., -0., -0., -0., -0.,
        -1., -1.,  0.,  0.,  0., -0.,  0.,  0.,  1.],
       [ 0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -1.,  0., -1.,
        -0., -0., -0., -0., -0.,  0.,  0., -0.,  1., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -1.,
        -0., -0., -0., -0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,
         1., -0.,  0.,  0., -0.,  0., -0., -0., -0.],
       [ 0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -1.,  1., -0.,  0., -0., -0., -0.,
        -0.,  0.,  0.,  0., -0.,  0., -0., -1., -0., -0., -0., -0.,  0.,
         1.,  0.,  1., -1.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -1.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0.,  0., -0., -0.,  0.,  0., -0.,  0.,  1.],
       [ 0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,
        -1.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0.,
        -1.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -1.,  0., -1.,  0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,
        -1., -1.,  0., -0., -1.,  1.,  0., -0.,  0.],
       [ 0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0.,  1.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,
         1., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -1.,
         1.,  0.,  1.,  0., -0.,  1.,  0.,  0.,  0., -0., -1.,  0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -1., -0., -0.,
        -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,
        -0., -0., -1., -0.,  1.,  0., -0.,  0.,  1.],
       [ 0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0., -1.,
         0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  1., -0.,  1.,  0., -0.,
        -0., -0., -0., -1.,  0., -0., -0., -0.,  0., -0., -0., -0., -0.,
         0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  0., -0.,
         1., -0.,  1., -0.,  0.,  0., -0.,  0., -1.],
       [ 0.,  0., -1., -1., -0., -0., -0.,  0., -0., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1., -1., -0.,
        -0.,  1., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0.,
        -0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0., -1., -0.,
         0., -0.,  0., -0.,  0.,  0.,  1., -0., -0., -1., -0.,  0.,  0.,
        -0.,  0.,  1.,  0.,  0.,  0., -0., -0., -1.],
       [-0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,
         0., -0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,
         1.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -1.,
        -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0., -0.,  0., -0., -1.,  0.,  0., -0.,  1., -0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  1.,  0.,  0.],
       [-0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  1., -0.,
        -0., -1.,  0., -1., -0., -1., -0., -0., -0.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  1.,  0.,  0., -1.,  0.,  1., -1., -0.,
        -0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -1.,
        -0.,  0.,  1.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -0., -1., -0., -0.,  0., -0.],
       [ 0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  1.,  0., -0.,  0.,
        -0.,  0.,  0.,  0.,  1., -1.,  0.,  0., -0., -1.,  0.,  0.,  0.,
        -0., -0., -0.,  0.,  1., -0., -1.,  0., -1.,  0., -0.,  0., -1.,
        -0., -0., -1., -0.,  0.,  0.,  0., -0., -1., -0., -0., -0.,  0.,
         0.,  0., -0., -0.,  0., -0.,  0.,  1.,  1.,  0.,  0., -0., -0.,
         0., -0., -0.,  0.,  0., -0., -0., -0.,  0.],
       [-0., -1.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,
        -0.,  0., -0., -0., -0.,  1., -0.,  0., -0., -0., -0.,  0.,  0.,
         0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -0.,  0.,  0.,  0.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0., -0., -1., -0., -0., -0.,
        -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0., -0.,  0.,  0., -0.,  1., -0., -0.],
       [ 0., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -1., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -0.,  0., -0., -1., -0., -0.,  0., -1.,
        -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  1., -0.,  0.,  0.,
        -0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  1.,  0.,
        -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.],
       [ 0., -0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
         0., -1.,  0., -1., -0., -1.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0., -0.,  1.,  0.,  0., -1.,  0., -0., -1.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -1.,
        -0., -0.,  1., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  1.,
        -0.,  0.,  0.,  0., -1., -0., -0., -0.,  0.],
       [-0., -0., -0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
         1.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0., -0., -0., -0., -0.,  1.,
        -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        -0., -1., -0., -0., -0.,  1., -0.,  0.,  0.,  0.,  0.,  0., -0.,
         0.,  0., -0., -0., -0., -0., -1., -0., -0.],
       [ 1., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,
        -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  0.,  1.,  0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  1.,
         0.,  1., -0., -0.,  1.,  0.,  0., -0., -0.,  0., -0., -0., -0.,
        -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0.,  0., -0.,  0., -0.,  0., -1., -0., -0.],
       [-0., -0., -0., -0.,  0.,  0.,  0.,  1., -0.,  1., -0.,  0.,  0.,
         0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  1.,
        -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -1.,  0., -1., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,
         0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0.,  1., -0., -0., -0.,  0., -0., -0.,  0.],
       [ 0., -0., -0., -0.,  0., -0., -1.,  0.,  0.,  0., -0., -0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  1., -0., -1.,  0.,  0., -0.,  0.,
         0., -1.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,
         0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  1.,  0., -0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  0., -1., -0.],
       [ 0., -0.,  0., -0.,  0.,  0., -1.,  1., -0., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0.,  0., -0.,  1.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0.,  0., -0.,  0., -0.,  0.,  1.,  0., -0.,  0.,  0., -0.,
        -1.,  0., -1., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  1.,
         0., -0.,  0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0., -0.,  0.,  0., -0., -0.,  0.,  0.,  0.],
       [ 1., -0.,  0.,  0., -0., -0.,  1., -0., -0., -0., -0., -0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,
         1.,  1., -0.,  0.,  0.,  0., -1.,  0., -0., -0., -0.,  1.,  0.,
         0.,  0.,  0.,  0., -0.,  0.,  0.,  1.,  0., -0.,  0.,  0., -0.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
        -0.,  1., -0., -0.,  1., -1., -0., -0.,  0.],
       [ 0., -0., -0., -0., -1.,  0., -1., -0., -0., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0., -1., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,
         0.,  0., -1.,  0.,  1., -0., -0., -0.,  0., -0.,  0.,  0., -0.,
        -1.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  1.,
        -1., -0., -1.,  1., -0.,  0., -0.,  1., -0., -0., -0., -0.,  0.,
        -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.],
       [ 0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0.,
         0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  1.,  1., -0.,
         0.,  0.,  0.,  1., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0.,  1., -1., -1., -0., -0.,  0.,  0.],
       [ 0., -0.,  0.,  0., -0.,  0., -0., -1., -1.,  0.,  0.,  0.,  1.,
        -0., -0.,  1., -0., -0., -0., -0.,  0., -1.,  0., -1., -0., -1.,
         0.,  0., -0.,  1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
         0.,  0., -1., -0., -0., -0.,  0.,  0., -0.],
       [-0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
        -0., -0., -0., -0., -0., -0., -0.,  0., -1., -0.,  0.,  1.,  0.,
         0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,
        -0.,  0., -0., -1.,  0.,  0.,  0.,  0.,  0.,  1., -0., -0.,  0.,
        -0.,  0., -1.,  1., -0., -0.,  0.,  0., -0.]], dtype=np.float32), np.array([[-0., -0., -0., -0., -0.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  0.,
        -0.,  0.,  1., -0., -0.,  0.,  0.,  0.,  0., -1., -0.,  0., -1.,
         0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,
        -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
         0.,  0.,  0., -0., -1., -0., -0., -0.,  1., -0.,  0., -0., -0.,
        -0., -0., -0.,  0., -0., -0.,  0., -0., -0.],
       [-0., -0.,  0.,  0.,  1.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0.,  0., -0., -0.,  1.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,
        -0., -0., -0., -0., -1.,  0., -0., -1.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  1., -0., -0.,  0., -1.,  0., -0., -0., -0.,  0., -0.,
        -1.,  0., -0., -0., -1., -0., -0.,  1., -0.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  1.,  0., -0., -0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,  1.,  0.,  0., -0.,
         1.,  0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
        -0., -0., -0., -0.,  0., -0., -1., -0.,  0.,  0., -0., -0.,  0.,
         0., -0.,  0.,  0.,  0.,  0., -0., -0., -1.,  0.,  0.,  0., -0.,
        -0., -1.,  0., -0., -0., -1.,  0., -0., -0.],
       [ 0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  1., -1.,
         0., -0., -0., -0.,  0.,  0., -0.,  0., -1.,  0., -0., -1.,  0.,
         0., -0., -0., -1., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0., -1., -0., -0., -0., -0.,  0.,  0., -0.,
         0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  1., -0., -0., -0.,
         0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.],
       [-0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  1.,  1.,  0.,
        -0.,  0.,  0., -0., -0., -0.,  1., -0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  1.,  0., -1., -0., -0.,  0., -0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  1.,
        -1.,  0.,  1.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  1.,
         0.,  0.,  0., -0., -0., -0.,  0., -0., -0.],
       [-0.,  0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  1., -0., -0.,
        -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -1., -0., -0.,
         0.,  0.,  0., -0., -0.,  1.,  0., -0., -0.,  0.,  0., -0., -0.,
         0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,
         0.,  0., -0.,  0., -0.,  0., -0.,  1.,  0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -0.,  1.,  0.,  1., -0.,  0., -0.,
         0.,  0., -1., -0.,  0.,  0.,  0.,  0., -0.,  1., -0., -0.,  1.,
         0.,  0., -0., -0., -0., -0.,  0., -0., -0., -0., -0., -0., -0.,
        -0., -0.,  1., -1., -0., -0., -0., -0., -1.,  0., -0.,  0., -0.,
         0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
         0., -0., -0.,  0.,  0., -0., -0., -0., -0.],
       [-0., -0.,  0., -0., -1.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0., -0., -0., -1.,  0., -0., -0.,  0., -0.,
        -0., -0., -0.,  0.,  1.,  0., -0.,  1., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0.,  0.,  1.,  0., -1.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -0.,  0., -1.,  0., -0.,  0., -0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0., -0., -0.,
         0.,  0., -0., -0., -0.,  0.,  1., -1., -0., -1., -0.,  0.,  0.,
         0.,  0.,  0., -0., -0., -0.,  1., -0.,  0., -0., -0., -0., -0.,
        -0., -0.,  0., -0.,  0., -0.,  1., -1.,  0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0.,  0., -0.,  0., -1., -0.,  0.,  0., -0., -0.,
        -0., -0.,  0.,  0., -0., -0., -0., -1., -0.],
       [-1., -1.,  0.,  0., -0., -1.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -1.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,
        -1.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,
         0.,  0., -0.,  0.,  1.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0.,  0.,  0., -0.,  1.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0., -0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0.,
         0.,  0., -0., -1.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0., -0., -0.,  0., -0., -0., -1., -0.,  1.,  1.,  0.,  1., -0.,
         0., -0.,  0., -0., -0., -0., -0.,  0., -1., -1.,  0.,  0., -0.,
        -0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -1., -0.,
        -0.,  0., -0.,  0., -0., -0., -0., -0.,  0.],
       [-0.,  1.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  1.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0.,  0., -0., -1.,  0.,  0., -0.,  0., -0.,
         0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  0., -0.,  0.,  1.,  0.,  0., -0.],
       [ 0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,
         0.,  0., -0.,  0.,  1.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,
         0., -0., -1., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -1.,
        -0., -1.,  0., -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,
        -0., -1.,  0.,  0., -0., -1., -0.,  0., -1.,  0., -0., -1., -0.,
         0.,  0.,  0., -0., -0.,  0., -0., -0., -0.],
       [-0., -0., -0., -0.,  1.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0.,  0., -0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         1.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  1., -1., -0.,
         0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.],
       [ 0.,  1., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
         0.,  1., -0., -0.,  0.,  1.,  0., -0., -0.,  1., -0., -0., -0.,
         0.,  0., -0., -0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,
         0., -0., -0., -0.,  0., -1.,  0., -0., -0.,  1., -0., -0., -0.,
        -0., -0.,  1.,  0., -0., -0.,  0., -0., -0.,  0.,  1.,  0.,  0.,
        -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0., -0., -0.,  0., -0., -1., -0.,
         0., -0., -0., -1.,  0.,  0., -0., -0., -0.,  0., -1.,  1.,  0.,
         0.,  0., -0.,  1., -0., -1., -0.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
         0., -0., -1.,  1., -0.,  0., -0.,  0., -0.],
       [-0., -0., -1.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -1.,  0.,
        -0.,  0.,  0., -1.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,
        -0., -0., -0., -0.,  0., -1.,  0., -0., -0., -0., -0.,  0., -0.,
         0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -1., -1.,  0.,
        -0., -0.,  0., -0., -0., -0., -1., -0.,  0., -0., -0.,  0., -1.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.],
       [ 0.,  0., -1., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,
         0., -1., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  1., -1.,  0.,
        -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0.,
         0., -0.,  0.,  0., -0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0.,
        -0., -0., -0.,  0., -1.,  1.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  0., -1., -0., -0.,  1.,  0., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,
         0.,  0.,  0.,  1., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,
         0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0., -0., -0.,  0.,  0.],
       [ 0., -0., -1., -0., -0., -1.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  0., -0., -0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  1., -0., -0., -0., -0.,  1.,  0.,  0.,
        -0., -0., -0., -0.,  0., -0., -0.,  0., -1.],
       [-0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0., -0., -1.,  0.,  1.,
        -1.,  0.,  1.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  1., -0.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  0., -1., -0., -0., -0.,  0.,  0.,  0.,
        -0.,  1., -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,
         1.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.],
       [-0.,  0.,  0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  1.,  0.,
         0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0., -1., -0.,  0.,  1., -0., -0., -0.,  0., -0.,  0.,  0., -0.,
         0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,
         0., -0.,  0., -0.,  0., -0., -0., -0.,  0.],
       [ 0.,  0.,  0.,  0.,  1., -0., -0., -0., -0., -0.,  0.,  0., -0.,
         0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  1., -0., -0.,  0.,  0.,  0., -1., -1.,  0.,
        -0., -0.,  0.,  1., -0.,  0., -1., -0., -0.,  0.,  0.,  0.,  0.,
        -0., -0., -0., -1., -0.,  0., -0.,  0., -0.],
       [-1., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0., -1., -0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,
         0.,  1., -0., -0., -0.,  0., -0., -1., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -0., -0., -0., -1.,  0., -0., -1., -0., -0., -0.,
         0., -0.,  0.,  0.,  0., -0., -0., -1.,  0.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 3
m = 4
p = 8
rank = 74

verify_tensor_decomposition(decomposition_348, n, m, p, rank)

##Rank-68 decomposition of <3,5,6> over Z


In [ ]:
#@title Data

decomposition_356 = (np.array([[-0.,  0., -0., -0.,  1.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0.,  1.,  1.,  0.,  0.,  0., -1.,  0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0., -1., -0., -0., -1., -0., -0., -0.,
        -0.,  0.,  1.,  0., -1., -1., -0., -1.,  0.,  1.,  1., -0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -1., -1.,  0.,  0., -0., -0.,  0.,
         0.,  0.,  0.],
       [-0.,  1.,  0.,  0.,  1.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,
        -0.,  0., -1., -0.,  0.,  1., -0., -0.,  0., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0.,
         0., -0.,  1.,  0., -0., -1.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,
         0., -0., -0.,  1., -0.,  0., -1., -0., -0., -0., -0.,  1.,  1.,
        -0., -0., -0.],
       [-0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0., -1., -0.,  0., -0.,  0., -1., -0.,  0.,  0.,  1.,  0.,  1.,
        -0., -1., -0.,  0.,  1.,  0.,  1.,  0.,  0.,  1., -0., -0., -0.,
         0., -0.,  0.,  0., -0., -0.,  0.,  1.,  0., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0.,  1., -0.,  0.,  1., -1.,  0.,  0., -0.,  1.,
         0.,  1., -0.],
       [-0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  1., -0.,  1.,  1.,
         0., -0., -0.,  0., -0.,  0., -0.,  0., -1., -1.,  0.,  0., -0.,
         0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -1.,
        -0., -0., -0.,  0.,  1., -0., -0.,  0.,  1.,  0., -0., -0.,  0.,
        -1.,  1., -1., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  1.,
        -0.,  0., -0.],
       [ 0., -0.,  0.,  0., -1.,  0., -0.,  0.,  0.,  1.,  0.,  1.,  0.,
        -0., -0.,  1.,  0.,  0., -1.,  0., -0., -0.,  0.,  1.,  0.,  1.,
         0., -1., -0., -0.,  1.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,
        -0.,  0., -1.,  0., -0.,  1.,  0.,  0., -0., -0., -1.,  0.,  0.,
        -0., -0.,  0., -0.,  1.,  0.,  1.,  0., -1., -0., -0.,  0., -0.,
        -0.,  1., -0.],
       [-1.,  0.,  0.,  0., -0., -0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., -0.,  0., -0., -0., -0., -0.,  0., -0., -0., -0., -1.,
        -0.,  0.,  0., -0.,  0.,  0.,  1., -0.,  1., -0.,  1.,  0., -0.,
         0., -0., -1.,  0.,  1.,  0.,  0.,  0., -0.,  0., -1.,  1.,  1.,
        -0., -0., -0.,  0., -0., -1., -0.,  1., -0., -0.,  0., -0., -0.,
         0., -0.,  0.],
       [-0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -1., -0.,  0.,
         0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0., -0.,  0., -0., -1., -0.,  0., -0.,  0., -1., -0.,  0., -0.,
        -1.,  0., -1.,  1., -0.,  1., -1., -0., -0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0., -1., -0.,  0., -0., -0.,  0.,  0., -1.,
         0., -0.,  1.],
       [ 0., -1., -0.,  0., -0., -0.,  1., -0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  1.,  0.,  0., -1., -0., -1., -0., -0., -1., -0.,  0., -0.,
        -1.,  0.,  0.,  1., -0.,  1., -0., -0.,  0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  1., -0.,  0., -1.,
         0.,  0.,  1.],
       [ 0.,  0., -0.,  1., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
        -1.,  0.,  0.,  1., -0., -0., -0., -1., -0., -0., -0.,  1.,  0.,
        -0., -1.,  0., -1., -0.,  0.,  1., -1., -0.,  0., -0.,  0., -0.,
        -0.,  0., -1.,  0.,  0., -0., -1., -0., -0.,  0., -0.,  0., -0.,
        -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -1.,  0.,  0.,
         0., -0., -0.],
       [ 0., -0., -1.,  0., -0., -0.,  1.,  0., -0., -1.,  1., -1., -0.,
        -0.,  0.,  0., -0.,  0.,  1., -1., -0., -0., -0., -0.,  0.,  0.,
         0.,  1., -1.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -1.,  0.,
        -0., -1., -0.],
       [-0.,  0., -0., -0.,  0., -0., -1., -0., -1., -0.,  0.,  0., -1.,
         1.,  0., -0., -0.,  1.,  0., -0., -0., -0.,  1.,  0.,  0., -1.,
        -0.,  0., -0.,  0., -0., -0., -0., -1.,  0.,  0.,  1., -0., -0.,
         0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0., -0., -1., -0.,  1.,
         0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -1.,
        -1., -0., -1.],
       [ 1., -0.,  0.,  0.,  0.,  0., -0., -0., -1.,  0., -1.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0., -0., -0.,  0.,  1.,  0., -1.,  0.,
         0.,  0.,  0.,  0., -1., -0., -0., -1.,  0.,  0., -0.,  0.,  0.,
         0., -1., -1.,  1.,  0.,  1., -0.,  0., -1., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -1.,
        -0., -0.,  0.],
       [ 0., -0.,  0.,  0., -0.,  0.,  1., -1., -0., -0., -0.,  0.,  1.,
        -1.,  0.,  0.,  0., -0., -0., -0.,  0., -1.,  0., -0.,  0.,  0.,
        -1.,  1.,  0.,  1., -1.,  0., -0., -0., -0.,  0., -1.,  0., -0.,
         0.,  0., -0.,  0.,  1.,  1.,  0., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,
         1.,  0.,  1.],
       [-0., -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -1.,
         0.,  0., -0., -0.,  1.,  0.,  0., -1.,  1.,  1.,  0., -0., -0.,
         1., -1.,  0., -1.,  0.,  0.,  0., -1., -0.,  0., -0., -0.,  0.,
        -0.,  0., -1., -0., -1.,  0.,  0., -0., -1., -0.,  0.,  0., -0.,
        -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -1.,  0., -1.,
         0.,  0., -0.],
       [-0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,
         0., -0., -1.,  0.,  0.,  0.,  0., -1., -0., -0., -1., -0., -1.,
        -0.,  0., -0., -0., -1., -1.,  0., -0., -0.,  0., -0., -1., -1.,
        -0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -0.,  0.,  1.,  0.,  0.,
        -0., -0., -1., -0., -1., -0., -1.,  0.,  0.,  0., -1.,  0., -0.,
         0., -0., -0.]], dtype=np.float32), np.array([[-0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -1.,
        -0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,
         0.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,
        -0., -0.,  0., -0., -1.,  0., -0., -1.,  0.,  0.,  0.,  0.,  0.,
         0., -0.,  0., -0.,  0.,  0., -0.,  1.,  0., -0., -0., -0., -0.,
        -0., -0., -0.],
       [ 1., -0., -0., -0.,  0., -0., -0.,  0., -1.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,
        -0., -0.,  0.,  0., -0., -0., -0., -0.,  1., -0., -0.,  0., -0.,
        -0.,  1., -0.,  0.,  0., -0., -0.,  0., -0., -1., -0.,  1., -0.,
        -0., -0., -0.,  1.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,
        -0., -0.,  0.],
       [-0., -0., -1.,  1., -0.,  0., -0.,  0., -0.,  0.,  1.,  1.,  0.,
         0.,  0., -0., -0., -0., -1.,  0., -0.,  0., -0., -0., -1.,  0.,
        -0., -0., -0.,  0., -0., -1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -1.,  0.,  0., -0., -1., -0., -0., -0., -0., -0., -0.,
         0.,  1., -1.,  0.,  0., -1., -0., -0.,  0.,  0.,  1.,  1.,  0.,
        -0., -0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -0., -1., -0.,  0., -0., -0.,  0.,
         0., -1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0.,  0.,  0.,  0., -1.,  0.,  1.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0.,
        -0., -0., -0., -0.,  0., -0.,  0., -1.,  0.,  0.,  0., -0., -0.,
         1.,  0.,  0.],
       [-0., -0.,  0., -0.,  0., -0., -0., -1., -0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  1., -0., -1.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  1.,  0., -0.,
         0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  1.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
         1.,  0., -0.],
       [ 1., -0.,  0.,  0.,  0., -0., -0., -0., -1., -0.,  0.,  0., -0.,
        -0., -0.,  1., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
         0.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  1.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -1.,  0., -1.,
         0., -0., -0., -0.,  0.,  0., -1.,  0., -0., -0.,  0., -0., -0.,
        -0., -0., -0.],
       [-0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  2.,
         1., -1., -0.,  1.,  0.,  0., -0., -0., -1.,  1., -0., -0., -0.,
         0., -0.,  0., -0., -0.,  0.,  1., -0., -0., -1.,  0.,  0., -0.,
         0.,  0.,  0.,  0.,  1., -0.,  0.,  1.,  0.,  1., -0.,  0., -0.,
        -0., -0., -0.,  0., -0.,  0., -0., -1., -0.,  0., -0., -0.,  1.,
        -1.,  0., -0.],
       [-1., -0., -0., -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,
         0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,
        -0., -0., -0.,  0., -0., -0.,  0., -0., -1., -0.,  0., -0., -0.,
        -0., -1.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -1.,  0.,
        -0., -0.,  0., -1., -0., -1., -0.,  0., -0.,  0., -0., -0., -0.,
        -0.,  0., -0.],
       [-0.,  0.,  1., -1.,  0., -0., -0.,  0., -0.,  0., -1.,  0.,  0.,
         0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  1., -0.,
        -0.,  0., -0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0.,  0., -0., -0.,  1.,  0., -0.,  0., -0.,  0., -0.,
        -0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0., -1., -0.,
        -0.,  0., -0.],
       [-0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
         0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
         0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0., -0., -0.,
         1., -1., -0., -1., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,
        -0.,  0.,  0., -1., -0.,  0., -0.,  0., -0.,  1.,  0., -0.,  0.,
        -0.,  0., -0.],
       [ 0., -0., -0.,  0., -0.,  0.,  1.,  1., -1.,  0., -0.,  0., -0.,
        -0., -0., -0., -0., -0.,  0., -1., -0.,  0.,  0., -2.,  0.,  1.,
         0.,  0.,  1., -0.,  0.,  0., -0.,  0., -0.,  0., -1.,  0., -0.,
        -0., -0., -0.,  0., -0., -1., -0.,  1., -0.,  0., -0., -0., -1.,
         0., -0., -0.,  0., -1., -0., -1., -0., -0., -0., -0.,  0.,  0.,
        -1.,  0.,  1.],
       [-0., -0.,  1.,  0.,  0.,  1.,  0.,  0., -0.,  0., -1., -0.,  0.,
        -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0., -0.,
        -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -0., -1.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -1., -0.,
        -0.,  0., -0.],
       [ 0.,  0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0., -1.,
        -1.,  1.,  0., -1., -0.,  0., -0., -0.,  1., -0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -1., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -1.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  1.,  0.,  0., -0., -0.,  0.,
         1., -0., -0.],
       [ 1., -0.,  0.,  0.,  0., -0., -0.,  0., -1., -0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  1.,  1.,  0.,  0., -0.,
        -1.,  1., -0., -0., -0., -0.,  0., -1.,  0., -1., -0.,  1.,  0.,
        -0., -0., -0.,  1., -0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.,
        -1.,  0.,  1.],
       [ 0.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -1.,  0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,
         0.,  1.,  0.,  1.,  0.,  1., -0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,
        -0., -1.,  1., -0., -0.,  0.,  0., -0., -0., -1., -1., -0., -0.,
        -0.,  1., -0.],
       [-0.,  0., -0., -0., -0., -0., -0., -1.,  0.,  0., -0., -0., -0.,
        -0., -1.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,
        -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0.,  0., -0.,
        -0., -0.,  0.],
       [-0.,  0.,  0.,  0., -0., -0., -1., -1., -0., -0.,  0., -0.,  0.,
         0., -0., -0., -0.,  0., -0.,  1.,  0., -0., -0.,  1., -0., -1.,
        -0.,  0., -1.,  0., -0.,  0., -0.,  0.,  0., -0.,  1., -0., -0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -1.,  0., -0., -0., -0.,  1.,
         0.,  0.,  0.,  0.,  1.,  0., -0., -0., -0., -0.,  0.,  0., -0.,
         0., -0.,  0.],
       [ 0.,  0., -1., -0.,  0., -1., -0., -1.,  0., -0.,  1.,  0.,  0.,
        -0., -0., -1., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0.,  0., -0., -0., -1., -0., -0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0., -1., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,
         0.,  0.,  0., -0.,  1., -0., -0., -0., -0.,  0., -0.,  1., -0.,
        -0., -0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -1.,
        -1.,  0.,  0., -1.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,
        -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
         1., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,
         1., -0.,  0.],
       [ 0.,  0., -0.,  1.,  0., -0.,  0.,  0.,  1.,  0., -0., -0., -0.,
        -0., -0.,  0., -0., -1., -0., -0.,  0.,  0., -1.,  0.,  0.,  0.,
         0.,  0., -0., -0.,  0.,  0.,  0., -1.,  0.,  0.,  0., -0.,  0.,
         0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        -1., -0., -0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  0.],
       [-0.,  0., -0.,  1., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,
         0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,
         0., -1.,  1.,  0., -0., -0., -0., -0.,  0.,  0., -1., -0., -0.,
         0.,  0.,  0.],
       [ 0.,  0., -0.,  1., -0.,  0., -0., -1., -0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0., -0.,
        -1.,  0., -0., -1.,  0., -0., -0.,  0., -0.,  0., -0., -0., -0.,
         0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,
        -1.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0., -0.,
         0., -0., -0.],
       [-0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.,  1.,
         1.,  0., -0., -0.,  0., -0.,  0.,  1.,  0.,  0., -0., -0., -0.,
        -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  1.,  1.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,
        -1.,  0., -0.],
       [-0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  1., -0.,
        -0.,  0.,  0.,  0., -0.,  1.,  0., -0., -0., -0., -0., -0., -0.,
         0.,  1., -0., -0.,  0., -0., -0.,  0., -1., -0.,  0.,  0.,  0.,
         0.,  0.,  1., -0.,  0.,  0., -0.,  0.,  0., -0., -1.,  0.,  0.,
         0., -0.,  0.],
       [ 0., -0., -0.,  0., -0., -0.,  0., -0., -0.,  1.,  0., -0., -0.,
        -0., -0., -0., -0.,  0.,  0.,  1.,  0., -0., -0.,  1.,  0.,  0.,
        -0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0., -1.,
        -0.,  0., -0., -0., -0.,  0.,  0., -1., -0., -0.,  0.,  0., -0.,
        -1.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0.,
         0., -0.,  0.],
       [ 0.,  0.,  1.,  0., -1., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0.,  0.,  0., -0.,  1., -0.,  0., -0.,  0.,  0., -0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -1.,  0.,
         0., -0.,  0., -0., -0.,  0., -0., -0., -0., -1.,  0.,  0., -0.,
         0., -0., -0., -0.,  0., -0.,  1., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.],
       [-0.,  0.,  1., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  1., -0.,
        -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,
         0., -0.,  0.,  0.,  0., -1., -0., -0.,  0.,  0.,  0.,  0.,  0.,
         0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0.,  1., -1.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0.],
       [-0.,  0.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
        -0.,  1.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -0., -0.,  0., -1.,  0.,  0., -0.,  0., -0., -0.,  0.,
         0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0.,  1.,  0., -0.,  0., -1., -0.,  0.,  0., -0.,
        -0.,  1.,  0.],
       [ 0.,  0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,
         0.,  0., -0., -0.,  0., -0., -1., -0.,  0., -0., -1.,  0., -0.,
         0.,  0.,  1., -0.,  0.,  0., -0., -0.,  0., -0., -0., -1.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  1., -0., -0., -0., -0., -0.,
         0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
         0., -0.,  0.],
       [-0., -0.,  1.,  0.,  0.,  1.,  0.,  0., -0., -0., -0.,  0., -0.,
         0., -0.,  1., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,
         0., -0., -0., -0., -0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
        -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -1., -0.,
        -0., -0., -0.]], dtype=np.float32), np.array([[ 0.,  1., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0.,
        -0., -0.,  0.,  0., -1., -0., -1., -0., -1.,  1., -0.,  0.,  0.,
        -1.,  0., -1.,  0., -0.,  0., -0., -0., -1.,  1., -0., -0.,  0.,
         1., -0., -0., -0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
        -1., -0.,  0., -1., -0.,  0., -0.,  1., -0.,  0.,  0., -0.,  1.,
         0., -0.,  0.],
       [ 0.,  1.,  0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0.,
        -0., -0., -0., -1., -0.,  0., -1.,  0., -0., -0.,  0., -0., -0.,
        -0., -0., -1., -0.,  0., -0.,  1.,  0., -1.,  1.,  0.,  0.,  0.,
         1., -0.,  0.,  0.,  0., -0.,  0.,  1.,  0., -0., -0.,  1.,  0.,
         0., -0., -0., -1.,  0.,  0., -0.,  1., -0., -0., -0.,  0., -0.,
         0., -0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  1.,
         1.,  0., -0.,  1., -1.,  0., -0., -1., -1.,  1.,  0., -0., -0.,
        -1., -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  1.,
         0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
        -1.,  0., -0., -0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0., -0.],
       [-0.,  1.,  0., -0.,  1.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,
        -0., -0., -0., -0., -1., -0., -0.,  0.,  0.,  1.,  0.,  0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  1.,  0., -0.,  0.,
         1.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0., -1., -0.,  0., -0.,
         0.,  0., -0., -1., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  1.,
        -0.,  0., -0.],
       [ 0.,  1., -0.,  0.,  1., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  0.,  1.,  1.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0., -0.,  0.,  1., -0., -0., -0., -0.,  0.,
         1.,  0., -1.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  1., -0.,
         0., -0., -0., -1.,  0.,  1.,  0.,  0., -0., -0.,  0., -0.,  0.,
         0., -0.,  0.],
       [-1., -1., -0., -0.,  1.,  0., -0., -0.,  1.,  0., -0.,  0., -0.,
         0., -0., -0.,  0., -1.,  0., -0., -0., -0.,  0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -1.,  0., -0., -0., -0.,  1., -0.,  0.,  0.,  0., -1., -1., -0.,
        -0., -0., -0.,  1., -0.,  0., -1.,  0.,  0.,  0.,  0., -0., -0.,
         0.,  0.,  1.],
       [ 0., -0., -1., -0., -1., -1.,  0., -0.,  0., -0.,  0.,  1.,  0.,
        -0., -0.,  0.,  0., -0., -1.,  0.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
         0., -1.,  0.,  0., -0.,  0., -0.,  0., -1.,  0., -0., -1., -0.,
         0.,  1.,  0.],
       [ 0., -0., -1.,  1., -1., -1.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -0., -0., -0., -1., -1.,  0.,  0., -0., -0., -0., -0.,  0.,
        -1.,  1.,  0., -1., -0.,  0.,  0., -1.,  0., -0.,  0., -0., -0.,
        -0., -0.,  1., -0., -0., -0., -1.,  0., -0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0., -0., -0., -0., -0., -1.,  0., -0., -1.,  0.,
        -0.,  1.,  0.],
       [ 0., -0., -0., -1.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0.,  1., -0.,  0., -0., -0.,  0.,  0., -1., -0.,
         1.,  0.,  0.,  1., -0.,  0.,  0.,  1., -0.,  0.,  0.,  0.,  0.,
         0., -0., -0.,  0., -0.,  0., -0., -0., -1.,  0.,  0., -0.,  0.,
        -0., -1.,  1., -0.,  0.,  0., -0., -0.,  0., -0.,  1., -0., -0.,
        -0.,  0.,  0.],
       [ 0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  1., -0., -0., -0., -0.,  0., -0., -1.,  0., -0., -0.,  0.,
        -1.,  0., -0.,  0.,  0., -0.,  0., -0., -1.,  0., -0.,  0.,  0.,
         0.,  0., -0., -0.,  1.,  0., -0., -0., -0.,  0., -0.,  1.,  0.,
        -0.,  0., -0., -1., -0.,  0.,  0.,  1.,  1.,  0.,  0.,  0., -0.,
         0.,  0.,  0.],
       [ 0.,  1., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
         0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,
         1., -1.,  0.,  1.,  0., -0., -0., -0., -1., -0., -0., -0.,  0.,
         0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  1., -0.,
        -0., -0., -0., -1.,  0., -0.,  0.,  0.,  1., -1., -0., -0.,  0.,
        -0., -1.,  0.],
       [-0., -1.,  0.,  0., -0., -0., -0.,  1., -0., -0., -0., -0., -0.,
         0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -1.,
        -1., -0., -0., -0., -1., -0.,  0., -0.,  1., -0.,  1.,  0.,  0.,
        -0., -0.,  0., -1.,  0.,  0.,  0.,  0., -0., -0.,  0., -1.,  0.,
         0.,  0., -0.,  1., -1.,  0., -0.,  0.,  1.,  0., -0.,  0., -0.,
         0., -0., -0.],
       [-0.,  0.,  0.,  0., -1.,  0., -0.,  0., -0.,  1.,  0., -0.,  0.,
        -0., -0.,  0., -0.,  0.,  0., -1., -0.,  0., -0., -1., -0., -1.,
         0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  1., -1.,
         0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  1., -0.,  1.,
        -0., -0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0., -0., -0.,  0.,
         0.,  0.,  0.],
       [ 0., -1., -0.,  0., -0.,  0., -1., -0.,  0., -0., -0., -0., -0.,
        -1.,  0.,  0., -1.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,
         0., -0., -1., -0.,  0., -0., -0.,  0.,  1.,  0.,  1.,  0., -0.,
        -1., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -1.,  0.,
        -0., -0., -0.,  1., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,
         1.,  0.,  1.],
       [ 0.,  1.,  0.,  0., -1.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
         1., -0., -0.,  1.,  0.,  0., -0., -1.,  0., -0.,  0., -0., -0.,
         0., -0., -0., -0.,  0.,  0.,  0., -0., -1., -0., -1.,  1., -0.,
         1., -0., -0., -0.,  0., -1.,  0., -0., -0.,  0.,  1.,  1.,  1.,
        -0., -0., -0., -1.,  0.,  0.,  1., -0., -0., -0., -0., -0., -0.,
        -1., -0., -1.],
       [ 0., -0.,  0., -0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -0., -0.,
         0., -0.,  1.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  1.,
        -0.,  0., -0., -0., -0.,  1., -0.,  0., -0.,  0.,  0., -0., -0.,
        -0., -0., -0.,  0., -0., -0.,  0., -0., -0.,  0., -1.,  0., -1.,
        -0.,  1., -1.,  0.,  1.,  0.,  0.,  0., -1.,  0., -0.,  0.,  0.,
        -0.,  0.,  0.],
       [-1., -1.,  0., -0., -0.,  1., -0.,  0.,  0., -0.,  1.,  0.,  0.,
        -0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  1., -0.,
        -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,
         0., -1., -0., -1.,  0., -0.,  1.,  0.,  0.,  0.,  0., -1., -0.,
        -0., -0., -0.,  1.,  0., -0., -0., -0., -0., -0., -0.,  1.,  0.,
         0.,  0., -0.],
       [ 1.,  1.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,
        -0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  1.,
        -0.,  0., -0., -0.,  1.,  1., -0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  1.,  0.,  1.,  0., -0.,  0.,  0.,  1.,  0.,  0.,  1., -1.,
         0.,  1., -1., -1.,  1., -0.,  0.,  0., -1.,  0.,  0., -0., -0.,
        -0.,  0.,  0.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 3
m = 5
p = 6
rank = 68

verify_tensor_decomposition(decomposition_356, n, m, p, rank)

## Rank-80 decomposition of <3,5,7> over Z

In [ ]:
#@title Data

decomposition_357 = (np.array([[ 0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -1.,
         0., -1., -0.,  0., -0., -0.,  1.,  0., -0., -0., -0., -0., -0.,
         0., -0.,  0.,  0.,  0., -1., -0.,  0.,  0.,  0.,  0.,  1., -0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  1., -0.,
        -0.,  0., -1.,  0.,  0., -0.,  0.,  0., -0.,  1., -0.,  0.,  1.,
         0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -1.,  1., -0.,
         0.,  1.],
       [-1., -1., -0.,  0., -0.,  0., -0.,  0., -1., -0., -0., -1.,  1.,
         1., -0., -0.,  0., -1., -0., -0., -0., -0.,  0.,  1.,  0.,  0.,
         0., -1.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,
         0., -0., -1.,  0., -0.,  0., -1.,  0., -0., -0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -0., -0., -1., -0., -0.,  0., -0.,  0., -0.,
         0.,  0., -1.,  0., -0.,  1.,  1., -0., -0., -1., -0., -0., -0.,
        -0., -1.],
       [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  1.,
         0.,  0.,  1., -0.,  0.,  0.,  1., -0., -0., -1., -0.,  0.,  0.,
         0., -0.,  0., -0.,  1., -0.,  0.,  0.,  1., -0., -0.,  0.,  1.,
         0., -0.,  0., -0., -1.,  0.,  0., -0.,  1., -0.,  0., -0., -1.,
        -0., -0., -0., -0., -0.,  0.,  0., -0., -1., -0., -0.,  0.,  0.,
        -0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,
        -1., -0.],
       [-1., -0., -0., -0.,  0.,  0., -0., -0., -1.,  0., -0., -0.,  0.,
         1., -1., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0., -0.,
         0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0., -0., -0.,  1., -0.,
         0., -0., -1.,  1.,  1.,  0.,  0.,  0.,  0., -0., -0.,  1., -0.,
         0., -0., -1., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  1.,
        -0.,  1., -0., -0., -0.,  0.,  0.,  0.,  0., -1., -1.,  1., -0.,
         0., -0.],
       [ 1., -0., -1., -0., -0., -0.,  0., -0.,  1.,  0.,  0., -0.,  0.,
        -1.,  1., -1.,  0., -1., -0., -0., -0., -0., -0., -0.,  0.,  0.,
         0., -1., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,
        -1.,  0.,  1., -0., -0., -0., -1.,  1.,  0., -0.,  0.,  0.,  1.,
         0., -0.,  0.,  0., -0., -0.,  0.,  0.,  1., -0.,  0., -0., -1.,
         0., -1.,  0.,  0., -0.,  1., -0.,  0., -0.,  1., -0., -1.,  0.,
        -0.,  0.],
       [-0.,  0., -0., -0.,  0.,  1., -0., -0.,  0.,  0., -0., -0., -0.,
         0.,  1.,  0., -0., -0.,  1., -0.,  0.,  0.,  0.,  0., -1., -0.,
         0., -0., -1., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
         0.,  0., -0.,  0.,  0.,  1.,  0., -0.,  0.,  0., -1.,  0.,  0.,
         0., -0., -0., -1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,
        -0., -0.],
       [ 0.,  0., -0.,  0., -0., -1., -0., -1., -0., -0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0., -0., -1., -0., -0.,  0., -0.,
        -1.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0.,  1.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0.,
         1., -0.,  0., -1.,  1., -1.,  0., -0., -0., -0.,  0., -0., -1.,
         0.,  0.],
       [-0., -0., -0., -0., -1.,  0., -1., -1.,  0.,  0., -0.,  0., -0.,
         0., -0., -1., -0., -0.,  1.,  0., -0., -1.,  0.,  0., -0., -1.,
         0.,  0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0., -1., -0., -0.,
         0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0., -0.,  0.,  1.,  0., -0., -0.,  0.,  0., -0.,  1.,  1.,  0.,
         1.,  0.,  0., -1., -0.,  0.,  0., -0., -1.,  0.,  0., -0., -1.,
         0., -0.],
       [ 0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -1.,  0., -0.,
        -1.,  1., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -1., -0.,
        -1.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,  1.,  0., -1., -0.,
         0., -0.,  0., -0., -0.,  1., -1.,  0., -0.,  0., -0.,  0.,  1.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -1., -0.,
        -0., -1., -0., -0.,  1.,  0.,  0., -0., -0., -0., -0., -0.,  0.,
         0.,  0.],
       [-1., -0.,  1.,  0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0.,  0.,
         1., -1.,  1.,  0.,  0.,  0.,  0.,  1.,  0., -0.,  0.,  0., -0.,
         0.,  1.,  0., -0.,  0.,  0., -1.,  0.,  0.,  0., -0.,  1., -0.,
        -0., -0., -0.,  0.,  0., -1.,  1., -1.,  0., -0.,  0., -0., -1.,
         0., -0.,  0., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0.,
        -0.,  1.,  0., -0., -0., -1., -0., -0., -0., -0., -0.,  1.,  0.,
        -0.,  0.],
       [ 0., -0., -0., -1., -0.,  0.,  1.,  0., -0.,  0.,  0.,  0., -0.,
         0.,  0.,  0., -1., -0.,  1., -1., -0., -0., -0., -0., -0., -1.,
        -0.,  0., -0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -1.,
         0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -1.,  0.,
         0., -1.,  0., -1., -1., -0., -0.,  1.,  0., -0.,  0.,  0.,  0.,
         0.,  0., -1., -1.,  0., -0., -0.,  1.,  0., -0., -0., -0.,  1.,
        -0., -1.],
       [-0.,  0.,  0.,  1., -0.,  0., -1., -0., -0.,  0., -0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,
         0., -0., -0.,  0., -0., -0.,  0.,  1.,  0., -0., -1.,  0.,  0.,
        -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  1.,  0.,  0.,  0.,
         0.,  1., -0.,  1.,  1., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,
        -0.,  0.,  1., -0.,  0.,  0., -1.,  0., -0.,  1.,  0.,  0., -1.,
        -0.,  1.],
       [ 0., -0.,  0.,  0.,  0.,  0., -1., -0.,  0.,  1.,  0., -0.,  0.,
        -0.,  0.,  0.,  0., -0.,  1., -1.,  0., -1.,  0., -0.,  0., -1.,
        -0., -0., -0., -1.,  0.,  0.,  1., -0., -1., -0., -1.,  0., -1.,
         0.,  1., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  1.,  0.,  1.,  0.,  0., -0.,  1., -0., -0.,  1.,  1.,  0.,
         1., -0., -0., -1., -0., -0., -0., -0., -1.,  0., -0., -0., -1.,
        -0., -0.],
       [ 0., -1.,  0.,  0., -0., -0.,  0., -0.,  0., -1.,  0., -0., -0.,
        -0., -0.,  0., -1.,  0., -0., -0., -0.,  1.,  0.,  0.,  0.,  0.,
        -0.,  0., -0.,  1., -1.,  0., -0.,  1.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -0., -1.,  0.,  0.,  0., -0.,  0.,  0.,  0., -1., -0.,
        -0., -0., -0., -0.,  0., -1.,  0.,  0., -0.,  0., -0., -1., -0.,
         0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  1.,  1.,  0.,  0.,
         0.,  0.],
       [-0.,  0., -0.,  0.,  0.,  0., -0.,  0., -1.,  0.,  0.,  0., -0.,
         0.,  0., -0.,  1., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0.,  0., -0., -0.,  0., -0., -1., -0.,  0., -0.,  0.,  0., -0.,
         1., -1.,  0., -0., -0.,  0., -0.,  0., -1.,  1., -0.,  0., -0.,
        -1., -0.,  1., -0.,  0., -0., -1., -0.,  0., -0., -1., -0., -0.,
        -1.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -1.,  0., -0.,  0.,
         0., -0.]], dtype=np.float32), np.array([[-0.,  0., -0., -0.,  0.,  0., -0.,  0., -1., -0.,  0., -0.,  0.,
         0., -0., -0., -1., -0.,  0.,  0.,  0.,  1., -0., -0., -0.,  0.,
        -1.,  0.,  0., -0., -0., -0.,  1.,  1., -0., -0.,  0., -0.,  0.,
        -1., -0., -0., -0.,  0.,  1., -0.,  0., -0.,  1.,  0., -1., -0.,
         0., -0., -1.,  0.,  0., -0., -1., -0., -0.,  0.,  0.,  1.,  0.,
        -1.,  0., -0., -0.,  0.,  0.,  1.,  0., -0.,  1.,  0., -0., -0.,
        -0., -0.],
       [-0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,
        -0.,  1., -0., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0., -1.,  0., -0.,  0., -0.,  0.,  0., -0.,  1., -1., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -1., -0., -0.,
         0., -0.,  0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,
        -0.,  1.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -1.,
        -0.,  0.],
       [-1.,  0.,  1.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
         1., -1.,  0.,  0., -0.,  1., -0.,  0.,  0.,  0., -0., -1., -1.,
         0., -1.,  1.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0.,  0., -0.,  1., -0.,  0.,  0., -0., -0., -1.,
        -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
         0., -1., -0.,  0.,  0., -0., -0.,  0., -1., -0., -0., -1.,  0.,
         1., -0.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,
        -0.,  1., -0.,  0., -0.,  1., -0.,  0.,  0.,  0., -0.,  0., -1.,
        -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -0.,
        -0., -0., -0.,  1.,  0., -0., -0., -0.,  0., -0., -0., -1.,  0.,
        -0., -0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  1.,  0., -0., -0.,  0., -0., -1., -1., -0.,  1., -0.,  0.,
        -0., -0.],
       [-0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,
         0., -1.,  0., -0.,  0.,  0., -1., -0.,  0.,  0.,  0., -0., -0.,
        -0., -0.,  1.,  0., -0., -0., -0., -0., -1., -0., -0.,  1.,  1.,
        -0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  1.,  0.,  0.,  0.,
         0., -1., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0., -0., -0.,
        -0., -0.],
       [ 0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,
         0., -0.,  0., -0.,  0., -1., -0., -0.,  0., -0., -0.,  0.,  1.,
         0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,
         0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0., -0., -0., -0., -0.,  1.,  1., -0.,  0., -0.,  0.,
        -0., -0.],
       [ 1., -0., -1.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,
        -1.,  1., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  1.,  0.,
        -0.,  1., -1., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0., -0.,  0., -1.,  0., -0., -0.,  1., -0.,  1.,
         0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -0.,
         0.,  1., -1.,  0., -0.,  0.,  1.,  0., -0., -0., -0.,  1., -0.,
        -1., -1.],
       [-0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,  1.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  1., -0., -0., -0.,
         1., -0., -0., -0., -0., -0., -1., -0.,  0.,  0., -1.,  0.,  0.,
        -1.,  0., -0., -0., -0.,  0.,  1., -0.,  1.,  0., -0.,  0.,  0.,
         0.,  0.],
       [-0.,  0.,  1., -0., -0.,  1.,  0.,  0., -0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0., -0., -0.,  0.,
        -0., -0.,  0.,  0., -0., -0.,  0., -0., -0., -0.,  2., -0., -0.,
         0.,  0.,  0.,  0.,  0., -0.,  1.,  1., -0., -0., -1.,  0., -1.,
         0.,  0., -0.,  1., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,
        -0.,  0.,  0.,  1., -1., -1., -0.,  0.,  0., -0., -0., -0., -1.,
         1., -0.],
       [ 0.,  0., -0., -0., -0., -1., -1., -0.,  0., -0.,  0., -0., -0.,
         0., -0., -0., -0., -0.,  1.,  0., -0., -0., -0., -0., -0., -1.,
         0., -1.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0., -0., -0., -1.,  0., -0.,  1., -0., -0.,
        -0., -0., -0., -1.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0., -0.,  0.,  0.,  1., -0.,  0., -2.,  0.,  0.,  0., -0.,
         0.,  0.],
       [ 0.,  1.,  1., -0., -0.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -0., -0., -0., -0., -0.,  1., -0., -0.,  0., -0.,
        -0.,  0.,  0.,  1.,  0.,  0.,  0.,  1., -0.,  0., -1., -0.,  0.,
        -0., -0., -0.,  0., -0., -0.,  1.,  1., -0., -0., -0., -0., -1.,
        -0.,  0.,  0.,  0.,  0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,
         0.,  0.,  0.,  0., -1., -1.,  1., -0., -1.,  0., -0.,  0.,  0.,
         1., -0.],
       [ 0., -0., -1., -1.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  1.,
        -0.,  0., -0.,  0.,  0., -0., -1.,  0., -0., -1., -0.,  0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  0.,  0., -1., -0., -0.,  0.,  1.,
         0., -0.,  0., -0., -0.,  0., -1., -1., -0., -0.,  0.,  0.,  1.,
         0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  1.,  0.,  0.,  0.,
         0., -0., -0.,  0.,  1.,  1.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -1.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -1.,  0.,
         0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0.,  1.,
        -0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0., -1., -0., -0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,
        -0., -0.,  0.,  0., -1., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
        -0., -0., -1., -1., -0., -0., -0.,  1.,  1., -0.,  0., -0.,  0.,
        -0.,  0.],
       [-0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0.,
         0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  1., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,
        -0., -0., -0.,  0., -0.,  0.,  0.,  1.,  0., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  0., -0., -1.,  1., -0.,  0., -0., -0., -0.,  0.,
         0.,  0.],
       [ 0.,  0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,
         0., -0.,  0., -0.,  0., -0., -0., -0., -1., -0., -0., -0.,  0.,
         0.,  1., -0.,  0., -0., -0.,  0.,  0.,  1., -0.,  0.,  0., -0.,
        -1.,  0.,  0., -0., -0.,  0., -0., -0., -0., -0.,  1.,  0.,  0.,
        -0., -0.,  0., -0.,  0., -0., -0., -0., -1.,  0., -0., -0.,  0.,
        -0., -0.],
       [ 0., -0., -1., -0., -0., -1.,  0.,  1.,  0., -0.,  0., -0.,  0.,
         0., -0., -1., -0.,  0.,  1., -0., -0.,  0.,  0., -0., -0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0., -1.,  0.,  0.,
         0., -0., -0.,  0., -0., -0., -0., -1., -0., -0.,  1., -0., -0.,
        -0., -0., -0., -1.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,
        -0., -0., -0., -1., -0., -0., -0., -0., -0., -0.,  0.,  0.,  1.,
        -1., -0.],
       [ 0.,  0.,  1.,  0., -1.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
        -0.,  0.,  1., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
         0., -0., -0., -0.,  0., -0., -0.,  1.,  0., -0.,  0., -0., -0.,
        -0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0., -0.,  0.,  0., -0., -0.,  0.,  1.,  0., -0.,  0., -0.,
         0.,  0.],
       [ 0., -0., -1., -0.,  0.,  0., -0., -0., -0., -1.,  0., -0.,  0.,
         0., -0., -1.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,
         0., -0.,  0., -1.,  1.,  0.,  0., -0., -1., -0.,  0., -0.,  0.,
        -0.,  0., -0., -0.,  0., -0., -0., -1., -0.,  0., -0.,  0.,  0.,
        -0.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -0.,  0., -0.,  1.,  0.,  0.,  0.,  0.,
        -1., -0.],
       [-0., -0.,  1., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0.,  1.,  0.,  0.,  0.,  0., -0.,  0.,  1., -0.,  0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  0.,  1.,  0.,  0., -0., -0.,
        -0.,  0., -0.,  0.,  0.,  0., -0.,  1.,  0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,
         0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,
        -0., -0.],
       [ 0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  1., -0.,  0., -1.,
        -0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  1.,
        -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0.,  0.,  0., -0., -0.,  0., -1., -0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -0.,  0., -1., -1., -0.,  0.,  0., -0.,
        -1.,  0.],
       [ 0., -0., -1.,  1.,  1.,  1.,  1., -1., -0., -0.,  0., -0., -0.,
        -0., -0., -1., -0.,  0., -1.,  0.,  0., -0., -1.,  0., -0.,  1.,
        -0.,  0., -0., -0., -0., -0.,  0.,  0., -1.,  0.,  1., -0., -1.,
        -0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0.,  0., -1., -0.,  0.,
         0., -1.,  0.,  1.,  1.,  0., -0.,  1., -0.,  0.,  0.,  0.,  0.,
         0., -0., -0.,  1., -0., -0.,  0.,  1.,  1., -0.,  0.,  0., -1.,
         1., -0.],
       [ 0., -0.,  0.,  0., -0., -0., -0., -0.,  1.,  0., -0., -0., -0.,
        -0., -0.,  0.,  0., -0., -0., -0.,  0., -1., -0.,  0., -0.,  0.,
         1.,  0.,  0., -0., -0.,  0., -0., -1., -0.,  0.,  0.,  0., -0.,
         1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -1., -0.,  0., -0.,
        -1.,  0.,  0., -0., -0., -0.,  1.,  0.,  0., -0.,  1., -1., -0.,
         1., -0., -0.,  0.,  0., -0., -1.,  0., -1., -1.,  0.,  0., -0.,
        -0., -0.],
       [-0.,  0., -1.,  0., -0., -1.,  0., -0., -0., -0., -0.,  0., -0.,
         0., -0., -0.,  0., -0.,  1., -0., -0., -1., -0.,  0., -0., -0.,
         1.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -1., -1., -0., -0.,
        -0., -0.,  0.,  0.,  0., -0.,  0., -1., -0., -0.,  1.,  0.,  1.,
        -0.,  0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0., -1., -0.,
        -0., -1.,  0., -1.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  1.,
        -1., -0.],
       [ 1.,  0., -1.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,
        -1., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  1.,  0., -1., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0.,  0.,  0., -0.,  0.,  0., -1.,  1., -0., -0.,  0.,  0.,  1.,
        -0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0.,
        -0.,  1., -0., -0.,  0.,  0., -0., -0.,  1., -0., -0., -0.,  0.,
        -1., -0.],
       [-0., -0., -1.,  0.,  0., -0., -0.,  0., -0., -0., -0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,
         0.,  0.,  0., -1.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0.,  0., -1., -0.,  0.,  0., -1.,  0.,  0., -0., -0.,  1.,
        -0., -0., -0., -0.,  0.,  1., -0., -0., -0., -0., -0.,  1., -0.,
         0., -1., -0.,  0., -0.,  0.,  0.,  0.,  1.,  0., -0.,  0.,  0.,
        -1., -0.],
       [-0., -0.,  1.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  1.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  1., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  1., -1., -0.,  0.,  1.,  0., -0.,  0.,  0., -1.,
        -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0.,  1., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0.,
         0., -0.],
       [-0.,  0.,  1.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0.,  1.,  0.,  1., -0.,  0., -0.,  0.,  0.,  1., -0.,
         0.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -1.,
        -0., -0., -0., -0., -0., -1., -0.,  0., -0., -1.,  0., -1.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0., -1., -0., -1.,  1.,  0.,
         1., -0.],
       [ 0.,  1., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -1., -0.,
         0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -1., -0., -0.,
        -0., -1.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0., -1., -0., -0.,  1., -1., -0., -0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,
         0., -1., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
         0.,  0.],
       [-0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,
         0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  1., -0., -0., -0., -0., -0., -0.,
         1., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,
        -1.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0., -0.,
        -0.,  0.,  0., -0.,  0.,  0., -0., -0., -1.,  0.,  0., -0., -0.,
        -0.,  0.],
       [-0., -0., -1.,  0.,  0., -1., -0., -0.,  0.,  0.,  1., -0.,  0.,
         0.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  1., -0., -0., -1., -1.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0., -1.,  0.,  0.,  1., -0.,  1.,
         0., -0.,  0., -1.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0.,
        -1., -0., -0., -1., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  1.,
        -1.,  0.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -0.,  0.,  1.,  0.,  0.,  0., -0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -0., -0., -0., -0.,
        -0., -0.,  0.,  0.,  0., -0., -0.,  1., -0., -0., -0.,  0.,  0.,
         0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,  0., -1.,  0., -0.,
         0., -0.,  0., -0., -0., -0., -0., -0.,  1.,  0., -0., -0., -0.,
        -0.,  0.],
       [-1., -0.,  0.,  0., -0.,  0., -0.,  0.,  1.,  0.,  0., -1.,  0.,
        -0.,  0., -0.,  0.,  1., -0., -0.,  1.,  0.,  0.,  1., -0., -0.,
        -0., -1.,  0.,  0.,  0.,  0., -1., -0.,  0.,  0., -0.,  0., -0.,
         1., -0., -1., -0.,  0.,  0., -0., -1., -0., -0., -0., -0., -0.,
         1.,  0.,  0., -0., -0.,  0., -1.,  0., -0.,  0., -1.,  0., -0.,
         0., -0., -0.,  0., -0.,  0., -0., -0.,  1.,  0., -0.,  0.,  0.,
        -0.,  0.],
       [ 0., -0.,  1., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  1., -0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,
        -1.,  0.,  0.,  0., -0., -0.,  0.,  1.,  1., -0., -0., -0., -0.,
         0., -0., -0., -0., -0., -0.,  0., -0., -1.,  0., -0.,  0., -0.,
        -0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,
        -0., -0.],
       [ 1.,  0., -0.,  0., -0.,  0., -0., -0., -1.,  0.,  0.,  1., -0.,
         0.,  0., -0.,  0., -1., -0., -0., -1., -0.,  0., -1.,  0.,  0.,
         0.,  1.,  0.,  0., -0.,  1.,  1., -0.,  0.,  0., -0.,  0., -0.,
        -0., -0.,  1.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,
        -1.,  0.,  1.,  0., -0.,  0.,  1.,  0.,  0., -0.,  1., -0., -1.,
         0.,  0., -0., -0., -0., -0.,  0., -0., -1., -0., -0.,  1.,  0.,
        -0.,  0.],
       [-0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0., -1., -0.,
        -0., -0., -0., -0.,  1., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0., -1., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,
         1., -0., -0., -0.,  0.,  0., -0., -1., -0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0., -1.,  0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0., -0.]], dtype=np.float32), np.array([[ 0., -0., -0.,  0., -0.,  0.,  0., -0., -1., -0., -0., -0., -0.,
         0.,  0.,  0., -0.,  1., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,
        -1.,  0., -1.,  0., -0.,  0.,  0.,  0.,  1., -0., -0., -0., -0.,
         1.,  0.,  1., -0.,  0., -0.,  1.,  0., -1.,  0.,  0., -0., -1.,
        -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0., -0.],
       [-0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -1.,  0., -0.,
        -0., -0., -0., -0., -0.,  0., -0.,  1.,  0.,  0., -0., -0., -0.,
        -1., -0., -0., -0., -0., -0., -1., -0.,  0.,  1., -0.,  0.,  0.,
        -0., -1., -0., -0., -0.,  1.,  0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  1., -0.,  0.,
        -1., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,
         0.,  0.],
       [ 0.,  0., -0.,  0., -0., -0.,  0., -0., -1.,  0., -0., -0., -0.,
         0.,  0.,  0.,  1., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,
         0.,  1., -1.,  0., -0.,  0.,  0., -0.,  0.,  1., -0., -0., -0.,
         1., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -1.,  0.,  0., -0.,
         0., -0.],
       [ 0.,  1., -0., -0., -0.,  0., -0., -0., -0., -0.,  1., -0.,  1.,
        -1.,  0.,  0.,  0., -0.,  0., -1.,  0.,  0., -0.,  0.,  0., -0.,
         0., -0., -0., -0., -1.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1.,
        -0.,  0.,  0.,  1., -0.,  0., -1., -0., -0.,  0.,  0., -0.,  1.,
        -0.,  0., -0.,  0.,  0.,  1., -0.,  1.,  0.,  0., -0., -0., -0.,
         0., -1.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  1.,  0.,  0.,
         1.,  0.],
       [-0., -0.,  0.,  0.,  0., -1., -1., -1., -0., -0.,  1., -0., -0.,
         0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -1.,  0.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
        -0.,  1., -0., -1.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,
         0.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  1.,
         0.,  0.],
       [-0., -0.,  0.,  0.,  0., -0.,  0.,  1., -0.,  0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0.,  0., -0., -0., -1., -0.,  0.,  0.,  0.,
        -0.,  0., -0., -0.,  0., -0.,  0.,  1., -0., -0., -1., -0., -0.,
         0., -0., -0., -0.,  0., -0., -0.,  0., -0., -1.,  0., -0., -0.,
        -0., -1.,  0., -0., -1., -0., -0.,  0.,  0.,  0., -0., -0., -0.,
         1.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -1.,
         0., -0.],
       [-1., -0.,  1.,  1.,  0.,  0.,  0., -0., -0.,  0.,  0., -1.,  1.,
         0., -0.,  0., -0.,  0., -0.,  0.,  1.,  0.,  1., -0., -0.,  0.,
        -0.,  1., -0., -0.,  0., -1., -0., -0., -0.,  0.,  0.,  0., -0.,
         0.,  0.,  1., -0.,  0.,  0.,  0.,  1., -0., -0., -0.,  0., -0.,
        -0.,  0.,  0., -0., -1., -0., -0.,  0.,  0., -0., -0., -0., -0.,
        -0., -0.,  1.,  0.,  0., -0., -0.,  0., -0., -0., -0., -1.,  0.,
         1., -1.],
       [-1.,  0.,  0.,  0.,  1., -0., -1., -0., -0., -0., -0.,  0., -0.,
         1.,  0., -0., -0.,  0.,  0., -0.,  1.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,
        -0.,  0.,  1.,  0., -0.,  0., -0.,  0.,  0., -0., -1., -0., -0.,
        -0.,  1., -0., -1.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  1.,
        -0.,  0.],
       [-0.,  0.,  0.,  0., -1.,  0.,  1.,  0., -0.,  0., -0., -0., -0.,
        -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  1.,
        -0., -0.,  0., -1., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
         1., -1.,  0., -0., -0.,  1.,  0., -0., -0., -0.,  1.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -0.,  1., -1.,  0.,  0.,  0., -0.,
        -0., -0.],
       [ 0., -1.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,
         0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  1.,  0., -0.,
         0.,  0.,  0.,  0.,  1., -1., -0.,  0., -0., -0.,  0., -0., -0.,
         0.,  0., -1., -1., -1., -0.,  0., -0., -0.,  0.,  0., -0., -0.,
         0.,  0.,  0.,  0., -0., -1., -0., -0.,  0., -0.,  0.,  0.,  1.,
         0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -1., -0., -0.,
        -0.,  0.],
       [ 1.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  1., -0., -0., -0.,
        -1.,  1.,  0.,  0., -0., -0., -0.,  0., -1., -0.,  0.,  0., -0.,
        -1.,  0., -1.,  1., -0.,  0., -0., -0., -0.,  0.,  0., -1., -0.,
         0.,  0., -1.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
         0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -1.,  1.,
        -0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  1.,  0.,
        -0.,  0.],
       [-0.,  0., -0.,  0., -0.,  0., -0.,  0., -1., -1.,  0., -0., -0.,
         0.,  0., -0., -1., -0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0., -0., -0.,  0., -0., -1.,  0.,  1., -0.,  0., -0.,  0., -0.,
        -0.,  0., -1.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,  1.,  0.,
         0.,  0.,  1., -0.,  0., -1., -0.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -1., -1., -0., -0.,
         0., -0.],
       [-0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  1.,
         0.,  0., -0.,  0., -0., -0., -1., -0.,  0., -1.,  0.,  0., -0.,
         0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -1.,
         0., -0., -0.,  0., -1., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -0.,  0.,  1., -1., -0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,
         0.,  0.],
       [-0., -0.,  1.,  0., -1., -1., -1., -1., -0.,  1., -0., -0., -0.,
        -0.,  0., -1., -0., -0., -0.,  0., -0., -1., -0., -0., -1., -0.,
        -1., -0., -1.,  1., -0., -0.,  0., -0., -0., -1., -0.,  0.,  0.,
         0., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -1.,
        -0.,  1.,  0., -1.,  0., -0., -0., -0., -1.,  0., -0., -1.,  0.,
        -0.,  0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  1.,
        -0.,  0.],
       [ 0., -0.,  0., -1.,  0., -0., -0., -0., -0., -1.,  0.,  0., -0.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0., -0., -1.,  0.,  0., -0.,
         0., -0.,  0.,  0., -1., -0.,  0.,  0., -1.,  0.,  0., -0., -1.,
        -0.,  1., -0.,  0., -0.,  0., -0., -0., -1., -0.,  0., -0., -0.,
        -0., -1.,  0.,  0., -0.,  0.,  0.,  1., -0., -0., -0., -0.,  0.,
         0., -0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0.],
       [-0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  1.,
        -0., -0.,  0., -0.,  0.,  0.,  1.,  0., -0., -0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0.,  0., -1.,  0.,  0., -1., -0.,  1., -0., -0.,  1.,
         0., -0.,  1.,  0., -0., -0., -0.,  0., -0.,  0., -0., -0., -0.,
        -0., -1.],
       [ 0., -0.,  0.,  0., -0., -1., -1., -0., -0.,  0.,  0.,  0.,  0.,
         0.,  1., -0.,  0.,  0., -1., -0., -0., -0.,  0.,  0.,  1.,  1.,
        -0., -0., -1.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -1., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -1., -0., -0.,
         0.,  1.,  0., -1., -0.,  0., -0.,  1., -0.,  0., -0., -0.,  1.,
        -0.,  0.,  0.,  1., -0., -0.,  0.,  0.,  0., -0.,  0.,  1.,  1.,
        -0., -0.],
       [-0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  0., -0., -0.,
        -0., -0., -0., -1., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0.,  0., -1., -0., -0.,  0., -0., -0.,  0.,  0.,
         0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0., -1.,  1.,  0., -1.,  0., -0., -1.,  0.,  0.,  0.,  0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  1., -0.,  0., -1., -0., -0.,
        -0.,  0.],
       [-0., -0.,  0.,  1.,  0., -0.,  0.,  0., -0.,  0.,  0., -1.,  1.,
        -0.,  0.,  0.,  0., -1., -0.,  0., -0.,  0.,  0., -1., -0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0., -0.,  1.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0., -1.,  0., -0.,  0., -0.,  0.,  0., -0.,  0.,
         0.,  0.,  1., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -1.],
       [-1.,  0., -0., -0.,  0.,  0., -1.,  0.,  0.,  0., -0., -0., -0.,
         1.,  0.,  0., -0., -1., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,
         0.,  0.,  1.,  0., -0.,  0., -1.,  0., -0.,  0., -1., -0.,  0.,
         0.,  1., -0., -1., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,
        -0.,  0., -0.,  0.,  1.,  1., -0.,  0.,  0.,  0., -0., -0.,  1.,
         0.,  0.],
       [-0., -1., -0.,  0., -0.,  0., -0.,  0.,  1.,  0., -0., -1.,  0.,
         0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  0., -1.,  0.,  0., -0.,  0., -0.,
        -0.,  0.,  1., -0., -0.,  0.,  0.,  0., -0., -1., -0.,  0.,  0.,
        -0., -1., -0.,  0., -1., -0.,  1.,  0., -0., -0., -0.,  0.,  0.,
         0.,  0.,  1., -0.,  0.,  0., -1., -0.,  0.,  1., -0., -0., -0.,
        -0., -0.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 3
m = 5
p = 7
rank = 80

verify_tensor_decomposition(decomposition_357, n, m, p, rank)

##Rank-48 decomposition of <4,4,4> over 0.5*C


In [ ]:
#@title Data
decomposition_444 = (np.array([[ 0.5+0.5j,  0.5+0.5j,  0. +0.j ,  0. -0.5j,  0.5+0.5j,  0. +0.j ,
         0. +0.5j,  0.5+0.5j,  0. -0.5j, -0.5+0.5j, -0.5+0.5j,  0.5+0.j ,
         0. +0.j , -0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j ,  0.5+0.5j,
         0. +0.5j,  0. -0.j ,  0. +0.j ,  0. +0.5j, -0.5-0.5j, -0. +0.j ,
        -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.5j,  0.5+0.5j,
        -0. +0.j ,  0.5+0.j , -0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,
        -0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j , -0.5-0.5j,  0.5-0.5j,
         0.5+0.5j,  0. +0.5j,  0. -0.j , -0.5+0.5j,  0.5-0.5j,  0.5+0.j ],
       [ 0.5+0.5j,  0. +0.j , -0.5+0.j , -0.5+0.j , -0.5-0.5j,  0. +0.j ,
         0. +0.j , -0.5-0.5j,  0. -0.5j,  0. -0.j ,  0.5-0.5j,  0.5+0.j ,
         0.5+0.5j,  0.5-0.5j, -0.5+0.j ,  0. +0.j , -0. +0.5j,  0.5+0.5j,
        -0. +0.5j,  0. +0.j ,  0.5+0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,
         0.5+0.j , -0. +0.j ,  0.5+0.5j,  0. -0.5j, -0.5+0.5j, -0. +0.j ,
         0.5+0.5j, -0.5+0.j ,  0. +0.j , -0. +0.5j,  0. +0.5j, -0. +0.j ,
        -0.5-0.5j,  0. -0.5j,  0.5-0.5j,  0. -0.5j, -0.5-0.5j,  0. -0.j ,
         0. -0.j ,  0.5+0.j ,  0. +0.j ,  0.5-0.5j,  0. +0.j , -0. +0.5j],
       [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,  0. +0.j ,  0.5-0.5j,
         0. +0.j ,  0. +0.j , -0.5-0.j ,  0. +0.j ,  0. -0.j ,  0. -0.5j,
        -0.5-0.5j, -0.5+0.5j,  0.5+0.j , -0.5+0.5j,  0. +0.5j,  0. +0.j ,
        -0.5+0.j ,  0.5-0.5j, -0.5-0.5j, -0.5-0.j ,  0. -0.j , -0.5-0.5j,
         0. -0.5j,  0.5-0.5j,  0.5+0.5j,  0.5+0.j ,  0. -0.j ,  0. +0.j ,
         0.5+0.5j,  0. -0.5j,  0.5+0.5j,  0. -0.5j, -0.5+0.j ,  0.5-0.5j,
        -0.5-0.5j,  0. -0.5j,  0.5-0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,
        -0. +0.j , -0.5-0.j ,  0.5-0.5j,  0. -0.j ,  0. +0.j , -0. +0.5j],
       [ 0. +0.j , -0.5+0.5j,  0. +0.j , -0.5-0.j ,  0. -0.j , -0.5-0.5j,
         0.5+0.j ,  0. +0.j ,  0. -0.5j, -0.5-0.5j,  0. +0.j , -0.5+0.j ,
         0. +0.j , -0. +0.j , -0.5+0.j ,  0.5+0.5j,  0. -0.5j,  0. +0.j ,
        -0. +0.5j,  0.5+0.5j,  0. +0.j ,  0. -0.5j, -0.5+0.5j,  0.5-0.5j,
        -0.5-0.j ,  0.5+0.5j,  0. +0.j , -0. +0.5j,  0. +0.j ,  0.5-0.5j,
         0. -0.j ,  0.5+0.j ,  0.5-0.5j,  0. -0.5j,  0. +0.5j,  0.5+0.5j,
         0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j , -0.5-0.5j,
        -0.5+0.5j, -0.5-0.j , -0.5-0.5j,  0. +0.j ,  0.5+0.5j,  0. -0.5j],
       [ 0.5-0.5j,  0.5+0.5j, -0. +0.j , -0. +0.5j, -0.5+0.5j,  0. +0.j ,
        -0.5+0.j , -0.5-0.5j,  0.5+0.j ,  0.5+0.5j, -0.5+0.5j, -0.5-0.j ,
        -0. +0.j ,  0. -0.j ,  0.5+0.j ,  0. -0.j , -0.5-0.j ,  0.5+0.5j,
        -0. +0.5j,  0. -0.j ,  0. +0.j ,  0. -0.5j,  0.5-0.5j,  0. -0.j ,
         0. -0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j , -0.5-0.5j, -0.5-0.5j,
        -0. +0.j ,  0.5+0.j ,  0. -0.j , -0.5+0.j , -0.5-0.j ,  0. -0.j ,
        -0. +0.j , -0. +0.5j, -0. +0.j , -0.5+0.j ,  0.5+0.5j, -0.5+0.5j,
         0.5-0.5j,  0.5+0.j , -0. +0.j ,  0.5+0.5j,  0.5-0.5j, -0. +0.5j],
       [ 0.5-0.5j, -0. +0.j ,  0. -0.5j, -0.5+0.j ,  0.5-0.5j,  0. +0.j ,
         0. -0.j ,  0.5+0.5j,  0.5+0.j ,  0. +0.j ,  0.5-0.5j, -0.5-0.j ,
        -0.5+0.5j,  0.5-0.5j,  0. -0.5j,  0. -0.j ,  0. -0.5j,  0.5+0.5j,
        -0. +0.5j,  0. +0.j ,  0.5+0.5j,  0. +0.5j,  0. +0.j ,  0. -0.j ,
        -0. +0.5j,  0. +0.j , -0.5-0.5j, -0.5+0.j , -0.5-0.5j, -0. +0.j ,
        -0.5-0.5j, -0.5+0.j ,  0. +0.j , -0. +0.5j,  0.5+0.j ,  0. +0.j ,
        -0.5+0.5j, -0.5+0.j , -0.5-0.5j,  0. +0.5j,  0.5+0.5j,  0. -0.j ,
         0. +0.j ,  0. +0.5j,  0. +0.j , -0.5-0.5j,  0. -0.j ,  0.5+0.j ],
       [ 0. -0.j , -0. +0.j , -0. +0.5j,  0.5+0.j ,  0. +0.j ,  0.5-0.5j,
         0. -0.j , -0. +0.j ,  0. -0.5j,  0. -0.j ,  0. -0.j ,  0. +0.5j,
         0.5-0.5j, -0.5+0.5j, -0. +0.5j,  0.5-0.5j,  0. -0.5j,  0. +0.j ,
        -0.5+0.j ,  0.5-0.5j, -0.5-0.5j,  0.5+0.j ,  0. +0.j ,  0.5-0.5j,
         0.5+0.j , -0.5-0.5j, -0.5-0.5j,  0. -0.5j,  0. -0.j ,  0. +0.j ,
        -0.5-0.5j,  0. -0.5j, -0.5+0.5j,  0. -0.5j, -0. +0.5j, -0.5+0.5j,
        -0.5+0.5j, -0.5+0.j , -0.5-0.5j, -0. +0.5j,  0. -0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.5j, -0.5-0.5j, -0. +0.j , -0. +0.j ,  0.5+0.j ],
       [ 0. +0.j , -0.5+0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5-0.5j,
         0. +0.5j,  0. -0.j ,  0.5+0.j , -0.5+0.5j,  0. +0.j ,  0.5+0.j ,
         0. +0.j ,  0. -0.j , -0. +0.5j, -0.5-0.5j,  0. -0.5j,  0. -0.j ,
         0. +0.5j,  0.5+0.5j,  0. -0.j ,  0. +0.5j, -0.5-0.5j,  0.5+0.5j,
         0. -0.5j,  0.5-0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.5j,
         0. +0.j ,  0.5+0.j ,  0.5+0.5j,  0. +0.5j,  0.5+0.j , -0.5-0.5j,
        -0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.5j,
         0.5+0.5j, -0. +0.5j, -0.5+0.5j,  0. +0.j ,  0.5+0.5j,  0.5+0.j ],
       [ 0.5-0.5j, -0.5-0.5j,  0. -0.j ,  0. -0.5j, -0.5+0.5j,  0. +0.j ,
         0.5+0.j , -0.5-0.5j, -0.5-0.j , -0.5-0.5j,  0.5-0.5j,  0.5+0.j ,
        -0. +0.j , -0. +0.j ,  0.5+0.j , -0. +0.j , -0.5-0.j ,  0.5+0.5j,
        -0. +0.5j,  0. +0.j ,  0. -0.j ,  0. -0.5j,  0.5-0.5j,  0. +0.j ,
         0. -0.5j,  0. +0.j ,  0. +0.j , -0.5-0.j ,  0.5+0.5j,  0.5+0.5j,
         0. +0.j , -0.5+0.j ,  0. -0.j , -0.5-0.j ,  0.5+0.j ,  0. -0.j ,
         0. +0.j , -0. +0.5j,  0. +0.j ,  0.5+0.j , -0.5-0.5j, -0.5+0.5j,
         0.5-0.5j, -0.5-0.j ,  0. -0.j , -0.5-0.5j,  0.5-0.5j,  0. -0.5j],
       [ 0.5-0.5j, -0. +0.j ,  0. +0.5j, -0.5+0.j ,  0.5-0.5j,  0. +0.j ,
         0. +0.j ,  0.5+0.5j, -0.5-0.j ,  0. +0.j , -0.5+0.5j,  0.5+0.j ,
        -0.5+0.5j,  0.5-0.5j, -0. +0.5j,  0. +0.j , -0. +0.5j,  0.5+0.5j,
        -0. +0.5j,  0. -0.j , -0.5-0.5j,  0. +0.5j,  0. +0.j ,  0. +0.j ,
         0. +0.5j,  0. +0.j ,  0.5+0.5j, -0.5-0.j ,  0.5+0.5j, -0. +0.j ,
        -0.5-0.5j,  0.5+0.j ,  0. +0.j ,  0. -0.5j, -0.5-0.j ,  0. +0.j ,
         0.5-0.5j,  0.5+0.j , -0.5-0.5j, -0. +0.5j, -0.5-0.5j,  0. +0.j ,
         0. -0.j , -0. +0.5j, -0. +0.j ,  0.5+0.5j,  0. +0.j ,  0.5+0.j ],
       [ 0. +0.j , -0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. -0.j , -0.5+0.5j,
        -0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.j ,  0. +0.5j,
         0.5-0.5j, -0.5+0.5j,  0. -0.5j,  0.5-0.5j, -0. +0.5j, -0. +0.j ,
         0.5+0.j ,  0.5-0.5j,  0.5+0.5j, -0.5+0.j ,  0. +0.j ,  0.5-0.5j,
        -0.5+0.j ,  0.5+0.5j,  0.5+0.5j, -0. +0.5j,  0. +0.j ,  0. +0.j ,
        -0.5-0.5j,  0. -0.5j,  0.5-0.5j,  0. +0.5j,  0. +0.5j,  0.5-0.5j,
         0.5-0.5j,  0.5+0.j , -0.5-0.5j, -0. +0.5j,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. -0.5j, -0.5-0.5j, -0. +0.j , -0. +0.j ,  0.5+0.j ],
       [-0. +0.j ,  0.5-0.5j,  0. +0.j , -0.5-0.j , -0. +0.j ,  0.5+0.5j,
         0. -0.5j,  0. +0.j ,  0.5+0.j ,  0.5-0.5j,  0. +0.j ,  0.5+0.j ,
        -0. +0.j , -0. +0.j , -0. +0.5j, -0.5-0.5j,  0. -0.5j,  0. +0.j ,
         0. -0.5j,  0.5+0.5j,  0. -0.j ,  0. -0.5j, -0.5-0.5j,  0.5+0.5j,
         0. +0.5j, -0.5+0.5j,  0. +0.j , -0.5-0.j ,  0. +0.j ,  0.5-0.5j,
         0. +0.j ,  0.5+0.j , -0.5-0.5j,  0. +0.5j,  0.5+0.j ,  0.5+0.5j,
        -0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.5j,
         0.5+0.5j,  0. -0.5j, -0.5+0.5j,  0. +0.j ,  0.5+0.5j, -0.5+0.j ],
       [ 0.5-0.5j,  0.5-0.5j, -0. +0.j , -0.5-0.j ,  0.5-0.5j, -0. +0.j ,
        -0.5+0.j , -0.5+0.5j,  0.5+0.j , -0.5-0.5j,  0.5+0.5j,  0. -0.5j,
         0. +0.j , -0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.5j,
        -0.5+0.j , -0. +0.j ,  0. -0.j , -0.5-0.j , -0.5+0.5j,  0. -0.j ,
        -0. +0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j , -0.5-0.5j,  0.5-0.5j,
         0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.5j,  0.5+0.j , -0. +0.j ,
         0. +0.j ,  0. -0.5j, -0. +0.j , -0. +0.5j, -0.5+0.5j,  0.5+0.5j,
         0.5-0.5j, -0.5+0.j ,  0. +0.j , -0.5-0.5j,  0.5+0.5j,  0. +0.5j],
       [ 0.5-0.5j, -0. +0.j ,  0. -0.5j,  0. -0.5j, -0.5+0.5j,  0. -0.j ,
         0. -0.j ,  0.5-0.5j,  0.5+0.j ,  0. -0.j , -0.5-0.5j,  0. -0.5j,
         0.5-0.5j,  0.5+0.5j,  0. -0.5j,  0. +0.j ,  0.5+0.j , -0.5+0.5j,
        -0.5+0.j ,  0. +0.j ,  0.5-0.5j,  0.5+0.j , -0. +0.j , -0. +0.j ,
         0. -0.5j,  0. +0.j ,  0.5-0.5j, -0.5+0.j , -0.5-0.5j, -0. +0.j ,
        -0.5+0.5j, -0. +0.5j,  0. -0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,
         0.5-0.5j,  0.5+0.j , -0.5-0.5j,  0.5+0.j , -0.5+0.5j,  0. +0.j ,
         0. +0.j ,  0. -0.5j, -0. +0.j ,  0.5+0.5j,  0. +0.j ,  0.5+0.j ],
       [ 0. +0.j ,  0. +0.j , -0. +0.5j, -0. +0.5j,  0. +0.j , -0.5-0.5j,
         0. -0.j ,  0. -0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
        -0.5+0.5j, -0.5-0.5j, -0. +0.5j, -0.5-0.5j,  0.5+0.j ,  0. -0.j ,
        -0. +0.5j,  0.5+0.5j, -0.5+0.5j, -0. +0.5j,  0. +0.j , -0.5+0.5j,
         0.5+0.j ,  0.5+0.5j,  0.5-0.5j,  0. +0.5j, -0. +0.j ,  0. +0.j ,
        -0.5+0.5j,  0.5+0.j , -0.5+0.5j, -0.5+0.j ,  0. +0.5j, -0.5-0.5j,
         0.5-0.5j,  0.5+0.j , -0.5-0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
         0. -0.j , -0. +0.5j, -0.5-0.5j, -0. +0.j ,  0. +0.j ,  0.5+0.j ],
       [-0. +0.j ,  0.5+0.5j,  0. -0.j , -0. +0.5j,  0. +0.j , -0.5+0.5j,
         0. +0.5j,  0. +0.j , -0.5+0.j ,  0.5-0.5j,  0. +0.j ,  0. -0.5j,
        -0. +0.j ,  0. +0.j , -0. +0.5j, -0.5+0.5j,  0.5+0.j ,  0. +0.j ,
         0.5+0.j , -0.5+0.5j, -0. +0.j , -0.5+0.j ,  0.5+0.5j, -0.5-0.5j,
         0. -0.5j, -0.5+0.5j,  0. -0.j , -0.5-0.j ,  0. +0.j , -0.5-0.5j,
        -0. +0.j , -0. +0.5j,  0.5+0.5j,  0.5+0.j ,  0.5+0.j ,  0.5-0.5j,
         0. -0.j , -0.5-0.j ,  0. +0.j , -0.5-0.j ,  0. -0.j ,  0.5-0.5j,
         0.5+0.5j,  0. -0.5j, -0.5+0.5j,  0. +0.j , -0.5+0.5j,  0.5+0.j ]],
      dtype=np.complex64), np.array([[-0.5-0.j , -0. +0.j ,  0. -0.j ,  0. -0.5j, -0.5-0.j ,  0. +0.j ,
         0. +0.j , -0.5-0.j ,  0.5+0.j , -0. +0.j ,  0. -0.j ,  0. -0.5j,
        -0.5+0.j , -0. +0.5j,  0. +0.j , -0. +0.5j,  0. +0.j ,  0. -0.5j,
         0. -0.j ,  0. -0.5j,  0. +0.j , -0. +0.j ,  0. -0.5j, -0.5-0.j ,
         0. -0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0.5+0.j ,  0.5+0.j ,  0. +0.j , -0. +0.j ,  0. +0.5j,  0. -0.j ,
         0. -0.j , -0. +0.j , -0. +0.5j,  0. -0.5j,  0. +0.j ,  0.5+0.j ,
        -0. +0.5j,  0.5+0.j ,  0. -0.5j,  0. +0.j , -0.5+0.j ,  0.5+0.j ],
       [ 0. +0.j , -0. +0.5j,  0.5+0.5j,  0. -0.j , -0. +0.j ,  0.5+0.j ,
        -0.5-0.5j,  0. -0.j ,  0. +0.j ,  0. +0.5j, -0. +0.5j,  0. -0.j ,
         0. -0.j ,  0. +0.j , -0.5+0.j ,  0. -0.j ,  0.5+0.j ,  0. -0.j ,
         0. -0.5j,  0. +0.j ,  0. -0.5j, -0.5+0.j ,  0. -0.j , -0. +0.j ,
         0. +0.5j,  0. +0.5j,  0.5+0.j , -0. +0.5j,  0.5+0.j ,  0. +0.5j,
         0. -0.j ,  0. -0.j , -0. +0.5j, -0.5-0.j ,  0. +0.j ,  0. -0.5j,
        -0.5-0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
         0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.j , -0. +0.j ],
       [ 0. -0.j ,  0. -0.j ,  0. -0.j ,  0. +0.5j,  0.5+0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j , -0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. -0.j ,
         0.5+0.j ,  0. -0.5j,  0. +0.j , -0. +0.j ,  0.5+0.j , -0. +0.5j,
        -0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j , -0. +0.5j,  0.5+0.j ,
         0. +0.j ,  0. +0.5j,  0.5+0.j , -0. +0.5j,  0. -0.j ,  0. +0.5j,
         0. -0.j , -0.5-0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0.5+0.j , -0. +0.5j, -0. +0.j ,  0. -0.j ,  0. +0.5j,  0. +0.j ,
        -0. +0.j , -0.5-0.j ,  0. -0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j ],
       [ 0. -0.j , -0. +0.5j,  0. -0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,
         0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.5j,  0. +0.5j, -0. +0.5j,
         0.5+0.j , -0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.j ,  0. -0.j ,
         0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.5j,  0.5+0.j ,
         0. +0.j ,  0. +0.5j,  0. -0.j ,  0. +0.5j,  0. -0.j ,  0. -0.j ,
         0.5+0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
        -0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,  0. -0.j , -0.5-0.j ,
         0. +0.j , -0.5-0.j ,  0. -0.j ,  0. -0.5j, -0. +0.j ,  0. +0.j ],
       [-0.5-0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
         0. -0.j ,  0.5+0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
        -0.5+0.j ,  0. -0.5j, -0.5+0.j , -0. +0.5j,  0.5+0.j ,  0. -0.5j,
         0. +0.j , -0. +0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5-0.j ,
         0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j ,
        -0.5+0.j , -0.5-0.j , -0. +0.j ,  0.5+0.j ,  0. -0.5j, -0. +0.j ,
        -0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.j ,  0. -0.j ,  0.5+0.j ,
         0. -0.5j, -0. +0.j , -0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. -0.j ],
       [ 0. +0.j , -0. +0.5j, -0.5-0.5j,  0. -0.5j,  0. +0.j ,  0.5+0.j ,
         0.5+0.5j,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. -0.5j, -0. +0.j ,
        -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0. +0.j ,
         0. -0.5j,  0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j , -0. +0.j ,
         0. -0.5j, -0. +0.5j, -0.5-0.j ,  0. +0.5j,  0.5+0.j ,  0. -0.5j,
         0. -0.j ,  0. -0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.5j,
        -0.5+0.j ,  0. +0.j ,  0. +0.j , -0. +0.5j,  0. +0.5j,  0. +0.j ,
         0. +0.j , -0.5-0.j ,  0. -0.j , -0. +0.5j, -0. +0.j ,  0.5+0.j ],
       [-0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.j , -0. +0.j ,
         0. -0.j ,  0. +0.j , -0.5+0.j , -0. +0.5j,  0. +0.j ,  0. -0.j ,
         0.5+0.j , -0. +0.5j,  0. -0.j , -0. +0.j , -0.5-0.j , -0. +0.5j,
         0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.5j,  0.5+0.j ,
         0. +0.j ,  0. +0.5j, -0.5-0.j , -0. +0.5j,  0. -0.j ,  0. -0.5j,
        -0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.5j,
        -0.5+0.j , -0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
         0. -0.j , -0.5-0.j , -0. +0.j ,  0. +0.5j, -0.5+0.j , -0. +0.j ],
       [ 0. -0.j ,  0. +0.5j,  0. +0.j , -0. +0.j , -0.5+0.j ,  0.5+0.j ,
         0. -0.j , -0.5-0.j , -0.5+0.j ,  0. +0.5j,  0. -0.5j, -0. +0.5j,
         0.5+0.j ,  0. -0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.j ,
         0. -0.5j, -0. +0.j ,  0. -0.5j,  0. -0.j ,  0. +0.5j,  0.5+0.j ,
         0. +0.j , -0. +0.5j,  0. +0.j ,  0. +0.5j,  0. -0.j , -0. +0.j ,
         0.5+0.j ,  0. -0.j , -0. +0.j , -0.5+0.j ,  0. +0.j ,  0. -0.j ,
        -0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,  0. -0.j , -0.5+0.j ,
         0. -0.j , -0.5-0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j , -0. +0.j ],
       [ 0.5+0.j ,  0. +0.j ,  0. -0.j , -0. +0.j ,  0.5+0.j ,  0. +0.j ,
         0. -0.j , -0.5-0.j , -0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,
         0.5+0.j , -0. +0.5j,  0.5+0.j ,  0. -0.5j,  0.5+0.j ,  0. -0.5j,
         0. -0.5j,  0. +0.5j, -0. +0.j , -0.5-0.j ,  0. -0.5j, -0.5+0.j ,
         0. -0.5j,  0. +0.j ,  0. -0.j ,  0. -0.5j,  0. -0.j , -0. +0.j ,
        -0.5+0.j , -0. +0.j , -0. +0.j , -0.5-0.j ,  0. +0.j ,  0. +0.j ,
        -0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.j ,  0. -0.j , -0.5-0.j ,
        -0. +0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. -0.j ],
       [ 0. -0.j , -0. +0.5j,  0.5+0.5j,  0. +0.5j,  0. +0.j ,  0.5+0.j ,
         0.5+0.5j, -0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. -0.5j, -0. +0.5j,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. +0.j , -0. +0.5j,  0. +0.j ,  0. +0.j , -0. +0.j ,
         0. +0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.5j,
        -0. +0.j , -0.5-0.j ,  0. -0.5j,  0. -0.j , -0. +0.5j,  0. -0.5j,
        -0.5-0.j ,  0. -0.j ,  0. +0.j ,  0. +0.5j,  0. -0.5j,  0. -0.j ,
         0. +0.j ,  0.5+0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. -0.j , -0. +0.5j, -0.5+0.j , -0. +0.j ,
         0. -0.j ,  0. -0.j ,  0. -0.j ,  0. -0.5j, -0. +0.j ,  0. +0.5j,
        -0.5-0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j , -0. +0.5j,
         0. +0.5j,  0. -0.5j,  0. -0.j ,  0. -0.j ,  0. +0.5j,  0.5+0.j ,
        -0. +0.5j,  0. -0.5j, -0.5-0.j ,  0. -0.j , -0. +0.j ,  0. +0.5j,
        -0. +0.j ,  0. +0.j , -0. +0.j ,  0. -0.j , -0. +0.5j,  0. -0.5j,
        -0.5-0.j , -0. +0.5j,  0. +0.j ,  0. -0.j ,  0. -0.5j, -0. +0.j ,
        -0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. -0.5j, -0.5-0.j , -0. +0.j ],
       [ 0. +0.j , -0. +0.5j,  0. -0.j ,  0. +0.j , -0.5-0.j ,  0.5+0.j ,
         0. -0.j ,  0.5+0.j ,  0. -0.j ,  0. -0.5j,  0. -0.5j,  0. +0.j ,
        -0.5+0.j ,  0. +0.j ,  0. +0.j , -0. +0.5j,  0. +0.j ,  0. -0.j ,
         0. -0.j ,  0. +0.j , -0. +0.5j,  0.5+0.j , -0. +0.5j,  0.5+0.j ,
         0. +0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0.5+0.j , -0.5-0.j , -0. +0.j ,  0.5+0.j ,  0. +0.5j,  0. +0.j ,
        -0.5-0.j , -0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,
        -0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.j ],
       [ 0. -0.5j,  0. +0.j , -0. +0.j ,  0.5+0.j , -0. +0.5j,  0. -0.j ,
        -0. +0.j , -0. +0.5j, -0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
         0. +0.5j, -0.5-0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,
         0.5+0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.5j,
         0.5+0.j , -0. +0.j ,  0. +0.j , -0.5+0.j , -0. +0.j , -0. +0.j ,
        -0. +0.5j, -0. +0.j , -0. +0.j ,  0. -0.j ,  0. -0.j , -0. +0.j ,
         0. +0.j ,  0. -0.j , -0.5-0.j , -0.5-0.j ,  0. +0.j , -0. +0.5j,
         0.5+0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j , -0. +0.5j,  0. +0.5j],
       [ 0. +0.j ,  0.5+0.j ,  0.5-0.5j,  0. +0.j ,  0. +0.j , -0. +0.5j,
         0.5-0.5j, -0. +0.j ,  0. -0.5j,  0.5+0.j , -0.5+0.j , -0.5+0.j ,
        -0. +0.j , -0. +0.j , -0. +0.5j,  0. -0.j ,  0. -0.5j,  0. +0.j ,
         0. -0.j ,  0. -0.j ,  0.5+0.j ,  0. -0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j, -0.5-0.j ,
         0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. -0.5j, -0.5-0.j , -0.5+0.j ,
         0. -0.5j, -0.5+0.j ,  0. +0.j ,  0. -0.j ,  0.5+0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.j , -0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.j ],
       [ 0. -0.j ,  0. -0.j , -0. +0.j , -0.5-0.j ,  0. -0.5j,  0. +0.j ,
        -0. +0.j , -0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5-0.j ,
         0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.j ,
        -0.5-0.j , -0.5-0.j ,  0. -0.j , -0. +0.j , -0.5-0.j ,  0. -0.5j,
        -0.5-0.j , -0.5-0.j ,  0. -0.5j,  0. -0.j , -0. +0.j , -0.5+0.j ,
         0. +0.j ,  0. +0.j , -0. +0.j ,  0. +0.j , -0.5+0.j , -0.5+0.j ,
         0. -0.5j, -0.5-0.j ,  0. -0.j , -0. +0.j ,  0.5+0.j , -0. +0.j ,
         0. +0.j , -0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. -0.5j, -0. +0.j ],
       [ 0. +0.j ,  0.5+0.j , -0. +0.j ,  0. +0.j ,  0. -0.5j, -0. +0.5j,
        -0. +0.j ,  0. -0.5j, -0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,
         0. -0.5j, -0. +0.j ,  0. +0.j , -0.5-0.j ,  0. +0.j ,  0. +0.j ,
         0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j, -0.5+0.j ,  0. -0.5j,
        -0.5+0.j , -0.5+0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. +0.j ,
         0. -0.5j,  0. +0.5j,  0. -0.j ,  0. -0.5j, -0.5-0.j ,  0. +0.j ,
         0. -0.5j, -0.5-0.j , -0. +0.j ,  0.5+0.j , -0. +0.j ,  0. -0.5j,
         0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ]],
      dtype=np.complex64), np.array([[-0. +0.5j,  0. -0.5j,  0. +0.j ,  0. -0.j , -0. +0.j , -0.5+0.j ,
         0. +0.j , -0. +0.j ,  0.5+0.j , -0. +0.5j,  0. +0.j , -0.5+0.5j,
        -0. +0.j , -0. +0.j ,  0.5+0.j ,  0. -0.5j, -0.5-0.5j,  0. +0.5j,
        -0.5-0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.j ,  0. +0.j ,
         0. -0.5j,  0. +0.j , -0. +0.5j, -0. +0.5j,  0.5+0.j ,  0. +0.j ,
        -0. +0.5j,  0. +0.j ,  0. -0.5j,  0. -0.j ,  0.5+0.j ,  0. -0.j ,
         0.5+0.j ,  0. -0.5j, -0.5-0.j ,  0.5-0.5j,  0. -0.5j,  0. +0.j ,
        -0.5-0.j , -0.5-0.j , -0.5+0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j ],
       [-0.5+0.j , -0.5-0.j , -0. +0.j ,  0. +0.j ,  0. +0.j , -0.5-0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.j ,  0.5-0.5j,
         0. +0.j , -0. +0.j ,  0. -0.5j, -0. +0.5j, -0.5+0.5j, -0. +0.5j,
        -0.5-0.5j,  0. +0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,  0. -0.j ,
        -0.5+0.j , -0. +0.j ,  0.5+0.j , -0.5-0.j ,  0. -0.5j,  0. +0.j ,
         0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
         0.5+0.j , -0.5-0.j , -0.5-0.j , -0.5-0.5j,  0. +0.5j, -0. +0.j ,
         0.5+0.j ,  0. +0.5j,  0. -0.5j,  0. -0.j , -0.5-0.j ,  0. -0.5j],
       [ 0. -0.5j,  0. +0.5j,  0. +0.j ,  0. -0.j ,  0. +0.j ,  0. +0.5j,
        -0. +0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j , -0. +0.j ,  0.5+0.5j,
        -0. +0.j ,  0. +0.j , -0. +0.5j, -0.5-0.j , -0.5-0.5j,  0.5+0.j ,
        -0.5+0.5j,  0. +0.j ,  0. -0.j ,  0. +0.j , -0. +0.j , -0. +0.j ,
        -0.5-0.j ,  0. -0.j , -0. +0.5j,  0.5+0.j , -0.5+0.j ,  0. +0.j ,
         0. -0.5j, -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,
         0. -0.5j, -0.5+0.j ,  0. -0.5j, -0.5+0.5j, -0.5-0.j ,  0. -0.j ,
         0. -0.5j, -0. +0.5j, -0.5-0.j ,  0. +0.j ,  0. -0.5j,  0. +0.5j],
       [ 0. -0.5j,  0. -0.5j, -0. +0.j ,  0. -0.j ,  0. +0.j ,  0.5+0.j ,
         0. +0.j ,  0. +0.j ,  0. +0.5j, -0. +0.5j,  0. +0.j , -0.5+0.5j,
         0. -0.j , -0. +0.j ,  0. -0.5j,  0. -0.5j,  0.5+0.5j,  0. -0.5j,
         0.5+0.5j,  0. -0.j ,  0. -0.j , -0. +0.j ,  0. +0.j ,  0. +0.j ,
         0.5+0.j , -0. +0.j ,  0. -0.5j,  0.5+0.j ,  0.5+0.j ,  0. -0.j ,
         0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. -0.5j, -0. +0.j ,
        -0.5+0.j ,  0.5+0.j , -0.5-0.j ,  0.5-0.5j,  0. -0.5j,  0. +0.j ,
         0.5+0.j ,  0. -0.5j, -0.5-0.j ,  0. +0.j , -0. +0.5j,  0. -0.5j],
       [ 0. -0.5j,  0. -0.j ,  0.5+0.j , -0.5-0.5j,  0. -0.j ,  0.5+0.j ,
         0.5+0.j ,  0. +0.j , -0.5-0.j , -0. +0.j , -0. +0.j ,  0.5-0.5j,
        -0.5+0.j , -0. +0.5j,  0. +0.5j, -0. +0.5j,  0. -0.j ,  0. -0.5j,
         0.5+0.5j, -0. +0.j ,  0.5+0.j , -0. +0.j , -0.5+0.j ,  0. -0.j ,
         0. +0.5j, -0. +0.j , -0. +0.j ,  0. -0.5j, -0.5-0.j ,  0. -0.5j,
        -0. +0.j ,  0. -0.j ,  0. +0.5j, -0.5-0.5j, -0.5+0.j ,  0. +0.j ,
         0. +0.j , -0.5-0.j ,  0. -0.j ,  0. +0.j ,  0. +0.5j, -0. +0.5j,
        -0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. -0.j ,  0. -0.5j],
       [ 0.5+0.j ,  0. -0.j , -0.5-0.j ,  0.5-0.5j,  0. -0.j ,  0.5+0.j ,
         0.5+0.j ,  0. -0.j , -0. +0.5j,  0. +0.j ,  0. -0.j , -0.5+0.5j,
         0.5+0.j , -0.5+0.j , -0.5-0.j ,  0. -0.5j, -0. +0.j ,  0. -0.5j,
         0.5+0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j ,  0. +0.j ,
         0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j , -0. +0.5j,  0.5+0.j ,
         0. +0.j ,  0. -0.j ,  0.5+0.j ,  0.5-0.5j,  0. -0.5j,  0. +0.j ,
         0. +0.j ,  0. -0.5j,  0. -0.j ,  0. -0.j ,  0. -0.5j, -0.5-0.j ,
         0. +0.j ,  0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. -0.j ,  0.5+0.j ],
       [ 0. +0.5j,  0. +0.j ,  0. +0.5j, -0.5-0.5j,  0. +0.j ,  0. -0.5j,
        -0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j , -0.5-0.5j,
         0. +0.5j, -0. +0.5j, -0.5+0.j ,  0.5+0.j ,  0. -0.j , -0.5-0.j ,
         0.5-0.5j, -0. +0.j , -0.5+0.j ,  0. -0.j , -0. +0.5j,  0. +0.j ,
         0.5+0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0.5+0.j ,  0. -0.5j,
         0. +0.j ,  0. +0.j , -0. +0.5j,  0.5+0.5j, -0. +0.5j,  0. -0.j ,
         0. -0.j , -0. +0.5j,  0. +0.j , -0. +0.j ,  0.5+0.j ,  0. -0.5j,
         0. +0.j , -0.5+0.j ,  0.5+0.j ,  0. -0.j ,  0. -0.j ,  0.5+0.j ],
       [-0. +0.5j,  0. +0.j , -0.5-0.j , -0.5-0.5j, -0. +0.j , -0.5+0.j ,
         0.5+0.j ,  0. -0.j ,  0. -0.5j,  0. +0.j ,  0. -0.j ,  0.5-0.5j,
        -0.5+0.j , -0. +0.5j, -0.5-0.j , -0. +0.5j,  0. +0.j , -0. +0.5j,
        -0.5-0.5j,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j , -0. +0.j ,
        -0.5-0.j ,  0. +0.j ,  0. -0.j , -0.5+0.j , -0.5+0.j ,  0. -0.5j,
        -0. +0.j ,  0. -0.j ,  0. -0.5j,  0.5+0.5j, -0. +0.5j,  0. -0.j ,
         0. -0.j ,  0. +0.5j, -0. +0.j ,  0. +0.j , -0. +0.5j,  0. -0.5j,
         0. +0.j , -0.5-0.j ,  0.5+0.j ,  0. -0.j , -0. +0.j ,  0.5+0.j ],
       [ 0. -0.j ,  0. +0.j , -0.5-0.j ,  0.5+0.j ,  0. +0.j , -0.5-0.j ,
         0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.j ,  0. -0.j ,  0. +0.5j,
         0.5+0.j ,  0. -0.5j,  0. -0.5j,  0. -0.5j, -0.5+0.j ,  0. -0.j ,
        -0.5+0.j , -0. +0.5j, -0.5-0.j , -0. +0.5j,  0. +0.j , -0.5+0.j ,
         0. -0.5j, -0.5-0.j , -0. +0.5j,  0.5+0.j , -0. +0.j ,  0. -0.j ,
        -0. +0.5j, -0.5-0.j ,  0. -0.5j,  0.5+0.j ,  0.5+0.j ,  0. +0.5j,
         0.5+0.j ,  0. -0.5j, -0.5-0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
         0. +0.j , -0. +0.5j, -0.5-0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ],
       [ 0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5-0.j ,  0. -0.j , -0.5-0.j ,
         0. +0.j ,  0. -0.j , -0.5+0.j ,  0. +0.j ,  0. -0.j ,  0. -0.5j,
        -0.5+0.j ,  0.5+0.j ,  0.5+0.j , -0. +0.5j, -0.5-0.j ,  0. -0.j ,
        -0.5+0.j , -0. +0.5j,  0. -0.5j,  0. -0.5j, -0. +0.j ,  0. +0.5j,
        -0.5-0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
         0.5+0.j , -0.5-0.j , -0.5-0.j , -0.5-0.j , -0. +0.5j,  0. -0.5j,
         0.5+0.j , -0.5+0.j , -0.5-0.j ,  0. -0.5j,  0. -0.j ,  0. -0.j ,
         0. -0.j , -0.5-0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j],
       [-0. +0.j ,  0. -0.j ,  0. -0.5j,  0.5+0.j ,  0. -0.j ,  0. +0.5j,
         0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.j ,  0. +0.j , -0. +0.5j,
         0. -0.5j,  0. -0.5j,  0.5+0.j , -0.5-0.j , -0.5+0.j ,  0. -0.j ,
        -0.5-0.j , -0.5+0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,
        -0.5+0.j ,  0.5+0.j , -0. +0.5j, -0. +0.5j,  0. +0.j ,  0. -0.j ,
         0. -0.5j,  0.5+0.j ,  0. -0.5j, -0.5+0.j ,  0. -0.5j, -0.5+0.j ,
         0. -0.5j, -0.5-0.j ,  0. -0.5j,  0. +0.5j,  0. +0.j ,  0. -0.j ,
         0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. -0.j ,  0. +0.j , -0. +0.5j],
       [-0. +0.j , -0. +0.j ,  0.5+0.j , -0. +0.5j,  0. +0.j ,  0.5+0.j ,
         0. +0.j ,  0. -0.j , -0.5-0.j ,  0. +0.j , -0. +0.j , -0.5-0.j ,
         0.5+0.j ,  0. -0.5j,  0.5+0.j ,  0. -0.5j, -0. +0.5j, -0. +0.j ,
        -0. +0.5j,  0. +0.5j,  0.5+0.j ,  0.5+0.j , -0. +0.j , -0.5-0.j ,
         0.5+0.j ,  0.5+0.j ,  0. -0.5j, -0. +0.5j,  0. -0.j ,  0. +0.j ,
        -0. +0.5j,  0. -0.5j,  0. +0.5j,  0. -0.5j,  0. -0.5j,  0. -0.5j,
        -0.5-0.j ,  0.5+0.j , -0.5-0.j ,  0.5+0.j , -0. +0.j ,  0. +0.j ,
         0. -0.j ,  0.5+0.j , -0.5+0.j ,  0. -0.j ,  0. +0.j ,  0. -0.5j],
       [-0. +0.5j,  0. -0.5j,  0. +0.j , -0. +0.5j,  0. -0.5j,  0. +0.j ,
        -0.5-0.j ,  0.5+0.j ,  0.5+0.j , -0. +0.5j, -0.5+0.j , -0.5-0.j ,
         0. +0.j , -0. +0.j ,  0.5+0.j , -0. +0.j ,  0. -0.5j,  0. +0.5j,
         0. -0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0.5+0.j , -0. +0.j ,
         0.5+0.j , -0. +0.j ,  0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.5j,
         0. +0.j ,  0. -0.5j, -0. +0.j , -0. +0.5j, -0. +0.5j,  0. +0.j ,
         0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. -0.5j,
        -0.5-0.j , -0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.5j],
       [-0.5+0.j , -0.5+0.j , -0. +0.j ,  0. +0.5j, -0.5-0.j ,  0. +0.j ,
        -0.5+0.j , -0.5-0.j ,  0. -0.5j,  0. -0.5j, -0.5+0.j ,  0.5+0.j ,
         0. +0.j , -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.5j,  0. +0.5j,
         0. -0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,  0. +0.j ,
         0. -0.5j,  0. -0.j ,  0. -0.j , -0.5+0.j ,  0. -0.5j, -0.5-0.j ,
        -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.5j, -0.5-0.j ,  0. +0.j ,
         0. +0.j , -0. +0.5j,  0. +0.j , -0.5+0.j ,  0. +0.5j,  0.5+0.j ,
         0.5+0.j , -0. +0.5j, -0. +0.j ,  0.5+0.j , -0.5-0.j , -0.5-0.j ],
       [ 0. -0.5j,  0. +0.5j, -0. +0.j , -0. +0.5j,  0. -0.5j,  0. -0.j ,
         0. -0.5j, -0. +0.5j,  0. -0.5j,  0.5+0.j ,  0. -0.5j,  0.5+0.j ,
        -0. +0.j , -0. +0.j , -0. +0.5j,  0. +0.j ,  0. -0.5j,  0.5+0.j ,
         0. +0.5j,  0. -0.j , -0. +0.j , -0.5-0.j ,  0. -0.5j,  0. +0.j ,
         0. +0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5-0.j ,  0. +0.5j,
        -0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j, -0.5+0.j , -0. +0.j ,
         0. -0.j ,  0. -0.5j, -0. +0.j , -0.5-0.j , -0.5-0.j ,  0. +0.5j,
         0. -0.5j,  0. +0.5j,  0. +0.j ,  0. -0.5j,  0. -0.5j, -0.5-0.j ],
       [ 0. -0.5j,  0. -0.5j,  0. -0.j ,  0.5+0.j ,  0. +0.5j, -0. +0.j ,
        -0.5-0.j , -0.5-0.j , -0. +0.5j,  0. +0.5j, -0.5-0.j ,  0. +0.5j,
         0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. -0.5j,
         0.5+0.j , -0. +0.j , -0. +0.j ,  0. -0.5j, -0.5+0.j ,  0. -0.j ,
         0. -0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j , -0. +0.5j,
         0. -0.j , -0.5-0.j ,  0. -0.j , -0.5+0.j , -0.5+0.j ,  0. +0.j ,
         0. -0.j ,  0. -0.5j,  0. -0.j ,  0. -0.5j,  0. -0.5j, -0. +0.5j,
         0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j, -0. +0.5j, -0.5-0.j ]],
      dtype=np.complex64))

In [ ]:
#@title Verification
n = 4
m = 4
p = 4
rank = 48

verify_tensor_decomposition(decomposition_444, m, n, p, rank)

##Rank-96 decomposition of <4,4,8> over 0.5*C


This decomposition can be obtained by doubling the rank-48 decomposition of <4,4,4> provided above.

## Rank-61 decomposition of <4,4,5> over Z

In [ ]:
#@title Data

decomposition_445 = (np.array([[-0.,  0., -0.,  0.,  0., -1.,  0.,  1., -0.,  0.,  0.,  0., -0.,
        -0., -0., -0., -1., -0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,
        -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  1., -0., -1.,
         1., -0., -1., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
         0., -0.,  1.,  0., -1., -1., -0.,  0., -1.],
       [ 0.,  1.,  0.,  0., -0.,  1., -1., -0.,  0.,  0., -1.,  0., -0.,
        -0.,  1., -0.,  0., -0., -1., -1., -0., -1., -1.,  0.,  0., -0.,
         0.,  0.,  1., -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  1., -1.,  0., -0.,  0.,  0., -1., -0.,  0., -0.,  0., -0.,
         0.,  0., -0., -0.,  0.,  0., -1.,  0., -0.],
       [-0., -0., -0.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0., -1.,  1.,
         0., -0., -0., -1.,  1.,  0.,  0., -0.,  0.,  0., -1.,  0.,  1.,
        -0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  1., -0., -1.,
         1., -0.,  0., -0.,  0., -1.,  1.,  0.,  0.,  0.,  0., -1.,  1.,
        -0., -0.,  1., -0., -1., -1.,  0.,  0., -1.],
       [-0., -0.,  0.,  0., -0., -0., -1., -0., -0.,  0., -1., -0.,  0.,
         0., -0., -0., -1., -0.,  0., -1.,  0.,  0.,  0., -0.,  0., -0.,
         0., -0.,  1.,  0., -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,
        -0.,  1., -1.,  0.,  0., -0., -1., -1.,  0.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.],
       [ 0., -0., -0.,  0., -1.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,
         0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0.,  0.,
         0., -0.,  1., -0., -0.,  0., -0., -0., -0., -0., -0., -0.,  1.,
        -0.,  0., -1., -0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  1.,  0.,
        -1., -0.,  0., -0., -1.,  0., -0.,  0.,  0.],
       [-0., -0., -0.,  0.,  1., -0.,  1., -0., -1., -1.,  0., -1., -0.,
         1.,  0.,  0.,  0.,  1., -0., -0.,  1., -0., -0.,  0., -1., -0.,
        -0.,  0., -0.,  1., -0.,  1., -0., -0.,  0.,  0.,  1.,  0.,  0.,
        -0.,  0.,  0., -0.,  1., -1., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0., -1.,  0., -0., -0.,  0.],
       [-0., -0.,  0., -0., -0., -0.,  1., -0.,  0.,  0., -0., -1., -0.,
        -0.,  0., -0.,  0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,
         1., -0., -0.,  1.,  0., -0.,  0., -0.,  0.,  0.,  1.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -1., -0., -0.,  0., -1., -0.,  0., -0.,
         0., -0., -0.,  0., -1.,  0.,  0.,  0.,  0.],
       [-0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -1.,  0.,  0.,
        -0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,
        -1., -0.,  1., -0., -0., -0., -0., -0.,  1., -0., -0., -0.,  1.,
         0., -0., -1.,  0., -0.,  0.,  0., -1.,  0., -1., -0.,  1., -1.,
        -1.,  0., -1.,  0.,  0.,  0.,  0.,  0., -0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -0., -1.,
         0.,  1.,  1.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,
         0.,  1., -0.,  0., -0.,  0.,  0., -1., -0.,  1.,  0., -0., -0.,
         0., -0., -1.,  0., -1.,  1.,  0.,  0.,  0., -1., -0.,  1.,  0.,
         0.,  0.,  0., -0.,  0., -0.,  0., -1., -0.],
       [-0.,  1., -0., -1., -0., -0.,  0., -0.,  0., -1.,  0.,  0., -0.,
        -0.,  0., -0.,  0.,  1., -0.,  0., -0.,  0., -0., -0., -1., -0.,
        -0., -1., -0., -0.,  1., -0.,  1.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  1., -0.,  1.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,
         0., -0.,  0.,  0., -0.,  0.,  0., -1., -0.,  1., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -1., -0., -0.,  0., -0., -1.,  0., -0., -0.,
        -0., -0., -0.,  1.,  0.,  0.,  0., -1.,  0.],
       [-0., -0., -0., -1., -0.,  0.,  0., -0., -0., -1.,  0.,  0.,  0.,
        -0., -0., -0., -1.,  1.,  1.,  0., -0., -0.,  1.,  0., -1.,  1.,
        -0.,  0., -0., -0.,  1., -0.,  1.,  0.,  0., -0., -0., -0., -0.,
        -0., -0.,  0., -0.,  0., -0.,  0.,  0., -1.,  0.,  0., -0., -0.,
         0., -0.,  0., -1.,  0.,  0., -0.,  0.,  0.],
       [-0., -0.,  0., -1.,  0.,  0., -0., -0., -0., -1.,  1., -0.,  0.,
        -0., -0., -0., -1., -0.,  1.,  0.,  0.,  1.,  0.,  0., -0., -0.,
        -0., -0.,  0., -0., -0.,  1.,  0.,  1., -0., -0., -0.,  0.,  0.,
         1., -1.,  0.,  0.,  0., -0., -0.,  0., -0.,  0., -1., -0., -0.,
         0.,  1.,  1.,  0., -1., -1.,  0.,  0.,  0.],
       [ 0.,  1., -1.,  0., -0., -0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,
        -0., -0.,  1.,  0., -0., -0., -1.,  0.,  0., -1., -0., -0.,  0.,
         0.,  0., -0., -0., -0., -0., -1., -0., -0., -0., -0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -0.,  0., -0., -0.,  0.,
         0., -1.,  0., -0., -0.,  0., -1.,  0.,  1.],
       [-1.,  1., -1.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
         0., -0.,  1.,  0., -0., -0., -1., -1., -0., -1., -0.,  1.,  0.,
         0., -0., -0.,  0.,  0., -0., -1.,  0.,  1., -0., -0.,  0., -0.,
        -0.,  0., -0., -1., -1.,  0.,  0., -1.,  1., -1.,  0., -0.,  0.,
         0.,  0., -0., -0.,  0.,  0., -1., -1.,  1.],
       [ 1., -0.,  0., -1., -0.,  0.,  0., -0.,  0., -1., -0.,  0., -0.,
         0.,  0., -0., -1., -0.,  1.,  0., -0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  0.,  1., -0.,  0.,  0.,  1., -0.,
        -0., -1.,  0., -0., -0.,  0.,  0., -1.,  0.,  0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0., -1.,  0., -0., -0.]], dtype=np.float32),
 np.array([[ 0., -0.,  0., -0., -0.,  1., -0., -0., -0.,  0., -1., -0.,  0.,
         0.,  1.,  0.,  0.,  0., -0.,  0.,  0.,  1., -0., -0., -0., -0.,
        -0.,  0.,  1.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0.,  1., -0., -0., -0.,  0., -0., -0.,  0.,  1.,  0.,  0.,
        -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.],
       [-0.,  0., -0., -0.,  1., -0., -0.,  0.,  1.,  0., -1., -0.,  0.,
        -1.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
        -0., -0.,  1., -0.,  0., -1., -0., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0.,  1.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,
         0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.],
       [ 1.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0., -1.,
         0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,  0., -0., -1.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  1.,  1.,  1., -0., -1., -0.,
         1.,  0., -0., -1.,  0., -0., -0.,  0., -1.,  0.,  0., -1., -0.,
         1., -0.,  1., -1., -0., -1.,  0., -0.,  0.],
       [ 0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  1., -1.,  0.,
        -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0., -1., -0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0., -1.,
        -0., -0., -1., -0., -0.,  1., -0., -0., -0.,  0., -0., -1., -1.,
         0.,  0.,  1., -0.,  1., -0., -0.,  0., -0.],
       [ 1.,  0.,  0.,  0., -0.,  0., -0.,  1., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -1., -0., -1.,
         0.,  0., -0.,  0., -0., -0., -0.,  1.,  1.,  1.,  0., -1.,  1.,
        -0., -0., -0., -1., -0.,  0.,  0., -0., -1.,  0., -1.,  0.,  1.,
         1.,  0., -0., -1.,  0., -1., -0.,  0., -0.],
       [-0.,  0.,  0.,  1.,  0., -0.,  0., -0., -0., -1.,  0.,  0., -0.,
         0., -0.,  0.,  0.,  0.,  1.,  0., -0.,  1., -0., -0., -0., -0.,
        -0., -0., -0.,  0., -0.,  1.,  0., -0.,  0., -0., -0., -0., -0.,
         0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  1.,  0.,  0.,
        -0., -1.,  0.,  0.,  0., -0.,  0., -0., -0.],
       [ 0.,  0., -0.,  1., -0.,  0., -0.,  0.,  1., -1.,  0.,  0., -0.,
        -1., -1., -0.,  0.,  0.,  1., -0., -0., -0., -0.,  0., -0.,  0.,
         0.,  1., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.],
       [-0., -1.,  0.,  1., -0.,  0., -0.,  0., -0., -1.,  0., -0.,  0.,
         0.,  0.,  1., -0., -0.,  1., -0.,  0., -0.,  1.,  0., -1.,  0.,
         0., -0., -0.,  0., -0., -0.,  1., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0., -1.,  0.,  0., -0., -1.,  0., -0.,  0., -0.,
        -0.,  0., -0.,  0.,  0.,  0., -0.,  1.,  0.],
       [-0.,  0., -0., -0., -0.,  0., -1.,  0.,  0., -0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0.,  0., -1., -1.,  0.,  1.,  1., -1.,  1.,
         1.,  0., -0.,  1., -1.,  0.,  0.,  0., -1.,  0., -0.,  0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  1., -0., -0., -0.,  0.,  0., -1.,
         0., -0.,  0.,  0., -0., -0.,  1., -0., -0.],
       [-0.,  0., -1., -0.,  0.,  0.,  1., -0., -0.,  0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  1., -0.,  0., -0., -1., -0., -1.,
        -1., -0.,  0., -1.,  1.,  0.,  1., -0.,  1., -0., -0., -0., -0.,
         0.,  0.,  0., -1.,  0., -0., -1., -0., -1.,  0.,  0.,  0.,  1.,
        -0., -0.,  0.,  0., -0., -0.,  0., -0., -0.],
       [ 0.,  0.,  1.,  0., -0., -1.,  0., -1.,  0., -0., -0.,  0., -0.,
        -0., -1.,  1., -0.,  1.,  0.,  0.,  0., -1., -0.,  0., -1., -0.,
        -0., -0.,  0.,  1., -1., -1., -0.,  0.,  0., -1., -1.,  0., -0.,
         0.,  0.,  0.,  0., -1., -1., -0., -0.,  0.,  1., -1.,  1., -0.,
        -1.,  1.,  0., -0.,  0., -0., -0.,  0., -1.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -1., -0., -0.,  0., -0.,
         1., -0., -0.,  0.,  1.,  0.,  0.,  0.,  0., -0.,  0., -1., -0.,
        -0.,  0.,  0., -0., -1., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0., -0.,  0.,  0., -1., -1.,  0.,  0., -0.,  1.,  0.,  1.,  0.,
        -1., -0.,  0., -0.,  0.,  0., -0.,  0.,  0.],
       [ 0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  1.,
        -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  1.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0., -0., -0.,
         0., -0., -0., -0.,  0.,  0.,  0., -0.,  1.,  1.,  0.,  1.,  0.,
        -1., -0.,  0.,  1., -0., -0., -0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  1., -0.,
         0., -0.,  0., -0., -1.,  0., -0.,  1., -0.,  0.,  0.,  1.,  0.,
        -1.,  0., -0., -1.,  1., -0., -0.,  0.,  1.,  0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  1.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.],
       [ 0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
        -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  1.,  0.,  1.,
         1., -0., -0., -0.,  0., -0.,  0.,  0., -1.,  0., -0., -0.,  0.,
        -0.,  0., -0.,  1., -0., -0., -0.,  0.,  1., -0., -0., -0., -1.,
         0.,  0.,  0.,  1.,  0., -0.,  0.,  0., -0.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  1., -0.,  0.,
        -0.,  0.,  0.,  0.,  0., -1.,  0., -0., -1., -0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  1.,  0.,
         0., -1.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,
        -0., -0.,  0., -0.,  0., -0., -0., -0.,  0.],
       [ 0., -0.,  0., -1., -1., -0., -0., -0., -1., -0.,  1.,  0.,  0.,
         1.,  1.,  0.,  0.,  0., -1.,  0., -0.,  0., -0., -0., -0., -0.,
         0., -1., -1.,  0.,  1.,  1., -0.,  0.,  0.,  0.,  0., -0., -0.,
         0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,
         1.,  0.,  0.,  0., -0., -0., -0.,  0., -0.],
       [-1., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0., -1., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,
         0.,  0.,  0., -0., -0., -0.,  0.,  0., -1.,  0., -0.,  1., -0.,
        -1.,  0., -0.,  1., -0., -0., -0.,  0.,  1.,  0., -0., -0.,  0.,
         0.,  0., -1., -0., -0.,  1.,  0.,  0.,  0.],
       [-0.,  0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,
         0.,  0., -0.,  0., -0., -0.,  1., -0., -0., -1., -1., -0., -1.,
         0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,
         0., -0., -0.,  0.,  0.,  0., -1., -1.,  0., -0.,  0., -0.,  1.,
         0., -0.,  0., -0.,  0., -0., -1., -0.,  0.],
       [-1.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  1., -0.,  1.,
        -0.,  0.,  0., -0.,  0., -0.,  0.,  0., -1.,  0.,  0., -0.,  0.,
         0., -0.,  0.,  1.,  0.,  0.,  1., -0.,  1.,  0., -0.,  0., -1.,
         0., -0.,  0., -0.,  0., -0., -0., -0., -0.]], dtype=np.float32),
 np.array([[ 0.,  0.,  0., -0.,  0., -1., -0., -1., -0.,  0., -0.,  0., -0.,
         0., -0.,  0.,  0.,  0., -0.,  0.,  0., -1., -0.,  0., -0.,  0.,
         0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0., -0.,  1.,  0.,
        -1., -1., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -1., -0., -0.,
         0.,  0., -0., -0., -0.,  1.,  0., -0., -0.],
       [ 0.,  0., -0., -0.,  1., -0.,  0.,  1., -0.,  0., -1.,  0.,  0.,
        -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  1.,  0., -0.,  1., -0., -0.,  0., -0., -1., -1., -0.,
         1.,  1., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  1.,  0.,  0.,
        -0.,  0.,  0., -0., -1., -1.,  0., -0., -0.],
       [ 0.,  1., -0., -1.,  0., -0., -0.,  1.,  0.,  0., -0., -0.,  0.,
        -0.,  1.,  1.,  0.,  0., -1.,  0.,  0.,  1.,  0., -0.,  0., -0.,
        -0.,  1.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -1., -0.,
         1., -0., -0., -0.,  0., -0.,  0., -0., -0., -0.,  1., -0., -0.,
         0.,  0.,  0., -0.,  0., -1., -0.,  0.,  1.],
       [ 0.,  0., -0.,  0.,  0., -0.,  0., -1.,  0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,
        -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0.,
        -1.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -1., -0., -0.,
        -0.,  1., -0., -0., -0.,  1., -0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -0.,  1., -0.,  1., -1.,  0., -0.,  0., -1.,
         1., -1., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,
        -0., -0., -1., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  1.,
        -0., -0., -1.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -1., -0.,
         1., -0.,  0.,  0., -0.,  0.,  0., -0., -0.],
       [ 0., -0.,  0.,  0., -1.,  0.,  0., -1., -0.,  0., -0.,  0.,  1.,
        -1., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0.,  1., -0., -1.,
         0.,  0., -0., -0., -0.,  1., -0., -0.,  0., -0.,  0.,  1.,  0.,
        -1., -0.,  0.,  0., -0., -0.,  0., -0.,  0.],
       [-0.,  0., -1., -0., -0.,  0.,  0.,  0.,  1.,  0.,  0., -0., -0.,
        -1.,  0., -1.,  0.,  0., -0.,  0.,  1.,  0.,  0., -0.,  1., -0.,
        -0., -1., -0.,  0.,  1., -0., -1., -0., -0.,  0.,  0., -0.,  0.,
         0., -0., -0., -0.,  1., -0.,  0.,  0., -0., -0.,  0., -0., -0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.],
       [-0.,  0.,  1.,  1., -0.,  0.,  0., -0.,  0.,  1., -0., -0.,  0.,
         1., -0., -0.,  0., -0., -0., -0., -1.,  0.,  0.,  0., -1., -0.,
         0.,  0., -0.,  0., -1., -1.,  1.,  0., -0.,  0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,
         0., -1., -0., -0.,  0.,  0., -0.,  0., -0.],
       [-0., -0., -0.,  0., -0., -0.,  0., -1., -0.,  0., -0., -0.,  0.,
         0., -0., -1.,  1., -0., -0., -0., -0., -0., -1.,  1.,  0.,  1.,
         0.,  0., -0.,  0., -0., -0.,  0., -1.,  0., -1.,  0.,  0.,  0.,
         0., -0., -0.,  1., -0., -0., -0.,  0.,  1., -0., -0., -0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  1., -1., -1.],
       [-0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0., -1.,
        -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        -1.,  0., -0.,  0., -1., -1., -0., -0.,  0., -1.,  0., -1., -0.,
        -0., -0.,  1., -0.,  1., -0.,  0., -1.,  0.],
       [-0., -1.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  1.,
        -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  1., -1.,  0., -1.,
         0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
         0., -0., -0., -1.,  0.,  0., -0.,  0., -1., -0.,  0., -0., -0.,
        -0., -0.,  0.,  0.,  0.,  0., -1.,  1.,  0.],
       [-0., -0.,  0.,  0., -0., -0., -0.,  1.,  0.,  0.,  0.,  0., -0.,
         0., -0.,  1.,  0., -0., -0.,  0., -0., -0., -0., -0., -0.,  0.,
         0., -0., -0., -0., -0., -0.,  0.,  1.,  0.,  1., -0.,  0.,  0.,
         1.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0., -1.,  0.,  1.,  1.],
       [ 0.,  0., -1., -0., -0.,  0.,  1., -0.,  0., -0., -0., -1.,  0.,
        -0.,  0., -0.,  0., -0.,  0., -1.,  0.,  0., -0., -0.,  0.,  0.,
        -0.,  0.,  0.,  1., -0.,  0.,  0., -0.,  0., -0.,  1., -0., -0.,
        -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -1., -0.,  1.],
       [ 0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -1., -0.,
         0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0., -0.,  1.,  0., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0., -0., -1.,  0.,  0.,  0., -0.,
         1., -0.,  0.,  1., -0., -0.,  0.,  0., -0., -0., -0., -0., -1.,
         0., -0., -1., -0., -1.,  0., -0., -0., -0.],
       [-0.,  1., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0.,
         0., -0., -0.,  0., -1.,  0., -0.,  1., -0., -1., -0.,  1., -0.,
        -0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0.,  1.,  1.,  0.,  0., -0., -0., -0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  1.,  0., -0.],
       [ 0., -0.,  1., -0., -0.,  0., -1.,  0., -0., -0.,  0.,  1.,  0.,
        -0., -0.,  0.,  0., -0.,  0.,  1., -0.,  0., -0., -1.,  0.,  0.,
        -0., -0., -0., -1.,  0.,  0., -0.,  0.,  1.,  0., -1., -0., -0.,
        -1., -0., -0., -1.,  0.,  0.,  0.,  1.,  0., -0.,  0., -0.,  1.,
        -0.,  0.,  1., -0., -0., -0., -0.,  0., -1.],
       [-0.,  0., -1.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0., -1., -0.,  0., -0., -1.,  0.,  0.,
         0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -0., -1., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -1., -0.,  1.],
       [-0.,  0.,  0.,  0., -0., -0.,  0.,  1.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0., -0.,  0., -0., -1.,  0., -0., -0.,  0.,  0.,
         1.,  0., -0., -1.,  0., -0.,  0.,  0., -1.,  0., -1.,  0.,  1.,
         1., -0.,  0.,  1., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,
        -0., -0., -1.,  0., -1.,  0., -0., -0., -0.],
       [-0.,  1.,  1.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -1.,
        -0., -0.,  1.,  0.,  0.,  0., -0., -0., -0., -1.,  1., -0.,  1.,
         0., -0.,  0., -0.,  0.,  0.,  1., -0., -0.,  1., -0.,  0.,  0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,
         0., -0., -0.,  1., -0.,  0.,  1.,  0., -0.],
       [-1.,  0.,  1., -0.,  0., -0.,  0., -1.,  0., -0.,  0., -0., -0.,
        -0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,
         0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,
        -1.,  0., -0., -1., -0., -0., -0., -0., -0., -0.,  0., -0., -0.,
         0., -0., -0.,  0., -0.,  1., -0.,  0., -1.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 4
m = 4
p = 5
rank = 61

verify_tensor_decomposition(decomposition_445, n, m, p, rank)

## Rank-85 decomposition of <4,4,7> over Z


In [ ]:
#@title Data

decomposition_447 = (np.array([[ 0., -0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,  1., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0., -1., -0.,  0.,  0., -0.,
        -0.,  1.,  0., -0.,  0.,  0., -1., -0., -1., -0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -0.,  0., -0.,  1.,  0.,  0., -0.,  1.,
         0.,  1.,  1., -0., -0., -1., -1.,  0., -0.,  0.,  0., -0., -0.,
         0.,  0.,  1., -0.,  0., -0.,  0., -1., -0., -0.,  0., -1., -0.,
         0.,  0.,  0., -0., -0., -0.,  1.],
       [ 0.,  0.,  0., -0.,  0., -1.,  0.,  0.,  0.,  0.,  1.,  0., -1.,
        -0.,  1.,  1., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0., -0.,
         0., -1.,  1., -1.,  0.,  1., -1.,  0., -0., -0., -0., -0., -0.,
         1., -0.,  0.,  0.,  0., -0., -1., -1., -0.,  0., -0.,  1.,  0.,
         0., -0.,  1.,  0.,  0.,  0., -1., -0., -0., -0.,  0., -0.,  1.,
         0., -0., -1.,  1.,  0.,  1.,  0.],
       [ 0., -0., -0., -0.,  0.,  1.,  0.,  0., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
         0., -0.,  0., -1.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
        -0.,  1.,  0., -0., -0., -1.,  1.,  0.,  0.,  0.,  0., -0., -0.,
        -1., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -1.,  0.,
         1., -1.,  0.,  0.,  1.,  0.,  0., -0.,  0., -0., -0., -0., -1.,
         1., -0., -0.,  0.,  1., -1., -0.],
       [-0., -0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,
        -0., -1.,  0., -0., -0.,  0., -0.,  0., -1., -0.,  0., -0., -0.,
         0., -0., -0.,  1., -0., -1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,
        -0.,  0., -1., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,
        -0., -0.,  1., -0.,  0., -1., -0.,  1.,  0., -0.,  0.,  0.,  0.,
        -1., -0., -0., -0., -1.,  0.,  1., -1.,  0., -0., -0., -1., -0.,
        -1.,  0.,  1., -1.,  0., -0., -0.],
       [ 0.,  0., -0., -0., -1., -0., -0., -0.,  1.,  1.,  0.,  0.,  0.,
         0., -0.,  0., -0.,  0.,  1.,  0.,  0., -0., -0., -0.,  0., -0.,
         0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1., -0., -0.,  0., -0.,
        -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,  1.,  0.,
        -0.,  0.,  0., -1., -1., -0., -0., -0.,  0.,  0., -0., -0.,  0.,
        -0., -0.,  0., -1.,  0., -0., -0., -0., -1., -1.,  0., -0., -0.,
        -0.,  0., -0.,  0.,  0., -0.,  0.],
       [-0.,  0., -1., -0.,  0., -0.,  0.,  0.,  1.,  0.,  0., -1.,  0.,
        -0.,  0., -0., -1., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,
        -0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,
         0.,  0.,  0., -0., -0., -1., -0., -0.,  0., -1., -1., -0.,  0.,
        -1.,  0., -0.,  0., -0., -1.,  1.,  1.,  1.,  1.,  0.,  0., -1.,
         0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
        -1., -0., -0., -1., -0.,  0.,  0.],
       [-0., -0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  1.,  0.,
        -0.,  0., -0.,  0.,  1.,  0., -0., -0.,  0., -1., -1., -0., -0.,
        -0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
        -1., -0.,  0., -0.,  1., -0.,  0.,  0.,  0.,  1.,  0., -0., -0.,
         1., -0., -0.,  0., -1.,  0.,  0., -1., -0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -0.,
        -0., -0.,  0., -0.,  0., -0., -0.],
       [-0.,  0.,  1., -0.,  0., -0., -0., -0., -0.,  1.,  0.,  0., -0.,
        -0., -0.,  0.,  1., -1.,  1.,  0.,  0., -0.,  1.,  1., -0., -0.,
        -1.,  0., -1., -1.,  0., -0., -0.,  0.,  1., -0.,  1., -1., -0.,
         1.,  0., -0., -0., -1., -0.,  0.,  0.,  0., -0.,  1., -0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0., -0., -1., -1., -0.,  0., -0.,
         1.,  0.,  0.,  0.,  0.,  1., -0.,  1.,  0.,  0., -0.,  1.,  0.,
         1.,  0., -0., -0., -0.,  0.,  0.],
       [ 0.,  0.,  0., -1., -0.,  0., -0., -0.,  0., -1., -1.,  0., -0.,
         0.,  0.,  0., -1., -0.,  0.,  1.,  1.,  1.,  0.,  0.,  0., -1.,
        -0.,  0.,  1., -0.,  0., -0., -0., -0., -0., -1.,  0.,  1., -0.,
        -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0., -0.,  0., -0., -0., -0., -1., -0., -0.,
         0.,  0.,  0., -0.,  0., -0.,  0.],
       [-0.,  1.,  1., -0.,  0.,  1.,  0., -0.,  0., -0., -1.,  1.,  1.,
        -0., -1.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,
         0.,  1.,  0.,  1.,  0.,  0., -0., -1., -0.,  0., -0.,  0., -0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0., -1.,  0., -1., -1., -0.,
        -0.,  0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,
         1., -0.,  1.,  0., -0.,  0.,  0.],
       [-0., -1., -1.,  0.,  0., -1.,  0.,  0., -0., -0., -0., -1.,  0.,
        -0., -0., -0., -0., -1., -0., -0.,  0.,  0.,  1., -0., -0.,  0.,
         0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -1.,  0.,  0.,
        -0., -1.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0.,  0., -0., -0., -0.,  0., -0.,  1., -0., -0.,  1., -0.,  0.,
        -1.,  1., -0.,  0., -1.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0.,
        -1., -0., -0.,  0.,  0., -0., -0.],
       [-0., -0., -0.,  0., -0., -0.,  0., -0., -0., -1.,  0., -0.,  0.,
        -0., -0., -0., -1.,  1.,  0., -0.,  1.,  1., -1.,  0., -0.,  0.,
         1.,  0.,  1., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  1.,  0.,
         0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1., -0.,  0., -0.,  0.,
        -0.,  0., -0., -0.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -1., -0., -0., -0.,  0.],
       [-0., -0., -0.,  0.,  0.,  0.,  1.,  1., -0.,  0., -0., -0., -0.,
        -0.,  0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,
        -0.,  1., -0.,  0., -1.,  0.,  0.,  0., -0.,  0., -0., -0.,  1.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1., -0.,
        -0.,  1., -0.,  0.,  0.,  0., -1.,  0., -0., -1., -0.,  0.,  0.,
         0., -0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,
         0., -1.,  0.,  0., -0., -0., -0.],
       [ 0.,  1., -0.,  1., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
         1., -0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  1.,  0.,
        -0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,  1.,
         0., -0., -0.,  0.,  0., -0.,  0., -1., -0.,  0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -1.,
        -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,  1.,  0., -0., -0.,
        -0., -0.,  0., -0.,  0.,  1., -1.],
       [ 0., -1., -1.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
        -0., -0., -0., -0., -1., -0.,  0., -1., -0.,  1., -1.,  0., -0.,
         0.,  0., -0., -0.,  1., -1., -0., -0., -1., -0.,  0., -0.,  0.,
         0., -0.,  0.,  1.,  0., -0.,  0.,  0., -0., -0., -0., -1., -0.,
         0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -1.,
         0., -0., -0., -0.,  1., -1., -0.],
       [ 1., -0.,  1.,  0., -0., -0., -0.,  1.,  0., -0.,  0., -0., -0.,
         0., -1.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  1.,
         0., -0.,  0.,  0.,  0., -1.,  1., -0., -0.,  0.,  1.,  0., -0.,
         1.,  0., -1., -0., -1., -0., -0., -0., -0., -0., -0., -0.,  0.,
         0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -1., -0., -0., -0.,
        -0.,  0., -0.,  1.,  0.,  1., -0., -0., -0.,  0.,  0., -0., -0.,
        -0., -1., -0., -1.,  0., -0., -0.]], dtype=np.float32), np.array([[ 0., -0., -0., -1., -0.,  0.,  0., -0., -0.,  1., -0.,  0., -0.,
         0.,  0., -0., -0., -0.,  0.,  1., -0.,  0.,  0.,  0., -1., -0.,
        -1.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -1.,  1.,
         0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,
         0., -1.,  0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,
        -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  1.,  1.,  0.,  0., -0.,
         0.,  0., -0.,  0.,  0., -0., -1.],
       [-0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  0., -0.,  1., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  1.,  0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,
         0.,  1., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,
        -1.,  1., -0.,  0.,  1., -0., -0., -1.,  0.,  0.,  0., -0.,  0.,
         0.,  0.,  0., -0., -0.,  0., -0.],
       [-0., -0., -0., -0., -1.,  0.,  1.,  0., -0., -1.,  0., -0., -0.,
        -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  1., -0., -0., -0.,  0.,
         1., -0.,  0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,  1., -0.,
        -0., -0., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,
        -1.,  1.,  0.,  0.,  1.,  0., -0., -1.,  0.,  0., -0., -0., -0.,
         0., -0., -0., -0., -0., -0., -0.],
       [ 0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  1., -0.,  0.,
         0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0., -0.,
         0.,  1.,  0., -0., -0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -1., -0.,  0.,  0.,  0., -0.,  0.,  0., -1., -0.,
        -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,
         0., -0.,  1., -0., -0., -0.,  0.,  0., -1.,  0., -0.,  0., -0.,
        -0.,  0., -0., -0.,  0.,  0., -0.],
       [ 0., -0., -0., -0.,  0., -0.,  0., -0., -0., -1., -1.,  0., -0.,
        -0., -0.,  0.,  0., -0.,  0., -1., -0., -0.,  0.,  0., -0.,  0.,
         1.,  0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  1.,  0.,
        -0., -0.,  0.,  1.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
         0., -0., -1., -0.,  0., -0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,
         0.,  0.,  0.,  0., -0., -0., -0.],
       [-0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  1.,  0.,  0.,
        -0.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -0., -1., -0., -0., -1., -0., -1.,  0.,  0., -0.,
        -0.,  0., -0., -1., -0.,  0.,  0.,  0., -1.,  0.,  0., -0., -0.,
         0.,  1., -0.,  1., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,
         0., -0.,  1.,  0., -0.,  0., -0., -0.,  1., -0., -1.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  0.],
       [-1., -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -0., -0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  1.,
        -0.,  1., -0., -0.,  0.,  0.,  1.,  0.,  0.,  0., -0., -0., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -1.,  0.,
         0., -1., -0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0., -0.,
         0., -0., -0.,  1., -0.,  0., -0.,  0.,  0., -0.,  1.,  0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  0.],
       [-0., -0., -0.,  1., -0., -0., -0., -0., -1.,  0.,  0., -0., -0.,
        -0.,  0., -1., -1., -0., -0., -1.,  0., -0., -0., -0.,  1., -0.,
         1., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  1.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0.,  0., -0.,  0.,
         0., -0., -0.,  0., -0.,  0., -0.,  0., -1., -1., -0., -0., -0.,
        -0., -0., -0.,  0., -0., -0., -0.],
       [-0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -0.,  1., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,
        -0., -1., -0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -1.,
        -0.,  0.,  0., -0., -0.,  1., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1., -1.,  0.,  0.,  0., -0., -0.,  1., -0., -0., -0.,  0.,  0.,
         1., -0.,  0., -0.,  0.,  0.,  1.],
       [-0., -1., -1.,  0., -0., -0., -1.,  0., -0., -0.,  0.,  0., -1.,
        -0., -0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  1.,  0., -1.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
         0.,  0., -0., -0., -0., -0., -0., -0.,  0., -1.,  0., -0., -0.,
        -0.,  0.,  0.,  0.,  0.,  1.,  0., -0., -0., -0., -0., -0., -0.,
        -0.,  0.,  0., -0., -0., -0., -0.],
       [-0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0.,  0.,  0.,  0.,
         0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,
        -0., -1.,  0., -0., -0., -0.,  0., -0., -0., -0., -0., -0., -0.,
         0.,  0.,  0.,  1.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0.,  0.,  0., -0., -0.,  0.,  1.,  0., -0.,  0., -0., -0.,  1.,
        -0.,  0., -1., -0.,  0., -0., -0., -0.,  1.,  1., -0., -0., -0.,
         0.,  0.,  0.,  0., -0., -0.,  0.],
       [-0., -0., -0., -1., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  1.,  0., -0.,  1.,  0., -0., -0.,  0.,  0., -0.,
        -1., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -1., -0.,
         0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0., -0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -1., -0., -0., -0., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0., -0.,  0.],
       [ 0., -0., -0.,  0., -0., -0., -0., -0., -1.,  0.,  0.,  0., -0.,
        -1., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0.,  0., -0.,  1.,  0.,  1.,  0., -0., -0., -0., -0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -1., -1.,
        -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -1., -1., -0., -0., -1.,
        -0., -0.,  0., -0., -1., -1., -0.],
       [-0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
        -1., -1.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0., -1., -0., -0.,  0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0., -1., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,
         0., -0., -0., -0., -0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -0.,
         0., -0., -1.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,
         0.,  0., -0., -1.,  0.,  0.,  0.],
       [-0.,  0., -1.,  1.,  0., -0., -0.,  0., -1.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -1., -0., -0., -1., -0., -0., -0., -0.,  1.,  0.,
         1., -0., -0., -1., -1., -0., -0.,  0., -0.,  0.,  0.,  1., -1.,
         1.,  0.,  0., -0.,  1.,  0., -0., -0.,  1., -1., -0.,  0., -0.,
        -0.,  0., -0., -0.,  0.,  1.,  0.,  0.,  1., -1., -1.,  0.,  0.,
         0., -0., -0.,  0., -0.,  0.,  0.,  0., -1., -1., -0.,  1., -0.,
         1., -0., -0.,  0.,  1.,  0.,  1.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,
         0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
         0., -0.,  0.,  1., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  0., -0., -1., -0., -0.,  0., -1.,
         0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,
         1., -1., -0., -0.,  0., -0., -0.,  1., -0., -0.,  0., -1.,  0.,
        -0.,  0.,  0., -0., -1.,  0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  0., -1., -0., -0., -0., -0.,  0.,  0.,
         0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,
         0.,  0.,  0., -0.,  1.,  0., -0.,  0.,  0., -0.,  1.,  0.,  0.,
        -1., -0., -0.,  0., -1., -0.,  0., -0.,  0., -0., -0.,  0., -0.,
         0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,
        -0., -1.,  0., -0.,  0.,  1., -0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0.],
       [-0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -1.,  0.,
         0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0., -1., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,
         0., -0.,  0.,  1.,  0.,  1., -0., -0.,  0., -0., -0., -0.,  0.,
        -1., -0., -0.,  1.,  1.,  0.,  1.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0., -1., -0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0.,  0.,  0.],
       [-0.,  0., -0.,  0.,  0., -1., -0.,  0., -0., -0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -1., -0.,  1.,  0.,  0.,  0., -0.,  0., -0.,
        -1., -0., -1., -0., -0., -0.,  0.,  0., -0.,  1.,  0., -1., -0.,
         0., -0.,  0., -1.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  1.,  0.,
        -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,
         0., -0., -0.,  0., -0.,  0.,  0.],
       [ 0.,  0., -0.,  0., -0.,  1., -0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0.,  1., -0., -0.,  0., -0., -0.,
         0.,  0., -0.,  1., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -1., -1.,  0.,  0., -0.,  0., -0., -0., -1.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -1.,  0., -0., -0.,  0.,
         0., -0.,  0.,  0., -1.,  0.,  0.],
       [-0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0., -1.,  0.,
         0., -1.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0., -1.,  0.,  0.,  0.,  1.,  0.,  1., -0., -0., -0.,  0.,  0.,
        -0.,  0., -1., -0., -0., -0.,  1.,  0., -0.,  0., -0., -0., -0.,
         0., -0., -0., -0., -0., -0.,  1., -1., -0.,  0.,  0.,  0., -0.,
         0., -0., -1.,  0.,  0.,  0., -1.,  0., -0., -0.,  0.,  0.,  1.,
        -0.,  0.,  1., -1.,  0., -0., -0.],
       [-0.,  0.,  0.,  1.,  0.,  0., -0.,  0., -1.,  0.,  0., -0.,  0.,
        -0., -0.,  0., -1., -0., -0., -1., -0., -0., -0.,  0.,  1.,  0.,
         1., -0., -0.,  0.,  0.,  1.,  1.,  0., -0.,  0., -0.,  1., -1.,
        -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,  1.,  0., -0.,
        -0.,  0.,  1., -0., -0.,  1.,  0., -0., -0., -1.,  0.,  0.,  0.,
         0., -0., -0.,  0., -0.,  0.,  0.,  0., -1., -1., -0., -0.,  0.,
        -0.,  1.,  0., -0., -0.,  0.,  1.],
       [ 0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,
        -0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,
         0., -0.,  0., -0., -0., -1., -1., -0.,  0., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -1.,
        -0.,  0., -1.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         1., -1., -0., -0., -1., -0., -0.,  1., -0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0.,  0., -0., -0.],
       [ 0.,  0.,  0.,  0., -0., -0., -1., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -1.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
         0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,
         1., -1.,  0., -0., -1.,  1., -0., -0., -0., -0., -0.,  0.,  0.,
        -0., -1., -0.,  0., -0.,  0.,  0.],
       [-0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -1.,  0.,
        -0.,  0., -0., -0.,  0., -1.,  0., -0.,  0., -1.,  0.,  0.,  0.,
        -0., -1., -0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,
        -0., -0.,  0.,  1.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,
         0.,  0., -0., -0., -0.,  0.,  1., -1.,  0., -0., -0., -0., -0.,
        -0.,  0., -1.,  1.,  0., -0.,  0., -0.,  1., -0., -0.,  0., -0.,
         0.,  0., -0., -1., -0., -0.,  0.],
       [-0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,
         0., -1.,  0., -0.,  0., -0.,  1., -1., -0., -0.,  0.,  0.,  1.,
        -1.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -1., -0.,
        -0.,  0.,  0., -1., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
         0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,
         0.,  1.,  0., -0.,  0., -0.,  0.,  1.,  0., -0.,  1.,  0., -1.,
        -0., -0., -1.,  0.,  1., -1., -1.,  1., -0.,  1., -0., -0.,  0.,
        -1., -0., -0.,  1.,  0., -0., -0.,  0.,  1., -0.,  0.,  0.,  0.,
        -0.,  0., -1., -1., -1.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,
         0., -0., -0.,  0.,  0.,  0., -0., -0., -1.,  0.,  0.,  1.,  0.,
        -0.,  1., -1., -0.,  0.,  0.,  0.],
       [ 1.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -1.,  0.,
         0., -1.,  0.,  0., -0.,  0., -0.,  0.,  0., -1.,  0., -0.,  0.,
         0., -1., -0., -0.,  0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,
        -0., -0., -0.,  0.,  0., -0.,  1., -1., -0.,  0., -0.,  0.,  0.,
         0., -0., -1., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -0.,  1., -1., -0., -0., -0.]], dtype=np.float32), np.array([[-0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,
        -0., -0., -1., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0.,
        -0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,
        -0.,  1., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  1., -0., -0.,
         0., -0.,  0.,  0.,  0., -1.,  0., -0.,  0.,  0., -1.,  0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
         1.,  0., -0., -0.,  0.,  0., -1.],
       [-0., -0.,  0., -0.,  0.,  0.,  0., -0., -1.,  0., -0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -1., -0.,
         0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,
        -0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -1.,  1., -0., -0.,
         0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -1.,
        -0.,  0.,  0., -0., -0., -0., -0., -0., -0., -1.,  0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -0.,  0.],
       [-0., -0.,  0.,  1., -0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0.,  1., -0.,  0.,  0.,  0., -0., -0., -0., -1.,  0.,
        -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0., -0., -0., -0., -0.,  1.,  0.,  0.,  1.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  0., -1.,  0., -1.,  0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  1.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,  0., -0.,
        -0., -0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  1., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  1.,
         0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -1.,  0.,  0.,
        -0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,  1.,  1.,  0., -0.,
        -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0., -0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -1.,  0., -0.,
        -0.,  0.,  0., -1., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  1.,  0.,  1.,
         0.,  0., -1., -0., -0., -1., -0., -0., -0.,  0., -1., -0., -0.,
         0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  1., -0.,
         1., -0., -0.,  0., -0., -0.,  0.],
       [ 0., -0.,  0.,  0., -1.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,
        -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -1.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  1., -0.,  0.,  0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -1.,  1.,  0., -0.,
        -0.,  0., -1., -0.,  0., -1., -0., -0., -0.,  0.,  0., -0.,  0.,
         0., -0., -0.,  0.,  0., -0.,  0.,  1.,  0.,  0.,  0.,  1., -0.,
        -0.,  0., -0., -0., -0., -0., -0.],
       [ 0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,
        -0.,  0.,  0., -0.,  0., -0., -1.,  0.,  1.,  0., -1., -0.,  0.,
         1.,  0., -0., -1., -0.,  0., -0.,  0., -0.,  0.,  0., -1.,  0.,
         0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  1., -0., -0.,
         0.,  0., -1.,  0., -0., -1.,  0., -0.,  0.,  0., -1.,  0.,  0.,
        -1., -0., -0.,  0., -1., -0.,  0.,  1., -0.,  0.,  0.,  1., -0.,
         1.,  0.,  0., -0., -0., -0., -0.],
       [-0., -1.,  1., -0., -0.,  0.,  0.,  1.,  0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  1., -0.,  0.,
         0.,  0., -0., -0., -1.,  0., -1., -0.,  0.,  0.,  0., -0.,  1.,
         1.,  0.,  0.,  0.,  1., -0.,  0.,  0.,  1.,  0., -1., -0., -1.,
        -0.,  1., -0., -0.,  0., -0., -0.,  0., -0.,  1.,  1., -0., -0.,
        -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
         0.,  0., -0.,  0., -0., -0., -1.],
       [-0., -0., -1., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  1.,
         0., -0.,  0., -0., -0., -0., -0., -0., -0., -0.,  1.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -1., -0., -0.,
         1., -0., -0., -0.,  1., -0.,  0., -0.,  0., -0.,  1.,  0., -1.,
         0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -1., -1.,  0., -0.,
        -1.,  1.,  0., -0.,  0.,  1., -0.,  1.,  0., -0., -0.,  0.,  0.,
        -0., -1., -0.,  0.,  0.,  0.,  0.],
       [-0., -0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,
         0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
         1.,  0., -0., -0., -0.,  0.,  0., -0., -0., -1.,  1.,  0.,  0.,
         0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -1.,  0., -0.,  0.,
         0., -0.,  0., -0.,  0.,  1., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0., -1., -0.,  0.,  0., -0., -0.],
       [-0., -0., -1., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,
         0., -0., -0., -0., -0.,  0.,  1., -0.,  0., -0.,  1.,  0.,  0.,
        -1.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -1.,  1.,  0.,
         1.,  0.,  0., -0.,  1., -0.,  0., -0., -0.,  0.,  1.,  0.,  0.,
        -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -1., -1., -0., -0.,
         0., -0.,  0., -0., -0.,  1.,  0., -0.,  0., -0., -0.,  0.,  0.,
         0., -1.,  0.,  0., -0.,  0., -0.],
       [-0., -1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -1., -0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
        -1.,  0.,  0.,  0., -1., -0., -0., -0.,  0.,  0., -1., -0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,  1., -0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,
        -0.,  1., -0.,  0., -0.,  0., -0.],
       [-0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -1., -0.,
         0.,  1., -0., -0., -0.,  1.,  1.,  1., -0.,  0., -0.,  0.,  0.,
         0., -0., -0., -0.,  0., -1., -1., -0.,  0., -0., -0.,  1.,  0.,
         0., -0.,  1., -1., -0.,  0., -1., -0., -0., -0.,  0.,  0., -1.,
        -0.,  0., -0., -1., -0.,  0.,  0.,  0.,  1., -1.,  0.,  0., -1.,
        -0., -0., -0.,  1.,  0., -0.,  0.],
       [-0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,
        -0.,  0., -0.,  0.,  0., -1., -0.,  0., -0., -0., -0., -1., -0.,
        -0., -0.,  0.,  0., -0., -0., -0.,  1.,  0., -0.,  0., -0., -0.,
         0.,  0., -0., -0.,  0., -1., -1., -0., -0.,  0., -0., -0., -0.,
        -1.,  0., -0., -1., -0., -0.,  0., -0., -0., -0., -0.,  0., -1.,
         0., -0., -0.,  0.,  0.,  0., -0.,  0.,  1., -1.,  0.,  0., -1.,
        -0., -0., -0., -0., -0., -0., -0.],
       [-0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,
        -0., -1., -0., -0., -0.,  0.,  0., -1.,  0.,  0., -0., -1.,  1.,
         0., -0.,  0.,  0.,  0., -0., -0.,  1.,  0., -0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0., -1., -1., -0., -0.,  0., -0.,  1., -0.,
        -1.,  0., -0., -1., -0., -0., -1.,  1.,  0., -0.,  0., -0., -1.,
        -0., -0.,  1., -1., -0.,  0., -0.,  0.,  1., -1., -1., -0., -1.,
         0., -0.,  1.,  1., -0., -0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  0., -1., -0.,  0., -0.,  0., -0.,
        -0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  1.,  0.,
         0.,  0.,  0., -0., -0.,  0.,  0., -1.,  0., -0.,  0., -0., -0.,
         0., -0., -0., -0.,  0., -0.,  1., -0.,  0., -0.,  0., -1., -0.,
        -0.,  0.,  0.,  1.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0.,
         0., -0.,  0.,  1., -0., -0., -0.,  0., -1.,  1., -0.,  0.,  1.,
        -0.,  0.,  0.,  0.,  0., -1., -0.],
       [ 0., -0., -0.,  1.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,
        -0.,  1.,  0.,  0., -0., -0., -1.,  0., -0., -0.,  1., -1., -1.,
        -0.,  1.,  0.,  1.,  0.,  1.,  1., -0.,  0., -1., -0., -0.,  0.,
         0., -0., -0.,  1.,  0.,  0.,  0.,  1., -1., -0., -0., -0.,  0.,
        -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -1.,  0.,
        -0.,  0., -1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  1., -1.,  0.,
        -0., -0.,  0., -0., -1.,  0.,  0.],
       [-0.,  0.,  0., -0., -1.,  0., -0., -0., -1., -1.,  0., -0., -0.,
        -0., -0., -0., -1., -0.,  0.,  0., -1.,  0., -0.,  1., -1.,  0.,
         0., -0., -1., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  1., -0.,
         0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,
         0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -1.,
        -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -1., -0.,  0.,  0.,
         0., -0., -0., -0.,  0., -0., -0.],
       [-0.,  0., -0.,  1.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,
        -0.,  0., -0., -0.,  1.,  0., -1., -1.,  0., -0.,  1., -1.,  0.,
        -0., -0., -1., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0., -0.,
         0.,  0., -0., -0.,  0.,  0.,  0.,  1.,  0., -0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0., -0., -0., -0.,  0.,  0., -0.],
       [ 0.,  0., -0., -1., -0.,  0.,  0., -1., -0., -0.,  0., -0., -0.,
        -0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0., -1.,  1.,  1.,
         0.,  0., -0.,  0.,  1.,  0., -0.,  0., -0.,  1.,  0.,  0., -0.,
        -1., -0., -0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -1., -0., -0.,
        -0.,  0., -0., -0., -0.,  0., -0.],
       [ 0., -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,
         0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  1.,  0.,  0.,
         0., -0.,  0.,  1.,  0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0.,
        -0.,  0., -0., -0., -0.,  0., -1., -0., -1., -0.,  0., -0.,  0.,
         0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,  0., -0., -1., -1.,
         0.,  0., -0., -0., -1., -0.,  0.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,
         0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  1.,  0., -0.,
        -0., -0.,  0., -0.,  0., -0., -0.,  1.,  0., -0.,  0., -0., -0.,
        -0., -0., -0.,  0., -0., -1., -1.,  0.,  0.,  0., -0., -0.,  0.,
        -1., -0., -0., -1.,  1., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
         0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0., -1.,
        -0.,  0., -0., -0.,  0., -0., -0.],
       [-0.,  0., -0., -0.,  0., -1.,  0.,  0., -0.,  0.,  0., -0.,  0.,
         0.,  0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,
         0.,  0., -1., -0.,  0.,  0.,  0.,  1., -0.,  1.,  0., -0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -1., -0.,  0., -0., -0., -0., -0.,
         0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  1.,  0.,
        -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -1.,
         0.,  0., -0., -0.,  0., -0., -0.],
       [-0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -1., -0., -0.,
        -0., -0., -0.,  0.,  1.,  0., -0., -1.,  0., -0., -0., -0.,  0.,
        -1.,  0., -0., -0., -1., -0.,  1., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  1.,
         0., -0.,  0.,  0., -0., -1., -0.],
       [-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0., -1., -1.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0., -1.,  0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0., -0.,
         0., -0., -1., -0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,
         0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0., -0.,  0., -0.],
       [-1., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0., -0.,  0., -0., -0.,  1.,  0., -0.,  0., -1.,  0., -0.,  0.,
         0., -0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -1.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  1.,  0.,  0.,  0., -0., -0.,
        -0.,  0., -0., -1., -0., -0., -1., -0.,  0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [-1., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,
        -0.,  1.,  0.,  0.,  0., -0., -0.,  1., -0.,  0.,  0., -0., -1.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0., -1., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0., -1., -0.,  0., -0., -0., -0., -0.,
        -0.,  0., -1.,  0.,  0.,  0., -0.],
       [ 1.,  0., -0., -0., -0.,  0., -0.,  1., -0., -0., -0.,  0.,  0.,
        -1.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0., -0.,  0., -0., -0., -1.,  0., -0., -0.,  0., -0.,  0.,
         0., -0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,
         0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -1.,
        -0.,  0., -0.,  0., -0.,  1.,  0.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 4
m = 4
p = 7
rank = 85

verify_tensor_decomposition(decomposition_447, n, m, p, rank)

##Rank-90 decomposition of <4,5,6> over Z

In [ ]:
#@title Data

decomposition_456 = (np.array([[ 0., -0.,  0.,  0.,  0.,  0., -1., -0.,  1.,  0.,  0., -0.,  0.,
        -0.,  0., -0., -0.,  0., -0., -1., -1.,  0., -0.,  0., -0., -1.,
        -0.,  0.,  1., -1., -0.,  0., -0.,  1., -0.,  1., -0.,  0.,  0.,
        -0.,  0.,  0., -1.,  1.,  0., -1., -0., -0.,  0.,  0., -0., -0.,
        -1.,  0., -0.,  1.,  0., -0.,  0., -0.,  0.,  1., -0., -0.,  1.,
        -0., -0.,  0., -0.,  0., -1.,  1., -0.,  0.,  1., -0., -0., -0.,
         0., -0.,  0.,  0.,  1.,  0.,  1., -0.,  0.,  0., -0.,  0.],
       [-0.,  0., -0., -0.,  0.,  1., -0., -0.,  0., -1.,  1.,  0., -1.,
         0.,  0.,  0.,  1., -1.,  1., -0.,  0., -1.,  0., -0., -0.,  0.,
         0., -0., -0., -0., -0., -0.,  0., -0., -0., -0., -1.,  0.,  0.,
         1.,  0.,  0., -1., -0.,  0.,  0.,  0., -0., -0.,  1.,  1.,  0.,
         0.,  0., -0., -0., -1., -0.,  0.,  0., -0.,  0.,  1.,  1.,  0.,
         0.,  1.,  0., -0., -1., -0., -0.,  0.,  0., -0.,  1.,  0., -1.,
         1., -0., -0.,  1., -0., -1., -0., -0.,  0., -0.,  1.,  1.],
       [ 0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -1., -0.,  0.,
        -0.,  1., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  1.,
         0., -0., -1.,  0., -0., -0.,  0., -0.,  1.,  0., -0., -1., -0.,
        -0., -0., -0., -0., -0., -1.,  0.,  0., -1., -0.,  0., -0.,  0.,
        -0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,
         0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0., -1., -0.],
       [ 0.,  0., -0.,  0., -0.,  0., -0., -0.,  1.,  0., -1., -0.,  0.,
        -1., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
         0., -0.,  0., -1.,  0.,  0., -0.,  1.,  0.,  0.,  0., -1.,  0.,
         0.,  0.,  1., -1.,  1., -0., -1., -0.,  0.,  0., -0.,  0., -0.,
        -1., -0.,  0., -0.,  0.,  0., -0.,  0.,  1., -0., -0., -0.,  1.,
         0., -1., -0.,  0.,  0., -1., -0., -1.,  0., -0., -0., -0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  1., -0., -0., -1., -0.,  0.],
       [-0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,
         1.,  0.,  0., -1.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0., -1., -0.,  1.,  1.,  0.,  0.,  0., -0., -0., -0.,  1., -0.,
        -0.,  1., -0., -0., -1., -0.,  1., -0., -1.,  0.,  0.,  0., -0.,
         1., -1., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0., -1.,
         0.,  0., -0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0., -0., -0.,
        -1.,  0.,  0.,  0., -0.,  1., -1.,  0., -0., -0.,  0., -1.],
       [ 0., -0.,  1.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -1.,  0.,
         0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -1.,  0., -0., -0.,
         0., -0., -0.,  0., -0., -1.,  1., -0., -0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  1.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  1.,
        -0., -0., -1., -0., -0., -0., -1., -0.,  0.,  0., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,
         0., -1.,  0.,  0.,  0., -0., -0.,  0., -1., -0.,  0., -0.],
       [-0.,  0.,  0., -0., -0., -0., -0., -0., -0.,  1., -0.,  0., -0.,
         0.,  1.,  0., -0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,
         0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,
        -0., -0., -0.,  1., -0.,  0.,  0., -0.,  0., -0., -1.,  0.,  0.,
         1.,  0.,  0.,  0.,  0., -0.,  1., -0.,  0.,  0., -1.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,
        -1., -0., -0., -1., -0., -0., -1.,  0., -0.,  0., -1., -1.],
       [-0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,
         0., -1., -0., -1., -0., -0.,  0., -0., -0., -0., -0.,  0., -0.,
        -1., -1.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
         0., -0., -0.,  0., -1.,  0.,  0.,  0., -1., -0.,  0., -0.,  0.,
        -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,
        -1., -0., -0., -0., -0.,  1., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0., -0.,  1.,  0., -0.,  1.,  0.,  1.,  0.],
       [ 0., -0., -0.,  1., -0.,  0., -0.,  0., -1.,  0.,  0., -0., -0.,
        -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,
        -0., -0., -0., -0.,  0.,  0.,  1.,  0.,  0., -0.,  1., -0.,  0.,
        -0., -0., -0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  1.,
        -0.,  1.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  0., -0.,  0., -0.,
        -0., -0.,  0., -0.,  0.,  0., -0.,  1.,  0., -0.,  0.,  0.,  0.,
         0., -0.,  0., -0.,  0., -0.,  0., -0., -1.,  0.,  0.,  0.],
       [-0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,
        -0., -0., -0.,  1.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         1.,  1.,  0., -1.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  0., -0.,  1., -0., -0.,  0.,  1.,  0.,  0.,  0., -1.,
        -1.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0., -0.,
         1., -0., -0., -0., -0., -1., -0.,  0.,  0., -0.,  0., -0.,  0.,
         1.,  0.,  0.,  0.,  0., -1.,  1., -0., -0., -0., -0.,  1.],
       [-0.,  0., -1., -0.,  1., -0.,  0.,  1., -0.,  0.,  0.,  1., -0.,
         0., -0., -0.,  0.,  0., -0.,  1., -0.,  0.,  1.,  0., -0.,  1.,
        -0., -0.,  0.,  1., -0.,  1., -1., -0.,  0., -1., -0., -0.,  0.,
         0., -0.,  0., -0., -1., -0.,  1.,  0., -0., -0., -0.,  0., -1.,
         1.,  0., -0., -1.,  0.,  0.,  1.,  0., -0., -0.,  0., -0., -1.,
         0.,  0., -0., -0., -0.,  1., -1.,  0.,  1., -1., -0., -0.,  0.,
         0.,  1., -0.,  0., -1.,  0., -1.,  0.,  1., -0.,  0., -0.],
       [-0.,  0.,  1.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0., -1.,  0., -1., -0., -1., -1., -0.,  1., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,
        -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -1., -0.,
        -1.,  0.,  0., -0., -0., -0., -1., -0.,  0.,  0., -0.,  0., -0.,
         0., -0., -0., -0.,  1., -0.,  0.,  0., -0.,  1., -1., -0.,  0.,
         0., -0.,  1., -0.,  0.,  0.,  1., -0.,  0., -0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -0., -0., -0., -1.,  0.,  0., -0., -0., -0.,
        -0.,  1., -0.,  1., -0.,  0.,  0., -0., -0., -1.,  0.,  0., -1.,
         1.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0.,
        -0., -0.,  0., -0.,  1., -0., -0., -1., -0., -0.,  0.,  1.,  0.,
         0., -0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0., -0.,  0.,  0.,  0., -1., -0.,  0., -0., -0.,  0.,  0.,  0.,
         0., -0.,  0., -0.,  0., -0., -0., -1., -1., -0.,  0., -0.],
       [-0., -0., -1., -0.,  0.,  0., -0.,  1.,  0., -0., -0.,  1.,  0.,
         0.,  0., -1., -0., -0., -0.,  1.,  0.,  0.,  1.,  1., -1.,  1.,
         0.,  0., -0.,  1., -1.,  1., -1., -1.,  0., -1., -0., -0.,  0.,
        -1.,  0., -0.,  0., -1.,  0.,  1., -0.,  0.,  0.,  0.,  0., -1.,
         1., -0., -0., -1., -0., -0.,  1., -0.,  0.,  0.,  0., -0., -1.,
        -0.,  0., -1., -0., -0.,  1., -1.,  0.,  1., -1.,  0., -0.,  0.,
         0., -0., -1., -0., -1.,  0., -1.,  1.,  1.,  0.,  0.,  0.],
       [ 0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
        -0.,  0.,  1., -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,
        -1.,  0.,  0.,  0., -0., -1.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0.,  0., -0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  1.,
         0., -0.,  0.,  1., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0., -0.,  0.,  0.,  0.,  0.,  0., -0., -1., -0.,  0.,  0.,  0.,
         0., -0.,  0., -0.,  1., -0.,  0.,  0., -0., -0., -0.,  0.],
       [-1.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0., -0., -1., -0.,  0., -0., -0.,  0.,  0.,
         0.,  0.,  1., -1.,  0.,  0., -0.,  0., -0.,  1., -0.,  0., -0.,
         0., -0., -0., -1.,  1., -0., -1.,  0., -0.,  0., -0.,  0.,  0.,
        -1.,  0., -0.,  1., -0., -0., -0., -0.,  0.,  0., -0., -0., -0.,
         0., -0., -0.,  0., -1., -0.,  1., -0.,  0., -0.,  0.,  0.,  0.,
         0., -0.,  0.,  1.,  0., -0.,  0.,  0., -0., -0., -0.,  0.],
       [-0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -1.,
         0.,  0., -0.,  1., -1.,  1., -0.,  0.,  0.,  0., -0., -0., -0.,
        -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  1., -0.,  0.,  0.,  0.,
         0.,  0., -0., -1., -0.,  0., -0.,  0.,  0., -0., -0.,  1., -0.,
        -0.,  0., -0.,  0., -1.,  0., -0., -1.,  0.,  0.,  1.,  0., -0.,
         0., -0., -0.,  0., -1.,  0., -0.,  0., -0., -0.,  1., -0., -1.,
         1.,  0.,  0.,  1., -0., -1., -0.,  0.,  0., -0.,  1.,  1.],
       [ 0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0.,  1.,
        -0., -0.,  0.,  0.,  1., -1., -0.,  0.,  0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0., -1., -0.,  0., -0.,  0.,
        -0.,  0.,  0., -0.,  0., -1.,  0.,  0.,  1.,  0., -0., -1.,  0.,
         0.,  0.,  0.,  0.,  1.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.,  1.,  0.,
        -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0.],
       [-1.,  1.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
        -0.,  0., -0.,  0., -0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0., -1.,  0., -0., -0.,  0.,  0.,  0.,  0., -1.,  0.,
        -0.,  0.,  1., -1.,  1.,  0., -1.,  0.,  0.,  0.,  0., -0.,  0.,
        -1., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,
         0., -1., -0.,  1., -1., -0., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0., -1., -0., -0.],
       [-0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0.,
         0., -0.,  0., -1., -0.,  0.,  0., -0.,  0., -0.,  0.,  0., -0.,
         0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
         0., -0., -0., -0., -1.,  1.,  1.,  0., -1.,  1.,  0., -0.,  0.,
         1.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -1.,  0.,  1.,
        -1.,  0., -0., -0., -0.,  1., -0., -0.,  0.,  0.,  0., -1.]],
      dtype=np.float32), np.array([[-1., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0., -0.,  0., -0., -0.,  1., -0.,  1.,  0., -0., -1.,
        -0., -0., -1.,  0., -0., -0., -0., -0., -0.,  1.,  0., -0., -0.,
        -0., -0., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,
        -0., -0.,  0., -0.,  0., -0., -0.,  0.,  0., -1., -0., -0.,  0.,
         0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0., -1.,  0.,  0.,  0., -0., -0.,  1., -0.,  0., -0., -0.],
       [-1., -1.,  0.,  0.,  0.,  0.,  0., -0., -1., -0., -0., -0., -0.,
         0., -0.,  1., -0.,  0., -0.,  0.,  1., -0.,  0.,  0., -0., -0.,
        -0., -0., -1., -0., -0.,  1., -0.,  0.,  0., -0., -0., -0., -0.,
         0., -0.,  0.,  1.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  0.,  1., -1.,  0., -0., -0.,  0., -0., -1., -1., -0., -0.,
        -0., -0.,  0.,  1.,  0.,  0.,  0.,  1.,  1., -0.,  0.,  0.,  0.,
         0., -0.,  0., -1., -1., -0.,  0.,  0.,  0.,  0.,  0., -0.],
       [-0., -0., -0.,  0., -0.,  0., -0.,  0., -1.,  0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
         0., -0., -0.,  1., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
         0., -0.,  1., -0., -0.,  0., -0.,  0.,  0.,  0., -1.,  0.,  0.,
         0.,  0.,  0.,  1.,  0., -0.,  0.,  1.,  0., -0., -0.,  0.,  0.,
         0.,  1., -0., -1.,  0., -0., -0.,  0.,  0., -0.,  0., -0.],
       [-1., -1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -1., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0.,  1., -0.,  0.,  0., -1., -0.,
        -0., -0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,
         0., -0.,  0., -0., -0.,  0., -1., -0., -0., -0., -0.,  0., -0.,
        -0., -1.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.],
       [-1., -1., -1., -0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0., -0.,  0.,  0., -0., -0., -1.,  1., -0.,  0., -0.,  0.,  0.,
        -0.,  0., -1., -0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0., -0.,  0., -0., -0., -0.,  0., -1., -0., -0.,  0.,
         0.,  0., -0., -0., -0., -0., -0., -0.,  0., -1., -0., -0.,  0.,
         0., -1.,  1.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.],
       [ 1.,  1., -0., -0., -1., -0., -1.,  0., -1.,  0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0., -0., -0., -1., -0.,  0., -0.,  0.,  0.,
         0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0., -0.,  1., -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,
        -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -1.,  0.,  0.,
        -0.,  0.,  0.,  1.,  0., -0., -0.,  1.,  0.,  0., -0., -0., -0.,
         0.,  0., -0., -1.,  0.,  0., -0., -0., -0.,  0., -0., -0.],
       [-0., -0., -1., -1., -0.,  1.,  0., -0., -0.,  0.,  0.,  1., -0.,
        -0., -0., -0., -0., -1.,  1., -0.,  0.,  1.,  0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -0.,  1., -0., -1.,  0., -1., -0.,  0.,
        -1., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -1., -0., -0.,
        -0., -0.,  0., -0.,  1.,  0., -1., -0., -0., -0., -0., -0.,  0.,
         0.,  0., -1.,  0.,  0., -0., -0., -0.,  0.,  0.,  0., -1., -0.,
         0., -0.,  1., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.],
       [-0., -0.,  0., -0.,  0.,  1.,  0., -0., -0.,  1.,  0., -0.,  0.,
         0., -0.,  0.,  0.,  0., -0.,  0., -0., -1., -0., -0.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,
         1.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  1., -0.,  0., -0.,
         0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,
         0.,  0.,  1.,  0., -0.,  0.,  0., -0., -0., -0., -1.,  0.,  1.,
        -0.,  0., -1., -0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.],
       [-0., -0.,  1.,  1.,  0., -0., -0., -0.,  0., -0., -0., -1., -0.,
        -0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,
        -0., -0.,  0., -0., -0.,  0., -1., -0., -0., -0.,  1., -0., -0.,
         0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  1., -0., -0., -0.,  1., -0., -0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,
         0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.],
       [ 0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  1., -0.,  0., -0., -0.,  0., -0.,
        -0., -0., -0., -1.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
        -1., -0.,  0.,  0.,  0.,  0.,  0., -1.,  0., -0., -0., -0., -0.,
        -0., -1.,  0.,  0.,  0.,  0., -1.,  0., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  1.,  0., -1.],
       [ 0.,  0., -1., -1., -0., -0., -0., -0.,  0.,  0., -0.,  1.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0., -0., -0.,  0., -0.,  0.,  1., -0., -0.,  0., -1., -0.,  0.,
        -1.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,
        -0.,  0., -0., -0.,  0.,  0., -1., -1., -0., -0., -0., -1.,  0.,
        -0., -0., -1.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
         0., -0.,  1., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0.],
       [-1.,  0., -0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0.,  0.,  1., -0., -0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,
        -1., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  1.,  0.,  0.,
        -0.,  0., -1.,  0., -1., -0., -0.,  0.,  0.,  0., -0., -0., -0.,
        -0., -0.,  1.,  1.,  0., -0., -0., -0., -0.,  0.,  0.,  0.],
       [ 0.,  0., -0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1., -1., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0.,  0., -1.,
         0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,
        -0., -0.,  0., -0.,  0., -1.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0.,  0., -1., -0., -0., -0., -0., -0.,  0.,  0., -0., -1.,  0.,
         0., -0.,  0.,  0.,  0.,  0., -0.,  1., -1.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,  1.,  0., -0., -1.,
         0.,  0., -0., -1., -0.,  1.,  0.,  0.,  0., -0.,  1.,  0.,  0.,
        -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0., -1.,
         0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  1., -0.,  1., -0.,
         0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,
        -0.,  0.,  1.,  0.,  0.,  0., -0.,  0., -0., -0., -1.,  0.,  1.,
         1., -0., -0., -0., -0., -1.,  0., -1., -0.,  0., -0., -0.],
       [ 0.,  0.,  0.,  1.,  0., -0.,  0., -0.,  0.,  0.,  0., -1.,  0.,
         0.,  0., -0., -0., -1., -0.,  0.,  0., -0., -1., -0., -0., -0.,
        -0., -0., -0.,  0., -0., -0., -1.,  0.,  0., -0.,  0., -0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,
         0., -0.,  0.,  0., -0.,  1., -0.,  0.,  0.,  0.,  1.,  0., -0.,
        -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,
         0.,  0., -0., -0., -0.,  0.,  0., -0.,  1., -0.,  1.,  0.],
       [ 0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0.,
        -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
        -0.,  0., -0., -1., -0.,  0., -0.,  0.,  0., -1.,  0.,  0., -0.,
        -0.,  0.,  1.,  0., -1.,  0., -0.,  0.,  1., -0., -0.,  0., -0.,
         0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0.,  0.,  0., -1.,  0., -0., -0., -0., -1., -0.,
        -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  1.,  0.,  0.],
       [ 0., -0., -1., -1.,  0., -0.,  0.,  0.,  0.,  0., -1.,  1., -0.,
         0., -0., -0.,  0., -0., -0., -1., -0.,  0., -0., -0., -0.,  1.,
        -0.,  1., -0.,  0., -0., -0.,  1.,  0.,  0., -1., -1.,  0.,  0.,
        -1.,  0.,  1.,  0., -1., -1., -0., -0.,  0.,  0.,  0., -0., -0.,
         1.,  0.,  0., -0.,  0.,  0., -1.,  0.,  1., -0., -0., -1.,  0.,
        -0.,  1., -1., -0., -0.,  1., -0.,  0.,  0., -1., -0., -1., -0.,
         0., -0.,  1.,  0.,  0., -0., -1., -0., -0., -0., -0., -0.],
       [-1., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,
         0., -1.,  0.,  0., -0., -1., -0.,  0.,  0., -0., -1., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -1.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0.,  0.,
         0.,  0., -1.,  0., -1., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,
         0., -0.,  0.,  1.,  0.,  0., -0.,  1.,  0.,  0., -0.,  0.],
       [-0.,  1., -0., -1., -0.,  0., -0., -0.,  0.,  0.,  0.,  1., -0.,
         0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0., -0., -1., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,  0.,  0.,  0., -1.,
        -0., -0., -1., -0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0., -1., -0.,  0., -1., -0.,  0.,  0.,  0.,
        -0.,  0., -1.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,
         0.,  1.,  0.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0.],
       [-0.,  1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -1., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,
        -0., -0., -0., -0., -1.,  0., -0.,  0., -0., -0.,  0.,  1., -0.,
        -0., -1.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,
         0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0.,  1., -1.,  0.,  0.,  0., -1., -0., -0., -0.,  0., -0.,
        -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.],
       [ 0., -0.,  0.,  1.,  0., -0., -0., -0., -0.,  0.,  0., -1., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,
         0., -0., -0., -0., -0.,  0., -1., -0., -0., -0., -0., -0., -0.,
        -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,
        -0.,  0.,  0., -1.,  0.,  0., -0., -1., -0., -0.,  0., -0.,  0.,
         0., -1., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.],
       [-0.,  1., -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  1., -0.,
        -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  1.,  0.,
         0.,  0.,  0.,  0., -0., -0.,  1., -0., -0.,  0.,  0.,  0., -0.,
        -0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,
        -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,
        -0.,  1.,  0., -0.,  0., -0., -0.,  0.,  0.,  1., -0.,  0.],
       [ 0.,  1.,  0., -1.,  0.,  0.,  0.,  0., -0.,  0., -1.,  1.,  0.,
         0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -0., -0.,  1., -0.,  0., -0., -1., -0., -0.,
        -1.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0.,  0., -0.,  0., -0., -0., -0., -0.,  0., -0., -1.,  0.,
         0.,  1., -1.,  0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.],
       [-0., -1., -0.,  0.,  0.,  0.,  1.,  0.,  1.,  0., -0.,  0., -0.,
         0., -0., -0., -0., -0.,  0., -0.,  1., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  1., -0., -0., -0.,  0., -0.,
         0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,
         0., -0., -1., -1., -0., -0., -0., -1.,  0.,  0.,  0.,  0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  0.],
       [ 0.,  0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0.,  0.,  1., -0.,
         1., -0.,  0., -0., -0., -0., -0., -0.,  0.,  1., -1.,  0., -1.,
         0.,  0.,  0.,  0.,  0., -0.,  1., -0., -0.,  1.,  0., -1., -1.,
         0., -0., -1.,  0.,  1.,  0., -1.,  1., -0.,  1., -0.,  0., -0.,
        -0.,  0., -0.,  1., -0., -1., -0., -0., -1., -0.,  0.,  0., -1.,
        -1., -0., -1., -0., -0., -1., -0.,  0., -0.,  0.,  0.,  0.,  0.,
         0., -0.,  0.,  0.,  1., -0., -0.,  1., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  1.,  0., -0., -0.,
        -0., -0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0.,  0., -0.,  0., -1.,  0.,  0.,  0., -0., -0., -0., -0., -0.,
        -0., -1., -0., -0.,  0., -0.,  0.,  0., -0.,  1., -0.,  0., -0.,
         0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0., -0.,
        -0., -0.,  1., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,
         1.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.],
       [ 0., -0., -0.,  1., -0., -0.,  0., -0.,  0.,  1., -0., -1.,  0.,
        -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,
         0., -0., -0., -0.,  0., -1., -1., -0., -0., -0., -0., -0., -0.,
         0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  1.,
         0., -1., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  1., -0., -0.,
        -0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,
         1., -0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.],
       [ 0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0., -0.,  0., -1., -0., -0., -0., -0.,  0.,  0., -0., -1., -0.,
         0.,  0., -0., -0., -0., -0., -1., -0., -0.,  1., -0., -0., -0.,
        -0., -0., -0.,  1.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -0., -1., -0.,  0.,  0., -0., -0., -0.,
         0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  1., -0.,  0.],
       [ 0.,  0., -1., -1., -0.,  0.,  0., -0.,  0., -0., -1.,  1., -0.,
         1., -0.,  0., -0., -0.,  0., -1.,  0., -0., -0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0., -0., -0.,  1.,  0.,  0.,  0., -1., -1., -0.,
        -1., -0., -0., -0., -0., -0., -1.,  0., -0.,  1., -0.,  0.,  0.,
         1.,  0., -0.,  1.,  0.,  0., -1., -0.,  0.,  0.,  0., -1., -1.,
         0.,  1., -1.,  0.,  0., -0., -0., -0.,  0., -1., -0., -0.,  0.,
         0.,  0.,  1.,  0.,  1.,  0., -1.,  0., -0., -0., -0.,  0.],
       [-1., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0., -1., -1., -1.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,
        -1.,  0.,  0.,  0.,  1.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  1.,  0.,  0.,
        -0., -0., -1., -0., -1., -0., -0., -0.,  0.,  0., -1., -0., -0.,
         0.,  0.,  0.,  1.,  0.,  0., -0., -0., -0., -0.,  0.,  0.]],
      dtype=np.float32), np.array([[ 0.,  0., -0.,  0.,  0., -0., -0.,  1.,  0.,  0., -0.,  0.,  1.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  1.,
         0., -1., -0., -0.,  0.,  0., -0., -0.,  1., -0., -0., -0., -1.,
        -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,
        -0., -0., -0., -0., -1., -0.,  0.,  0., -1., -0., -0., -0.,  0.,
        -1., -0., -0.,  0.,  0., -1.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0., -0.],
       [-0., -0.,  0.,  0.,  0., -0., -0.,  1.,  0.,  0., -0.,  0.,  1.,
        -0., -0., -0., -0., -1.,  0., -0., -0., -0., -1.,  0., -0., -0.,
        -0., -0.,  0.,  0., -0., -0., -0., -0., -0., -0., -0.,  0., -1.,
         0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,
         0.,  0., -0., -0., -1., -1., -0., -0.,  0., -0., -0.,  0., -0.,
        -1.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
         0.,  0., -0., -0.,  0.,  0., -0., -0., -1., -0.,  1.,  0.],
       [ 0.,  0.,  0.,  0., -0., -0., -0.,  1., -0.,  0., -0.,  0.,  1.,
         0., -0., -0.,  0., -0., -1., -0.,  0.,  0., -0., -1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0., -1.,
         0., -0.,  0., -0., -0., -0., -0.,  1., -0.,  0.,  0.,  1.,  0.,
         0., -0.,  0., -0., -1., -0.,  0., -0., -0., -0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,
        -0., -0., -0., -0.,  0., -0.,  0., -1., -0., -0.,  0.,  0.],
       [ 0., -0., -0., -0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -1.,
         0., -0., -0., -0.,  0., -0.,  0.,  0., -1.,  1., -0.,  0.,  0.,
         0., -0., -1., -0., -1., -1.,  0., -0., -1.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0., -0.,  1.,  0.,  0., -0.,  0.,
        -0.,  0., -0., -0.,  0.,  1.,  0.,  0., -0.,  0.,  0., -1., -0.,
        -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  1., -0., -0., -0.,  0.,  0.,  0., -1.,
         1.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -1., -0.,  0., -0.,  0.,
         0., -1.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,
         0.,  0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0., -1.,
         0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  1.,  0.,  0., -0.,  0.,
        -0., -0., -0., -0., -1., -0.,  0.,  0., -0., -0., -0., -0.],
       [ 0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  1.,  0.,  0., -0.,
         0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0., -0., -1., -0.,  0.,  0., -0.,  0.,  0., -0.,
         0., -1., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  1.,
         0.,  1.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0., -0.,  0., -0.,  0.,  0.,  1., -0., -0., -0., -1.,
         1., -0.,  0.,  0.,  0., -1., -0., -0., -0.,  0.,  0., -0.],
       [ 0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,
         0., -0.,  1., -1.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,
         1., -0., -0., -0.,  1., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,
        -0., -1.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0., -0., -0., -0.,  0., -0.,  1., -0.,  1., -0., -1.,
         0., -0., -0., -0.,  0., -1., -0.,  0.,  0.,  0., -0., -0.],
       [-0.,  0., -0., -0., -0., -1.,  0.,  0.,  0., -0., -0., -0.,  0.,
        -1.,  0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0., -0., -1.,  0.,
         0.,  0., -0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0., -0.,
        -0.,  0., -0., -1., -1., -0.,  0., -0.,  0., -0.,  0., -0.,  1.,
        -0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -1.,
        -0., -0., -0., -0.,  1.,  0., -0., -0.,  0., -0., -0., -0.],
       [-0., -0., -0.,  1.,  0., -0., -1., -0., -1., -0.,  0., -0.,  1.,
        -1.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  1.,  0., -1.,  0., -1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  1.,  0.,  0., -1., -1.,  0., -0., -0.,  0.,  0., -1.,  1.,
        -0.,  0., -0.,  0.,  0.,  0., -0.,  1., -1., -0., -0., -0.,  0.,
        -0.,  0., -0.,  0.,  1., -0.,  0., -0., -0.,  0.,  0.,  0.],
       [-0.,  0.,  0.,  1.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,  1.,
        -0., -0., -0., -0., -1.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,
         0.,  0., -0.,  0.,  0.,  1., -0., -0., -0.,  0., -1.,  0., -1.,
        -0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0., -1.,  0., -1.,
         0.,  0., -1.,  0., -1., -1., -0., -0.,  0.,  0.,  0., -1., -0.,
         0., -0.,  0., -0., -0., -0., -0.,  0., -1.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  1.,  0.],
       [ 0., -0.,  1., -0.,  0.,  0., -0.,  0., -0., -0., -0., -1.,  1.,
        -0., -0.,  0.,  0., -0., -1.,  0., -0., -1.,  1., -1., -1., -0.,
        -0.,  0., -0.,  0., -0.,  1.,  0., -0.,  0.,  0., -0., -0., -1.,
         1., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  1.,  0.,
        -0., -0., -1.,  0., -1., -0.,  0.,  0.,  0.,  0.,  0., -1., -0.,
        -0.,  0., -0., -0.,  0.,  0.,  0.,  0., -1., -0., -0.,  0.,  0.,
         0.,  1.,  1.,  0.,  0.,  0., -0., -1.,  0., -0., -0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,  1., -0.,  0.,  0.,
         1.,  0.,  0., -0., -1., -0., -0., -1., -0.,  0., -0., -0.,  0.,
         0.,  0.,  1., -0., -0.,  0., -0., -0., -1.,  0.,  0.,  1., -0.,
        -0.,  0., -0.,  0.,  0., -0., -1.,  0., -0., -1., -1.,  0.,  0.,
         0., -0., -0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  1.,  0., -1.,
         0.,  0.,  0., -1., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,
        -0., -0., -0., -1., -1., -0.,  0.,  0.,  0.,  0., -0.,  0.],
       [ 0., -1.,  0.,  0., -0., -0.,  1., -0.,  0., -0., -1., -0., -0.,
        -1.,  0.,  0.,  0.,  0.,  0., -0., -1., -0., -0.,  0., -0.,  0.,
        -0., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -1.,  0.,
         0., -0., -1.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
         0.,  0., -0.,  0., -0., -0., -0., -0.,  1.,  1., -0., -0., -0.,
        -0., -1.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -1.,  0.,  0.],
       [-0.,  0.,  0.,  1., -0., -0.,  0.,  0., -0., -0., -0.,  1.,  0.,
        -0.,  0.,  0.,  0., -0., -0., -1.,  0., -0.,  0., -0.,  0.,  1.,
         0.,  0., -1.,  1.,  0., -0.,  1.,  0., -0., -1.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0., -0.,
        -1., -0., -0.,  1., -0.,  0., -0.,  0., -0.,  1.,  0., -0., -1.,
         0.,  0.,  0., -0., -0., -1.,  1., -0.,  0., -1., -0.,  0., -0.,
         0.,  0., -0., -0., -1.,  0.,  1.,  0., -0., -0.,  0., -0.],
       [-0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -0.,  0., -1.,  0., -0.,  0., -0., -1.,  1.,
        -0.,  0., -1.,  0., -0.,  0., -0.,  0., -0., -1., -0.,  0.,  0.,
        -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         0., -0.,  0.,  1.,  0., -0.,  0., -0.,  0.,  1., -0.,  0.,  0.,
         0., -0., -0.,  0., -0., -0.,  1.,  0., -0., -1.,  0.,  0., -0.,
        -0.,  0.,  0., -0., -1., -0.,  0., -0., -0.,  0.,  0., -0.],
       [-0.,  1.,  0.,  0.,  0.,  0., -1.,  0., -0., -0.,  1.,  0.,  0.,
         1.,  0., -0., -0., -0.,  0., -1.,  1., -0., -0., -0., -0.,  1.,
         0., -0., -1., -0.,  0.,  0., -0.,  0., -0., -1.,  0.,  1.,  0.,
         0.,  0.,  1., -0.,  1., -0., -1.,  0.,  1.,  0.,  0.,  0.,  0.,
        -1., -0.,  0.,  1.,  0.,  0., -0., -0., -1., -0., -0.,  0., -1.,
         0.,  1.,  0., -0., -0., -1., -0., -0., -0., -1., -0., -0., -0.,
         0.,  0., -0., -0., -1., -0.,  1.,  0., -0.,  0.,  0., -1.],
       [ 0., -0., -0.,  0., -0., -0., -0., -1., -0., -0.,  1., -0., -0.,
        -0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -1.,
         0.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,
        -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,
        -0., -0.,  0.,  0., -0., -0.,  0.,  0.,  0., -1.,  0., -1.,  1.,
         1., -0., -0., -0., -0.,  1., -0., -0., -1.,  0.,  0., -0., -0.,
        -0., -0.,  0.,  0.,  1., -0., -0., -0.,  0.,  0.,  0.,  0.],
       [-0., -0., -1., -0.,  0.,  0., -0., -1.,  0.,  0., -0., -0., -0.,
         0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -1.,
        -0., -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -1., -0., -0.,
         0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -1., -0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -1.,  0.,  0., -1.,  0., -1.,  1.,
         1.,  0., -0.,  0., -0.,  1.,  0.,  0., -1.,  1., -0., -0., -0.,
        -0., -0.,  0.,  0.,  1., -0., -1., -0.,  0.,  0.,  0.,  0.],
       [-0.,  0., -0., -0., -0.,  0., -0., -1.,  0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -1., -0.,  0., -0., -1.,
         0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,
         1.,  0., -0.,  0.,  0., -0.,  0., -1., -0., -0., -0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -1.,  0., -1., -0.,
        -0., -0., -0., -0., -0.,  0.,  0.,  0., -1.,  1., -0., -0.,  0.,
        -0.,  0.,  1.,  0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0., -1., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  1., -0.,  0., -0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,
         0.,  0., -0.,  0.,  0.,  1.,  0., -0., -0.,  0., -0.,  0., -0.,
         1., -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0., -0.,
        -0., -1., -0., -0.,  0.,  0.,  0., -0., -0.,  1., -0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0., -0., -1., -0.,  0.,  0.,  0.,  1.],
       [ 0., -0.,  0.,  0.,  0.,  1.,  1.,  0.,  0., -0., -0., -0., -2.,
         0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0., -0.,
        -0., -0.,  0.,  0.,  1.,  0., -0.,  1., -2., -0., -0., -0.,  1.,
        -1., -1., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., -0., -0.,  2., -0., -0.,  0., -0.,  0.,  0.,  1.,  0.,
        -0.,  0.,  1.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.],
       [ 0., -0., -0.,  0.,  0.,  0., -0., -0., -1., -0., -0.,  0., -1.,
         0.,  1., -0., -1.,  0.,  1.,  0., -0.,  1., -0.,  1.,  0., -0.,
        -0., -0.,  0.,  0.,  1., -0., -0.,  1., -0., -0.,  0., -0.,  1.,
        -1., -1.,  0.,  1.,  0.,  0., -0.,  0.,  0., -0., -0., -1., -0.,
         0.,  0.,  1., -0.,  1.,  0.,  0.,  0., -0.,  0., -0.,  1.,  0.,
        -0.,  0.,  1., -0.,  1., -0., -0., -0., -0., -0.,  1., -0., -1.,
         0., -0.,  0., -1., -0., -1., -0., -0., -0., -0.,  0.,  0.],
       [-0., -0., -0., -0., -1.,  0.,  0., -0., -0.,  0.,  0.,  0., -1.,
         0., -0., -0., -1., -0.,  1.,  0.,  0.,  1.,  0.,  1.,  0., -0.,
         1.,  0., -0.,  0.,  1.,  0.,  0., -0., -0.,  0.,  0., -0.,  1.,
        -1., -1.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -1.,  0.,
        -0.,  0., -0., -0.,  1., -0., -0., -0.,  0., -0., -0.,  1., -0.,
        -0.,  0.,  1.,  0., -0., -0.,  0., -0.,  0.,  0.,  1., -0., -1.,
        -0., -0., -0.,  0.,  0., -1.,  0., -0.,  0.,  0.,  0.,  0.],
       [-1., -0., -0., -0., -0., -1., -1.,  0., -0.,  0., -0., -0., -0.,
         0.,  0.,  0., -0., -0.,  1., -0.,  1.,  1., -0., -0.,  0.,  0.,
        -0.,  0., -1., -0., -0.,  0.,  0., -0.,  2.,  0., -0., -0., -0.,
         0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,
        -0.,  0.,  0.,  0., -1., -0., -0., -0., -0., -0.,  0.,  0., -0.,
        -0.,  0., -0., -0.,  1.,  0., -0., -0., -0., -0.,  1., -0., -1.,
        -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],
      dtype=np.float32))

In [ ]:
#@title Verification
n = 4
m = 5
p = 6
rank = 90

verify_tensor_decomposition(decomposition_456, n, m, p, rank)

## Rank-93 decomposition of <5,5,5> over Z



In [ ]:
#@title Data

decomposition_555 = (np.array([[ 0.,  0.,  0., -1., -0., -0.,  0.,  0., -0., -0., -0.,  1.,  0.,
         0.,  0.,  0., -0., -0.,  1.,  0., -0., -1., -0.,  0.,  0., -0.,
         0.,  0.,  0., -1., -1.,  1., -1.,  0., -0., -0.,  0., -0., -0.,
        -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,
        -1., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,  1.,
        -0., -0.,  0.,  0., -0.,  0.,  0., -1.,  0., -0., -0.,  0., -0.,
        -0., -1.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
         0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,
         0., -0.,  0.,  0.,  0., -1., -0., -1.,  1., -0., -0., -1., -1.,
        -1.,  0.,  1.,  1.,  1., -1.,  1., -0.,  0.,  0., -0., -0., -0.,
        -0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0.,  0., -1.,  1., -0.,
         1.,  0., -0.,  0., -0.,  1.,  0., -0., -0.,  0., -0., -0., -1.,
         1.,  0.,  0.,  0.,  1.,  0.,  0.,  1., -0.,  1., -0., -0.,  1.,
        -1.,  0., -1.,  0.,  0., -0., -0.,  0., -0., -0., -1., -0., -1.,
         0.,  0.],
       [-0.,  0., -0.,  0.,  1.,  0.,  0., -0.,  0.,  1., -1., -0.,  0.,
         1.,  0., -1.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0., -1.,
        -0., -0.,  0., -0., -0., -1.,  1., -1.,  0., -1., -0., -1.,  0.,
        -0., -1., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0.,
         1.,  0., -0.,  0., -1., -0.,  0.,  0., -0.,  0., -0., -1.,  0.,
        -0., -0., -0., -0.,  0.,  0.,  0.,  1.,  0.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -0.,  1., -0.,  0.,  0., -0., -1., -0., -0.,  0.,
         1.,  0.],
       [-0., -0.,  1., -1.,  0., -0.,  0., -0.,  0.,  0., -0.,  1., -0.,
        -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0., -0., -1.,
        -1.,  1.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,
         0., -0.,  0.,  0., -0.,  0., -1., -0.,  1., -0., -0.,  1., -0.,
        -0., -0.,  0.,  0.,  0.,  1., -0.,  0., -0.,  0.,  0.,  0., -0.,
         0., -1.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0., -1.,  0.,  0.,  0.,  1.,  1., -0.,  0.,  0., -0., -0., -1.,
        -0., -1.],
       [ 0.,  0.,  0., -0., -1., -0.,  0., -0.,  0., -1.,  0., -0.,  0.,
        -1.,  0.,  1.,  0.,  0.,  0.,  0., -0.,  1., -0., -0.,  0.,  0.,
        -0., -0., -0.,  1.,  1., -0.,  0.,  1., -0.,  1., -0.,  1.,  0.,
        -0.,  1., -0., -0.,  0., -0.,  0.,  0., -0.,  0., -1.,  1., -0.,
         0.,  0., -0., -0.,  1.,  1.,  0.,  0.,  0.,  0., -0.,  1.,  0.,
        -0.,  0., -0., -0.,  0.,  1., -0.,  0., -0., -0.,  0.,  0.,  0.,
         0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,  1., -0., -0.,  0.,
        -1., -0.],
       [ 0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,
        -0.,  1., -0.,  0.,  0., -0., -0., -0.,  0., -0., -1., -0.,  0.,
        -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,
        -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,
         0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  1., -0., -0., -0.,  0.,
        -1.,  0., -1., -0., -1.,  0., -0.,  1., -0.,  0.,  1., -0., -0.,
         0.,  1., -0., -1.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
         0.,  1.],
       [-0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0.,  0., -0., -1.,  0.,
        -0., -1.,  0.,  0., -0., -0.,  0.,  0.,  0.,  1.,  1.,  1., -0.,
         0.,  0., -1.,  0.,  0.,  0., -0., -0., -1., -0., -0.,  0.,  1.,
        -0.,  0., -0.,  0., -0., -0.,  0.,  0., -0., -1.,  0., -0., -0.,
        -0.,  0.,  1., -0., -0.,  0., -0.,  0., -1.,  0., -0.,  0.,  0.,
         0.,  0.,  0., -0.,  0.,  0.,  0., -1., -0., -0., -0.,  0., -1.,
        -0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,
         0., -0.],
       [-0.,  0., -0.,  0., -1.,  0., -0.,  0., -0.,  0.,  0., -1., -0.,
         0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,  1.,  1., -0.,
         0.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.,  1.,  0.,
        -0.,  1., -0., -0., -0.,  0., -0.,  0.,  0., -1.,  0., -0.,  1.,
         0., -0.,  1., -0.,  1.,  0.,  0., -0.,  0.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  0.,  0., -1.,  0.,  0.,  0.,  0., -1.,
        -0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0., -0., -0., -1.,  0.,
        -0.,  0.],
       [ 0.,  1., -1., -0., -0., -0., -0., -0.,  1., -0.,  0., -1., -1.,
        -0., -0., -0., -0.,  0., -0., -1., -0.,  0.,  0.,  0., -0.,  0.,
        -0., -1., -0.,  0., -0.,  0.,  0., -0., -1., -0.,  0.,  0.,  1.,
        -0.,  0.,  1.,  0., -0.,  0.,  1., -0., -1.,  0.,  0., -0., -0.,
         0., -0.,  1.,  0., -0.,  0., -0.,  0.,  0., -0.,  1., -0.,  0.,
         0.,  1., -1., -0.,  0., -0., -1., -0., -0.,  0.,  0.,  0.,  0.,
        -1.,  1.,  0., -0., -0., -1., -1., -1., -0., -0., -1.,  0., -0.,
         0.,  1.],
       [-0.,  0.,  0., -0.,  1.,  1.,  0.,  1., -0.,  1., -0., -0.,  0.,
        -0.,  0., -1.,  1.,  0.,  0., -0.,  0., -1.,  0., -0., -0., -0.,
        -0., -0., -1., -0., -1.,  0., -0., -0., -1.,  0., -0., -1.,  0.,
         0., -1., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -1.,  0.,
         0., -0.,  0., -1., -1., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0.,  0., -0.,  0.,  0.,  0., -0., -0., -1.,  0., -0.,  0., -0.,
         0.,  0.],
       [ 0.,  0., -0.,  1., -0., -0., -0., -0.,  0., -0., -0., -1., -0.,
         1.,  0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0.,  0., -0., -0.,  0., -1.,  1., -0.,  0.,  0., -0., -1.,  0.,
        -1., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,
         1., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0.,  0.,  0., -0., -0.,  0., -0.,  1., -0., -0.,  0., -1.,  0.,
         0.,  1., -0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0., -0.,
        -1.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,  1.,  1.,
        -0., -0., -1., -0., -0.,  1., -1., -0., -0.,  0.,  1.,  1.,  0.,
         1., -0.,  0., -1., -0., -0.,  0., -1.,  0.,  0.,  0.,  0., -0.,
        -1., -0., -0.,  0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  1., -0.,
        -1., -0., -0.,  0., -0.,  0.,  0., -1., -0.,  0., -0.,  1.,  0.,
         1.,  0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         0.,  0.],
       [ 0., -0., -0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0.,
        -1., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  1.,
        -0., -0.,  0., -0.,  0.,  1., -1., -0., -0.,  1., -0.,  1.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -1., -0.,  0., -0.,  1.,  0.,  0.,  0.,  0., -0.,  0.,  1.,  0.,
         0.,  0., -0.,  1., -0.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.,
         0., -0.,  0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,  0., -0.,  0., -1.,  0.,
         0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  1.,
         0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  1.,  0., -0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0., -0., -0.,  0., -0.,
        -0., -0., -0., -0.,  1.,  0.,  0.,  0.,  0., -0., -0.,  1., -0.,
         0.,  1., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,
        -0.,  1., -0.,  0.,  0., -1., -1.,  0., -0., -0.,  0., -0., -0.,
         0., -0.],
       [-1., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
        -0.,  0., -0., -0.,  0., -1., -0., -0.,  0., -0.,  0.,  0.,  0.,
        -1., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -1.,  1.,  0.,  0.,
         1., -0., -0., -1.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
        -0., -0.,  0., -1., -0., -1., -0., -0., -0., -0.,  0.,  0.,  0.,
        -0., -0., -1.,  0., -1., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
         0.,  0.],
       [-0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0., -1., -0., -0., -0., -0., -0.,  1.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  1., -0., -0., -0.,  0.,
         0., -1.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0., -0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,  0.,
         0., -1., -0., -0.,  0.,  1., -0.,  1.,  0., -0.,  0., -0., -0.,
         0., -1.],
       [-0.,  0.,  1.,  0.,  0.,  0., -0.,  0., -1., -0., -0.,  1.,  0.,
        -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -1.,  0., -0.,
         0., -0.,  0., -0.,  0., -0., -0., -0.,  1., -0.,  0.,  0.,  1.,
        -0., -0., -0.,  0.,  0.,  1., -1.,  0., -0.,  0., -0.,  0., -1.,
         0.,  0., -1.,  0., -0.,  0.,  1., -1., -1.,  1.,  0., -0., -0.,
         0., -0.,  0., -0., -0.,  0.,  1., -1.,  0.,  0.,  0., -0., -0.,
        -0.,  0., -0.,  0.,  0.,  0.,  1.,  0.,  1., -0., -0., -0., -0.,
         0., -0.],
       [ 0.,  0.,  1., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  1.,  0.,
         0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -1., -0., -0.,
         0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  1.,  0.,
         0.,  1.,  0., -0.,  0.,  1., -1., -0., -0., -0., -0.,  0.,  0.,
        -0., -0., -1.,  0.,  1.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  0., -0.,  1., -1.,  0., -0., -0.,  0.,  0.,
         0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,
         0.,  0.],
       [ 0., -0.,  1., -0., -0.,  0., -0., -0., -1., -0.,  0.,  1., -0.,
         0., -0.,  0., -0., -1., -0., -0.,  0., -0.,  0., -0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,
        -0.,  0.,  0., -0.,  1., -0., -1., -0.,  1., -0.,  0.,  0.,  0.,
        -0.,  0., -1., -0., -0.,  0., -0.,  0., -0.,  0., -1., -0., -0.,
         0.,  0.,  1.,  0.,  0., -0.,  1., -0., -0.,  0.,  0., -0.,  0.,
         0., -1., -0., -0.,  0.,  1.,  1.,  1.,  0., -0.,  0.,  0., -0.,
         0., -1.],
       [-0.,  0.,  0., -0.,  0., -0.,  0.,  1., -1.,  1., -0., -0., -0.,
         0., -0., -0., -0.,  0.,  0., -0., -0., -1.,  0., -0., -0., -0.,
         0., -0.,  0.,  0.,  0.,  0., -0., -0.,  1., -0., -0., -1.,  0.,
        -0., -1., -0., -0., -0.,  0., -0., -0.,  0.,  0.,  0., -1.,  0.,
        -0., -0., -0.,  0., -1., -0.,  1.,  0., -0., -0.,  0.,  0., -0.,
         0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
         0.,  0., -0., -0., -0.,  0.,  1.,  0.,  0., -0.,  0., -0., -0.,
        -0., -0.],
       [-0., -0., -0., -1., -0.,  0., -0.,  0.,  0., -0., -0.,  1., -0.,
         0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0., -1., -0., -0.,
        -0.,  0., -0., -0., -0.,  1., -1.,  0., -0.,  0.,  0.,  0.,  0.,
         0.,  0., -0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
        -1., -0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,
         0.,  0., -1.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0., -0., -1., -0.,
        -0., -0.,  0., -0.,  0., -0., -0., -0., -0.,  1.,  1., -0., -1.,
        -0., -0., -0., -0., -0., -1.,  1.,  0., -1., -0., -0.,  0.,  0.,
         0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -1.,  0.,  0.,  0.,
         1.,  0.,  1.,  0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,  0.,  1.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  0., -0.,
         0.,  0.],
       [-0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -1.,  0.,
         1., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  1.,  0., -1.,
        -0.,  0., -0., -0.,  0., -1.,  1.,  0., -0., -1., -0., -0., -0.,
        -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
         1., -0.,  1., -0., -0.,  0., -0., -0., -0., -0.,  0., -1., -0.,
        -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.,
         0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,
         1., -0.],
       [-0., -0.,  0., -1., -0.,  0.,  0., -0.,  1.,  0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0., -1.,
         0., -0., -0., -0., -0., -0.,  0.,  0., -1., -0.,  0.,  0., -0.,
        -0.,  0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  1.,  0., -0., -0.,  0.,  0.,  0., -0.,  1., -0.,  0.,
         0., -0., -1.,  0.,  0., -0., -1., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0., -0.,  0., -0., -1., -0., -0., -0., -0.,  0.,
        -0., -0.],
       [ 0.,  0., -0., -0., -0.,  0., -1., -0., -0., -0.,  0.,  0.,  0.,
        -1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,
        -0., -0., -0.,  1., -0., -0., -0.,  1., -1.,  1.,  0., -0., -0.,
         0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0., -1., -0.,  0.,
         0., -0.,  0., -0., -0.,  1.,  0., -0., -0.,  0.,  0.,  1., -0.,
        -0.,  0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0.,
        -1.,  0.]], dtype=np.float32), np.array([[ 0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  1.,
        -0.,  0.,  0., -0.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  0.,  0.,
         1., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -1., -1.,  1., -1.,
         0., -0., -0., -0.,  0., -0.,  0.,  1.,  0., -0.,  1., -1., -0.,
        -1., -1.,  0.,  0.,  1., -1.,  0., -0.,  1., -0., -0., -1., -0.,
        -0.,  0.,  0.,  0., -0.,  0., -0., -1., -1., -0., -0.,  0., -0.,
         1., -0., -1.,  0.,  0., -0., -0., -0.,  0., -0.,  1.,  0., -1.,
         0.,  0.],
       [-0., -0.,  1.,  1.,  0., -0.,  0.,  0., -1., -0.,  0.,  1., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0., -1., -0.,  0.,  1., -0.,  0.,
         0.,  0., -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0.,
        -0., -0.,  0., -0., -0.,  1., -1.,  0.,  1., -0., -0.,  0., -0.,
        -0.,  0., -1.,  0., -0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0., -1., -0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0.,  1., -0.,  0.,  0., -0., -0., -0.,
        -0., -1.],
       [ 0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,
         1., -0., -0., -0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,
        -0.,  0., -0.,  1.,  0., -1.,  0.,  0.,  0., -1., -0., -0.,  0.,
        -0., -0., -0., -1., -0.,  0.,  0., -0., -0., -0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  1.,  0., -0.,
         0.,  0., -1., -0., -0.,  0., -0.,  0., -1.,  0.,  1.,  0., -0.,
        -0., -0.,  0.,  0.,  0., -0.,  0.,  1.,  0., -0., -0., -0., -0.,
        -0.,  0.],
       [-0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0., -0., -0., -0., -1., -0., -1., -0., -0.,  0., -0., -0.,
         0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0., -0., -0.,
         1., -0.,  0., -1.,  0., -0., -0., -1., -0.,  0., -0.,  0.,  0.,
        -0.,  1.,  0., -0., -0., -0., -0.,  0., -0.,  0.,  1., -0.,  1.,
         1., -0., -1.,  0., -1.,  0., -0., -0.,  0., -0.,  0.,  0., -0.,
        -0., -0.,  1., -1., -0., -0.,  0.,  1., -0., -0.,  0.,  0.,  0.,
         0.,  0.],
       [-0.,  0., -0., -1.,  0., -0.,  0.,  0.,  1.,  0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0., -1., -0.,  0., -0., -0.,  0.,  0., -0.,
        -0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,  0.,
         1.,  0., -0., -1., -0.,  0., -0., -1.,  0., -0.,  0., -0.,  0.,
         0.,  0., -0.,  0., -0.,  0., -1., -0.,  0.,  0.,  1.,  0.,  0.,
         1., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,
        -0., -1.,  1., -0., -0.,  1., -1.,  0.,  0., -0.,  0., -0.,  0.,
        -0.,  0.],
       [ 0.,  1., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,  1.,
         0., -0., -0., -0., -0.,  0.,  0.,  1., -0., -0., -0.,  0.,  0.,
         0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  0., -1.,
        -0., -0., -0.,  0., -0.,  0., -0.,  1.,  0.,  0.,  0., -0., -0.,
        -0., -0.,  0.,  0.,  0., -0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
        -0., -0., -0., -0.,  0., -0.,  0., -0., -1., -1., -0.,  0., -0.,
         1., -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  1.,  0.,  0.,
        -0.,  0.],
       [-0.,  0., -0.,  0., -0.,  1.,  0., -0., -0., -0.,  0., -0.,  0.,
        -0., -0.,  0., -1.,  0., -0., -0., -1.,  0., -0.,  0., -1.,  0.,
         0.,  0., -1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
        -0., -0.,  0., -0.,  0.,  0., -0.,  0., -0.,  1.,  0., -0., -1.,
         0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,
         0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0., -1.,
         0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  1., -0., -0.,  1., -0.,
         0., -0.],
       [-1., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,
        -0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -1., -0., -0., -0.,
         0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,
         0., -0., -0., -1.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,
        -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  1.,  0., -0., -0.,
         0.,  0., -0.,  0.,  0., -0., -0.,  0., -1., -1., -0., -0.,  0.,
         0.,  0.,  1., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0., -0.,
        -0., -0.],
       [-1., -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0., -0.,
         0.,  1., -0.,  0., -0.,  0.,  0., -1., -0., -1.,  0., -0.,  0.,
         0., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -0., -0.,  0., -1., -0., -0., -0., -1.,  0., -0., -0.,  0., -0.,
        -0.,  0., -0., -0., -0., -0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,
         1.,  0., -0., -0., -1., -0.,  0.,  0., -0.,  0., -1.,  0.,  0.,
         0., -0.,  1., -1.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,
         0.,  0.],
       [-1.,  0.,  0., -0.,  0., -1., -0., -0., -0.,  0.,  0.,  0., -0.,
         0.,  0.,  0.,  1., -0., -0., -0., -0., -0., -1., -0.,  0., -0.,
        -0.,  0.,  1., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,
        -0.,  0.,  0., -1., -0., -0.,  0., -1., -0.,  0.,  0.,  0., -0.,
         0., -0., -0.,  0., -0.,  0., -0., -1.,  0., -0., -0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0., -0., -0., -1., -0., -0., -0., -0.,
         0., -0.],
       [-0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  1.,  0.,  0.,  0.,
        -0., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,
         1., -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -1., -1.,  0.,  0.,
        -0.,  1., -0.,  0., -0., -1., -0., -0., -0., -0.,  1., -1., -1.,
        -0., -0.,  0., -0.,  1., -1.,  0., -1.,  0., -1., -0., -1.,  0.,
         0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  1., -0., -0., -0.,
        -0.,  0., -1.,  0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -1.,
         0., -0.],
       [-0., -0., -0.,  1., -0., -1.,  0., -0., -1.,  0., -1.,  1., -0.,
         0., -0., -0.,  1.,  0., -0., -0.,  0., -0.,  0., -0.,  1.,  0.,
         0.,  0.,  1., -0.,  0., -0.,  1., -0.,  0., -0., -0., -0.,  0.,
         0., -0., -0.,  0.,  0.,  1., -1.,  0.,  0., -1.,  0.,  0.,  1.,
        -0.,  0., -1.,  0., -0.,  0.,  1.,  0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -0.,  0., -0., -1., -0., -0.,  0., -0.,  0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  1., -0., -1.,  0.,  0., -1., -0.,
        -0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  1.,  0., -0., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  1., -0., -1.,  0., -0., -0.,
         0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -1.,  1.,  0., -0.,
         0.,  0., -1.,  0., -0.,  0.,  0., -0.,  0., -1.,  1.,  0.,  0.,
         0., -0., -0., -0., -0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,
         0., -0., -1., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,
        -1.,  0.],
       [ 1., -0.,  0.,  0.,  1.,  0.,  0., -0., -0., -1.,  0., -0., -0.,
        -1.,  0., -1.,  0., -0., -1., -0.,  0., -1.,  1., -0.,  0.,  0.,
        -0., -0., -0., -1., -1., -0.,  0.,  1.,  0.,  0., -0.,  1.,  0.,
         1., -1., -0., -0., -0.,  1., -0., -0.,  0., -1.,  0.,  0.,  1.,
        -0., -0., -1., -0.,  0., -0.,  0.,  1.,  0., -0.,  1.,  0.,  1.,
         0., -0., -0.,  0.,  0., -0., -1.,  0.,  0.,  0., -0.,  1.,  0.,
         0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -1.,  0.,
        -1., -0.],
       [ 1., -0., -0., -1., -0.,  1.,  0., -0.,  1., -0.,  0., -1., -0.,
        -0., -0.,  0., -1.,  0.,  0., -0., -0.,  0.,  1., -0., -1., -0.,
        -0.,  0., -1.,  0., -0., -0., -1.,  0.,  0.,  0.,  0.,  0., -0.,
         0.,  0., -0., -0., -0., -0.,  1., -0.,  0., -0., -0., -0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  1., -0., -0.,
         0., -0.,  0., -1., -0., -0., -0., -0.,  0., -0., -0.,  0., -0.,
         0., -0.,  0., -0., -1., -0., -1.,  0.,  1.,  0.,  0., -0., -0.,
         0.,  0.],
       [-0., -1., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -1.,
         0.,  0., -0.,  0.,  0.,  0., -0., -1., -0., -0.,  0., -0., -1.,
        -1.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  1.,  1.,  0.,  0.,
         0., -0., -0., -0.,  1.,  0.,  0., -1., -0., -0., -1., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  1.,  0., -0.,  0.,  0.,  0.,  1., -0.,
         0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0., -0.,  0.,
        -1.,  0.,  1.,  0., -0.,  0., -0., -0.,  0.,  0., -1.,  0.,  1.,
        -0.,  0.],
       [ 0.,  0., -1., -0.,  0., -0.,  0.,  0.,  1., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -0.,  0., -0.,  0.,
         0.,  1., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -1.,  1., -0.,  0., -0., -0., -0., -0.,
        -0., -0.,  0., -0.,  0., -0., -1., -0., -0.,  0., -0.,  0., -0.,
         0., -0.,  0., -0., -0.,  0.,  1., -0.,  0., -0.,  0.,  0., -0.,
         0., -0.,  0.,  0.,  0., -0., -1., -0.,  0.,  0.,  1., -0., -1.,
        -0., -0.],
       [-0., -1., -0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -1.,
         0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  1.,  0., -0.,  0.,
        -0., -0.,  1.,  1.,  0.,  0., -0.,  0., -0.,  0., -1., -0.,  0.,
        -0.,  0., -0.,  0.,  0.,  1.,  0., -0., -0., -0., -1.,  1., -0.,
         0., -0.,  0., -0.,  0., -0., -0., -0.,  1., -0., -0., -0., -0.,
        -0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
        -0.,  0.],
       [-0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -1., -0., -1.,  1., -0.,  0., -0., -0.,  0.,
         1.,  1.,  0., -0.,  0.,  0.,  0., -0.,  0., -0., -1., -0.,  0.,
         0., -0., -0.,  1.,  0., -0.,  0.,  1.,  1., -0., -0., -0., -0.,
        -0., -1.,  0.,  0., -0., -0.,  0., -0., -0.,  0., -1.,  0.,  0.,
         0., -1., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,
         1.,  0., -1., -0., -0.,  1., -0., -1.,  0.,  0.,  1., -0., -1.,
        -0., -0.],
       [-0.,  0.,  0., -0.,  0., -0., -0.,  0., -1.,  0.,  0.,  0., -0.,
        -0., -0.,  0., -0.,  0.,  0.,  0., -0., -0., -0., -0., -0., -0.,
         1.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0., -1.,  0.,  0.,
         0.,  0., -0.,  1.,  0., -0.,  0.,  1.,  0., -0., -0.,  0.,  0.,
        -0.,  0.,  0.,  0.,  0., -0.,  1.,  0.,  0.,  0., -1., -0., -0.,
        -0., -1., -0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
         1.,  0., -1.,  0.,  0.,  0.,  1., -0.,  0., -0.,  0.,  0., -0.,
         0.,  0.],
       [-0.,  0.,  0., -0.,  0., -0., -0., -1., -0.,  0.,  0.,  0., -0.,
        -0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
         1., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,
         0., -0., -0., -0., -0., -0., -0.,  0., -0., -0.,  1., -1., -0.,
         0., -0., -0., -0.,  0., -1.,  1., -1., -0., -1., -0., -0., -0.,
         0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  1., -0., -0.,  0.,
        -0.,  0., -1., -0.,  0., -0.,  0., -0., -1.,  0.,  0.,  0., -1.,
        -0.,  0.],
       [ 0., -0., -0.,  0., -0.,  0.,  1.,  1.,  0.,  1., -1., -0.,  0.,
         0.,  0.,  1.,  1., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0., -1., -0., -0.,  0.,  0., -0.,
        -0., -0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,
        -0., -0.,  0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,
        -0.,  0., -0.,  0., -0., -1.,  0.,  0.,  0., -0., -0.,  0., -0.,
        -0., -0.,  0.,  0., -1., -0.,  0., -0.,  0., -1., -0., -0., -0.,
        -0.,  0.],
       [ 1.,  0.,  0., -0., -0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,
         0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  1., -0., -0., -0.,
        -0.,  0.,  0., -0., -0., -0., -0.,  0., -1.,  0., -0.,  0., -0.,
        -0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0.,  1., -0., -0.,
         0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -1.,  1.,  0., -0.,
         0., -0., -0., -0., -0., -0., -0., -0.,  0.,  1., -0.,  0.,  0.,
        -0.,  0., -1.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0., -0.],
       [ 1.,  0.,  0., -0.,  0.,  1.,  1.,  1.,  1.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -0., -1.,  0.,  0., -1.,  1., -0.,  0.,  0.,
         0., -0., -0., -1., -1., -0.,  0., -0., -1., -0.,  0.,  0., -0.,
         0.,  0., -0.,  0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  1., -0., -0., -1.,  1., -0.,  0.,  1., -0.,  1.,
        -0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,
         0., -0., -0.,  0., -0., -0.,  0.,  0.,  1.,  0., -0., -0., -0.,
         0.,  0.],
       [ 1., -0., -0.,  0., -0.,  1.,  0., -0.,  1.,  0., -0., -0., -0.,
         0.,  0.,  0., -1., -0.,  0., -0., -0., -0.,  1.,  0., -0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0., -0., -1., -0., -0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0., -0., -1.,  1., -0., -0.,  1.,  0., -0.,
        -0., -0.,  0.,  0., -0.,  1., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0., -0.,  0., -0.,  1., -0., -0., -0., -0.,
         0., -0.]], dtype=np.float32), np.array([[-0., -0., -0.,  0.,  0., -0.,  0., -1.,  0.,  1., -0.,  0.,  1.,
        -0., -0.,  1., -0., -0., -0.,  0.,  0.,  1., -0., -0., -0., -0.,
        -0.,  1., -0.,  0., -1., -0., -0., -0., -0., -0.,  0., -0.,  0.,
        -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -1., -0.,
         0., -0.,  0., -1., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,
        -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -1.,  0., -1.,
         0., -0.],
       [ 0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  1.,
        -0., -1.,  0., -1., -0., -0., -0., -0., -0., -0., -0.,  0., -0.,
         0., -0., -0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0., -1.,
         0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0., -0.,  0., -1.,
         0., -0., -0., -1., -0.,  0., -0.,  1.,  1.,  0., -0.,  0., -0.,
        -0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,
         0., -0.,  0., -0.,  0.,  0., -0., -0.,  1.,  0., -0.,  1.,  0.,
        -0.,  0.],
       [-0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,
        -0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,
        -1.,  1.,  0., -0., -0., -0., -0.,  0.,  0.,  0.,  1., -1.,  0.,
        -0., -1.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0., -0.,
         0.,  0.,  0., -0.,  1., -0., -0., -0., -0., -0., -0., -0.,  0.,
         0., -1.,  0., -0.,  0., -0., -0., -0.,  0., -0., -0.,  1.,  0.,
         1., -0.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -1., -0., -1.,
        -0.,  0.],
       [-0.,  0.,  0.,  0.,  0., -0.,  0., -1.,  0.,  1., -0.,  0.,  0.,
         0.,  1.,  1.,  1., -0.,  0.,  0., -0.,  1., -0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -1.,  0.,  0., -0., -0.,  0.,  0.,  1.,  0.,
        -0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,
         0., -0., -0., -0.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0.,  0.,
         0., -0.,  0.,  0.,  0., -0., -0., -1.,  0.,  0.,  0., -1.,  0.,
        -0., -0.,  0., -0., -0., -0.,  0., -0., -1., -0.,  0., -1.,  0.,
         0.,  0.],
       [ 0., -1., -0.,  0.,  0., -0., -0.,  1., -0., -1., -0., -0., -1.,
         0.,  0., -1., -0.,  0., -0., -0.,  0., -1.,  0., -0., -0.,  1.,
         0.,  0.,  0.,  0.,  1., -0., -0., -0., -0.,  0., -0., -1., -0.,
         0., -1., -1.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  1.,  0.,
         1., -0.,  0.,  1.,  1., -1.,  0.,  0., -0., -0.,  0.,  1.,  0.,
        -0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  1.,  0.,
         0., -0., -0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,
        -0.,  0.],
       [ 0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  1.,  0.,  1.,
        -0.,  0.,  0.,  0.,  1.,  0., -1.,  1., -0., -0.,  0.,  0., -0.,
        -0.,  1., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -0., -0., -1., -0., -0., -0.,  0.,
        -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0., -0.,  0., -0., -0.,  0.,  0.,
        -0.,  0.,  0., -1., -0., -0.,  0.,  0.,  0., -1., -1., -0.,  0.,
         0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0.,  0., -0., -0.,  1.,
         0., -0., -1.,  0.,  0., -0., -1.,  1., -0., -0.,  0., -0.,  0.,
        -0.,  0., -0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0.,  0.,
         0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -0., -0., -0., -0.,
         0., -0., -0., -0.,  1.,  0., -0., -0.,  0.,  0., -0.,  0.,  1.,
         0., -0., -0., -1., -0.,  0., -0., -0.,  0., -1., -1.,  1.,  0.,
         0.,  0.],
       [ 0., -0., -1.,  0., -0., -0., -0., -0.,  0.,  0.,  1., -1.,  0.,
        -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0., -1., -1.,  0.,
         0., -0., -0.,  0., -0.,  0., -1.,  0., -0., -0., -0., -0., -0.,
         0., -0., -0.,  0.,  0., -0.,  1.,  0., -0.,  0., -0.,  0.,  0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,
        -0., -0.,  0., -1., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -1.,
        -0., -0., -0.,  0.,  1.,  0., -0., -0.,  0.,  0.,  0.,  0., -0.,
         0.,  1.],
       [ 0., -0., -1., -0., -1.,  0.,  0., -0., -0.,  1., -0., -0.,  0.,
        -0.,  0.,  1.,  0., -1., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -0., -1.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0.,  1., -0.,  0.,  0.,  1., -0.,  0.,  1., -0., -0., -0.,  1.,
        -0.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,  0.,
         0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
         0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -1., -0.,
        -0., -0.],
       [ 0.,  0., -1., -0.,  0.,  0.,  0.,  0., -0.,  0., -0., -0., -1.,
        -0., -0., -0., -0., -1., -0.,  1., -1.,  0., -0., -1., -0., -0.,
         0., -1., -0.,  0.,  0., -0.,  0., -1.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  1., -1., -0.,  0., -0.,
         0., -0., -1.,  0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  0., -1.,  0., -1., -0.,  0., -0.,  0., -0., -1.,
        -0., -0., -0.,  1., -0., -0., -0.,  0.,  0.,  1.,  1.,  0., -0.,
         1., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  1.,  0., -0., -0.,  0.,  0., -1.,
         0., -0., -1., -0., -0., -0., -0.,  0., -0., -0.,  0., -0., -0.,
        -0., -1., -0., -1.,  1., -0.,  0., -1., -0., -0., -0.,  0.,  0.,
        -0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0., -1., -0.,  0.,
        -0., -0., -0.,  1.,  0.,  1.,  0., -0., -0.,  0.,  0., -0., -0.,
         0., -0.,  0., -0., -0.,  0., -0., -0., -0., -1.,  0., -0., -0.,
         0.,  0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,  1., -0.,  1.,
         0.,  0.],
       [ 0., -1.,  0., -0.,  0.,  1.,  0., -0., -0.,  0.,  0.,  0., -1.,
        -0.,  1.,  0.,  1.,  0., -0.,  0.,  0.,  0., -1., -0., -0.,  0.,
        -0.,  0.,  0., -0.,  0.,  0., -0.,  0., -0., -0.,  0.,  0., -0.,
         0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  1.,  0.,  0., -0.,
        -0.,  0.,  0.,  1.,  0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  0.,  1., -0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,
         0., -0.],
       [ 0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
         1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1., -0.,  0.,
        -0.,  0.,  0.,  1., -0., -0., -0., -1.,  0., -0., -0., -0., -0.,
        -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,  1.,  0.,
        -0.,  1.,  0.,  0.,  0.,  0.,  0., -0.,  0., -1.,  0., -1.,  0.,
        -1.,  0.,  1.,  0., -0., -0., -0.,  0.,  0.,  0.,  1.,  0.,  1.,
        -1.,  0.],
       [-0.,  0., -0.,  0., -0., -0.,  0.,  1., -1., -1., -0., -0.,  0.,
        -0., -1., -1., -1.,  0., -0.,  0.,  0., -1., -0., -0.,  0., -0.,
         0., -0., -0.,  0.,  1.,  0.,  0., -0., -0., -0.,  0., -1.,  0.,
        -0., -1.,  1., -0., -0., -1., -0., -0.,  0.,  0., -0.,  0., -1.,
        -0.,  1., -0.,  0., -0.,  0.,  1.,  1.,  1.,  1.,  1., -0., -0.,
         0.,  0.,  0., -0., -0., -0., -1.,  1.,  0., -0., -0.,  1., -0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  1.,  1.,  0.,  0.,  1., -0.,
        -0., -0.],
       [ 0.,  1., -0.,  0.,  0.,  0., -1.,  0., -0., -0., -0., -0.,  1.,
         1., -0.,  1.,  0., -0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,
         0., -0., -0.,  1., -1., -1., -0.,  1., -0., -0., -0.,  0.,  0.,
         0., -0.,  1., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,  0.,
         0., -0., -0., -1.,  0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,
         0., -0.,  0.,  0., -0.,  0.,  0.,  0., -1.,  0.,  0., -1.,  0.,
        -0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,  0.,
        -1.,  0.],
       [ 0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,
         0., -0.,  1., -0., -1., -0.,  1.,  0., -0.,  0.,  0., -0., -0.,
        -0.,  0., -0., -0., -1.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,
        -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  1.,  0., -0., -0.,  0.,
        -0.,  0.,  0., -1., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  1.,
        -0., -0.,  0.,  0., -1., -0., -0., -0., -0., -0., -0., -0.,  0.,
        -0.,  0., -0.,  1.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.,  0.,
         0., -1.],
       [-0., -0., -0., -0., -1.,  0.,  0., -0., -0., -0.,  0., -0., -0.,
         0., -1.,  1.,  0., -0.,  0.,  1., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0., -0.,
         0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,  0., -0., -0., -0.,
        -0.,  0.,  0., -1.,  0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,
         0., -0.,  0.,  0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,
         0., -0., -0.,  1.,  0., -0., -0.,  0.,  0.,  1., -0., -0.,  0.,
        -0., -0.],
       [-0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,
         0., -0.,  0., -0.,  0.,  1.,  0.,  0., -0., -0., -0., -0., -0.,
         0.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -1., -0., -0.,  0.,  0.,  0., -0., -0.,  1., -0., -0.,  0., -0.,
         0., -0., -0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  1.,
        -1., -0.,  0., -0., -1., -0., -0., -0., -0.,  0.,  0.,  1.,  0.,
        -0.,  1.,  0.,  0.,  0., -1., -0., -0., -0., -0., -0., -0.,  0.,
         0., -1.],
       [-0.,  0., -0., -0.,  1., -0., -0., -0.,  0.,  0., -0.,  0., -0.,
         0.,  1., -0.,  0.,  1., -0.,  0.,  0.,  1., -0., -0.,  0.,  0.,
         0., -0.,  0.,  0., -1., -0., -0.,  0., -0., -0.,  0.,  1., -0.,
         0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0., -0., -0.,
        -0., -1.,  0.,  0.,  0.,  0., -0.,  0., -1., -0.,  0.,  0.,  0.,
        -0., -0.,  0.,  0.,  0., -0., -0., -1., -0., -0.,  0., -1., -0.,
         0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,
         0., -0.],
       [-0., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0.,  0.,  0.,
        -1.,  0., -1.,  0.,  1.,  0., -1.,  0.,  0., -0.,  0.,  0., -0.,
        -0.,  0.,  0., -1.,  1.,  1., -0.,  0.,  0., -0., -0.,  0.,  0.,
         0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,
        -0., -0.,  0.,  1.,  0.,  0., -0., -0., -0.,  0.,  0., -0.,  0.,
        -0., -0.,  1.,  0., -0., -0., -0.,  0., -0., -0., -1.,  1.,  0.,
         0., -0., -0., -1.,  0., -0.,  0.,  1., -0., -1., -0., -0., -0.,
         0., -0.],
       [-1., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        -0.,  0.,  0.,  0.,  1., -1., -1., -0., -0., -0., -0.,  0., -0.,
        -1.,  1.,  0., -0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0., -0., -0., -0., -0.,  0., -1., -0., -0., -0., -0.,
        -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -1.,
         0.,  0., -0., -0.,  1.,  1., -0., -0.,  0.,  1.,  0., -0., -0.,
        -0.,  0., -1., -1., -1.,  0., -0.,  0.,  0., -1., -1.,  0., -1.,
        -0.,  1.],
       [ 0., -0., -0.,  0.,  1.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.,
        -0.,  0., -1., -1.,  0.,  0., -1., -0., -0.,  0., -0., -1.,  0.,
         0., -0., -1., -0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,
         0.,  0.,  0.,  0., -0.,  0., -0., -1., -0.,  0.,  0.,  0.,  0.,
         0.,  0., -0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0.,
        -1.,  0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0.,  0., -0.,
        -1., -0., -0., -1., -0.,  0., -0.,  0.,  0., -1.,  0.,  1., -0.,
         0., -0.],
       [-1., -0.,  0.,  0., -0., -0., -0.,  0., -0.,  0., -0., -0.,  0.,
        -0.,  0.,  0.,  0., -0., -1.,  0.,  0., -0., -0., -0.,  0.,  0.,
        -1.,  1., -0.,  0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
        -0.,  0.,  0.,  0.,  0.,  0., -0.,  1., -1.,  0.,  0., -0., -0.,
         0., -0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0., -0., -0., -1.,
         1., -1.,  0., -1.,  1.,  0., -0.,  0.,  0.,  1., -0., -0.,  0.,
         1., -1., -1.,  0., -0.,  1., -0., -0., -0.,  0., -1.,  0., -1.,
         0.,  1.],
       [-0.,  0., -0., -0., -1., -0.,  0., -1.,  0.,  1., -0.,  0.,  0.,
        -0.,  0.,  1.,  1., -1., -0., -0.,  0., -0., -0.,  0., -0.,  0.,
        -0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0.,  0.,  0., -0.,
        -0.,  1.,  0.,  0.,  0.,  1., -1., -0., -0., -0., -0.,  0.,  1.,
        -0., -0.,  0.,  0.,  0., -0., -1.,  0., -0., -0.,  0.,  0.,  0.,
         0., -1.,  0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0., -0.,
        -0., -0.,  0., -0.,  0.,  1.,  1.,  0., -1., -0.,  0., -1.,  0.,
        -0.,  0.],
       [-0.,  0.,  0.,  1., -0., -1.,  1.,  0., -1.,  0.,  0.,  1., -1.,
        -0., -0.,  0., -0., -1., -0.,  1., -0., -0.,  0.,  0.,  1.,  0.,
        -0., -0.,  1.,  0.,  0.,  0., -0., -1.,  1.,  0., -0., -0.,  0.,
         0.,  0.,  0.,  0., -0., -0., -1.,  1., -0., -1., -0., -0., -0.,
         0., -0., -1.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1., -1., -0., -0.,  0., -0., -1., -0., -0., -0., -0.,  0., -0.,
         1., -1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0., -0., -0.,
         1., -0.]], dtype=np.float32))

In [ ]:
#@title Verification
n = 5
m = 5
p = 5
rank = 93

verify_tensor_decomposition(decomposition_555, n, m, p, rank)

# Kissing number in dimension 11

This section contains the description of a sphere packing in dimension 11, discovered by AlphaEvolve, which shows that the kissing number in dimension 11 is at least 593. That is, it shows that one can arrange 593 nonoverlapping unit spheres in dimension 11 so that they all touch a common unit sphere.

In [ ]:
#@title Data
import numpy as np
import itertools

sphere_centers = np.array([[ -126140549599, -4345934944399,   470267207045,   263582420739,
   -214242641164,  6323834592662, -5619609064852, -1309426459793,
    402713598698,  2669019955225,  -434830723701],
 [ 4995872033121, -3686427151476, -2854162550811,  1374657172375,
  -1039798804579,  2439226626247, -3321352498532, -4613529709005,
   1873895977070, -2351434851488, -1740061033492],
 [ -148878709727,  6009198311530,  3423705484214, -3908379065200,
  -2039242115428,   740038679089, -1903248798804, -3229808619747,
    -67448542119,  2608302674396,  3361875535061],
 [-3613653483807,  -421965187481,  4386947861356,  4668066279141,
   2659643185631,  -793481959702,  -782545296485,  3044001988734,
   -210044264644,   732930704465,  5250176454220],
 [ 2145601851578,  2286907266338, -6008630565540,  -951491851263,
  -1128202168949, -2351035905822,   338686846605,     2773377260,
  -3167080097124, -2121084647622,  5631442707990],
 [ 2905871765916, -8266251805063,   212038997927,  -422612434475,
  -2967768531066,   917651742589, -1199305228751,  -428118279913,
   1046645354339,   629591995938,  3199722319491],
 [ 4101374281925,   140190714949,  7791359990956,  2617604236962,
   2117466506344,  1449617600436,   733498008403,  1078801552552,
  -1108251520223, -2353519449875,  -740114970023],
 [-4114734847133,  -711568804977, -1096919699416, -2179625341981,
  -3277074074174,  3846872693149, -1343845317573, -4029410886259,
   -760938481950,  4323960137424,  3708537789894],
 [ 3749853727043,  -920834513083, -2932082478667,   453628989161,
  -3091441942608,  6398413670681,  2921263471507,   652306845488,
   3309571127773, -1260515535742, -2071139339072],
 [-5502577676278, -1227448640831, -3806962231330,  -700239558766,
   -634751583528,   218595111082,    36644542589, -4043060252879,
  -4497661105875, -2852762755592,  2840093077865],
 [ 6431155805256,  2472183772617,  1852141756766, -3447375436498,
  -2150831199996, -1035995564180,  -582868441525,    -7570292152,
  -2212911701699,  4854953687339, -1645298702536],
 [ 5457305326303,  1209487773311,  3838331900010,   672841580707,
    640009753701,  -223039211201,   -29010721761,  4023147410765,
   4562777370943,  2817042290572, -2858123518221],
 [  947994596354,  -493063270445,  -891551370805,  1943900881453,
   4808222709486,  4576541247433,   121556039265, -2928496070412,
   2867556918429,  5676016954336, -1090838534345],
 [ 2487751367449, -4435069885581,  1203527162971, -7039059911283,
  -2789489469568, -1507124796768,   408730098019,  2199308898207,
  -2690789476039,  -886790999066,   246907513281],
 [ 1258393365327,  1558236082994, -2901279533502, -4588804764838,
    608788356464,    99979762711, -2072852177408, -5237300731547,
   2206084299760,  5428740521977,  -262884833863],
 [-2167736272270, -3517473061426, -2581444334945, -4875068237304,
   1316469763812, -5045504207465,  1421790102552, -1042617987235,
  -2433031020177,  1273290801609,  3828642503948],
 [-1748341091839,    84738171505,   491018476370,  4552281419561,
   -872666527835,  3684504527423, -4225299894261, -2607643958170,
   1747681580888,   -38267500797,  5824587161476],
 [-1190649965834, -4307368301932,  -590143473885,  1188487865738,
   1457925043503,  5074991372962,  -835020531624,  4328426802452,
   1938681290116,   658478446157,  5173244484302],
 [ 1157198248631, -3276093261966,  4564492907322,    71866472319,
   6569376668375,  2635857316126, -2535627189471,   666339223157,
   -370257470349,  2316182402253,  2146785163984],
 [-3008255020828,  3938057143766, -5277192922587,   593373194319,
    313114162483,  4123287795940,  1308453155442, -2544545673675,
   1482541632927,  -605248563516,  4403486139737],
 [ 2953917709599,  -440143619596,  -971489179779,  2328335269090,
   2051703342736, -1981036966235,  1096402607228, -8032142516776,
   1008884833142,  2051765017859, -2374176269185],
 [ -260449853094, -1557229191143, -2449904390021,  7358451024369,
   -307525874663, -2110837486887, -1514682853981, -3642026943603,
   -341788837820, -3181174130215, -2648178657093],
 [-4525545210765, -4561293261761, -2207465574022, -3656150761762,
  -1355527319115, -1325108959607, -2606830665241,  4286163870684,
  -2272423058824,  2270296581664, -1181276273370],
 [-4989184462869, -2211501213310, -3272216512438,  2932576526381,
  -4053121684822,  -741725216911, -1173137805746,  2181007677136,
  -2973820944500, -3480396984635, -2615745041603],
 [ 3602451243212,  1363170942462, -5335236817537, -5099315765146,
   1671801957653,  -733157544910, -1557322490939,  3489905697601,
  -2568635080802,  2482181047248,     9540974612],
 [ -117106507206, -1557986758662, -1613271333904,  3599924179366,
  -4726770521248,  -122491678154,  3003684160051, -2037648443680,
  -1689648678287, -6017572885613,  2720160888071],
 [  656948042076,  5303758392177,  1223385282487,  4208826988200,
  -2322770619544, -1931938705746,   194867179198,  -149785416225,
   -975234488353, -6149937849011, -2065621641353],
 [-2822884624434,  1200686951368,  -740362280165,  2549091748238,
   6473313457564,   198867059897, -5879821576926,  -497127882332,
   -583143250935, -1042480487756,  2313887084841],
 [ 2071972627918,  1689375919671,  2561773181285,  -215304283110,
   3678351753570,  -481570249929, -4708236211267,  -713440165548,
  -2024413488114, -6510658512889, -1821622848642],
 [ 1540576902788,   553319845306,  5192033407329,  3790472513109,
   -989994836245, -6751185485237,    63271511119,  1311050643347,
   -364367110835, -1981107885558,  1912214169188],
 [ 1043725858252,  1320396911651, -3526919864724, -4284357877757,
   1470366412833,  4775346083201,  1368745214981,  3701799302088,
   2580249413700, -3202483086523,  2985971881393],
 [ 2328981318412,   782277770357,  1613319529120,  6708838015807,
   2409644663632,  -164697695288, -2876391291223,  1405607425611,
   2563177561828, -4390790783825,  2102566367890],
 [-2898694569620, -1993213074975,  4767991564340,  -549183545871,
  -1716073975618, -1376827268341,  3962365401068, -1804381043737,
    364314410734, -2366368669369, -5921165923758],
 [-2821980692939, -2907092870119,   526484763216,  -283219412226,
   1816656338961,  4630761564997,  5094816643786,   822612609763,
  -1628794974160,  4497378787802, -2995063685088],
 [ 3981893553028,  6726656929267,  4509428321078, -1718973949677,
   1235696414272, -3105774701931,  -561577844505,   806709024917,
    670699852230, -1703554165369,  -340483077176],
 [     958740882,   492343935025, -4877156942860,  3610017499034,
   1303862660779,  2282799096236, -4071872206188, -4319287760002,
  -3382516964626,  3040863514676,   320653772456],
 [ 2668842066670, -3328224345221,  1245442050272,   807268381271,
   6338371398067, -1028063433204,  2718246800023,  1487663933151,
    641493721090,  2991619189786, -4404795224958],
 [  491355192733, -1231352019317,  2214609261146, -4246171762475,
   3060427833343, -6012660473699,   369697486180, -1926711613647,
   1222190747138, -4895424389279,  -694078468931],
 [-3936231732764,  3646035008845,  3917837393650, -2302689585385,
   -631606199075, -1190907291088, -1462454097742, -1026315700574,
  -3403121373084,  4358068529006, -3870064655165],
 [ -891693194815, -3159218172959,   -36815057657, -5392159737721,
   2695159539664,  4846777170940, -2774317263822,  2475655478805,
  -3507372987730, -1040335371092,  1477389122043],
 [-1909886271559, -3025194791002,   273889648930,    22368016637,
   1769343232414, -6711034204699,  1807271442431,  5498553429397,
  -2113841429687,  -965335059539,  -236521072887],
 [ 1421740442238, -4445189941322, -6335315158518,  -671536406806,
   2646092406700,  -638731456569,  -171872290986, -2000918638949,
   2251948982557,  2831806435076, -3619165775254],
 [-3348523808774, -3469324536046,   -76708109001,  1992777744163,
  -5049711606642,   581253431088, -3927998298964,  2296310725868,
   2561198403600,  3752242898555,  2365461437834],
 [  779042434547, -1416461625219,  1452341221883,  1439971242011,
   1285662467259,  2679379627727,  1926099800655, -1269351586896,
   2663459313927, -8084575368127, -2568350771324],
 [-3980831743069,  3637325407505, -1572468831509, -2438291232144,
  -3064062866153, -2476525715560,  4274880397145, -2716987458463,
  -1175083650793,  1660991539240,  4145711223394],
 [ 6967450268144,  2625304133625, -1401474346935,  1995863610145,
    763569629212, -3804173165507,   997773279985,  1860188439504,
   1442735815840,  1895879894694,  3664470120161],
 [ 3015840597743,  1234709422899, -6952463247052, -1298276880914,
  -1224429458570, -1308306783583,   972871140414,  2450262921520,
   5290015394161,  -560255643005,  -947955712831],
 [ -936946977342,  1116679407669,  -396124541576, -5766839382952,
   2927744211058, -1495168010655,  2861812090255,  3711225318549,
  -3673521240929, -3839839100081,  1858038850084],
 [ 1308510697687, -4987098459834, -5143597325210, -1869372147752,
   -978771635962,  3179840980566, -3422701861744,  3615532701633,
   1765018082471, -1744159035660, -1204896731728],
 [-1646730796381,  4421637104002,    37254831639,  4273667291949,
   -655020690827, -2640710658714,  1397627767521, -1309174476974,
   1380616410107, -2725730878362,  6250309303605],
 [ -627474373499,  -451185628535,  1125092239854,  1228838206359,
     26093010133, -2939555208566,  3370434196692,   667250715880,
  -7209365367712, -1252881093849,  4757673344364],
 [ -797420603404, -1227566289334,  3086021511548,  -505080489488,
   5478728185869,  4126710775002,   391076993732,   139481050671,
  -3647775845204, -2041750080177, -4835799186849],
 [ 4092334232882,   729220292791, -1168955935796, -3085250972755,
     82524549053,  5275269956420,  3666568911398,  2396726507160,
  -4862220845505,   358540800263, -1021153633823],
 [ 1756033032922, -1110377613430, -1249242600393,  2221908240877,
   -400217238665, -3426298784081,  6127013124122,  4593495905370,
   3265502690632,  1973356712550, -2021902581270],
 [-3725619678320, -2329459894461, -5070699197459,  1534301307867,
  -1037135676352, -3205158454204,  6159879240390,   559487604350,
  -1262339301196,  -862647461628,   826929767550],
 [-2274391587253,  -270713035752, -1330852865067,  -377690887956,
  -2008376261801,  4177033325257, -2292376198981,   547802074232,
  -7458187593121,  2184009598005, -2325794304528],
 [-1452869174536, -1495810745460,  3812294435170,  4056541456396,
  -1452578046306, -4834849869320, -1310486979894, -3907635677827,
  -2007229588715,  2874984727132, -3145687074383],
 [ 1794443008446,  -307543586686,  3579316950366,  4062541899933,
    930653960952,  1957509312467, -2003233153037,   999980394766,
  -3454594428395,  5958959150112, -3196146826361],
 [-1460561567534,  -916399636812,  4669810481354,  -882217985126,
   2825371943033,  4288562595538,  4110001703233,  3676245445344,
  -2776098864617,  -863483729868,  3034437958314],
 [-3278999167478, -1273384556037,   681284367484,  4877315854280,
   3219855412980, -1975019416928,   259552774384, -5776746289350,
  -1601605704779,   635789280058,  3563508544421],
 [ 3932398091349, -3647552677504, -3915156254426,  2300359279437,
    632049048201,  1190512933335,  1463079709812,  1024625838059,
   3408695534850, -4361112761007,  3868556093990],
 [ 3405094292821,  1309788195129,  2321854096188, -1882365030199,
   4076617425181,  6149102664930,  -142554192789, -1909405757679,
   -914754807934, -3485849481473,  2582854034473],
 [-2463493552091, -6182166095073,   698027052685,  5496165743777,
  -2223299965981, -3647879802948,   628799524443,   494699815054,
  -1002913242476,  -295445251725,  2243571445095],
 [ -729861928381,   580654590161,   736639591977, -1815111899007,
  -4839349528509, -4562434137639,  -151406490905,  3024687926959,
  -3192175323619, -5507105846599,  1169574501828],
 [ -944932369434, -3799932528535,  1269141877026, -1606029533014,
  -5042197520192,  1753508262775, -5757405933996,  1676387264954,
  -2659952371999, -2907761426523,   700425645446],
 [ -313412211989, -1472067839495,  2014918908248, -2994085643890,
  -3631601113505,   798675905998, -5151770012594, -4504700612925,
   4645803566421,   250361628870,  1550709051811],
 [ 2806524625245, -4911847282592,  2447813070402,  -867370776942,
   2711443083233, -1173006619677,  4693314097340,  1244183268233,
  -3834905781184, -3651786699409,  -952863639178],
 [ 5102102592725,    26765691116,  -331028209281, -2142605004855,
    884206303205, -6425311667055,  -950525299835,   262446895670,
  -4980233069174, -1053283462981,   562620518775],
 [ 1098954479782, -1109038396398,  1216752926622,  2017980461121,
  -7349092644345,  3487054908779,  1652822263220, -2099842431090,
   2295686494056,  1022158793060,  3519196027836],
 [-4358075667015, -3089508657759,  4802679891308, -4810354941838,
   4228949294611, -1392241901612,  1450152791677,   688600723787,
   -547780846332,   728656385898, -1423515444254],
 [-3368253979501, -1295226691162, -2347476680864,  1904689227772,
  -4081008654975, -6145377037947,   136414911035,  1925567171135,
    861357305081,  3514943751022, -2568129787920],
 [ 2231100299994, -4273492332869, -4596691742304,  3642873481075,
  -5001877658405, -1406691322979,  2412453357834, -1290390423599,
   2512433823420,   586628501749, -1104670360982],
 [-2575376667766,  -960065509919,  -340750042435, -3074326806221,
   5178574195494, -1775696830323, -2226386336847,  5177679668223,
    209007473419,  1808719719559,  4222211433121],
 [-3578650785329,  -760578973239, -3477963098591, -2048910481829,
   3686410787928, -3078459201831,  3150278587757,  2610639587537,
    384617180163,  5179605695847, -1880968201088],
 [-1453680890271,  -673688713741, -4079118060194,  6145074907773,
   2493898205845,  1386249452566,   358139860104,  1448264243842,
  -3561849396434, -1272570622967,  4284699431295],
 [ 3328068080720, -2292691401219,  1230258674419, -4178966504561,
   1952019534393, -5264768742287,  1386218014402,  2997466555571,
   2074496226701,  4120699917727,   985876677105],
 [ 2539767610843,    94765789165,  3854189084765,  1251736924810,
    288435856169, -2041205772562, -2845981392459, -4127782598606,
   4941323231817, -2767998454739, -3956333511561],
 [ 1067212670401,   -37369907419,  1058370258904,  -923482582767,
  -1672287471130,  1249103492221, -4785063288729, -5636554728192,
  -1540216098658,  2012650649787, -5607074089976],
 [ 2693775044582,  2069745791644,  2364479048946,  -861476754182,
   -369013705992, -6661486740994, -1991744606252, -3971515305028,
   1993645640090,  3364267727602,  1606968451704],
 [-1060152337718, -1462090647628,  4542812980901,  -128973585724,
   4479127796943,  2331049535271,  4969221611534, -4599291027477,
   1873218159741, -1057368339060,  -338512596613],
 [ 1828308410223, -5297151224782,  4618727669960,   -42450901494,
   1921798353915, -4281708414571,   722583002058, -2788707550640,
  -1892464744087,  1898148718778,  3123025242617],
 [-2974029255356,   -91718543918, -2789616442708,  -417822031141,
  -4472352298733,  4615977419360,  5600790978158, -2574878657168,
  -1894939235615,  -308842022774,  -442698710154],
 [ 1645018045050, -4422289912096,   -36103964862, -4274703871227,
    655235768532,  2640591254105, -1397378950407,  1308418198678,
  -1378166081991,  2724363162782, -6250975307286],
 [  433691886277,  2523890861790,  2284499873225, -3703627868154,
   -518512695555, -4938318927311,   257841134127,  1840194784536,
    184975564588,  -457909300815,  6793639833180],
 [ 3325316992112, -2486276617276,  1123273178383,  4294677510387,
  -1936032844161, -2776149438784,  -572327617931, -4803422897516,
   2993538631796, -1489999107747,  4125247597048],
 [ 2393744590379,  1049137728331,  5122417881385, -1088629195634,
   5105537095120, -2435589424350, -1707333453883, -3641898816074,
  -2375980295032,  1693223337093, -3009270903611],
 [   28402196897,  1416027777891,  3440429273442,   203961451796,
  -3940869930791,  2347105807829,  3199355476507,   922594830306,
  -4951120644270,  4761531968667,  2608391131483],
 [ -813227623899, -5357941337067, -1112669738057, -4304097048545,
   2345298447370,  1911013266658,  -174960144340,    85301205749,
   1204772867285,  6020219044345,  1995755808449],
 [  996433361992,  3887894764559,   -61480148325,  3439374904618,
   4912048980888, -5693090961926,  -928200494105, -3021190348807,
    299266571934, -2320681651943,  -233525862680],
 [ 2453568242579, -1987414549672, -4478808100402, -1731895787954,
   5278273305930, -3861728068050,  4222749196955,  1356061448728,
    966666015199, -1824003862323,   515695447649],
 [-2664724735182, -1495474258595,  2944595283119, -2919860019644,
  -2308099678557,  -818004793386,  2276432949055,  4387646832659,
   2527039579293,  4206496484419, -4353847210838],
 [ 2103427320648,  3064498166470, -2140607330478,  -214240562173,
   2185849739279,  3694612769131, -3992827456340,  1545977029970,
  -1036921145639,  4214246024451,  5095140041841],
 [-4722758966585,  1206959806325, -2218480002577, -2968990772048,
   4375690183492,  2762753454305,  -688043457918, -1384779569811,
    297008499206, -5649005618160, -1154239157355],
 [-2609408155367,   294141147806,  2298399803607, -1460949423253,
  -6206872086624,  3044769045395,   637152553136,  1635378523205,
   2401015430092, -4948485353184, -2135126898687],
 [ 3867944404697,  2385613461166,  4971200333675, -1447973537461,
   1019789091334,  3219226368043, -6182937104131,  -497220049095,
   1055477569740,   975358261409,  -770526134964],
 [  768762538555,  -861169573712, -3921515618745,   -22343685772,
  -3124204955684,   646496969665,  2154751469802,  5008107566907,
  -2780174801716,  4768474481990, -3594186429160],
 [-5829157725768, -1845011523627,  5847631650732,  -217123128185,
  -2762905325685, -2720689317598, -2609714245707,  2013419676738,
   1074499522941,  -866115882344,  -754946967281],
 [-1311298215779,  -486042628530, -2763893228432,  1526263490041,
   2632583689192, -3621344081841,  1368629186733,    20242687756,
  -3847994704863, -7116704442527,  -838416785108],
 [-4079307778929, -2848117033354,   180600291286,  3916227562361,
   1526038123223, -1150669460319,  3432721011725, -2556067021564,
   4840526733645,  2294106198479, -3035542058866],
 [  129849177296,  -895608414736,   739412179265, -3313217288439,
   1347265236537, -1350474981128,  8576800884753,   753529921447,
  -1070437463712,  2225825733472,  1946412591514],
 [ 1022976878190,   -45968183236, -4432158106829, -1058813047828,
  -4104820558761,   -36539537494,   952233064667, -7327066689318,
    687280832481,  -684194666759,  2408791472363],
 [  726100104866, -3850927701095, -5622396324275, -2466174786399,
  -2197077396011, -5197407057482,  -329370102200,  1038406981314,
   -989600011579, -2648022596253, -2435463239897],
 [ -165613374687,   552413143883, -1161665499778, -3926523097504,
   7762169581077,  3071383417728,  3006518773230,   704634129572,
   -541373108803,  1699918236493,   705136590918],
 [-2463707289161,     7334104944, -5254000589214, -1875851987073,
    575650872497,   193540586190, -4724021792447,    88382347075,
  -1404280131318,  -733098432327, -6132233566685],
 [-1068227955381,    36991913966, -1057698833338,   922844025654,
   1672425887977, -1249208449583,  4785238023229,  5636085487623,
   1541691946288, -2013439331427,  5606684229854],
 [ 2220517143322,   198705222749,   247604570500,  1479799498957,
   2489049282415,   814044846502,  3030144160090,  4590197712109,
   -561532095582,  7075362333048,  2300632700445],
 [ -156780111086,  2736931326151,   965004659863, -1831034245257,
  -3726675002694,  5210673474784,  1453882908988,  -361014855400,
   -136594164620,  3121012600730, -5929417932372],
 [-2949947628793,  2988361859580,  -212814868864,   903455432497,
  -2046005627366, -1876329775279,  6503528569315,  -277099644684,
  -1478109831366,  3366507572296, -4231829163727],
 [ 2045601175700, -1564278540916,  2774622143877,  -964561825269,
  -1253678239635,   552184872386,  3180838585545, -2911804386954,
  -6180323347514,  2204084547490, -4605681208142],
 [   55250421914, -7350188953636,  -641569884634, -1083937595197,
  -1091999530200, -2998210223345,  3830530928145,   490910348930,
  -1415454125947,  3413215490274, -2374004857697],
 [-5017343821737,  3677836937740,  2869185859152, -1387660752676,
   1042242648295, -2441379816081,  3324919305925,  4604118406050,
  -1842614982590,  2334378964167,  1731413185941],
 [ 4334119519474, -2595497670494, -1222943617868,  2697227087236,
  -2416592134269, -1744364781539, -4568500959292,  2410005633708,
   5468207643593,    52150443924,   492042756756],
 [ 2658920838129,  5641550619343,   442931548469, -1551669065028,
  -5542562030946,   579266633195, -3639887265300, -1186440616563,
   1501365270434, -1381312597597, -2936851176766],
 [ 1437696918439, -3901230133907,  5939123390411,   696495681099,
  -5307342745969,  -526543773849,  1855636404266,    60757669874,
  -3092578936240, -1971580173004,  1269547930945],
 [-1279198726336, -4494764829706,  1092293047395,  2850892770464,
  -3897572176262,  1684173370526, -2252708806981, -6118712302944,
  -2711864726485,  -639474370511,   732162729451],
 [ 2571548192351,  1062970499840,  5931954641540, -4827888529490,
    682286003349,  -442106562559,  2804703382020, -2023617203399,
  -3776836195923, -1111938053742,  2374317197047],
 [ 1987507400483,  2047084168016, -3002059435990,  1244072815805,
  -2542649156248,   176634825883,  7225040063273, -1155681023105,
   3321068587346,  -876174069295,  3077163956589],
 [-1461334920223, -5683558990067,  1458287405745, -3140510195587,
   -547138900823,  2658674539715,  3374598110250, -2852546789839,
  -4372452460752,   550693577616,  2694030473882],
 [ 2789684933477,  2418905805336,  3920543907820,  -531513340616,
   2693654810086,  1969467153943, -1396197508824,  5134579301295,
   2611529296474, -1045763809032,  4832372637798],
 [ 3650744567980, -4644629683627, -3699694543658,  2612553277032,
    519883653911, -1881417480910,   943481947691, -3790800920758,
  -4416251706114,  -408269127764,  2417850592918],
 [ 1436248473698, -6015909485541, -1585609778223,  -728279221654,
  -4061556287510,  2506355530637,  1623620972154,  -769589298542,
  -2935741997700, -3367508117025, -3568797013361],
 [ -644923126395, -2161877991026, -2767262539037, -5203279765784,
  -2774283868680,   612706970955, -3061605161094,  2123312292904,
    925203316429,   593138457327,  6084138308239],
 [  252405123934,  2206139084474, -4595639419590,    85111351867,
  -4782930422763,  -721735987789, -5272745523900,  4265015464793,
  -2085605756537,   408606626432,  -189359683156],
 [ 7053112995937,  -721686034826,  1665017815644,  3883332184132,
   -338594544788,  4332759688617,  -659180497339,   353063806376,
    -22371930753,  2334890792379,  2642228581106],
 [-2358290547732, -5112615059945,  1377813744578, -1209079881217,
   -964930564226, -3896741748046, -1290056495803, -1441252411451,
  -6180287803578, -2142260186258, -1515589161384],
 [ 2501097637297,  4123491123748,  4271158567686,  3389529547678,
  -3347701183653,  -578176305504, -2597493200365,    60111207331,
   4259394584395,  1251306377062,  3001572968103],
 [-3905847290960, -1488771800312, -2053247525676,   -90486946243,
  -1054900276341,  1382249202756, -2234835133169,  -955881342876,
    812023295000,  7153556583528, -4188021476324],
 [ 3894547097085,  2814755872739,  3839420830742,  1947626151590,
   1950087300099, -1442382122810, -2803223032330, -1748644554391,
  -3530905638874,   850911700296,  5327737153655],
 [ 5123151777017,  -404226532300,  3075206595856, -5134854073886,
   4638569941373,   465818284017,  -707657296426,  3205407676666,
   -665340777495, -1290658656591, -1774565213611],
 [ 1916117580566, -4093266494671, -5821590032850,  1263077964827,
   -497586117275, -2100172023666,  -682742738160,  3330924624014,
   -388213188429,  3880245331412,  3558210516981],
 [ 3401202788258,   810363437763, -5274150637157,  -928677899866,
  -2090316636540, -4612443348259,  2952211444677, -1956336686949,
  -1824801816226,  4161313173728,  -512573096978],
 [-6969575388595, -1819483463926, -1381333443046, -4002918516902,
    798912659548,  2344670318129,  3174898396419,  2144891020890,
   1607026711522,  -356366181421,  2580164864728],
 [  276997024511,  1029528219459,  -412748310462, -1893341701947,
   -764769161531, -2085125184027,  -779580331768, -5726504265913,
  -6450058000710,  3162432669128,  2273938482379],
 [-2006373209666, -2054493934083,  3015218165908, -1255518043090,
   2544926372296,  -178539759170, -7221911766451,  1147400549023,
  -3293605634553,   861176827029, -3084659981606],
 [ 1304027205357,  3324730053896,  -243321857757,  5642880845393,
  -2736638803105, -4801593338556,  2713255478063, -2296124924557,
   2924603069862,  1363744529287, -1323946982056],
 [ 1313958374220,  1081499628340,  4072809931350,  -172775867819,
   6779271255119, -3685975109542,  3079865241934,  1989147097123,
   -817455291683,  -329359888877,  2592743966517],
 [-1567783876789, -7384170965839,  3311780178305,  1327667060418,
    389718199022,  2636863689822,  -630307270725,  3399475322545,
  -3073903523076,   297542749995, -1299777063762],
 [-1504145081860,  -538826415777, -5217302414788, -3768713739576,
    985738688357,  6755155777720,   -69482645562, -1295245807684,
    311515406667,  2010148660475, -1897294536706],
 [-2082074589211, -4576770136906,  3060705179466, -1792361754528,
  -1929011305802, -1520940025193,  2120807677184,  4188855567239,
   -838713657044,  2987248595378,  4942862485457],
 [-1118160218577, -3932752314296,   145244832515, -3513492678208,
  -4897566395071,  5677719152947,   949385945492,  2966833768506,
   -121064747752,  2223158760683,   185123993948],
 [-2885810971379,  3921536918930,  1089916425343, -3072576745427,
  -1665297542424,  7397430565361,    96402644487,   723506423342,
  -1998710348122,  -790422177651,  1737077257039],
 [ 4824082055808,  3789854052554,   424544297045, -3281115028812,
  -2102394783795,  1626782367763, -3185969944244,  2411113323944,
  -3913025740612, -2319500673832,  2774360181845],
 [ -947768904213,   842278199349,  5370428414965,  -548373867687,
   3148686234987,  -737691635251, -2420660381267, -3985317011510,
   2700872971740, -3297024906174,  4345599604186],
 [-2483741090108, -4116493995975, -4283394863320, -3379040828527,
   3345662921050,   580094565410,  2594282927079,   -52488891633,
  -4284802307248, -1237329360849, -2994423995937],
 [-1098108373298,  3299541851152, -4605910635694,   -36129840727,
  -6576311788859, -2629800174936,  2525791880294,  -640513138117,
    284228178609, -2269218166260, -2123139014045],
 [-2219471355897, -6471651795388,   353742166342,   171015677393,
  -1471849306206, -1991285927739, -3589311739944, -1466374989604,
   3344329836540,   137984938737, -4546287909462],
 [ 2073182936613,   466328932539, -2791012405298, -3313588062417,
   3530568343429,  2216032040681,  2265865618174, -4346765336882,
   5293052453049, -2639133395520,  -573740069659],
 [  214459878412,    19398313189,  6305390579996, -3608487383784,
  -1989842234019,  3278350146195, -1223967877264,  3305477784465,
  -3617311292070,  -114403255569, -2635362871908],
 [  426321805749,  1137571596648,  -934133552682,  6233342815683,
    -48487549937, -4002542435362, -4889499143122,   703316459258,
   -975659268269,  3514709109664,  2251081605192],
 [ 2468565829596,  5155658424646, -1453270930266,  1276928271948,
    949302333786,  3903659962438,  1275142291747,  1489697404928,
   6018387529292,  2226900271067,  1554937693002],
 [ 2715506054844,  3482379220419, -1729557792903, -3606698334999,
   5937100067475, -1554453997960, -2096963359890,   -92230435877,
   1000574089023, -2751065007208,  3722722845190],
 [ 2634743781118,   983127432399,   299209752327,  3110492334599,
  -5185766104877,  1781713739917,  2216061204070, -5151489900582,
   -295402636687, -1761199754849, -4197899465723],
 [ 2077555732930,   283559312757, -2760538621384,   417300252388,
   5927503430910, -3533754014174, -1001230518933, -1223879516797,
  -2575219126684,  4882369112087,  2686317858147],
 [ 1168466204065,  3822971229690,  5029915887937, -1996991926131,
   3051301848435,  1525811987757,  4531837892615,  1633064178647,
   -970556221271,  2800621530552, -3332129692759],
 [-4333405640177,  2595803503469,  1222444806936, -2696811891176,
   2416496360838,  1744487793328,  4568361186596, -2409674071826,
  -5469243574926,   -51579532841,  -491734613348],
 [-2467630147606,  1981816676657,  4488642911129,  1723347036133,
  -5276606227799,  3860287962711, -4220417788713, -1362249656124,
   -946152098270,  1812822630005,  -521332300988],
 [-2241001781198,   -40972216213, -4450445816436, -1830900109084,
    583181392280,  2373038873262,  2677962053627,  2142841574856,
  -4906715376677,  3515275831386,  4201176544176],
 [ 3566116295130,   755631566161,  3486726151340,  2041359806509,
  -3684933736093,  3077109175398, -3148124103181, -2616165387334,
   -366438653115, -5189581246319,  1875913477081],
 [ 1169756207380,  4450182319364, -1017069826463, -2917632011119,
   3909587453197, -1692287000160,  2268879223154,  6069084923479,
   2871909406721,   554215792900,  -771496783680],
 [ 4545643710456, -2839300538053,  3196872591342,    48408958543,
  -5617612880939,  2851958971972, -2444934555215,   415011797637,
   1397334760407,   814873213931, -3549001395164],
 [ 1369104443879,   508626624505,  2723357463617, -1490777202574,
  -2639761015160,  3626693760144, -1377786536700,     5248435782,
   3763602126491,  7162511645981,   861345608385],
 [ 2440131171132,  -819355214094, -1455810769125,  -601774225686,
    415079064241,  -101645112079, -2020387519010,  5515392925687,
  -6019543383458, -2866131618202, -3429160170996],
 [ 3404935097258, -1475777975123, -1624555070751, -3064900974353,
    510268804990, -3238776936905, -1707968738275,  6124575621962,
   1783373494878, -3781535254565,  2353218057669],
 [  480568627596,  2119861335709, -2131094079761, -6433151784280,
  -4224803047866, -4467323734356, -2216351640087, -2233676451907,
   -916228259636,  -116451429973,   887544630281],
 [ 1347340652764, -2425982744085,  1305009012156,  4911066205519,
   4558713915155, -3083628705040, -1527150020658,  1092432828781,
  -5648998324766,  -375301281712,  -780636116878],
 [ -645955447182,  -573412844198,   291621049695, -2851751604036,
  -6178291223918, -2042697299143, -1653484381312,  1144334097639,
  -3019079742109,   208920449350, -5957391471904],
 [  643872500447, -3889012601565,  2040770000899, -5164439015432,
   3090436298776,   367803434308,  2592146963002,  -244364271528,
   3210036084153, -1870466177752,  4832892053998],
 [-3489708056613, -3565689589022,   421784280972,  4382563289631,
    822777100831,  1565898196956,  4032550397657, -2172439184083,
  -4750428726450, -2287378729032, -1953049895133],
 [ 1510363990797, -5503286576671,  -695235405965,  2412916532682,
   -599696475475,  2886241647929,   526584803957,  3231823250970,
   3842006991659,  4548131061588, -2503718301923],
 [ 1226958249894, -2773201525784,    91343677483,   909406635811,
   2053979594022, -3961278384597, -6239413826246, -5274237794401,
  -1407879540787, -1106055587451,   323498175969],
 [ -754260506420,  4933069749976,   -59774358666,  -581839150403,
   5435911970421,  1679010859911,  -379642495186, -6244963647764,
   -688048364149,   979863489122,  1344747317080],
 [-4466010900252,  1515606562093,   561844871854,  3992079077555,
   1161311755868,  1990294207776,  6492035972605,  -485351519506,
   -252078871908,  1773771427134,  3256301020790],
 [ 1800158889991, -4861798591592, -2635881641748,  3375896166871,
  -2368461074210, -1053116459665, -4421962342629,   680808497648,
  -3685268727265,  2081875272447, -3182211796350],
 [ -700568222425, -2059326140183,   969918550260, -2331659486757,
   6358449782368,  -702075227607,  -515434118465, -3327861842950,
  -4776211464527, -2744252145422,  2506941608789],
 [-2707039197917,  2328954056365,  4390223123745,   -73968588581,
   2565737297374, -5510058522172,   906559073621,   430873253591,
   3999558820412,  1152761661004, -3562554875280],
 [-1024528772430,    45347036460,  4433248776900,  1057871180179,
   4104998210599,    36381024375,  -951957791240,  7326387516491,
   -684996202517,   682937228904, -2409432229150],
 [ 4067741710999, -2566271397519, -3256492925837, -4386848618314,
   1886947461174,   944956097375,  -461004655030,  -898479697604,
  -1812451258699, -6184293797316,   121525174369],
 [ 1315366752583,   800997282555,  4566987593264, -6895175556057,
   -699226297501, -1451542326893,  -328352244017, -1928137156462,
   3132392662035,  1114118165728, -3428239828999],
 [ 2144150939231,  1554529906139,  1257893659262,  -353150017642,
   2410536308459, -3423149484282,  1256725264007,   506784869501,
   7845923503507, -2077267572141,  2457775205201],
 [ 6273929664447,  1986709087805, -4968720868214,  -284932371925,
   2539623792742,  2613082443799,  1482794570863, -3206489923520,
  -1170089623014,  1375141403761,  1703863316471],
 [ 5113286992062,  -262519979148, -1205611478137,  4656492845107,
   1575614313201,  2901229953062,  1651661934635,    53279269656,
  -4041644034623, -4390196272182, -1187317734356],
 [ 3978464984049, -3638245937238,  1574138884919,  2436835214069,
   3064334755163,  2476356395527, -4274505026665,  2715967838611,
   1178536371096, -1662873006476, -4146618641126],
 [ 3350770810383, -6123869174151, -1791629481518,  -101508661756,
   3779184489497,  5473608843151,  1686970086472,  -501827889720,
   -750568283023,  -129282778282,  -360452279019],
 [  474331919901,  1699069111081,  1367225341153, -3383752095864,
   4667453902113,   151303632031, -3058063988967,  2202965005450,
   1165308647261,  6289559138375, -2576773702522],
 [-2005176305822,  3377760252313,  2836300312586,  4011574435132,
   4826243835769,  1579668182124,  1692136264531, -1004400577170,
  -2904950274070, -4442988350488,  1610486381879],
 [-1169046415570, -2309917566597,  5637115874876,  3572460874826,
    816413649445, -2106133162533,  5137698753257,  2181218129160,
  -2153312905457,  2606837452574, -1034268872376],
 [ 2474067050690, -1851274671465,  2426267545725,  1659327951792,
   -906277412481,  2756227575435,  3811770018044,  4421784098451,
   -842976716678,  -721246831441, -6170974772918],
 [ -750066121913, -1354049898449, -3732247107281,  1991239201577,
   4538863103650,  1845719188469, -3121715318647,  4478824024069,
  -3273309473431,  3661450392158, -1332495781935],
 [ 3433238320095, -3537063600267, -2635906680144,   261302237613,
  -2759443456705,  3386536019652,   949511068238, -2950096444044,
  -2047497480251,  5963423843188,  -493262884399],
 [-4105722097165,  -141913655347, -7788314434300, -2620215156652,
  -2116962346128, -1450024151373,  -732778649936, -1080692149409,
   1114563921787,  2350123655098,   738406232725],
 [ 2200620897542,   705226451779,  2723763190353,  3782000913440,
  -1482406284152,  3009948542426,  5720928376417, -1025025527459,
   2998989283179,  3937668088876, -1844267429759],
 [-4001613157751,  3378934508215,   409158451737,  5494366397472,
  -3327412136028,  1079984219353, -2631470696356,  4020893842667,
   1536948449196, -1726419747723,  1241540389589],
 [  888673029462,  2258870927984,  2595865553564,  5350662068447,
   2747261226114,  -588630358854,  3020228413670, -2016224262166,
  -1280527406098,  -400562929971, -5988204141971],
 [ -784448296601,  -864748064806,   559798999754,  1688253545921,
    240653265587,  1648793052913,   -17280348977,  5756292949770,
   6805727075975, -3089022346053, -1923969062188],
 [-2732454644794,  1183497496595,   127337376432, -1243692020947,
   2061785440351,  2186341094142, -1357024113404,  1082754121094,
  -1857273260280, -3914021204478,  7665252583820],
 [-1236793825863, -2333318951451,   211305234637,  2283356120529,
  -2629243609853, -5271440586761,  1723316268126,  -415823216033,
   4601979636555, -5151650335502, -1485309781336],
 [ 2821642382102,  2906951566716,  -526257070986,   283003410036,
  -1816610494370, -4630780065571, -5094770360351,  -822804971694,
   1629306854930, -4497641960600,  2994931491381],
 [ 5757045691793,   187120190092,  3814983805307,   225282311117,
    632342828617, -4181668994943,  5188218310499, -1815462636634,
    503375258259,   788144965033, -1800630830432],
 [ 2678559131102,  -555673908285,  1769279437351,  2948585676564,
   4394538007670,  2199184553610,  4013515708219, -2604227000232,
  -4707767382466,  2690870493540,  2061431279089],
 [  379190190073, -2714425600161,  5334870609655, -1772639083263,
   1735877144851, -1931477068771, -2679193262334,  3669975714046,
  -3492163017355, -3229146038935,  3297234694602],
 [ 1518980188259,  4863668733161,  2236438537843, -1739700465146,
  -2429011831799, -2695460666052,  4238729018848,  1324838215596,
   3326618888075,  4511503228801,  1304426815049],
 [-2033226249345,  -405083959238,   463453849128,  2230853372887,
   1182847258739, -6433452357703,  3685783962573,   116931247361,
   1510246758153,  3814439635060,  4158424407346],
 [ -951517934441,  1675539718712, -1336285940935, -1645421904301,
  -2336910100273, -2747786607563, -2184017666043,  1806679730034,
  -2597175549173,  7708279758434,  2141425780852],
 [-1083879915157, -4640600124494,  4167562887098, -2758177942872,
  -3592361677113,  -279859805331, -2652509510016, -1477927943039,
  -2277043972392,  4944375002945,  -691862540020],
 [-3898439760039,  -257945608795,   323292417029,   151022835822,
   4501929287046,  -634192149356,  5746049858676,  2593260480066,
   2269947530578, -3311852390405, -2841763833478],
 [  141864048396, -2742792494647,  -954616381445,  1821875393772,
   3728473011512, -5212023826256, -1451554866067,   354522574070,
    158471783785, -3132772093270,  5923484286679],
 [ 1071769248680,   334438752973, -3760862096385,  3089607262596,
   3359666382977,  6606247254319, -2968708276988,   215987412493,
   2515190151398, -2201877893616,   292432061488],
 [-1230530900302,  2771727322867,   -88812425037,  -911548894030,
  -2053580890639,  3960865394137,  6240083048958,  5272634654412,
   1413139988991,  1103161460414,  -324993353976],
 [  -86998330244,  1058762300965, -5641212497529, -3083499428291,
   3245614970178, -3524730716672,  -698745775891, -5232054699530,
  -1592394393518, -1715033802990, -1114791669436],
 [ 4809659318974,  -149233818971, -5314475619759,  2436109542017,
   3216650845950, -1668718122847, -4689437464696,   970733506968,
  -1702018773990, -1901992922741,   295923281742],
 [  569079240735,  -615142865604, -1688765527873, -3981623417149,
  -2356414755070, -2577678633195,  5070931344481, -3371149123615,
  -4372302060906, -3469182077811,  -405136618554],
 [ -900259136377,   812233153653, -2247324243777,  5222932711560,
  -2659657768943,  5571312149424,  -298641624508,  2116878044993,
  -1279522203441,  4522091249610,  1195898864850],
 [  902141610581,  3298918161640, -3342757931425,  -682634882119,
   3097205669747,  1059949340361,  1339024616970, -1293379261324,
  -4136728035009,  5057105421452, -4449093393518],
 [  -73353143388,  -815510154657,  2852365403161, -1111929579292,
   1441350269881, -2025644047242,  3093622515804, -3125572943450,
    285987548666,  7905010645558, -1365933247070],
 [ 2465951952348,  6328200956618,    18797023916,   194486333831,
   1417719448242,  1679612858144,  4017966457568,  1466430593215,
  -2758941377335,   454029325736,  4784493105961],
 [-4109098251814,  4344566738087,  1241576641643, -1336655497978,
  -2058584846899,   801204721188, -4497140463018,  1970940831292,
  -3318281163024, -3749067380961, -2618551918968],
 [ 1648454925486,  4747015799246, -3471514447645, -3561631541054,
    201564745507, -2919312907498, -1240784771082,  1963570428022,
   -488321627252, -5111461208275, -3112577709569],
 [-2551031962307,  5068904330152,  2119147294801, -3762883681579,
   2615881186145,  1034576539254,  4104081891625,  -547162595789,
   2990336877677, -2732468275682,  2771968680612],
 [-3245165420869,  -170031800418,  2008342982470,  3956781345125,
  -1036193413253, -2028762948150, -4275902223348, -1111504962239,
  -3408998867350, -5018672581355,  2869446580626],
 [ 2180933012478,  -546194667748,  3575643651836,  2455368469823,
    389037085827, -2570081736166,  1314204101992, -5721618825168,
  -3927732407186, -4401729105455,  -328809010366],
 [ 3945590192550, -1756704785784,  -137454169152, -1789453902934,
  -2979714527466,  1252082126175,  2555339462242,  4431903196124,
   4945122565474,  1738074490083,  3745956922482],
 [ 2259619080308,  4590487947210, -2251005923542, -1947039143183,
  -2494199878457,  3514421935768,  2391274822796, -2570700740949,
   -561771487022, -5792662736345,   440604120366],
 [  682425135818,   587302795151,  -317869972891,  2871329834968,
   6175415094364,  2049437638613,  1643654972210, -1128116887017,
   2969019410103,  -180019819214,  5973785736564],
 [-4671784204114, -3636066015725, -4541700480367,  2150041272614,
   3184135409938, -4505677577062, -2292223611054,  -700095384547,
  -1667730528470,   331259751111,  -792720660805],
 [  931616503018, -1118731315337,   399845528585,  5763565676285,
  -2927098487698,  1494653556681, -2860994615124, -3713538466516,
   3681274686970,  3835698208950, -1860120472308],
 [-2660324687047,  -191977827658,  4155438805119,  1558555138949,
  -4256397452858, -2912318234252, -1266917535702, -2756313453107,
  -1000147214125,  2275249532232,  5586692736924],
 [-3399778343912,  6159145363346, -3063878260556, -4689453897494,
    165191722795,  -292410107528, -1867435094055,  2612690604473,
   -333003423324,    77320108261,  2930013664243],
 [  531147775146, -2170328967690, -2293201254125,  5367543080577,
    121326811130,  4429621544785,  3170478944287, -5098314400427,
   1619930635276,  -576670726531,  1516748305541],
 [  327608737716,  3811429821160, -3932622782162,  -826291283558,
   4066076816263,  3047328830449, -1581534415805,   320899075464,
  -5970199045313, -2078284885189,   911827996371],
 [ 4518785347160,  -388700493896, -2490485817584,  5359924247762,
   2449270871360,   498109529865,  1367907884520,  -424892833282,
   5044293942255,  -116702307384, -3277333634716],
 [-6525162079463, -1181354204005,   624956783856,   358842171217,
   3469986050917,   255099458968,  -915621384173,  3284176993775,
  -5185525820581, -2168287046212,   431412415300],
 [-2941652726062,  3110504473894,   821394292308, -3588079192813,
   2753639156092, -2618552523411, -5756456571727, -2919392392152,
   2519754256098, -1237747428259, -2035256728565],
 [ 5048423056420, -2313715010096,  -761025759937, -1804248447970,
   3211761556085,   726679387348,  1117003513562,  2661551201487,
  -3278751911642,   339342731636,  5940141443264],
 [-4042579749446, -6750360135928, -4466751606843,  1681946903260,
  -1228916050969,  3098704888002,   571813546750,  -833218627166,
   -581856657591,  1654660302331,   315409511224],
 [-1035333893018, -1442118298059, -5617578658270,  1864867197505,
   3156317086217,  3338066033401,  4743005829326,  2175921776371,
   3365096684350,  1379639723178,   500414999183],
 [  -11427126637,  7367594985755,   610962666431,  1110263111105,
   1087256905897,  3002689697382, -3838425077956,  -471264988323,
   1351394896230, -3377921610308,  2392062008724],
 [ 1358649557230,  -462924873050,  2962306138089,  4756468859486,
   3530257458269,  3657347279216, -3369408194109, -5111547699335,
   -689873076083, -1041547577939, -1288422971606],
 [-1702441183899,  2379753603282, -1977514619740, -6150487283618,
  -3941633834821,  2272934930821,  1398178305322, -1214292023464,
   5041711426171,   290921398220,   252589314494],
 [-1789898852424, -6045978389703, -2407782197450, -1215663486795,
   -669694319880,  -197516091123,   875316813228,  3264967464244,
  -2738960314402, -4523497111234,  3617945509846],
 [ -690529249041,  -551543126829, -1669071497316, -3584852110174,
   -472677388588, -4932348321410,  4822503098792, -3153350238000,
   4686045472618,  1009143113924, -1689075334590],
 [-3403775108391,  1476258452330,  1623721323956,  3065601595098,
   -510393100108,  3238892394793,  1707768905743, -6124079953293,
  -1785076817159,  3782462175439, -2352725566758],
 [ 2396927824505, -1801977200565,  1370352050487, -4811131694918,
  -3379559646078, -1225803264230,  2281600137297, -3168189949940,
    886080762421,  3919090990916,  4655234166250],
 [-1503549869037,  1212735318063,  1076597747714, -2066580829533,
    368020916081,  3447110116613, -6157411151982, -4483552814227,
  -3629149843989, -1778234244327,  2114705819539],
 [ 4462028326936, -1517183397319,  -559064623985, -3994470582582,
  -1160814586133, -1990740040881, -6491343119880,   483609499542,
    257883507284, -1776945142174, -3257932052113],
 [ 3614730897172,   422382984848, -4387680174406, -4667356932295,
  -2659814430804,   793535112010,   782431041005, -3043534264706,
    208539181470,  -732100135520, -5249789136291],
 [-1647666307448, -1710410004391,  3156752345801,  4341228612939,
   -578114935938,  -138079452183,  2126032864910,  5033459758869,
  -1675582532975, -5703048434848,    83595080804],
 [  -71611541693, -2208735633139, -3149735813939,   174588134464,
   3916295713013, -1950257452486, -3625851960195,  -222009405516,
   5048747575851, -4908852220951, -1800386877248],
 [ -757479774973,  1842556725253, -5335399910162,  2409191958230,
   1896290788129, -4407192826846,  2176298076868,   267030096684,
    748547596232,  -790242619193, -5722661824073],
 [-1708067341610,  3778225389386, -5739863819085,  -849033045991,
   5333610382016,   503226217254, -1805456153799,  -183182963432,
   3486590152969,  1757208882586, -1379371915984],
 [-1372830112462,   -26521209511, -2124080741896,  1688948421360,
  -2531925342951, -7620134073784,   133402939606, -3907749557550,
  -2114497263376, -1692611313949,  1910221036734],
 [ 6964785946583,  2532233513343,  -655135734296,  -587153760045,
    750596564141,  1991363355268, -2056984275819,  1289181637869,
   3749221382394, -4405034503841,  -650008468001],
 [-1729882284524, -1344800787370, -1317512698711, -2649263530170,
  -2705350082060, -1070400469548, -2711924816032, -4646510085482,
    934003050063, -6279053663051, -2955746292466],
 [-1274233595323,  5412938535530, -5424784225187,  2834538558180,
    -96685163611, -1338972520537,  2243850168311,  2928084938293,
   -273640652930,  3877623040267,  1044965244792],
 [ 2898149547620, -2990432543296,   247745768225,  -937802201036,
   2045083409460,  1891270956134, -6501832935269,   252749135059,
   1534982709221, -3375257889106,  4227489575329],
 [-2269865736199, -4594524252363,  2258191837167,  1940818929425,
   2495428789378, -3515406037013, -2389572202205,  2566217428711,
    576709992011,  5784526702722,  -444671457975],
 [-2245342351779,  3384856863440, -3568161851038, -1754563009282,
  -1186980873045, -3900929284580,   629702048159,  1640587456022,
  -6769181311722,   545232058360, -1362935439985],
 [-1437312260121, -1847697111240, -3049619556769, -7835377675664,
   2880335165688, -1331949009768,  -945504108148,   875356400851,
   2987731743090,  -466280584155, -1718203090590],
 [ -278532810318,  2179880920650, -2011458323430,  1781485798113,
  -2859562978183,  1726766616329, -1351503498302, -2376402753467,
   7188032423483, -4181862900095,   403013457950],
 [-4066954318074,  2566602669850,  3255939520713,  4387344278805,
  -1887028511272,  -944891597795,   460897739197,   898848728882,
   1811289058759,  6184899102792,  -121259486508],
 [ 2345548484243,  1039005251056,  -365378882558, -1226402450103,
   2673509261365, -3721614510411,  4030589000797, -5334143996077,
   -142721473165, -1006686008224,  5005068766952],
 [ 4341459031588,  4484224767691,  2332866018647,  3544769452216,
   1375523081600,  1309149831450,  2636685662476, -4361177622051,
   2528756086563, -2417713797695,  1103603852692],
 [  750664436098, -2148943522252,  4008663719182,  1762059704738,
  -1869060518421,  7708557739294,  1945574678840, -1790633954168,
  -2004669972305,  -687062970437, -1116542776830],
 [ 4329647159964,  1262405360551,  2440979596453,  -879866742125,
  -1927676754522, -2131337489055,  3443513668473,  6450458962922,
  -3211827601687,   305451932838,   891100160463],
 [ 2193404209828, -3317090496399,   122821783843,  -978804771565,
   2462904558619,  2600348067812, -1585390611262, -6654730960632,
    297410577503,  1642402629124,  4560722388450],
 [-2409481474165, -1064732547599,   410769153834,  1188086537465,
  -2665858946672,  3714006215367, -4019149086204,  5305475113973,
    235643584566,   955830409373, -5030810211646],
 [ -845161422135,  3925130330303, -2077708073922,  5179976208925,
  -3200810750409,  -723860328055, -1555830229190,  -337924643751,
  -3427840044559,  2255422092479, -4752793863350],
 [-1168114860623, -3822826448574, -5030151779254,  1997211921609,
  -3051362506056, -1525745276917, -4531922239602, -1632910473648,
    970029768864, -2800300569351,  3332289694638],
 [ -581794081007, -2595079464845, -3123555538763,    -6773249377,
  -6874918744212, -1391281133731,   532315418523,  -112614262325,
  -5048355315029,  1671291716108,  2322738285131],
 [ 2338012133440, -1336683953743, -3115196655662,   613370202792,
  -2374543281464,  5833833429157, -1156933975158, -1302463235103,
  -4390871575893, -1174256566617,  4393255672731],
 [ 3891039367281,  2404273366108,  -564822641231, -1541134362753,
  -1423497284225,  1936471871239,  -454771626662, -2158090210442,
   3181298476237, -1668814410250,  7269052382347],
 [-2922513445504,  8259732626733,  -200487366516,   412388800769,
   2969837437489,  -919097665817,  1201968335381,   420914703816,
  -1022507411409,  -642740879182, -3206284945675],
 [  413661737289,  1618152381840,  2340126452014, -7269061056510,
    290262527334,  2130166416306,  1486826857866,  3708222574137,
    117242906970,  3307994981739,  2712013942179],
 [-3403108019382,  5241633649204, -3092714393079, -2995093677834,
  -2318344914351,  1801341565116,  -992301959469, -4822779014794,
  -2128535474010,   262292422313, -2223912831128],
 [-3171278517329,  1036972010098,  -163496287217, -3097558909873,
   3999139351372,   601001380339,  1711936961218, -3465559911694,
   1956544678212,  1185749576185, -6535813520357],
 [ 2579929796494,  1485575743088,  4112675668364,  1154406856819,
  -2756282810637,  -860769984091, -4468232066595,  5422364409564,
    699865213083, -3564103950269, -1411548383478],
 [ 1316062228161, -3831074514174,  5731057985101,  3994159989126,
  -1191850856492,  -436276944900, -5541018219297,   -41243974686,
    181432992600,   610688088654,  1439173634386],
 [-2667525179584,  4965987709141, -2544676139347,   951835957644,
  -2728676268728,  1186067930727, -4715622352279, -1182564942954,
   3632600866661,  3760780591228,  1007304374680],
 [ 2453277201468,  -491631561492,   786969316878,  3033260699249,
  -4049219737779,  -289916809971, -1342406267171,  3788364791102,
  -2045268192298,  -519415590365,  6843451545319],
 [  366765670903, -1200692417917, -1429467588701, -1496779860285,
   1391348379768,  4134240253425,  1275738234114, -7138858643012,
  -3162845062064, -2649152656940, -2345184615911],
 [  914341258819, -1781328446830,  5225313203447, -2312903454284,
  -1915917711758,  4421371639718, -2200482167067,  -197760517690,
   -979348654132,   914118035842,  5782922630427],
 [ -782180416810,  2052558781591,  2470844921564, -5517471215525,
    -84460969577, -4465076441973, -3115934462584,  4968069001646,
  -1239746927806,   356061150341, -1621444586483],
 [ 2355142884752, -1017634208111, -1736346495382, -3010885033044,
   6053769566543,  2670424431904, -4806248354162, -2995681693375,
   -210057713591,      889905356, -2332347547099],
 [-2645384386648, -3900273217887,  1163765540114, -3009688646456,
    666557069730,  -688711131228, -1335498699546,  3861250470782,
    304172533591, -5656942428963, -4203841385314],
 [-2615974363572,  1031046264966,   468394941374, -4180199374433,
   3651122727237,  4505931176975, -3351586434352, -1101201563112,
   4177012293650,  1099521763330,  3108680079658],
 [ -136897195983, -1081886913472,  -657609981089,  6686299501168,
  -1254690698723,   245381970475,  1924363243205,  1922466896425,
   5223373108159,  1821950711408,  3746342618508],
 [ 2424156496536, -2198726064092, -2884756557450,   903276598819,
    975991121091,   758060057008,  7373144182348, -2427240019001,
   -338080780553,  -425095511148, -4251396499489],
 [-6325490988486,  2598656058563, -1207708054039, -1211334362887,
  -5737679409125,  1245158976971,   244137249672,  2617541998517,
  -1033672732075,  2668395449981,  -859029787814],
 [-2772603497675, -5685522069233,  -361553801111,  1482882840208,
   5553003984629,  -593738427560,  3659694568985,  1136926900543,
  -1335602128256,  1289653471776,  2887293051407],
 [-4095417183648,  4839098221725,  -465435241876, -2479348325824,
   1395032968339,  2992221080202,  2556976049731,  -799439188010,
   1550778200364,  5741502567849,    19751812624],
 [-6908934403500,  1531241071359, -4706719133239,  2128676614342,
   1734681290776,  2186163589365,  1764336556458,  -387799201584,
  -1686620354607,  1057206905840, -2867643679802],
 [-2140503593039,   561984352407, -3604049075333, -2430617584736,
   -394188466297,  2573789311727, -1320334788844,  5739390159704,
   3868688028118,  4433547836917,   344374165714],
 [-1893666539574,  1659730122758,   946544030408,   -76189385339,
     20559221468,   571107687498,  2129920519145, -5678538610919,
   6180191631216,  2276775563167,  3502877120755],
 [-2794745255581,   372415077265,  2972006867864, -2348892576132,
   3501669213866,   161804686635,  1734915599637, -2037765678905,
  -1587118565352,  3999728050819,  6303453485896],
 [ -968718573149,  5689743280001,  1952593569916, -5300349284827,
  -3695445383604,  -448512674898,  3189704448263,  1223541944931,
  -2084765359924, -1293321077701, -1798445430447],
 [-1525591131074, -4866300238791, -2231803949435,  1735707161763,
   2429785224139,  2694759754916, -4237612395193, -1327733563264,
  -3316991137642, -4516772902518, -1307125169001],
 [-4466940599372, -1647027909491,  2531558090634, -3028270226670,
    910724778055,   192877798160,   344148556492, -1765984879127,
   6830022701532, -3059479270272, -1281072994700],
 [ 1541635846422,  1362353363053,  -198072058243,  -316239710940,
  -4095584293555,  5659216425601, -4483688773694,  3175096487836,
   3329121932947,  -665912022636,  2261940867051],
 [ -437105715614, -2149437760686, -3236107456867,  6005716132078,
   1287626975401, -1604415904986,  2739810393095,  -592828819392,
  -2493162444723,  5324874624305, -1409804529466],
 [ 1397647866505,  7312735332159, -3192838943251, -1429753386275,
   -370962974316, -2656027893092,   659706429203, -3466751520431,
   3313054721054,  -432560781839,  1234078745832],
 [ -293708146266,  4440039470869,   432358239719, -5887353939842,
   3500817164191, -4164343477674,  1982663104286, -1491944365746,
  -1311915602379,  2801491320417,   167661571787],
 [-6163626891441,  -136261226569, -2176276951445,   622796697888,
  -2011615445323, -3964961823186, -1210594725027,  2980786895674,
   -904406545254,  -448590894926,  5072449632079],
 [  385073785918,  2021446299101,   936796987436,  3667782702405,
  -1063153037472,  5814427729415,  2760137975099,  1761773698065,
   1632899051403, -3975044593693,  4160269612796],
 [  109714879158,  -120550740434,  6360099365214, -3047956367250,
   -457719420703, -1620627096654,  3283373657054,  4116387541021,
   2998192254453, -3266540573839,  -109342959566],
 [ 2816298045683,  5843722832103,  2492182163736,  1654309583123,
   -639788473056,  3284375995947,  -433160286357, -2696718584479,
  -5261469995038,  -372036857907, -1579178515081],
 [ 4664308006134,  -395601340473, -1392181786846,    91212707612,
  -4217859713708,  -609179267588, -5649307507006, -2511802374320,
  -1501802234682,  2648283927022,  3238387930209],
 [ 4735606214913,     6260817852,  1358663626156,    84369215894,
   3699432657352,  2648486372163,  6050758041547,  2492903514265,
   2945068298062, -1838695677162,   374376326382],
 [ 5417322970567, -2065248616302,  -519393174767, -5889580242966,
   -750460081537,  4570124985149, -1835866028015,  -901827283530,
   1979879462268,   962684788193,   975100689454],
 [ -911512194716,   565281354632,   -70196043250,  1921790595333,
   -415630650321, -6556423890214, -4662785919001,  1404684530327,
  -1785704456781, -2705015278984, -4215613080425],
 [-1418676255514, -2529630289464,  2732865434791,    42265025956,
  -3126937889025,  -229710682698, -1326022638311,  1808071533522,
   4989560834424, -3938532880436,  5371430314699],
 [-2154770173062, -1655489775893,  6405363148531,   546322166172,
   1272231190292,  2843001042335, -1745559831129, -1136676524727,
   3206356279165,  2677931970476, -4449370082130],
 [ 3167144720180,  3400465286298,   209022404118, -2100076134849,
   5075564382316,  -609283725349,  3961560836596, -2381212947343,
  -2292497261332, -3903117524233, -2442422997698],
 [ -288840861068,  1594920895700, -1527469002679,  2656358099910,
   3564908985902,     4911908294,  5131175474934,  4519124677249,
  -5082505477121,   280208763309, -1616468280120],
 [ 2158636040022,  3513858812998,  2587785249793,  4869552631870,
  -1315394109919,  5044557560708, -1420213024157,  1038640418726,
   2446280530658, -1280515133208, -3832271169069],
 [ -161207628592,  1424471926574, -2257894915604, -7117922473962,
   1060663107915,  2350965116796,  1047783051663, -3719425115065,
  -1954167589074,  -747088199132,  4027537035630],
 [ 2970021068420,    90202386471,  2792402654131,   415340747262,
   4472871111723, -4616263033670, -5600231136442,  2573077535170,
   1900778557469,   305707229798,   441162857580],
 [  175691387717,    12983192677,   811036497227, -3740551373609,
  -4422972150623,  1993335857013,  4512377635188,  1617341950259,
  -1397734151434, -2806884824984,  5382729663013],
 [ 2260116487747,   -31441253112,  4462528514067,  5599611951951,
   3850480954428, -2189852968882,   216461989754, -1718012042775,
   2842688854909,  3517794520944,   738397699658],
 [ 3510207103853, -6112307016110,  3003367419328,  4758336321777,
   -190958098368,   297496767962,  1871066810970, -2557954314180,
    169590084297,     4470543314, -2908332380072],
 [-4056853878728,  2088231267532,  2306312239637,  1295837021287,
  -5357441368825, -1313994440561,  1542310809410, -3065882661537,
  -5088082843024, -1662287496516, -1150674485511],
 [-4196431697300, -1517019629998,  -164153174035,  1261121010131,
   4237229602749,  6496602069360,     5994676524, -2510946868028,
  -2367512895552,   453987039040,  2489010883646],
 [-3812995858287,  -446150972528,  1029791281779,  3279436620691,
    277503140652, -3743425449490, -4126554994269, -1774179768184,
   6042491720568,   293725272166,  1606998591499],
 [-3777648277536,   909754842129,  2951619154550,  -470607509255,
   3094849665213, -6401365658946, -2916636724929,  -664522333345,
  -3269020243665,  1238465293599,  2060088943158],
 [-4168834440227,  3619474011711, -2374851631283,  1294784688238,
   1360399529916, -4471080017925,  -235437613868, -4339691111753,
    240612553421,  4458017410701, -1241947169126],
 [-3956823318298, -4744591949417,   -90763831885, -3724676022218,
  -5814213810699, -2038890118243,  1502989889968, -1565786109659,
   1831121126423, -1007317820938,   954166256105],
 [  762150529743,    -4708941881, -1237543599039, -2548647433955,
    145343016379,  3056651884393, -3645833955480,  -334735077049,
   6467857952495,   653714211045, -5134497014594],
 [ -750814928953,  2148867259446, -4008573099659, -1762178348707,
   1869101610402, -7708544182978, -1945580238424,  1790570978275,
   2004905471401,   686933180014,  1116499587732],
 [ 4083148129093, -4439835127532,  5235858794965, -2414215105043,
     82444671866,  2444225410125,  3331102481012,  1208773168150,
     63373379130,  3437159794961,  -128913033634],
 [ -773814964513,  -222898812318,  3568293909779, -2909424648814,
  -3401226932222, -6584682394845,  2936928259888,   -88014711735,
  -2948957039760,  2421586839552,  -184232961144],
 [ -815764090125,  1944969591284, -4012422151591,  1422208542377,
   1299999338763,  -259239012726, -3562650436673,  3226679983667,
   4740167343852, -1370950528653,  5312210159669],
 [ 1604476820817, -4429392743063, -5529218278565, -4407828402920,
  -1777715771558,  1355318695197,  4339320904962,  -380491261787,
    843969667101,    30503134227,  1766372319428],
 [ 1900004359421,  3193119945072,   879428022823,  2140011849071,
   1518589953933, -5905485332377,   915396365000,  2769300395353,
  -2479123920671,  4561357908670, -2863744936299],
 [-1774755740880,  -548200206493, -2969017914696, -3529848401167,
   1437887855970, -2989713695603, -5746601802979,  1192653194931,
  -3584817931168, -3609419357469,  1986008658016],
 [-1625213889932,  4421119860839,  5543654927871,  4395226412743,
   1780196769319, -1357444062802, -4335770064592,   371441570966,
   -813723749688,   -46935696425, -1774620718433],
 [ 3715263790834, -2507441356067,   916933214642, -1140143779240,
  -1613057939667,  1637675316378, -4053977038224,  5891778174886,
  -2113162615126,  4002114501913,   924535278642],
 [ 3559719361406,  3593611015819,  -470681462291, -4340318713557,
   -831229915233, -1558565867977, -4044317435861,  2202917506989,
   4648560925856,  2343087643899,  1981376332508],
 [-5149326266331,   425278277647, -2358074079089, -4157768090984,
   4060365081707,   363375547590, -3004681046676, -1898453697484,
  -3150592017271,  3351575609492,   214239081516],
 [-1885574942447, -3414768961929, -2237633145328,  1009180982198,
  -1258984424464,    83569138486,  2748852970600, -2104550426355,
    845115067889,   278675248156,  8024293216243],
 [-2616397571692,  -557163011524,  3825473785314,  -264303622221,
   3632263036396,  1472196243522,  3117970572795,  2829316578089,
   5699698566962,  3100518803696,  1718335559607],
 [-2001171217114,  3769109590606,  4843242734316, -2207726731685,
     20425655823,  3295100129708,  -666418356220, -4366181203628,
    719822543059, -3684959001579, -2999207897268],
 [ -416440675793,  3966595907281,  5404744038843,  2659058883616,
   2152299922905,  5211934991105,   283496230637,  -898514893236,
    536756726137,  2884298612590,  2553271449435],
 [  335206399612,  2429619549291, -1339859634853, -2402137072018,
   3268570292751,  2164671951558, -6122786239087,  5227011286623,
   1193309401770, -1771237023356, -1295498554102],
 [-4535646281527,   381955225200,  2502373372548, -5370195363497,
  -2447289461544,  -499927932217, -1365080082561,   417493881829,
  -5019672401578,   103242561889,  3270547937599],
 [ 1320294581199,  4358886860887,   498976946644, -1109624493511,
  -1473532016269, -5062368760463,   813784390254, -4271521122302,
  -2127754988860,  -555742065341, -5121863011168],
 [  319954682508,  1644930456827, -4048011124793, -3586693983952,
   2262601583584, -1140899394730,  2693872741624,   452698386592,
   3558061465140,  3516425128220,  5388238523679],
 [  576511139734,  1064716969756,  3503843991704, -3344260267626,
  -7471489976141,  -941661781581, -1519077763886,  3002302559918,
    663039771013,  1605726864235,  2005922589557],
 [ 4250970101161, -2011115256118, -2442314255295, -1179417975855,
   5333733062342,  1334393134560, -1574038371390,  3150621287536,
   4805338266734,  1816225427635,  1226974967377],
 [ 4983683345524,  1852620815355, -3743000131140,  1791614456828,
   -567655115089,  2415036182526, -3770153436093,  1353588916043,
    865467369408,  4540416849173, -3308391327027],
 [  653875695174, -2088800981864,  -396296213420,   574777457479,
   6775229964712,  1746981249329,  -217765778774,  4178116225117,
   -436521096935, -5077695128414,  1512824196198],
 [ 6338261765354,  1103907876255,  -495193503598,  -471410004876,
  -3451108817434,  -276797009173,   953344819002, -3358985667948,
   5441258081614,  2023964355435,  -508173452306],
 [ 2536643074779, -3692952713363, -1574451231015,  3508714524187,
  -2094648436830,  2337704889084,  5790495109123,  2579128237328,
  -2852311810587,   988057334451,  2447749092585],
 [ 1107940979252,  4585036193841,  2271843849403,  -965363534190,
   1620650767048,  3533508575939,  1463699849379,  4695963528445,
  -1064452003406, -5208530782449, -2021923639670],
 [-5180192026516,  -682001819054,  3365130146399, -1146825897072,
    833223008031,  3879974708325, -2289723829805,  3278672359678,
  -1386497537237,  4975083926998,  1283100178528],
 [-1173309723898,  1972452553536,  2451361482700,   640154543412,
  -1461649408553,  3335640582544,  2487877767457, -5842578928918,
  -2412034499213, -1801630687007,  5065751601085],
 [-1861323243252,  7134713267597,  1043143150418,  2407895374026,
  -4374893776181,  2563292203874,  2861480832555, -1695338029534,
   1327685478187,   348924414534,   297206271745],
 [-3160682237136, -1293519066942,  7051223326360,  1212818181618,
   1240982054689,  1292819826873,  -946419879752, -2511751426302,
  -5074675901170,   439922048758,   885621582510],
 [ 2188964509271,  3776040312346,   599471719195, -3057340945717,
  -1053173182133,  1489062314445,  5484454041137, -5695084326043,
   -279665896719,  2114489660823,  -924268607476],
 [-2476893720297,    34883886820, -3998421890342, -3476509194583,
   -303750020300, -2556222197138,  1420669576580, -1107807790381,
   3628549044514, -5341968597365,  3770306885879],
 [ 5580243107323,   870088036249, -3653724378671,  1377611388532,
   -854444995382, -3805450560520,  2204562554778, -3085267538823,
    792883755047, -4628634029265, -1101316943758],
 [ -443073828773, -2527575553027, -2277870089923,  3697915945056,
    519660245092,  4937386511160,  -256261999794, -1844351342821,
   -171254223366,   450416274550, -6797384473024],
 [-1614063693797, -5687794257250,   631118897913,   614539461954,
   3872879641616,   667496606968, -1141353611169, -4459858887340,
  -3008967932029,  3376122946463, -2679170226183],
 [-1077503001626, -3179364363585, -6355780313745, -1760217878449,
   4672660384934,  1558429221160, -1551256510307,  -821218393081,
     54578924800,  -901856944419,  4132401386776],
 [  474012047316,  -418492364892,   935216300467,  4101362436092,
  -7784530923283, -3021692423649, -3072488345516,  -563620609805,
     92653544193, -1445327407673,  -565170540846],
 [ -727291774990, -1199846378894,  1522043036747, -6424513445156,
    690286592565,  4094735859791,  4355456852779,    48083592644,
    762082613769, -3107376393562, -2826339611100],
 [-4785539510058,  -195122238138,   -87015655469,  1355600434426,
  -1894721462168,  6343721346916,   895517818241,    69205432358,
   5249358217569,  1388620904496, -1044879626466],
 [ 1403217816097, -2759955368282,   901952695799, -2830822998223,
  -2369047932369, -1967888992644, -1727162378428, -3656626225376,
  -2806465725575, -3912460635556,  5706484915651],
 [  900933303598,  3782303821132, -1238220147270,  1579435167099,
   5047188700148, -1757508394950,  5764458100935, -1695557742093,
   2723593858156,  2873081511427,  -717836154020],
 [-5591360915532, -2299262997255,  1023283552056,  6123785470364,
    542599258027,  1749957055294, -3776886321330,  -428384580942,
  -1507417469899,  1669069049272, -1427924670054],
 [-4320802718315, -4408013786734,  -951706198440,  -404213484961,
   5292587337125,  3116007011938, -1327154063206,  1881795726692,
   3074349217745,   540156817422, -2839732225352],
 [ -833007262294,  4275286650201,  1266984179126,  3194602883065,
  -2303239298218, -1413718556553, -3341802901801, -7042332430762,
    194373106103,  -911851377636,   531440032197],
 [ -503498478826,   641644035566,  1642707177561,  4021124148244,
   2349026710612,  2585350178490, -5082390229479,  3399573781690,
   4277067791291,  3520956838531,   432036574956],
 [-1075519744251,  -657866402426,  2279633816118, -1583392676961,
   -872446027515,  3055025066170,   613936841094,  -472056376972,
  -5999177194784, -6602001415793,   660273253248],
 [ 5458222620096,   726821298710,  2895790786049, -2423330058281,
  -2830370552335,  -594589717906,  1054714318464,    54724324427,
  -2323222848422, -5038937807997, -3892710328751],
 [ 4925348223428, -4238808160264,  1883829333124,  -696988168671,
   1612785621524,  -904493180526, -3575203377554, -2438749496283,
   2613419908066,  4676413794434, -1700373938607],
 [  789135193341,   876771811315,  2994340714412,  8164545731350,
  -2280921572686,   485848730350,  1031053925182, -1120268957357,
  -3550513617260,   344133324190,  1581566277484],
 [ 1148805594811, -1982179045540, -2434235799568,  -655088635527,
   1464554557163, -3338144399259, -2483848910744,  5831855229633,
   2447698493683,  1782260254513, -5075501792497],
 [  759758233706, -5381479911596,  5463015225044, -3259655955371,
    419547755608,  1674286216232, -1590599768081, -3251143232941,
    385436596648, -3472422706879, -1306550293429],
 [ 5850570351917, -2844347723006,  1152180391276, -1028321395205,
  -1803891869981,  2328543877781,  4261580125936, -4060654542183,
    -83855060725, -2791238368926,  2042396090035],
 [ 1021382773756,  -522249914981,    -6452390609, -1854192881588,
    401939677286,  6565397550965,  4646202339613, -1356365726256,
   1625089296953,  2790761783288,  4257201916257],
 [ -916864521284,  4882119279119, -3037114480034, -3598840689545,
  -1856465954116, -2498825285240,  -490199796195,  1425503727696,
   1881741783977,  3800615982163, -4816634486056],
 [ 3228333574343, -3775735676247, -1337034225973,  3272909243213,
   1640070794750, -7339972984229,  -162276523455,  -567635823245,
   1489156798299,  1078987798315, -1589561147745],
 [ 1009310519958,  6374344250331,  1321053132808,   278696672762,
  -1448952220730,  2205414665864, -1469892187714,  5270798397805,
  -2122826126532,  3721495116100, -1124741123393],
 [ 3884090917285,   989485866732, -2756010325512,  2997744873691,
    613459920307,  3384135229556,   581056758324,  7411009364290,
    161905946025,  -432413260354,   223835611333],
 [   -6416220723,    62298005769,  3509315492983, -4613741636480,
   2127034776469,  5178045787303,  -446050563912, -3615014478561,
  -2191147944382,  4018469496647,  -917179569092],
 [ -367232182889, -2442285948819,  1361271870123,  2381968867543,
  -3265462546848, -2167879350239,  6127632329518, -5240087527930,
  -1146828021226,  1745666144638,  1281263630456],
 [-1251017490073, -4642095748751, -2171086712114,   879411328420,
  -1608892765072, -3552415305994, -1436676767663, -4762639888399,
   1269985074694,  5097038000814,  1962471083642],
 [ -864633438705,  2007639737117,   549361299094,  -700145530508,
  -6756070553071, -1777328588218,   244120353697, -4279423278929,
    741244752520,  4912354536451, -1603955937975],
 [-2047245747673,  4353208118734,  4466611782840, -3548383600830,
   4999839198597,  1454755980125, -2458960259203,  1357907326478,
  -2777115559538,  -417104527389,  1193844659660],
 [ 3675941328561, -3486332771489,  -160820464138, -5684858841984,
   3339235986597, -1121182615107,  2683706915249, -4143352587015,
  -1057443892780,  1469492407342, -1390920364707],
 [  993837045276,  -438636370991, -8717966456784,  1766684229717,
  -1069629155048,  2133201807113,  1194040007098,   219210991929,
  -2630650088855, -1554823751875, -1785927066321],
 [-1367498723704,  6042596120959,  1537745030550,   770188839683,
   4053118337687, -2500307022642, -1634141626027,   799918225857,
   2835543828405,  3421800261642,  3594511289418],
 [  982823525219,  1200708454647,  1945809394940,  4024567441855,
    792309730526,  4512350191374, -4653124964118,  3383451539119,
  -4439864514732, -1275738318566,  1476739467597],
 [ 1106754610890, -2394320131487, -4897587067212,   292142796785,
  -1555079481674,  3451029272877, -1532062569004, -1802822589670,
   5499495710060,  2971643738652,  3157568531073],
 [-4345262074852,  1492498015734,  2537782846725,  2481763074580,
   6679536299018,   -19914908484,   942943074417,  -290013121900,
  -1700796117075,  3988954204029, -1377440865793],
 [-5432043423830, -2493218984646,  -841654449445,  3599258880566,
   -348924243872,   484952891758,  2333307556399,  6303940442131,
    332828845772,  2211691559452,  -265415436677],
 [-6529936089701, -2957302376195,  1140672433647, -3261543281125,
  -1437066055571,  3684338528247, -1442955524331, -2187329993338,
   -974580392896,  -619358671550, -3582742616297],
 [-1116902797656, -6743508315541, -1817602116827,  -434253387542,
   2035342670101, -1329262058964,  2502140641844, -4715189368996,
   1732358794722, -3514538145381,  -170437457561],
 [ 1029206877767,  1439657260208,  5621866832294, -1868589302411,
  -3155615871098, -3338652844717, -4742001464113, -2178623887785,
  -3356167984795, -1384513098671,  -502864970606],
 [  320429667982, -3946305475847,   930598799560, -1497434135835,
  -2953541775817, -6590623479233, -4481515560774,  1560827004200,
   -104091485319,   698423516255,  2457062236060],
 [-2978034195832,   430650162067,   988431484766, -2343115725466,
  -2048729141708,  1978716107729, -1092585951018,  8021485138991,
   -973519475706, -2070889786771,  2364691006950],
 [-2712219526408,  3310764168929, -1214934419161,  -833709350239,
  -6332709171084,  1023517680586, -2711222605694, -1506671847696,
   -578139731924, -3026126981594,  4387288104757],
 [  -48903615025, -4144457369607,  3793232874556, -4122724679103,
    752987362722,   452285299083, -3863667028717,  3829371623270,
   4513875875097,   831222628119,    77709434942],
 [ -666721920985,  1088966220636,  1634692132718,  1319565594151,
  -1376175422408, -4184119051681, -1246328486604,  7021352075837,
   3601519529610,  2399648537017,  2230129144183],
 [ -450541913644,  2941073593038,  2689672157268,   794082250564,
   6644727806254,  2166885399331, -1002974015615,    45546992483,
   4739837281813, -2258073274905, -2421035200007],
 [ 5497457887295, -3331913189351,  2120338954097,  -812017651650,
   1035748752659,  1874777695388, -3730526907220, -1404709821964,
  -5727686966612,  -471728256119,  -120698876988],
 [ 4191754506567,  1514866414373,   167435490699, -1263952486962,
  -4236564175885, -6497242769183,    -5043047408,  2508947347210,
   2374411993924,  -457576869117, -2490797587303],
 [-2398785273054,  1801277758244, -1369097835142,  4809980281419,
   3379854663624,  1225743869128, -2281363772418,  3167390354183,
   -883402227426, -3920501243994, -4655889932572],
 [ 1980737205664,  3053404339992,  -323943331253,    20867510076,
  -1777393188178,  6718044530180, -1819063817558, -5467050226464,
   2011280082124,  1021475674295,   265493871800],
 [ 1599869272571,  3377719330174,  6001091289038,  2081212663557,
  -4732197244452, -1506882132131,  1470853105028,  1047753312878,
   -814439480096,  1323626136882, -3917398298387],
 [ 4083313123547, -3653278918306,  2434503600644, -1346087693035,
  -1349610752951,  4461562678186,   249213063476,  4301172841954,
   -116976792089, -4525602358660,  1207228751899],
 [ 3505121213808,   -54964681561,  6330466537123,   936869135621,
  -2245033441714,  1052802142894,   -56606474689, -5736285575286,
    -98364524252,  2725251134633,   514937353695],
 [-3826788354731, -1549339011126, -2816648506289,   823919117818,
   1815495823546,  2433189957751, -3034549349066, -6366145289961,
   3649625483909,  -303992348639, -1406405494359],
 [-2879258515750,    18437974896, -1788210766892, -2916196539595,
  -3765367125024, -2883992614280, -4247599018791,  2690967326177,
   5052300992980, -2295362186083, -1197661048381],
 [   85474414588, -1059316670792,  5642243025391,  3082520650536,
  -3245404137150,  3524664313803,   698971571389,  5231397282827,
   1594622910311,  1713869112672,  1114247507789],
 [-2571844755038, -1063082595609, -5931752930656,  4827706638804,
   -682261633973,   442064640260, -2804656597975,  2023493164644,
   3777253088939,  1111726471262, -2374431254449],
 [-1451865255754,  1434879333664,  -222934803148, -6032820997610,
  -2754518011731,  2875699145837, -5660480943146,   761341028648,
   -493538755737,  2870569005255, -1557925806967],
 [-3597540747759, -2287267736214,   358447275322,  1718727051754,
   1388467521087, -1907068375136,   404805979464,  2287631596342,
  -3608626971544,  1902172164019, -7151312756131],
 [-3056848776557,  3910743207505,   275583471771,   670911991649,
   2757070913723,  5404126904843, -4416514500153,  -892044492083,
   -608187213609,  2020639000540, -3643548954192],
 [-2751876334569, -1550898849626, -3990090800205, -1259031734331,
   2776101594701,   846161726660,  4491773842344, -5498344852050,
   -446646521187,  3420148409171,  1337324771812],
 [ 2126034257521,  -566761087013, -3202512097426,  -955255048591,
  -1957779120038,   -98981002933,  5343663038295,  4232773194344,
   -325280990750, -5643641570750, -1311436791707],
 [ 2020061716733,   399958811246,  -454228609744, -2238824832238,
  -1181283168353,  6432062026078, -3683641723228,  -122700509248,
  -1491086193600, -3824909829059, -4163691453763],
 [ 2144395714720,  1048885248121, -2961713568381,   862342851146,
  -3262508813018,  -962393920531, -3358586808125, -2840852389626,
  -6185303554465, -3784145993324, -1134711364446],
 [-7141283903264,   551834985988, -1228956085633, -3427703495950,
    -79846573493, -4673140541172,  1013599254678,  -369132877622,
   -117029324101, -2203279605677, -2749847384473],
 [ 2870850102456,  -342530910644, -3025183865207,  2395044282745,
  -3510929228914,  -154488423091, -1747033003302,  2071335827742,
   1476515852691, -3939638776374, -6273676579891],
 [ 5455606189908,  5161643859152, -1963267172191,  1962918350939,
  -1506569440660,   152020201372,  4080574760052,  1488678826955,
   -919845174390,   710634322540, -3657197615865],
 [ 1666038769592, -1144981454845,  2413758433311, -6344060479149,
   2341156973304, -1828752695926, -4756556500031, -1554846357901,
  -1327644699779,  2221633325014,  3045320137186],
 [  392718162477,  4676739216630,    77425302885,  1417772730177,
   2597757214163, -3343608510592, -2224972537263,  5445498862369,
  -3671191664282, -1706044551920,  2652838335825],
 [ 3724160656674, -4061502039499,  1023701547041, -1786358074463,
  -4415514817647, -4179494048051,  -316437645212, -4851662235880,
   -425996373440,  -300935959329, -2126041476570],
 [-5364912915398,  3342606599752, -1736109834737,   750035218692,
  -1193393702016, -2098009179891,  2967467968446,  1770139858878,
   6204110394335,    -6003298785,   460376157721],
 [-2393626640652, -1049091665942, -5122478167056,  1088697622782,
  -5105574070497,  2435637978174,  1707308916357,  3641923304831,
   2375835061606, -1693161318275,  3009285224425],
 [-5137934506729,   398416258386, -3064932278532,  5125884313080,
  -4636778408632,  -467235353649,   710025386347, -3211870763345,
    686833201600,  1279034186343,  1768743224370],
 [-8499479993195,  2896057304196,  2300168240403,   381215387805,
   1211236786556,   912532188414, -1081041844269, -2481562130878,
   1138235680116,  -391226534827,  1691254723136],
 [ 4033905237261, -4871081723045,   527943332897,  2438280532272,
  -1392948478908, -2994883443748, -2550199728912,   785654231548,
  -1459817677244, -5797650556225,   -44856538943],
 [ 1798605426441,  1782407567411,  -127633268326,  2585887676284,
  -6406453074812,  3248170520160,  1039266846782,  3233776473994,
  -4249459430586, -2128554974155, -1079650933168],
 [-1524879561131, -1348790227653,   185509040260,   329713680789,
   4083642895560, -5661792069994,  4485155389945, -3159021193549,
  -3361256296350,   675473040821, -2264953045065],
 [  415419510304, -3420403757556,  4999987567914,    53920267208,
  -2835870013600, -2987658367312,  1388260272541, -1118843304886,
   6096363081975,  2392770253011,   295213822969],
 [ 5427952107234,  2491529961799,   844488585689, -3601761663589,
    349389474711,  -485436373845, -2332590907306, -6305750705375,
   -326824207595, -2214997798918,   263710502677],
 [ 5410465593212,  2284182716040, -1830665274215, -2386366134810,
   3881010473360, -3743818152518,  -734591050674, -1191235830145,
   2912744793533,  1133676778387, -3956880585831],
 [-1519966253416,  5499495095313,   701947719793, -2418777804335,
    600867197418, -2887137450951,  -525054769308, -3236045504713,
  -3827991322506, -4555732971445,  2499908413479],
 [ -220262711735,   -21686679736, -6301349010170,  3604978483523,
   1990534831616, -3278942072131,  1224944290631, -3308018241872,
   3625732944572,   109840321507,  2633054746403],
 [-4610510376437,  2813617129952, -3151596993439,   -88494734667,
   5624819370040, -2857888666106,  2454279412895,  -443157633996,
  -1303047844300,  -865935395631,  3523149627136],
 [ 1591898023250,  2469941412807, -5931303111509, -3304855274361,
   -876943572352,  2133776999267, -5194459233467, -1995517495373,
   1538603604609, -2276570139335,  1194245096370],
 [-5854130447786,  -227585494107, -3740308754695,  -288690684276,
   -617533036803,  4168516334893, -5171221429765,  1767360391392,
   -361225444154,  -867392702819,  1760194993347],
 [-2073846400455, -4864443057694,  3768191015592,  3279233077955,
   -127179189053,  2890636750283,  1306208403755, -2158360568070,
   1156806226339,  4731226850683,  2936725638233],
 [ 4380402244915,  1718336099159, -2490203993633,  2962792650483,
   -609296656817,   -88183771411,  -528514858921,  1835636669616,
  -6875078311611,  3073438996418,  1462704076511],
 [ 1808548016756, -7156596836882, -1005793147626, -2439553382253,
   4381084164538, -2569424600789, -2852359039920,  1672380191489,
  -1251321929623,  -391277073385,  -318876263817],
 [ 3050479025203, -1112913585673,   583195931072, -2411256428011,
  -6500725040245,  -177287830264,  5844127903559,   597134495908,
    253318857497,  1220454761246, -2224137621907],
 [ 2081026457771,  4576326717134, -3059971157905,  1791758352530,
   1929136083966,  1520812361717, -2120604198642, -4189323624596,
    840252242315, -2988107401845, -4943288443379],
 [  374633385085,  4064005181314, -3643593868621,  3916456117646,
    731274504682, -1390998890607,  3891975595409, -3931218738608,
  -4577567048717,  -775483008911,   319402267704],
 [-2835830561561,  6330276368832,  1469030071465,   402575127112,
  -3894140626988, -5490744757063, -1778186499571,   725501790480,
     51664708241,   528979064893,   531180438358],
 [ -407377873713, -1054311025402, -2688240679126,  -400274214617,
   3049297157362,  2934281209053,  6032558300772, -1387844054570,
  -2051260454929, -4417782970588,  3374173701078],
 [  684982537657, -4959087143757,   106581022743,   541496847341,
  -5426086784485, -1687822946375,   394145178486,  6213307814871,
    787828294802, -1036369422923, -1373765538818],
 [ 2436940108262,   -17959317412,  5272745875180,  1859736836422,
   -572591553586,  -196355684278,  4728622193259,  -100172273328,
   1443302561770,   711844762922,  6121630990387],
 [  223238508073, -5979661064274, -3475906742063,  3953573669553,
   2030325741149,  -732468271069,  1890864460635,  3262533163758,
    -41044222721, -2549195724778, -3332092252171],
 [-1774327400291, -5768496654472,  2193622125140,   685714364485,
  -1578315854585,  2649364870676,  5560588530162,  1445106797385,
   3397613217391, -2045535048810,   217585806122],
 [ 4658041545519,  3630655850203,  4551294374768, -2158382540553,
  -3182453315019,  4504288573550,  2294451726266,   694082709894,
   1687704540905,  -342007246067,   787331961924],
 [-2465951597827, -1864199930418, -2270856798900,   -22595841598,
  -3664322575097,   410068708466,  4800776928729,   516836974097,
   2660325854959,  6212676802611,  1643732824380],
 [  359846782248,  1044409315702, -2799990553119,   262881376730,
  -5481381738941, -4220241657587,  -294707117620,  -304138737129,
   4283597060107,  1718359722968,  4640405561466],
 [  276745595187, -4446743546668,  -420469848217,  5877043034202,
  -3498803237174,  4162570629251, -1979809669506,  1484541890021,
   1336631902838, -2815013300598,  -174470552559],
 [ 5102172976455,  4259278196381,   646116293517,  2524356648706,
   6102964143435,  1254045003892,  -957282256921,  1873342983288,
  -1573270140866,  1489934935322, -1047305543583],
 [-3026034433384,  4955504388403,  4333613435828, -2343097902290,
  -1131529879827,  1679795382350, -2094050038152,  2926412538950,
   4863771372296,   454595688732, -1049686222394],
 [ 1413569263672,  5664984198517, -1424796239863,  3110895550026,
    552999270359, -2663403492361, -3366801478812,  2831708540909,
   4442817380487,  -589019091899, -2713492019071],
 [-1196383610997, -2317590034549,   488155267722, -1612569113807,
   6813504685085, -2793562988534,  -651338217079, -3533045594143,
   4362308711116,  1960841909849,   591300922110],
 [-6225964858091,  2664169319390,  -865309055360,   819974520373,
   1830992948396, -2381820387657, -4175153119521,  3871551417818,
    656947569754,  2466614045870, -2180885828971],
 [ 1769560500047,  6037989935725,  2421935205712,  1203216048338,
    672174795176,   195496479510,  -872012665847, -3273813092151,
   2768564775195,  4507389782533, -3626020480703],
 [-5047746784092,  2453874805493,   605654139215,  1972416669127,
  -2665071988780,  -422111933646, -2198438897961, -1931046200821,
   3511718736225,  -872604552173, -5941804355173],
 [-3328181454099,  2292646681385, -1230196268189,  4178873463660,
  -1952023152818,  5264778689679, -1386241486348, -2997510223232,
  -2074354162481, -4120755939342,  -985908910127],
 [-2128722363465,  3342442538371,  -168399550678,  1018145277613,
  -2470638175877, -2594108650010,  1574952019260,  6683203855831,
   -391972896290, -1591148785755, -4535096826577],
 [-5201779311507, -5745047858643,  1148085852099, -1972658788736,
   1626021341995,  -443420182872, -3687980730355, -1722445336866,
   1126867930447,  -728342002724,  3676619806419],
 [ 1025623808061,  2255781933926,  3434448250170,  1588728653730,
  -1672468315028,  1399827775510,  7540246372905,  -415840499655,
  -2622968673028, -3238462954919,  -622524241645],
 [-2330888881958,  -783041291864, -1611954800518, -6710003615179,
  -2409429846717,   164483025058,  2876722005477, -1406424960130,
  -2560402079163,  4389258928773, -2103339319444],
 [  981353914797,  4600570118021, -4095848035320,  2695778349891,
   3604971925019,   268230099620,  2669684981719,  1433109276176,
   2427347451707, -5026339548167,   650854878249],
 [ -577346357670, -1065056354349, -3503260666052,  3343754775146,
   7471599453113,   941566908039,  1519206252295, -3002675888721,
   -661863774295, -1606353643907, -2006230896461],
 [-1530270045251, -1165038214408, -3910191264386,    42757199935,
  -6749578664506,  3655208406975, -3037805775025, -2087407439549,
   1129782270412,   146803346119, -2684898890488],
 [ 1124648695462,   619841252926, -4343167328282,  5126149833326,
  -2012444152893, -4492681408815,   228998673850,  4372547039269,
    461299896200, -2790084369521,  1344378474112],
 [-4934004008631,  4235433403434, -1877806736947,   691697509341,
  -1611705070234,   903752864656,  3576555598625,  2434967123936,
  -2600892774175, -4683201034835,  1697030426423],
 [ 2586198910446,   161208459786, -4102174412712, -1602983963062,
   4264078758861,  2903065212760,  1279685247477,  2722901928486,
   1107788946521, -2335008002528, -5616653031119],
 [ 3230590122770,   164261854428, -1998144233601, -3965645048161,
   1037949781618,  2027214434103,  4278313498655,  1105158967135,
   3430220607132,  5007148562462, -2875265811886],
 [ 6761580377743,  1346954619468,  1557368388076,  4516065361338,
  -1017022327120, -2592367674560, -2820017724509, -1495826289031,
  -1863906044044,   102998318679, -2866539059950],
 [-1377509159657,   455485182438, -2949168797229, -4767997985688,
  -3528005486674, -3659205157792,  3372542508170,  5103307954216,
    717304332569,  1026608060109,  1280917880731],
 [ 4576680007827,  3360652753004,  1053724557420,  1053718472281,
  -5382930598816, -2808844103951,  2350474900212, -2196771652022,
  -3356114285095,  -478194092956,  2613636825032],
 [ 2852163474361, -1135949180783,  -211725018832,  1315891010135,
  -2075288507285, -2173478161662,  1336495634981, -1030302943854,
   1682346975780,  4009979323683, -7616402455926],
 [ 3857915221829, -4463500003492, -1062016014312,  1187751209208,
   2073567876700,  -835202429467,  4556863313623, -2101419111944,
   3696351907824,  3543026342009,  2502151362963],
 [  401382018489,  1051931974710,  2692405464188,   396635193722,
  -3048558168232, -2934833001129, -6031609308015,  1385264991156,
   2059965050099,  4413057494559, -3376567915437],
 [ -169167788834,   880019797476,  -711861100149,  3289188658280,
  -1342563601245,  1346475859727, -8570110050068,  -770689718251,
   1127827874954, -2257027985665, -1961898300042],
 [ 3930207101923, -4788272198497,   656309023670, -2770544407001,
    557594079672, -1221527037189,  1949795656742,   754170867605,
   5202682587302, -2373954082924, -3826510589359],
 [-1070179288209,  2415714774160,  4871960243032,  -273380831179,
   1555236788003, -3439007596191,  1518887854371,  1816423812368,
  -5553714292496, -2942765660316, -3138907782101],
 [ 3940018399641, -2506993712381,  1927018684527,  3369813266746,
  -4298165936640, -3300625203781,   924147333853,  1210577413806,
   -875664104610,  5450673764923,   978600957183],
 [-4957499786340,  -560490955659, -3196530206184,  2729154604869,
   2729502871664,   637868740008, -1108714879636,   155258764813,
   1587348802021,  5408907770018,  4068977713458],
 [-2489799141264,  4434234188279, -1202064872044,  7037820836620,
   2789740578487,  1506913373001,  -408374381403, -2200198212345,
   2693826864962,   885099416630,  -247788096976],
 [ -389022934133,  2710548948948, -5327964571766,  1766636537686,
  -1734693563450,  1930537485456,  2680796992755, -3674273444797,
   3506538711694,  3221342484833, -3301123849731],
 [-1316371471221,  4984144857377,  5149034340701,  1864552720515,
    979887311061, -3180597467149,  3423896005233, -3618977271654,
  -1753560603942,  1737977565955,  1201821181321],
 [-3962236789207,  1750259608810,   148960340831,  1779306059384,
   2981987855892, -1253539672826, -2552909536048, -4439189885805,
  -4920945901236, -1751087652709, -3752301685105],
 [-1246240693862,    23064000757, -1867078026750,  4661938680281,
   6095716390748, -3242714454691,   273216436183,  4017729577537,
   2942885259625,   791597255920,   223038496552],
 [  -30649670188, -3871015798898,   704807144470, -2320378138230,
  -2357264920905,  3134715677961,  2576542245250, -5342097663914,
   3860660551080,  1647128407834, -3309975816110],
 [ 6894364675615, -1536991055335,  4716871019965, -2137498887887,
  -1732935394892, -2187576148777, -1761947809958,   381412063761,
   1707817215756, -1068696752044,  2861883710692],
 [  112529695580,  1072157541057,   674722309129, -6700999288825,
   1257537007593,  -247891340853, -1920166825819, -1933231865155,
  -5187788661630, -1841296457101, -3756111585811],
 [-3458907956722, -2289333716918,  1197674347308, -2043672756747,
  -3032049510746,    79839440230,  4925347625983,  3887792418169,
  -4954357866607,  -935659668029, -1785801254471],
 [-1895276345846,  5269942247678, -4571502851511,      573694574,
  -1913509641237,  4275131330747,  -712911549218,  2758847108459,
   1991204978412, -1950884204095, -3147761728208],
 [  911004404572, -4884438497405,  3041146246306,  3595341898145,
   1857144844875,  2498224328604,   491146027412, -1428032983801,
  -1873303763754, -3805218707120,  4814323097766],
 [ 2004940182490, -3377861273990, -2836125064246, -4011742049221,
  -4826198407407, -1579685127785, -1692090129551,  1004315098601,
   2905310241165,  4442787437574, -1610585800723],
 [ 6321039821476, -2600451384479,  1210817283665,  1208651410377,
   5738198705036, -1245592945410,  -243411859957, -2619475786472,
   1040117842899, -2671921119669,   857236814176],
 [ 1464911252042, -1429635132299,   214089203797,  6040223039937,
   2753642908473, -2874026574776,  5657954751029,  -756132119662,
    475762436227, -2860233689147,  1563968488668],
 [ 2024222593676,  3280398978682,  -776485984108,  3547678058283,
  -1037485022957,   101765635860,  1222606531210, -3699551759045,
   -414345348428,  6291285314975,  3991543685779],
 [  418325636565,  -976628268689,  1960130308613, -3131328225267,
   3158424345724, -1238270377957,   224716563958,  3089029508549,
  -6964846779600,  3950714977405,   -71241800321],
 [ 1489527748759, -1672027828684,     6450993624,  3553007823045,
  -3447768798791, -4787063826997,  3771831286954,   782420859434,
  -3615334610695, -1957132225043, -3977276642890],
 [-1828623208758,  -278174491750,  2607271135651,  2827484184789,
  -3362214664270, -2695356375558, -2098354025067,  4762092301817,
  -5709012503524,  1853552845219,  -328174437859],
 [  406777144877,  1082675837846,  8160639820723, -1302165462571,
    570799717723, -2118686180845, -1205705957770,  1864618075055,
    220710737538,  4169787838341,  1785960103011],
 [ 5431184070137,  -309737848981,  2160845782374,  4325354834855,
  -4093168843177,  -335438513349,  2957437208333,  2026067676495,
   2741298099837, -3128261655508,   -98395198705],
 [-3801575303236, -1326300030816,  2902747383118, -3164052910919,
    102088557005, -2691942486225,   232441481562, -7596192477934,
    157609214931,   281536406512,  -484284223556],
 [  870030828371,  4241678915053,   808293549586, -1616247604007,
   -787092901633,  -132753496525,  3552033927118,   233893752987,
   5370834839860, -2915968611203, -5227626775815],
 [ -506870497010, -2130343626318,  2149499315013,  6417195180367,
   4227919514470,  4464540212850,  2220798395012,  2222111890082,
    954554218823,    95595711536,  -898061942099],
 [ -324237004813,  3944786027795,  -927906648754,  1495096796314,
   2953976428372,  6590277965583,  4482138600796, -1562494879528,
    109672701189,  -701466848923, -2458541732536],
 [ 1245292199359,   -23444566233,  1867756989833, -4662601885381,
  -6095537007032,  3242683991266,  -273108949196, -4018152292284,
  -2941412956581,  -792344336969,  -223379123929],
 [   71316212551,   814716464961, -2850928767408,  1110667068218,
  -1441092269598,  2025468463968, -3093331235645,  3124664682535,
   -283034304041, -7906608771207,  1365176650977],
 [-1428258126828,  4442614282252,  6339864332681,   667625196526,
  -2645336875445,   638054729150,   172956580553,  1998043476530,
  -2242537274146, -2836925815891,  3616561855795],
 [ 1862005254671,  3405517571734,  2253927404268, -1023599065883,
   1261979031615,   -85706852298, -2745053539391,  2094330190451,
   -810787164646,  -297173455525, -8033696546314],
 [-1405446876079,  2759059261876,  -900391877185,  2829467448571,
   2369321616397,  1967666102388,  1727535445022,  3655643684313,
   2809717869209,  3910704397889, -5707371709276],
 [-5674064524733, -2501010713368, -2027147460036,  4003059107952,
   1467189200434,  1306497371464,   747398537831,   -26249408325,
   3599012165758, -4470903015833,  2013800548899],
 [-4061251660559,  3970924364989,  -826536677181,  1630672788067,
   4420910313958,  4159312858739,   360275634236,  4772058270784,
    840163189121,    56015589920,  1984957418361],
 [ 3100369373946,  3618829690233, -3943659757447, -2085962908171,
  -5284398817946,  2817370743566,  1310074672458,  1099853485048,
   -660564537771,  3373159711192,  2604825104656],
 [-1599835715474,  1171222746699, -2460125766453,  6384414730274,
  -2349076021091,  1835123130434,  4745784133927,  1583862847820,
   1231102912196, -2169672507598, -3019234328815],
 [ 4315279050541, -1504471071267, -2516854702418, -2499608888206,
  -6675867671970,    16443473889,  -937797981903,   277048238808,
   1744117134227, -4012814526763,  1365286042378],
 [-2576691553872, -4168355303390,  4537168270309,  3289883489257,
   4545811286758, -2343742404114,  -883746419592,  -423757223990,
   1797956997202, -3357840962479, -1717670949614],
 [-2770757495943, -2100276262903, -2310355957864,   815198757292,
    378008579111,  6654075747375,  2004799154313,  3938478545291,
  -1882585389522, -3425197886948, -1637953937110],
 [-1957929560953,   632058228489,  3085463734040,  1058548099344,
   1935515723895,   113529956234, -5368507010752, -4158778252136,
     79583267059,  5775251335043,  1375182887763],
 [ 1043608214197,   645438379821, -2257458783399,  1563856913321,
    876487024162, -3057822807242,  -609133909223,   458157961937,
   6045544209448,  6576656903033,  -672338544894],
 [-4142674083739,   191452447235,  5844843854843, -3328434054873,
  -2950087353136,  2629530078902,  3529773720027, -1643439769921,
   1641203888894,  1727359214850,  1051648120926],
 [-2536425192805, -3412028873887,  1603269936707,  3714143636044,
  -5958326431609,  1573982825063,  2066214946180,   169698750068,
  -1260855487109,  2893560055930, -3649617067653],
 [-3370910924887, -2664833577526, -4212224537043, -1664159625471,
  -2346009074864,   941991345314,  3116568323270,  1337408376752,
   3417509333245, -1300488029900, -5455268464385],
 [  658394595659, -5845490182112, -1734937401395,  5132881559044,
   3731320560325,   406316780591, -3110371669968, -1368398831925,
   2549752914186,  1048224195587,  1674960469699],
 [-4069018215699,  2876255152390,  1999107398479,   288371782958,
   2154913562127, -3129067806294, -1567216762796,  2815472413652,
   2748385177040, -6162870836304,   859237607542],
 [-4081397478017,  4440571166733, -5237098386667,  2415216733177,
    -82617245215, -2443974120286, -3331468168048, -1207989621760,
    -65932252178, -3435715154676,   129657560479],
 [ 4540975362633, -2962280943473, -3727641214052, -3334039256515,
    570980206559, -3755688295128, -2042533319394, -2636445564667,
   3169252319979,  -272285070269,  3149763104314],
 [-1970685170517, -3212396105451, -1122959696365, -2522166126002,
  -1796870295135,  5719246575631,   184448827929, -3166927469451,
   2390181077127, -4324669286196,  2785111405287],
 [  200644095783, -1408319683626,  2230072881087,  7141211846352,
  -1065713117235, -2345629089454, -1055615018478,  3737240815498,
   1897700894272,   777942571718, -4010083583079],
 [ 3207211747233, -3860205876030,  5140569835651,  -471207339362,
   -337229668079, -4107979119479, -1338074993769,  2629126458727,
  -1774101172768,   760813963139, -4329765470951],
 [ 2143894337056,  6169806192886, -1781372515887,  -719475094542,
   1035253435287, -2386252505629, -5559323574118, -1325703604772,
  -3012161951007,  2175050982007,  -642487683129],
 [-1100041253761, -4027322736527,   229361570027,  1637503933953,
    608768884422,   512867167535, -3736704991334,    67174202189,
  -5291870130711,  3555218083799,  4959030247620],
 [ 1420617715787,   900534716736, -4641612714007,   858580710316,
  -2821052711788, -4292847420429, -4102875100962, -3693473208841,
   2833909355485,   831540662000, -3050318865731],
 [-4069141454652,  2397030807707, -1277633907502,  1139048765540,
    -81027369382,  -511257043713,  4392208537723, -5619448373269,
   2158427361261, -4069786076345, -1551431016185],
 [-2187452393712,  2289066386737,  2720374946090,  -754365626582,
  -1007844680787,  -738792081057, -7404298713659,  2534366005211,
    -13257462581,   611993086866,  4338524326161],
 [-2261267990905,    31011633178, -4461713731260, -5600361739838,
  -3850335661845,  2189742342684,  -216324100412,  1717505044705,
  -2840927392525, -3518702564105,  -738885782573],
 [ 3835162625076,  4018145639189, -3969032215791,  5489480336209,
  -3320287131143,  1771420196090, -1544616085553, -1452735700983,
     93447194857, -1212172345514,  1766865015618],
 [-3082861861208,  3058375781651,  1165336696419,  -124017674997,
  -2636642417196,  2258840895000,   782669515714,  2647187849760,
   2623806578654,  2724227790305,  6766735682713],
 [-3924384276199, -1257058873969,  4552529697420,   387469317643,
   1545295213894,  5125273879835, -3071156583424,  1850357772371,
   1429128901788, -4262910977786,   645241204936],
 [ 3701699616746,  2383614266719, -1367424358250,  2190400968495,
   3001798884658,   -56795233975, -4965441320966, -3781381548309,
   4601491412457,  1125849679522,  1881052978621],
 [-2189333732830, -3776207341475,  -599211629311,  3057114021789,
   1053223858244, -1489095677808, -5484378375890,  5694926621678,
    280207726420, -2114773711882,   924126154637],
 [ 2665403576355, -2944060824958,  3157438546205,  1078691117758,
   1382923782302,  4459945621588,  -505584133266, -1501254246342,
   6777564021800, -1030807481720,  1335749241587],
 [ 4212558911551,   750153481199,  1028258998901,  2239063169609,
   3265243301827, -3837339068027,  1328070175434,  4072573976198,
    618748560534, -4246823513919, -3669594372664],
 [  -93602350411, -1556291309329,  3891657186953,  3723295327173,
  -2291127866238,  1160236616592, -2729402661166,  -356981559810,
  -3888216948351, -3338076730838, -5303564299718],
 [-1401779354450,  2278330163184, -1845651571694,  -265759043783,
  -1100547800936,  1597321845900,  2809607617862,  -311530978355,
  -4080685213673, -3942194199680, -6739487572218],
 [-2719919362899, -2391373190823, -3969228213732,   574141537015,
  -2702607686033, -1963447676327,  1386211653292, -5103770747637,
  -2713538834489,  1100638362345, -4805724744400],
 [ 2523998535100,  6205802308153,  -740529115356, -5459372639947,
   2215638830763,  3653337752122,  -638121391235,  -468164566807,
    914489609231,   343023507309, -2219921307491],
 [-2350524571905,  1019407106351,  1732790704046,  3013675873791,
  -6054456149167, -2669548888846,  4804552750018,  2998031686008,
    203759526543,     2877308034,  2335521273375],
 [ 5028263042894,  2227314771702,  3244450087185, -2909303881378,
   4048893324798,   746603634549,  1165671661409, -2163688951411,
   2917552225784,  3511807906586,  2632286844261],
 [-4714905505924,  2895973072505,  3843863522651,  3233205036684,
   -552014538054,  3737708661071,  2071463682670,  2568124973805,
  -2931446631079,   135373795872, -3221953642295],
 [ -399808288793, -2027286498904,  -926551441885, -3676835599757,
   1065045172846, -5815841324681, -2757928855920, -1768235050712,
  -1611389885744,  3963506395496, -4165996467535],
 [  637654470917,  1884384072534,  3454373034475, -2817649664288,
  -4340794990017, -2094754538411,  2347442561387, -4183501013377,
   3191236618832, -4027802664085,  1876379562685],
 [ 1169509591568,   888130497109, -2780123094578,  2233387019770,
   2466835615100,   826095556511, -1869080414836, -5231726772561,
   -402940143583, -5836576954281,  3642284968454],
 [ 1920808507726,   -17069658569,  -611385986731, -4447880127311,
    851659797175, -3668042836712,  4197058334701,  2683661104136,
  -1998312720929,   174016363969, -5756043708775],
 [   26883657634, -4931237083796, -3796335053355, -4723730642380,
  -1415711385171,   720768095098, -3722199717928, -3871322798266,
  -2669188419925,   653348669329,  -198240881117],
 [ 1376090713691,    52794369058,  2105420292898, -1682550534643,
   2542698783425,  7624383483088,  -148779774205,  3913076935775,
   2098374493774,  1721371826116, -1882198869441],
 [  681296782274,  2051665305196,  -956475448024,  2319926209191,
  -6356153701200,   700058790681,   518621453633,  3319478656765,
   4804216979985,  2728945451941, -2514612942808],
 [ 1618110226052, -2193439357575,  1695044091886,   397656341641,
   1073388847499, -1576671686419, -2843336252998,   407189794653,
   3766298781170,  4112633385347,  6822906738995],
 [ 6164789949614,   136739143580,  2175454310590,  -622114272459,
   2011485522508,  3965112522977,  1210393364110, -2980277210958,
    902750629627,   449521686336, -5071953818337],
 [ 3881218273012,  1479027175218,  2070483335377,    75560330693,
   1057802184062, -1384772852312,  2238953203018,   945086030287,
   -776171777608, -7173099477397,  4178226133483],
 [ 3235814803974,  1256294864562,  -650900085810, -4903438216654,
  -3214890352777,  1970650069612,  -252075430182,  5758156457942,
   1664750015810,  -670281194160, -3581101717788],
 [ 3378074137115, -5251527924600,  3110186772230,  2979998202959,
   2321062449968, -1804075580466,   996582370415,  4811856309508,
   2164952889186,  -282367400925,  2213709332985],
 [  574905043173, -4386090861270, -1092429566069, -3352980169208,
   2327865872888,  1391131336277,  3388203163344,  6943674361107,
    166333751023,   709426251370,  -645073660217],
 [-5093483115816,  -172520412035, -1119927279528,  -298914389540,
  -3684232686653, -2715581429324, -5996134939421, -2654472304753,
  -2448900299536,  1566386998418,  -533468096138],
 [ -694829797925,  1077657175865,  2426418664382, -6898838565028,
   -630728354927,   891813183909, -3004655039870,   712908865280,
   1510580937205, -5343119532124,  1820512127979],
 [-2431689551338, -5427049186581, -2338557330124,  -815424187913,
    981962290892, -3062993059108,  1128316606362,  2535891852174,
   6084947796540,   617493934047,  1744232021160],
 [-5435703869515, -2294163621518,  1848370199428,  2371054182793,
  -3877945120584,  3741103307158,   738838875423,  1180167173884,
  -2875974256951, -1153626673599,  3946706754350],
 [ 8219585512141, -3454414521062, -2804725743538,  -219356185092,
  -1174314716502,  -448706854993,   982240147012,  2239687163655,
  -1333597836815,   384385903109, -1761603754385],
 [ 1591055287346,  5678683783494,  -615012178719,  -628567142182,
  -3870188383064,  -669880486908,  1145281971929,  4449831881766,
   3042481569158, -3394493236261,  2669963658827],
 [-3754248407042,  5294073804022,  -296736245623,  2912063924725,
   -791691392173,   751000806695, -1982989567750,  -407567361095,
  -5153438369097,  2080369983896,  3624255814498],
 [-1109208301621,  1105002101950, -1209577854326, -2024207507329,
   7350334972180, -3488128656973, -1651140239034,  2095339824418,
  -2280768401335, -1030270576505, -3523292766568],
 [-5069180086335, -1838030708503,  3563978996722,  -302815316047,
    440743640453, -3187355375955,  4347216998557, -1328131210369,
    353915357918, -3762804268885,  3582142309006],
 [-3661582264231,   -11651104094, -6216800774575, -1027515731357,
   2255363625886, -1074441912663,    87463515947,  5662135623408,
    329269044501, -2851689332257,  -582416329253],
 [   56248471539,  4946765000660,  3758738313061,  4784112795759,
   1394864083549,  -728062773091,  3727632364237,  3905942835080,
   2549768126638,  -577510027537,   219804846640],
 [ 5587427890418,  2297669997514, -1020505798020, -6126153847699,
   -542119172009, -1750346906402,  3777573935242,   426662882168,
   1513176033026, -1672209304069,  1426348449094],
 [  259175505863,  4398571312812,  -563569394904,  -182986310141,
    198425934183, -6310433112917,  5597938858309,  1367486286215,
   -595778885575, -2563939288918,   487534331752],
 [-4802134395185,   159246504059,  1550301624727, -4853626304168,
   -500450446506, -3445176305248, -1086016460941,  -198494483359,
   3935712232757,  4174986491713,  2166600367239],
 [-2955758367798,   197182840185,  5575429750308,  4821829786482,
  -2151221731203,  1377147185183,  1369265209861, -2672300951780,
   3787808035786, -2629332593910,   296485252008],
 [ -907637263048, -2208932588521, -3516691104412, -1517457288318,
   1659108833526, -1386977275547, -7560769135554,   467600743208,
   2452516555656,  3332072477470,   670257831686],
 [-5343273458688,  2094742859491,   467642153205,  5934014125908,
    741852808966, -4562356494190,  1823039549615,   934097259328,
  -2087710423337,  -903735244984,  -945273930686],
 [-7240969135778, -2639902878880,   849460636225,   414183625708,
   -726198001149, -2021521159959,  2097041348792, -1405395811621,
  -3323825049514,  4169543904047,   535356746516],
 [-2450513019016,  1863099990948, -2437818958899, -1649975566093,
    913002479189, -2735144788138, -3826123267412, -4408375983513,
    804673891768,   752627625256,  6185218403530],
 [-1335399479000,  3823343667766, -5717460499457, -4005933034396,
   1194120926048,   434347561966,  5544143288165,    32763093488,
   -153238809945,  -626002138389, -1446843381290],
 [ 3384149124219, -3775174772430,  -505726264120,  -482507757887,
  -2778435182795, -5352559874474,  4355591684268,  1038732361168,
    127898700843, -1750769851358,  3782290455385],
 [ 3090835195085, -3054792832893, -1170619376721,   128325298949,
   2636523183051, -2257212225518,  -784444777154, -2644511696348,
  -2635072545182, -2717234652626, -6763588745282],
 [-3729763407548,  2337035244907,  -846339999856, -4541567007554,
   1974270756041,  2750510194087,   636060278562,  4678194935433,
  -2434295636669,  1169387614202, -4270669080080],
 [ -182352232618,   -15639990365,  -806361709654,  3736501768658,
   4423778632034, -1993974011012, -4511280703919, -1620258902598,
   1407442349625,  2801596965688, -5385383608572]], dtype=np.int64)

In [ ]:
#@title Verification

def compute_squared_norm(point: list[int]) -> int:
  """Returns the squared norm of an integer vector using exact computation."""
  return sum(pow(int(x), 2) for x in point)

def verify_sphere_packing(sphere_centers: np.ndarray):
  """Checks that after normalizing, the points correspond to a valid sphere packing for kissing numbers.

  Args:
    sphere_centers: the list of sphere centers, of shape [num_spheres, dimension].

  Raises:
    AssertionError: if the sphere packing is not a valid kissing configuration.
  """
  num_spheres = sphere_centers.shape[0]
  dimension = sphere_centers.shape[1]

  # Rounding to integers to guarantee exact computation throughout.
  sphere_centers = np.around(sphere_centers).astype(np.int64)

  # Checks that the minimum pairwise distance between centers >= the maximum norm of the centers.
  max_squared_norm = max([compute_squared_norm(list(center)) for center in sphere_centers])
  min_squared_distance = min(compute_squared_norm(list(a - b)) for a, b in itertools.combinations(sphere_centers, 2))
  assert min_squared_distance >= max_squared_norm, f'Verification failed because the minimum squared distance = {min_squared_distance} < {max_squared_norm} = maximum squared norm.'

  # Checks that for each pair of centers, each of their norms is greater than their dot product.
  # This guarantees that after normalizing (scaling each center by a number >= 1), the minimum distance will not decrease.
  for a, b in itertools.combinations(sphere_centers, 2):
    a_squared_norm = compute_squared_norm(list(a))
    b_squared_norm = compute_squared_norm(list(b))
    dot_product = sum([int(a[i]) * int(b[i]) for i in range(dimension)])
    assert min(a_squared_norm, b_squared_norm) >= dot_product, f'Verification failed because the points {a} and {b} do not satisfy min ({a_squared_norm}, {b_squared_norm}) >= {dot_product}.'

  print(f'Verified the sphere packing showing kissing number in dimension {dimension} is at least {num_spheres}...details below:')
  print(f'  The minimum disance between all pairs of centers equals  sqrt({min_squared_distance}).')
  print(f'  Each center has integer coordinates and has norm at most sqrt({max_squared_norm}). We normalize all the centers so all their norms are equal to this value.')
  print(f'  Since for each pair of points, each of their norms is greater than their dot product, this normalization can only increase their mutual distance.')
  print(f'  Thus, after normalization, we have a set of {num_spheres} points in dimension {dimension} with the same norm, and their minimum distance >= their norm.')
  print(f'  After scaling down so they all have norm 2, this gives a valid kissing configuration.')

verify_sphere_packing(sphere_centers)

# Autocorrelation inequalities

For any function $f:\mathbb{R} \rightarrow \mathbb{R}$, define the *autoconvolution* of $f$, written $f*f$, as
$$f*f (t) := \int_\mathbb{R} f(t-x) f(x)\ dx.$$


In [ ]:
#@title Visualization function (run this cell before running any of the visualizations below)

import numpy as np
import matplotlib.pyplot as plt


def plot_step_function(step_heights_input: list[float]):
  num_steps = len(step_heights_input)

  # Generate x values for plotting (need to plot steps properly).
  step_edges_plot = np.linspace(-0.25, 0.25, num_steps + 1)
  x_plot = np.array([])
  y_plot = np.array([])

  for i in range(num_steps):
    x_start = step_edges_plot[i]
    x_end = step_edges_plot[i+1]
    x_step_vals = np.linspace(x_start, x_end, 100)  # Points within each step.
    y_step_vals = np.full_like(x_step_vals, step_heights_input[i])
    x_plot = np.concatenate((x_plot, x_step_vals))
    y_plot = np.concatenate((y_plot, y_step_vals))

  # Plot the step function.
  plt.figure(figsize=(16, 9))
  plt.plot(x_plot, y_plot)
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.title("The discovered step function")
  plt.xlim([-0.3, 0.3])  # Adjust x-axis limits if needed.
  plt.ylim([-1, max(step_heights_input) * 1.2])  # Adjust y-axis limits.
  plt.grid(True)
  plt.step(step_edges_plot[:-1], step_heights_input, where='post', color='green', linewidth=2)  # Overlay with plt.step for clarity.
  plt.show()

## First autocorrelation inequality, the constant C

Let $C$ denote the largest constant for which one has
\begin{equation}
 \max_{-1/2 \leq t \leq 1/2} f*f(t) \geq C \left(\int_{-1/4}^{1/4} f(x)\ dx\right)^2
\end{equation}
for all non-negative $f: \mathbb{R} \rightarrow \mathbb{R}$.  This problem arises in additive combinatorics, relating to the size of Sidon sets.  It is currently known that
$$ 1.28 \leq C \leq 1.5098$$
with the lower bound proven by [Cloninger and Steinerberger (2017)](https://www.ams.org/journals/proc/2017-145-08/S0002-9939-2017-13690-9/S0002-9939-2017-13690-9.pdf) and the upper bound achieved by [Matolcsi and Vinuesa (2010)](https://www.sciencedirect.com/science/article/pii/S0022247X10006001) via a step function construction. AlphaEvolve found a step function (with all intervals being equally spaced) that gives a better upper bound of $1.5053$.

In [ ]:
#@title Data

step_function_heights_1 = [9.00854017233134, 4.581788824134047, 5.954983797866223, 3.7314786029786324, 4.25817159660483, 3.544987049547799, 0.08876194700959494, 0.0491697316439318, 1.7439263266999894, 3.8548164683263795, 3.621038728073569, 4.04587668218835, 4.68211946529981, 5.5904236142896675, 4.737832747546433, 3.1594093823451055, 1.902874129984629, 2.7870307391136304, 3.277574995692391, 1.8981329099596054, 1.526040859367755, 2.305128838504833, 5.17673786436095, 4.583218228762042, 3.9910761392791887, 2.784600928752006, 5.450687602543662, 6.170368723277989, 7.045569321986071e-16, 7.149948549556939e-15, 0.0, 0.0, 0.0, 0.0, 1.2580295353835013e-15, 0.0, 0.0, 0.0, 0.0, 3.873037303627252e-15, 0.0, 0.0, 2.020385656008168e-06, 0.000293922119342568, 0.0, 4.9514916125368726e-15, 7.282654612521097e-16, 1.906059354629418e-14, 0.0, 3.3528418595404916e-15, 1.5099558045700925e-15, 4.901439953827422e-15, 0.0, 8.851999542886555e-15, 0.0, 0.0, 0.0005211322699854395, 0.3757576289315001, 0.25176470069965495, 4.1179587840945515e-06, 0.0, 2.946431316197597e-15, 0.0, 1.0333089131925899e-16, 2.591940622467849e-15, 0.0, 6.852171628124262e-15, 0.0, 0.0, 1.3885601200927435e-14, 2.5015636739088256e-15, 1.4382184696274247e-14, 1.235388698636516e-15, 9.328196456283097e-15, 6.938490364750181e-15, 5.581796597296351e-17, 0.0, 0.0, 5.1220388613389905e-15, 0.0, 6.085199919293191e-15, 0.0, 0.0, 1.0633201915504476e-14, 6.240893078396387e-16, 0.0, 9.242385301100576e-15, 2.1818685641605435e-15, 0.0, 3.841626602268906e-15, 0.0013592097228050644, 8.120066974555713e-15, 8.479388423870961e-16, 2.5924005380166956e-15, 0.0, 2.6610672065525727e-15, 0.0, 1.233819156251431e-14, 8.819083406210366e-15, 0.0, 4.492323424835768e-16, 0.0, 3.0916450306058138e-15, 0.0, 0.0, 3.404186949211756e-15, 4.54126650881379e-15, 1.462631558763152e-14, 0.0, 0.0, 0.0, 1.4460597710909072e-15, 9.521734973996671e-15, 0.0, 4.559858799705722e-15, 7.864867909828807e-16, 0.0, 1.7856864350178655e-16, 0.00021045010164189585, 0.26541232693216404, 0.8094426381528257, 0.5750041584597478, 0.23313281323505236, 3.6007277514467585e-05, 0.0, 0.7828826491881691, 0.43382874037802, 1.3263698571911402, 0.5441713262465393, 0.9864380574571914, 0.6776516652004773, 0.5910950602641856, 0.507419190418916, 0.5231329501406576, 0.9391246115133585, 0.4508771959372286, 0.28283039994676146, 1.2889986480406397, 0.9649046182943108, 1.4104382244415803, 1.3916682358533747, 0.8743196646011149, 0.7627485335443527, 0.2103862254578538, 0.14545209168646947, 0.019762475547189184, 1.2279396984729254, 0.012006361768949678, 1.7677675926679783, 0.9303739918691369, 1.0966313889580412, 0.40142701455261154, 0.1477985748190306, 0.1310850821272394, 0.0027642064206369592, 0.6718883532064702, 0.287789791442545, 1.1886491680958895, 0.6459736548490735, 0.88966666001013, 0.36931312374260505, 0.6840914190936884, 0.38692129734520775, 0.8050006872194091, 0.26610729268169875, 0.002941709304056364, 0.5150673486621109, 0.4049854152265144, 1.1607178193685956, 1.7547854228356075, 0.0, 0.8531817250969695, 2.3845552035650363e-05, 0.035208188035124974, 0.06799207369201249, 0.14050016250524128, 0.4862562534194792, 1.508781726996261, 0.46943710673489225, 0.22962993226722195, 1.589825945710927e-11, 3.51517770993058e-15, 2.4398590319680178e-15, 1.1666504235544564e-06, 0.0021946672216711, 0.34171503722540436, 0.4703022197366691, 0.1313974666218601, 0.11754826815054241, 0.0, 2.2387234387833643e-16, 0.0, 7.192783695625604e-05, 0.4486935802226264, 1.234691190028419, 2.8985055264499153, 1.0234017394012231, 2.7375379465420373, 0.5899927642043619, 1.4461499611411766, 0.7033498408537826, 1.6505029216035125, 0.9593634797752735, 0.009302210703764222, 0.0004181359389419785, 0.0, 0.0, 0.0023430720926212976, 0.42801036705183393, 0.6031743194865573, 1.8862845950884395, 1.0944504439060767, 1.3978223736063145, 0.13603422891356853, 0.8568768273359568, 0.5287328963079988, 0.04201038853661816, 0.5746932650501643, 0.7698787794362285, 2.2478052766496255, 1.3267115762262056, 1.3819155415467284, 1.210307904386098, 1.2050374056121944, 0.973960636675429, 0.13506178694552, 0.0017211602091930576, 1.2080793667302383, 0.9431703684918005, 0.004927152124127672, 0.26457949335968395, 0.219096730428291, 0.8972094379125464, 1.009247390062118, 2.5396761105116816, 2.0567929964131704, 2.5384945885180765, 2.051772820060434, 2.841483226472209, 2.5484575236736253, 2.900405077014117, 2.7293223781158513, 2.8016507480694623, 2.5235338506952227, 2.842495616436774, 3.6113040879253218, 2.4409992918997654, 2.8613737519007785, 2.0376653653073236, 2.873716631081072, 2.7431139992026585, 2.3176851657187343, 2.963845077577065, 2.1297112056154828, 3.1281786712157276, 1.559962066888169, 1.5175735153572592, 1.8986372289826554, 2.422172211485286, 1.4024751115172904, 1.6645681102200025, 1.0890488631004245, 0.9551468779062758, 0.4210663124027455, 0.7844656815643463, 1.3849725648239561, 1.1400002207678432, 1.2589535564861496, 0.00010847583255872839, 0.33022246693439483, 0.009991411612394792, 3.897603693807049e-14, 0.0, 0.0, 4.615098985648224e-16, 0.0, 0.0, 0.00019552451607645426, 7.535959259635103e-15, 0.0, 0.0, 0.0, 3.391322478636254e-15, 0.0, 5.23262076392476e-16, 0.0, 0.0013138779575907105, 3.8754445450268335e-05, 0.6732744675805906, 0.1403000505612687, 0.1066587972153134, 3.248799681911591e-05, 0.0002955183459383166, 0.003362359949825627, 1.258993392593319, 0.6721840514813638, 1.4023077519116312, 0.26971646568749497, 1.5317811712427716e-06, 0.0, 0.0004643255998670316, 0.21977013225378167, 0.4192480303816599, 1.5193826730071023, 1.140986767032434e-14, 0.00023172541086911792, 0.0, 2.8126101180648514e-16, 0.3250641832764737, 0.2631639184319745, 9.581064115586714e-06, 6.911241239536039e-08, 2.1977413193942923e-15, 0.0, 0.009284626840990289, 0.38181362260215157, 0.9229241174303233, 1.2389265199825776, 4.003701357117134e-15, 0.0018347819223694948, 0.09741977675536181, 0.7177732728524777, 0.893918190086622, 0.7592094697415344, 0.33941833076021116, 0.9895237780462212, 1.2103201657651075, 0.1562357060123791, 0.05779942219135546, 0.046035988201598356, 0.20605598834789451, 0.7087789468745198, 0.9835248618606901, 0.15343068950958472, 0.7458575667928422, 1.1650897270704061, 0.8603305066335794, 0.21035575411632462, 0.1359607526732069, 0.27083984880209855, 0.6401471037748572, 3.021195033008772e-11, 0.2302514662496976, 0.6133656015904811, 0.0014526177283533382, 0.2833008515927792, 0.6173666814379335, 0.35174441102605253, 0.42994053621927014, 0.17019592622467353, 0.18730260272562482, 0.20732692125660646, 0.0006488239072035948, 0.0008689690193708769, 0.0060827028748053225, 0.42803050608386095, 0.5802023431307022, 0.07735146147149399, 0.2677857202504946, 0.0009184606747345728, 0.43037124020293993, 0.3617534843366182, 0.3772422594534307, 0.2584947755294139, 0.001169636807952639, 1.9643151751270573e-08, 0.0, 0.0, 0.0, 2.0182994696828576e-15, 0.06977916421885891, 0.2579938169821856, 0.37671532887212333, 0.024565028057913885, 0.12617825469188657, 0.6340448467030686, 0.5754643941945634, 0.13048109721771595, 1.4341543078480776, 1.3216214397106167, 0.4415648326891368, 0.8216927039920507, 0.9032727935119587, 1.7656705299211204, 1.627512140396068, 1.5918752695978569, 2.051239519025683, 1.9657798505455604, 2.8356702084496526, 2.5590427999121284, 2.982526516926714, 2.6272105137933197, 2.960027241401188, 3.3362990605756897, 3.131536821850587, 3.159111523746814, 2.7752077904104517, 1.0508812833436538, 0.21463498939778325, 0.6073490769165661, 0.267278502966685, 0.3440305423365897, 1.0124744653679076, 1.342256457731422, 0.2717826360960419, 1.6632114845610049, 1.7120322558818795, 1.575579186781418, 2.202279058543633, 2.005531691208538, 1.891973046186819, 2.2982791457282916, 2.4252951011748562, 1.3467523990450576, 1.394123734140854, 1.306121994024973, 1.3563331357893305, 1.908587215779497, 2.2410142807813824, 2.1400334226110425, 2.0238641943935187, 2.4448713165408282, 0.001517101822563153, 9.482987339023494e-15, 0.0, 1.5827373963789938e-15, 1.580023867552888, 2.5839838573569534, 1.5871346370389885, 4.1337617125689725, 0.0016876790012300346, 2.337637442823331, 1.9268402331708496, 2.509223443618991, 2.8573979857307554, 2.7429627532040692, 2.3184117402885605, 2.2519888495692886, 1.441733890843454, 2.3283267069330638, 2.090507069768287, 1.616388780668859, 0.30852077577914405, 1.2418308849676503, 0.749579822648432, 2.0216862557918627, 1.8471265276557536, 1.9409844374088654, 2.029630658555306, 1.7488835640200255, 1.4429217698293368, 0.09853693097516952, 1.5685094105495399, 0.060035092997817674, 1.1562109869575399, 0.9883011451997243, 1.257630809337659, 1.6997562951967606, 0.4508041502784602, 0.3164090446061367, 1.4182969827012353, 1.3595162629204571, 1.475081471520821, 3.021289456736385, 3.0956206508951407, 2.481681959913101, 2.1308362149915583, 2.9008847410243757, 2.909122424144527, 2.7204695218309363, 2.087469496170989, 1.3538856364999683, 0.2008306196121235, 1.600964614957816, 1.4250387287265247, 1.6911311698607534, 1.1526705582269934, 0.7292975452608408, 0.4173852602091413, 0.2662677349829448, 1.5122097133171966, 1.0836674077065491, 1.031414686946613, 0.8173974802808452, 0.7095506482976092, 0.5949976898998941, 0.29670773819783247, 0.5083829941296425, 0.6440505445429058, 0.053833868680306, 7.794970013943188e-16, 0.7045208512938965, 1.6120699276991068, 1.5318231382310976, 1.7340273755678486, 2.4381603392169247, 2.6170276892155027, 2.5906147953962844, 3.407886846404895, 3.6212633825479905, 2.329416094716585, 2.958660792491976, 2.6669305434977773, 2.1590860132417564, 2.4937418622856145, 2.5589089762124786, 1.3338118137548678, 1.1942787402156965, 1.7418035300505763, 1.4081188229888932, 1.2487225960375548, 1.4492676314261193, 1.2654783371285574, 1.1685912162797853, 1.0148303874944786, 1.1962440020710763, 1.305708313372589, 0.6602148155530632, 0.337166044389861, 0.8396055147211476, 0.8562349502018103, 0.588778548048956, 0.7049070769530035, 1.2538977263308646, 1.4831897704424597, 1.4593441911500031, 2.1621717599542505, 2.4273857543891015, 2.426355640271325, 2.83832034285733, 2.7641303296460444, 2.2050969080359004, 2.6355562577584215, 3.1005626046243817, 2.4089187966341488, 1.8919645338161346, 1.8840157076492403, 1.344761629829863, 1.404294123950026, 1.8721961393692923, 1.3226408636613955, 0.4215497636181964, 0.5726863357586803, 1.0258923965461795, 1.1819610363504558, 0.8368490648663582, 0.6515561348082733, 0.6685731745760881, 0.5334870649826413, 0.8710519187832059, 0.6669646197224997, 0.5260752114304805, 0.3876797985565807, 0.03621327582895155, 0.46897871650384915, 0.8718533580569904, 0.7009452451531725, 1.4931853849244896, 1.8652719440498333, 1.6631794982365034, 1.494779190512575, 2.508688004725345, 3.0433643835464537, 3.2533878501144433, 3.579790260747532, 2.164640103097207, 0.6698924809914789, 2.1342050222506663, 2.5814605344559984, 1.6583152357630657, 1.3111552900920307, 1.20851491437197, 0.3334479204279151, 0.0027238985981172218, 0.7485037657977041, 0.23706880539492062, 0.3990097623354095, 4.751136081369487e-05, 1.5362095500430528, 0.46926869783190056, 0.0007246232360620678, 0.0, 5.239717734593537e-16, 9.938359637204445e-16, 0.0, 1.7385067095755083e-16, 4.106727240038999e-15, 0.10511094949367368, 0.026846967487429776, 0.0796163088839284, 0.8797518497354565, 2.616397453997683, 3.9912371044774604, 3.6233174077890604, 1.5672138389164023, 1.8304904251881515, 2.748948532497653, 3.287747311072218, 4.0926517829783675, 6.029811160308903]


In [ ]:
#@title Verification
import numpy as np

convolution_1 = np.convolve(step_function_heights_1, step_function_heights_1)
C_upper_bound = 2 * len(step_function_heights_1) * np.max(convolution_1) / np.sum(step_function_heights_1)**2
print(f"The discovered step function shows C <= {C_upper_bound}.")

In [ ]:
#@title Visualization
plot_step_function(step_function_heights_1)

## Second autocorrelation inequality, the constant C'
Let $C'$ be the smallest constant for which one has
$$ \|f*f\|_2^2 \leq C' \|f*f\|_1 \|f*f\|_\infty$$
for all non-negative $f:\mathbb{R} \rightarrow \mathbb{R}$.
It is known that
$$ 0.88922 \leq C' \leq 1,$$
with the lower bound coming from a step function construction by [Matolcsi and Vinuesa (2010)](https://www.sciencedirect.com/science/article/pii/S0022247X10006001). AlphaEvolve found a  step function that improves the lower bound to $0.8962$.

In [ ]:
#@title Data
import numpy as np

heights_sequence_2 = [0.0022217502753395443, 0.798058737836952, 0.4369294656327977, 1.1704958412868685, 1.3413665690827143, 1.5342366222696133, 1.7690742844401723, 1.9329450122360183, 2.2225113878900893, 1.9363966992163675, 2.0382191032475467, 2.2010898310433933, 2.0229605588392388, 2.029541518023742, 2.2636974412575626, 1.9622346498507677, 2.0781053776466134, 2.9856571697702514, 3.4418422600649374, 3.3477129878607825, 3.253250196453988, 3.420135507780267, 3.2509579118114464, 3.2308578066681575, 3.4707132763246245, 2.6462657430572087, 0.9614362498214617, 0, 0.0008733532713782356, 0.00041056186458359313, 0.00029587319086208687, 5.039012949497012e-06, 0, 0.5858888998745988, 6.741440691998236, 7.934548956206666e-06, 0.00013382382526231794, 4.551621108101551e-06, 0.0008898629473865954, 1.083008496291632e-05, 0.0006121618352774956, 0.0011493704284828532, 7.157034681754761, 9.111886252846807, 3.3127569806426527, 8.556232703271356e-06, 0.00017950056213609822, 2.7122354902710758e-06, 1.4036462843158317e-05, 1.1451768709981007e-05]
convolution_2 = np.convolve(heights_sequence_2, heights_sequence_2)

In [ ]:
#@title Verification

# Calculate the 2-norm squared: ||f*f||_2^2
num_points = len(convolution_2)
x_points = np.linspace(-0.5, 0.5, num_points + 2)
x_intervals = np.diff(x_points) # Width of each interval
y_points = np.concatenate(([0], convolution_2, [0]))
l2_norm_squared = 0.0
for i in range(len(convolution_2) + 1):  # Iterate through intervals
  y1 = y_points[i]
  y2 = y_points[i+1]
  h = x_intervals[i]
  # Integral of (mx + c)^2 = h/3 * (y1^2 + y1*y2 + y2^2) where m = (y2-y1)/h, c = y1 - m*x1, interval is [x1, x2], y1 = mx1+c, y2=mx2+c
  interval_l2_squared = (h / 3) * (y1**2 + y1 * y2 + y2**2)
  l2_norm_squared += interval_l2_squared

# Calculate the 1-norm: ||f*f||_1
norm_1 = np.sum(np.abs(convolution_2)) / (len(convolution_2) + 1)

# Calculate the infinity-norm: ||f*f||_inf
norm_inf = np.max(np.abs(convolution_2))
C_lower_bound = l2_norm_squared / (norm_1 * norm_inf)

print(f"This step function shows that C' >= {C_lower_bound}")

In [ ]:
#@title Visualization
plot_step_function(heights_sequence_2)

## Third autocorrelation inequality, the constant C''

Let $C''$ be the largest constant for which one has
$$ \max_{t \in \mathbb{R}} \left|f*f(t)\right| \geq C'' \left(\int f(x)\ dx\right)^2$$
for any function $f: \mathbb{R} \rightarrow \mathbb{R}$. Then there is a step function that gives the upper bound $C'' \leq 1.45810$, see page 75 of [Vinuesa's thesis (2009)](https://www.icmat.es/Thesis/CVinuesa.pdf). AlphaEvolve discovered a step function that improves this bound to $1.4556$.

In [ ]:
#@title Data
import numpy as np

height_sequence_3 = np.array([1.2065002259, 0.4579577248, 0.5163353850, 0.4072619373, 0.3357781229, 0.2857531610, 0.2820916949, 0.2605397021, -0.0945455083, 0.0263675774, 0.0066053345, 0.4231327261, 0.1460589430, 0.2693078743, 0.0894390737, 0.1956637306, 0.2479890347, -0.0252634907, 0.1662117356, -0.2218493784, 0.0739312998, 0.2415490072, 0.1698246070, 0.3250002134, 0.0020647116, -0.0688859195, 0.0585905178, 0.0939453029, -0.3060000075, -0.1837206775, -0.2352337979, 0.0835844954, -0.0058309576, -0.0832913333, -0.0166100706, -0.0593295469, -0.0358427923, -0.0137145337, -0.2200581708, -0.0020420409, -0.0154295569, -0.1668919220, 0.1254567765, 0.0104704418, -0.1281078357, -0.0961891358, -0.0180100824, 0.0285695118, -0.1876421178, -0.0503548117, 0.0705665536, -0.0849054288, -0.1712106082, 0.1330779045, -0.0395224921, -0.1763118729, 0.1943874403, 0.3136366436, 0.2907763337, 0.0972482020, 0.2859490417, 0.0544729197, 0.2859115117, 0.1089164067, 0.0171270233, -0.0434679259, -0.1024364047, -0.1167575690, -0.0003774918, -0.0344795844, -0.0972136207, -0.1226383339, -0.0252432553, 0.0400353923, 0.0455096903, 0.0122078271, -0.0826149005, 0.0068314042, -0.1352404062, 0.1294605758, -0.0250363591, 0.0356465906, 0.0085605877, -0.0734465832, 0.0239223522, 0.0560616111, 0.1051222588, 0.1186521152, 0.2114356563, 0.0580833852, 0.1532644654, 0.0340677496, 0.0584859676, 0.0676309739, 0.0501669055, 0.0098826564, -0.0699961802, -0.0655442615, -0.2431379908, 0.0191721224, -0.0101123101, 0.0395025740, 0.0369236983, -0.0405410938, 0.0043783628, -0.0022377932, 0.1882810103, -0.0392647889, 0.1110883981, -0.0056550805, -0.0165795658, 0.0484482116, 0.1040965030, 0.0315331695, 0.0290282853, 0.1700089425, 0.0788158652, 0.3053423957, 0.1407591357, 0.1288565749, 0.0731018735, 0.1212327910, 0.0265656763, -0.0026874534, 0.1448773580, -0.1569172936, -0.0332279397, -0.0961953570, -0.1057990671, -0.0208089846, 0.0622545893, 0.0086508578, 0.1134542356, 0.0796647542, -0.0118068228, 0.1183252889, 0.0322959545, 0.1390562294, 0.1334327477, 0.0790933677, -0.0121592574, 0.0355197021, 0.0680023155, 0.1367771272, 0.2764105186, 0.0917362039, 0.1789370907, 0.0801451602, 0.1507336094, 0.1499095182, 0.1338559365, 0.0883362440, 0.0130685381, -0.0010501160, 0.0434483442, -0.0452555246, -0.0160911253, 0.0001037672, -0.0215276925, 0.0922451592, 0.0914967270, 0.1247376137, 0.0674269368, 0.1233065439, 0.0662811693, 0.1056929066, 0.1236410207, 0.1092652789, 0.1253119938, 0.1350819190, 0.0760669548, 0.0975781877, 0.3030153656, -0.0405615005, 0.0736039858, 0.1516256451, 0.2310415709, 0.0903196002, 0.1368532961, 0.1558915065, 0.1395537845, 0.1134530692, 0.1672360224, 0.1779020155, 0.1110537453, 0.3179860968, 0.2418612890, 0.1879389889, 0.1759242853, 0.1751315796, 0.0707347421, 0.1498901200, 0.0009217008, 0.1217537983, 0.0774209961, 0.1585992473, 0.0621134399, 0.1515100010, 0.0780855072, 0.0997527327, 0.1158696157, 0.0293743767, 0.0551188205, 0.0597680188, -0.2775720739, 0.0759704568, 0.1250316677, 0.2132197633, -0.0252846511, -0.2298263857, -0.1916394076, -0.0828138996, -0.4176293110, -0.7396661838, -0.5511700738, -0.5448611785, -0.1727176925, -0.1101347125, 0.2043220162, -0.1082288708, -0.1748688117, 0.0862407071, -0.0531903867, -0.0616308223, 0.0700887776, 0.2081066310, 0.0954712433, -0.2241273540, 0.0753693148, 0.0747931780, -0.0124247160, -0.0919875318, 0.0869789932, 0.0228948007, 0.0330558155, 0.0365086857, 0.0425990861, 0.0946517743, 0.0461941250, 0.0314389948, 0.1547977504, 0.0995246754, 0.0972214450, 0.0642031653, 0.1125917823, 0.0823779771, 0.1138470746, 0.0561091801, 0.0318706784, 0.0083585533, 0.0141864593, 0.0550841887, 0.0552072046, 0.0056539829, 0.0160881555, 0.0233437573, 0.0715285236, 0.0809609974, 0.0733507409, 0.0740536003, 0.0214532872, 0.0210531815, 0.0613127802, 0.1098196117, 0.0679995089, 0.0841437462, 0.0422816392, 0.0795029912, 0.1219842632, 0.0629392926, 0.0442479973, 0.1754482798, 0.1892151967, 0.0506690589, 0.1127489948, 0.0489164780, 0.1195561384, 0.0803870907, 0.1088302988, 0.0086773701, 0.0350070450, -0.0039451117, -0.0491785508, 0.0777041816, 0.1256031497, 0.0758115755, 0.0338649690, 0.0774479672, 0.1600267712, 0.1116514343, 0.0393425971, 0.0880571065, 0.1752844511, 0.0567110352, 0.0848220320, 0.1023379468, 0.1731927552, 0.1015859496, 0.0456034593, 0.1484852609, 0.2335600810, 0.2680922857, 0.0964039102, 0.1370762748, 0.1194107677, 0.0705488401, 0.0492937128, 0.0010280509, 0.0748767880, 0.0413276428, -0.0120807694, -0.1011133614, 0.0378549584, 0.0742557513, 0.0834599468, 0.0828277891, 0.1937349434, 0.0722637117, 0.0539880627, 0.1092967592, 0.1314066075, 0.2276951671, 0.0993954217, 0.0506623717, 0.0665635040, 0.1475022076, 0.1348793959, 0.1120191289, 0.2268202779, 0.1968416912, 0.1785939824, 0.0653614828, 0.1308139129, 0.1845238365, 0.0400508726, -0.0030416083, 0.0612302085, 0.0446805476, 0.0413828618, 0.0087997710, -0.0288662567, 0.0106587067, 0.0459708143, 0.0777709776, 0.1302908220, 0.1269325239, 0.1144772737, 0.1448279485, 0.0854580145, 0.0780628793, 0.1185972450, 0.1083017344, 0.1382659716, 0.1161843882, 0.1031074870, 0.1081799914, 0.0968144848, 0.1369405176, 0.1597198239, 0.1076990897, 0.0998522537, 0.0980860040, 0.0856864888, 0.1147330385, 0.1090380435, 0.0478343797, 0.1104568134, 0.0535653627, 0.0395945072, 0.0615590595, 0.0846094168, 0.1366808400, 0.0979556009, 0.1150407810, 0.1093077855, 0.0701841815, 0.1536579376, 0.0281163265, -0.0188640466, 0.1095483493, 0.2435488950, 0.2224846591, 0.0349037817, 0.0471722551, 0.1366470882, 0.1619649438, -0.0201219126, 0.0776987862, 0.1770040649, 0.1642189494, 0.1510757616, 0.2256426202, 0.1989086542, 0.1778289411, 0.1869462368, 0.2779359655, 0.2578238178, 0.3850396218, 0.4772987312, 0.5429612326])
convolution_3 = np.convolve(height_sequence_3, height_sequence_3)

In [ ]:
#@title Verification
C_upper_bound = abs(2 * len(height_sequence_3) * np.max(convolution_3) / (np.sum(height_sequence_3)**2))
print(f"The discovered step function shows C'' <= {C_upper_bound}")

In [ ]:
#@title Visualization
plot_step_function(height_sequence_3)

# Packing disjoint circles inside a unit square to maximize sum of radii

Given a positive integer $n$, the problem is to pack $n$ disjoint circles inside a unit square so as to maximize the sum of their radii. The state of the art can be found on [Erich Friedman's homepage](https://erich-friedman.github.io/packing/cirRsqu/).

* For $n=26$, the SOTA was $2.634$, and AlphaEvolve improved it to $2.635$ (Construction 1).
* For $n=32$, the SOTA was $2.936$, and AlphaEvolve improved it to $2.937$ (Construction 2).

The constructions found by AlphaEvolve are shown below.

In [ ]:
#@title Verification function
import numpy as np
import itertools

def verify_circles(circles: np.ndarray):
  """Checks that the circles are disjoint and lie inside a unit square.

    Args:
      circles: A numpy array of shape (num_circles, 3), where each row is
        of the form (x, y, radius), specifying a circle.

    Raises:
      AssertionError if the circles are not disjoint or do not lie inside the
      unit square.
  """
  # Check pairwise disjointness.
  for circle1, circle2 in itertools.combinations(circles, 2):
    center_distance = np.sqrt((circle1[0] - circle2[0])**2 + (circle1[1] - circle2[1])**2)
    radii_sum = circle1[2] + circle2[2]
    assert center_distance >= radii_sum, f"Circles are NOT disjoint: {circle1} and {circle2}."

  # Check all circles lie inside the unit square [0,1]x[0,1].
  for circle in circles:
    assert (0 <= min(circle[0], circle[1]) - circle[2] and max(circle[0],circle[1]) + circle[2] <= 1), f"Circle {circle} is NOT fully inside the unit square."

In [ ]:
#@title Visualization function
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_circles(circles: np.ndarray):
  """Plots the circles."""
  fig, ax = plt.subplots(1, figsize=(10, 10))
  ax.set_xlim(0, 1)
  ax.set_ylim(0, 1)
  ax.set_aspect('equal')  # Make axes scaled equally.

  # Draw unit square boundary.
  rect = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor='black', facecolor='none')
  ax.add_patch(rect)

  # Draw the circles.
  for circle in circles:
    circ = patches.Circle((circle[0], circle[1]), circle[2], edgecolor='blue', facecolor='skyblue', alpha=0.5)
    ax.add_patch(circ)

  plt.title(f'A collection of {len(circles)} disjoint circles packed into the unit square to maximize the sum of radii')
  plt.show()

In [ ]:
#@title Construction 1: data

construction_1 = np.array([[0.68180423, 0.90401948, 0.09598051040194801], [0.90598057, 0.49972596, 0.094019420598057], [0.08464495, 0.08464502, 0.084644941535505], [0.4818405, 0.1034156, 0.10341558965844], [0.48234279, 0.89652324, 0.10347674965232401], [0.88928295, 0.11071646, 0.11071644892835401], [0.07852351, 0.50033803, 0.07852350214764901], [0.27330428, 0.1051224, 0.10512238948776001], [0.38153556, 0.70211016, 0.115517618448237], [0.13252625, 0.70386646, 0.132047406795258], [0.59610341, 0.72720176, 0.10051153994884501], [0.59576502, 0.2725969, 0.10054788994521], [0.88902242, 0.88902318, 0.11097680890231801], [0.68087256, 0.09573004, 0.095730030426996], [0.53098174, 0.49996661, 0.13587084641291403], [0.90742967, 0.68631006, 0.09257029074297], [0.74197859, 0.40386139, 0.095943250405674], [0.08471273, 0.91528735, 0.08471263152873601], [0.76300349, 0.7598366, 0.069494603050539], [0.27358888, 0.89472069, 0.105279299472069], [0.13242993, 0.29639184, 0.132426596757339], [0.90745313, 0.31316575, 0.092546580745341], [0.76284098, 0.23948486, 0.06975189302481001], [0.38095848, 0.29797266, 0.115741008425898], [0.74207757, 0.59573774, 0.09593309040669], [0.27141024, 0.50032568, 0.114361818563817]])

In [ ]:
#@title Construction 1: verification
print(f"Construction 1 has {len(construction_1)} circles.")
verify_circles(construction_1)
print(f"The circles are disjoint and lie inside the unit square.")
sum_radii = np.sum(construction_1[:, 2])
print(f"Construction 1 sum of radii: {sum_radii}")

In [ ]:
#@title Construction 1: visualization
plot_circles(construction_1)

In [ ]:
#@title Construction 2: data

construction_2 = np.array([[0.09076163, 0.40381803, 0.090761620923837], [0.07310993, 0.92689178, 0.07310821268917801], [0.08745017, 0.22570576, 0.087381421261857], [0.24855246, 0.30880277, 0.093428060657193], [0.4079865, 0.06300614, 0.063006133699386], [0.47646318, 0.90136179, 0.09863820013617901], [0.89604966, 0.10309934, 0.10309932969006601], [0.9066386, 0.68096117, 0.09336139066386], [0.08962002, 0.76509474, 0.0895289910471], [0.06973669, 0.06965159, 0.06965158303484101], [0.40979823, 0.21756451, 0.09156283084371601], [0.25742466, 0.88393887, 0.11606111839388701], [0.09064689, 0.58506214, 0.090482500951749], [0.90294698, 0.30231577, 0.09623644037635501], [0.57265603, 0.10585396, 0.105853949414604], [0.74007588, 0.40129314, 0.09435083056491601], [0.57539962, 0.71183255, 0.115160168483982], [0.7367635, 0.21592191, 0.09104997089500201], [0.41096972, 0.40263617, 0.093512520648747], [0.88664452, 0.88667032, 0.113317128668286], [0.57582722, 0.49961748, 0.09705531029446801], [0.24962585, 0.49417195, 0.09194421080557799], [0.90546338, 0.49309632, 0.094507120549287], [0.67381348, 0.90149423, 0.09850576014942301], [0.24310147, 0.1077195, 0.10771948922805], [0.40815297, 0.5886157, 0.09248833075116601], [0.24737889, 0.6771266, 0.090994980900501], [0.75801377, 0.7532924, 0.07192969280703], [0.73526642, 0.06243992, 0.062439303756069], [0.57415412, 0.30715219, 0.095403150459684], [0.39239379, 0.75259664, 0.07223814277618501], [0.7439361, 0.58879735, 0.093166630683336]])

In [ ]:
#@title Construction 2: verification
print(f"Construction 2 has {len(construction_2)} circles.")
verify_circles(construction_2)
print(f"The circles are disjoint and lie inside the unit square.")
sum_radii = np.sum(construction_2[:, 2])
print(f"Construction 2 sum of radii: {sum_radii}")

In [ ]:
#@title Construction 2: visualization
plot_circles(construction_2)

# Packing disjoint circles inside a rectangle of perimeter 4 to maximize sum of radii

Given a positive integer $n$, the problem is to pack $n$ disjoint circles inside a rectangle of perimeter 4 so as to maximize the sum of their radii. The state of the art can be found on [Erich Friedman's homepage](https://erich-friedman.github.io/packing/cirRrec/).

For $n=21$, the SOTA was $2.364$, and AlphaEvolve improved it to $2.365$. The construction found by AlphaEvolve is shown below.

In [ ]:
#@title Data
import numpy as np

circles = np.array([[0.12396161, 0.12396161, 0.12396159760383901], [0.0747415, 0.31647214, 0.07474149252585001], [0.12740019, 0.51163445, 0.127400177259981], [0.0747415, 0.70679675, 0.07474149252585001], [0.12396161, 0.89930729, 0.12396159760383901], [0.36284141, 0.11508312, 0.115083108491688], [0.25916864, 0.31452627, 0.10969588903041], [0.37244262, 0.51163445, 0.117642228235776], [0.25916864, 0.70874263, 0.10969588903041], [0.3628414, 0.90818577, 0.11508311849168701], [0.59083599, 0.11292169, 0.11292167870783101], [0.48083152, 0.30914434, 0.112032298796769], [0.62891008, 0.51163445, 0.138825206117478], [0.48083152, 0.71412455, 0.112032298796769], [0.59083598, 0.9103472, 0.11292168870783001], [0.67852787, 0.2919303, 0.086412051358794], [0.85766825, 0.39257159, 0.11906284809371401], [0.85766825, 0.6306973, 0.11906284809371401], [0.67852787, 0.73133859, 0.086412051358794], [0.83965981, 0.8861976, 0.13707128629287002], [0.83965981, 0.1370713, 0.13707128629287002]])

In [ ]:
#@title Verification
import itertools

def minimum_circumscribing_rectangle(circles: np.ndarray) -> tuple[float, float]:
  """Returns the width and height of the minimum circumscribing rectangle.

  Args:
    circles: A numpy array of shape (num_circles, 3), where each row is of the
      form (x, y, radius), specifying a circle.

  Returns:
    A tuple (width, height) of the minimum circumscribing rectangle.
  """
  min_x = np.min(circles[:, 0] - circles[:, 2])
  max_x = np.max(circles[:, 0] + circles[:, 2])
  min_y = np.min(circles[:, 1] - circles[:, 2])
  max_y = np.max(circles[:, 1] + circles[:, 2])
  return max_x - min_x, max_y - min_y


def verify_circles_disjoint(circles: np.ndarray):
  """Checks that circles are disjoint.

  Args:
    circles: A numpy array of shape (num_circles, 3), where each row is of the
      form (x, y, radius), specifying a circle.

  Raises:
    AssertionError: if the circles are not disjoint.
  """
  for circle1, circle2 in itertools.combinations(circles, 2):
    center_distance = np.sqrt((circle1[0] - circle2[0])**2 + (circle1[1] - circle2[1])**2)
    radii_sum = circle1[2] + circle2[2]
    assert center_distance >= radii_sum, f"Circles are NOT disjoint: {circle1} and {circle2}."


num_circles = len(circles)
verify_circles_disjoint(circles)
print(f"Construction has {num_circles} disjoint circles.")
width, height = minimum_circumscribing_rectangle(circles)
print(f"Perimeter of minimum circumscribing rectangle: {2 * (width + height)}")
assert (width + height) <= 2, "The circles cannot be contained within a rectangle with perimeter 4."
sum_radii = np.sum(circles[:, 2])
print('Sum of radii:', sum_radii)

In [ ]:
#@title Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_circles_rectangle(circles: np.ndarray):
  width, height = minimum_circumscribing_rectangle(circles)
  _, ax = plt.subplots(1, figsize=(10, 10))
  ax.set_xlim(0, width)
  ax.set_ylim(0, height)
  ax.set_aspect('equal') # Make axes scaled equally.

  # Draw rectangle boundary.
  rect = patches.Rectangle((0, 0), width, height, linewidth=1, edgecolor='black', facecolor='none')
  ax.add_patch(rect)

  # Draw the circles.
  for circle in circles:
    circ = patches.Circle((circle[0], circle[1]), circle[2], edgecolor='blue', facecolor='skyblue', alpha=0.5)
    ax.add_patch(circ)

  plt.title(f"{len(circles)} disjoint circles packed inside a rectangle of perimeter {round(2 * width + 2 * height,6)}")
  plt.show()

plot_circles_rectangle(circles)